In [1]:
import numpy as np
import pandas as pd
import scipy.sparse

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
import string
from os import listdir
import os
import errno
import subprocess
from collections import defaultdict
import operator
import re
import pickle
import time
from collections import OrderedDict
import six # needed for Google Cloud client

!pip install tqdm
from tqdm import tqdm as ProgressBar
!pip install unidecode
import unidecode

#!pip install --upgrade --ignore-installed google-cloud-language
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

In [2]:
def translate_non_alphanumerics(to_translate, translate_to=u''):
    # https://stackoverflow.com/a/1324274/2491761
    not_letters = u'!"#%\'()&*+,-./:;<=>?@[\]^_`{|}~0123456789'
    if isinstance(to_translate, unicode):
        #print "unicode instance"
        translate_table = dict((ord(char), unicode(translate_to)) for char in not_letters)
    else:
        #print "not unicode instance"
        assert isinstance(to_translate, str)
        #translate_table = string.maketrans(not_letters, translate_to *len(not_letters))
        return to_translate.translate(None, string.punctuation + string.digits)
    return to_translate.translate(translate_table)

def extract_ingredients(full_ingred_list, strip_punct=True):
    # From: https://en.wikibooks.org/wiki/Cookbook:Units_of_measurement
    meas_units = [
        'teaspoon', 'teaspoons',
        't',
        'tsp', 'tsps',
        'tablespoon','tablespoons',
        'tbl',
        'tbs',
        'tbsp', 'tbsps',
        'fl', 'fluid',
        'oz', 'ozs',
        'ounce', 'ounces',
        'cup', 'cups',
        'c',
        'pint', 'pints',
        'p',
        'pt',
        'quart', 'quarts',
        'qt', 'qts',
        'q', 'qs',
        'gallon', 'gallons',
        'gal', 'gals',
        'jiggers', 'jigger',
        'ml',
        'milliliter', 'milliliters',
        'millilitre', 'millilitres',
        'cc',
        'l',
        'liter', 'liters',
        'litre', 'litres',
        'pinch', 'pinches',        
        'pound', 'pounds',
        'lb', 'lbs',
        'mg', 'mgs',
        'milligram', 'milligrams',
        'milligramme', 'milligrammes',
        'g', 'gs',
        'gram', 'grams',
        'gramme', 'grammes',
        'kg', 'kgs',
        'kilogram', 'kilograms',
        'kilogramme', 'kilogrammes',
        'mm', 'mms',
        'millimeter', 'millimeters',
        'millimetre', 'millimetres',
        'cm', 'cms',
        'centimeter', 'centimeters',
        'centimetre', 'centimetres',
        'm', 'ms',
        'meter', 'meters',
        'metre', 'metres',
        'inch', 'inches',
        'in', 'ins',
        ###
        'loaf', 'loaves',
        'pouch', 'pouches',
        'wedge', 'wedges',
        'drop', 'drops',
        'amount', 'amounts',
        'bulk', 'bulks',
        'coating', 'coatings',
        'carton', 'cartons',
        'count',
        'dusting', 'dustings',
        'roll', 'rolls',
        'hint', 'hints',
        'round', 'rounds',
        'cube', 'cubes',
        'husk', 'husks',
        'envelope', 'envelopes',
        'container', 'containers',
        'dash', 'dashes',
        'bitesize', 'bitesized',
        'bite', 'sized',
        'size',
        'each',
        'taste', 'desired',
        'can', 'cans',
        'unit', 'units',
        'box', 'boxes',
        'tub', 'tubs',
        'slab', 'slabs',
        'sprig', 'sprigs',
        'stalk', 'stalks',
        'matchstick', 'matchsticks',
        'balls',
        'clove', 'cloves',
        'slice', 'slices',
        'head', 'heads',
        'spear', 'spears',
        'chunk', 'chunks',
        'piece', 'pieces',
        'jar', 'jars',
        'package', 'packages',
        'pack', 'packs',
        'packet', 'packets',
        'bunch', 'bunches',
        'tube', 'tubes',
        'jug', 'jugs',
        'bottle', 'bottles',
        'stick', 'sticks',
        'strip', 'strips',
        'bag', 'bags',
        'dash', 'dashes',
        'container', 'containers',
        'envelope', 'envelopes',
        'sleeve', 'sleeves',
        'rounds',
        'sheet', 'sheets',
        'squares',
        'semicircles', 'semicircle',
        'circles', 'circle',
        'moons', 'halfmoons',
        'strands', 'strands',
        'ribbons', 'ribbon',
        'whole',
        'extra', 'extras',
        'dozen', 'dozens',
        'half', 'halves',
        'third', 'thirds',
        'fourth', 'fourths',
        'quarter', 'quarters',
        'fifth', 'fifths',
        'eighth', 'eighths',
    ]
    
    preparatory_descriptions = [
        'baked',
        'beat', 'beaten', 'wellbeaten',
        'blanched',
        'blended',
        'boiled',
        'bottled',
        'broiled',
        'broken',
        'browned',
        'canned',
        'chilled',
        'chipped',
        'chopped',
        'cleaned',
        'converted',
        'cooked',
        'cooled',
        'cored',
        'crispcooked',
        'crumbled',
        'crushed',
        'cubed',
        'cut',
        'defrosted',
        'deshelled',
        'desilked',
        'deveined',
        'diced', 'dice',
        'dissolved',
        'divided',
        'drained',
        'dried',
        'filleted',
        'filtered',
        'flaked',
        'fresh',
        'fried',
        'frozen',
        'grated',
        'grilled',
        'ground',
        'halved',
        'hardened',
        'heated',
        'hulled',
        'husked',
        'individual',
        'jarred',
        'juiced',
        'julienned', 'julienne', 'juliennecut', 'juliennesliced',
        'kneaded', 'kneading',
        'marinated',
        'mashed',
        'matchstickcut',
        'melted',
        'microwaved',
        'minced',
        'mixed',
        'opened',
        'packed',
        'peeled',
        'pitted',
        'pounded',
        'prepared',
        'pressed',
        'prewashed',
        'processed',
        'pulled',
        'quartered',
        'raw',
        'refrigerated',
        'removed',
        'rinsed',
        'ripe',
        'ripened',
        'ripped',
        'roasted',
        'rolled', 'rolling',
        'salted',
        'saved',
        'scored',
        'scrubbed',
        'seasoned',
        'sectioned',
        'seeded',
        'separated',
        'shaved',
        'shelled',
        'shredded',
        'shucked',
        'sifted',
        'smashed',
        'smoked',
        'snipped',
        'sliced',
        'slivered',
        'softened',
        'split',
        'squeezed',
        'steamed',
        'stemmed',
        'strained',
        'soaked',
        'sweetened',
        'thawed',
        'toasted',
        'torn',
        'trimmed',
        'uncored',
        'uncooked',
        'undrained',
        'unhusked',
        'unopened',
        'unpeeled',
        'unsalted',
        'unsweetened',
        'unwashed',
        'unwrapped',
        'use', 'used',
        'warmed',
        'washed',
        'wrapped',
        'zested',
    ]
    
    other_words_to_filter = [
        # articles:
        'a', 'an', 'the',
        # adverbs:
        'accordingly', 'according',
        'all',
        'approximately',
        'artificially ',
        'coarsely',
        'crisply',
        'crosswise',
        'diagonally', 
        'finely',
        'freshly',
        'horizontally',
        'left',
        'lengthwise', 
        'lightly',
        'like',
        'moreorless ',
        'nearly',
        'overnight',
        'piecewise',
        'plus',
        'right',
        'roughly',
        'sideways',
        'slightly',
        'stiffly',
        'then',
        'thickly',
        'thinly',
        'vertically',
        'very',
        'well',
        'widthwise', 
        # adjectives:
        'additional ',
        'aged',
        'allpurpose',
        'another',
        'any',
        'best',
        'certified',
        'chunky',
        'classic',
        'coarse', 
        'cold',
        'complete',
        'crisp',
        'diagonal',
        'discarded',
        'dry',
        'extralarge',
        'extrasmall', 
        'even',
        'fatfree',
        'favourite', 'favorite',
        'fine', 
        'firm',
        'flat',
        'fresh',
        'hard',
        'hot',
        'inchesthick', 'inchthick',
        'included',
        'inner',
        'jumbo',
        'large', 
        'lean',
        'leftover',
        'less',
        'long',
        'lowcalorie',
        'lowfat',
        'lowsodium',
        'matchsticksize',
        'medium',
        'moist',
        'more',
        'multicolored',
        'natural',
        'needed',
        'new',
        'nonfat',
        'old',
        'oldfashioned',
        'only',
        'optional',
        'organic',
        'original',
        'other',
        'outer',
        'pure',
        'real',
        'reduced',
        'reserved',
        'rough',
        'sharp',
        'short',
        'small',
        'stale',
        'soft',
        'such',
        'supreme',
        'thick',
        'thickcut',
        'thicklycut',
        'thicklysliced',
        'thicksliced',
        'thin',
        'thincut',
        'thinlycut',
        'thinlysliced',
        'thinsliced',
        'tough',
        'young',
        'your',
        # prepositions:
        'about',
        'above',
        'across',
        'against',
        'apart',
        'as',
        'at',
        'away',
        'below',
        'by',
        'for',
        'from',
        'in',
        'into',
        'of',
        'off',
        'on',
        'out',
        'to',
        'until',
        'with',
        'without',
        # conjunctions:
        'and',
        'but',
        'if',
        'or',
        'so',
        # abbreviations:
        'eg', 'ie',
        # nouns:
        'accompaniments',
        'blossoms',
        'bottoms', 'bottom',
        'choice',
        'directions',
        'ends', 'end',
        'fists', 'fist',
        'hands', 'hand',
        'hours', 'hour',
        'ingredients',
        'list',
        'minutes', 'minute',
        'slicer',
        'peeler',
        'product',
        'purpose',
        'reserve',
        'room',
        'seconds', 'second',
        'spirals', 'spiral',
        'temperature',
        'thickness',
        'thinness',
        'tops', 'top',
        # verbs:
        'add',
        'alternating',
        'are',
        'be',
        'discarding',
        'including',
        'is',
        'sit',
        'using',
    ]
    
    #print "Before..."
    #print full_ingred_list
    
    # strip punctuation chars
    # all lowercase
    if strip_punct:
        cleaned_ingred_list = [translate_non_alphanumerics(s).lower() for s in full_ingred_list]
    else:
        cleaned_ingred_list = full_ingred_list
    
    #print "length of cleaned_ingred_list: {}".format(len(cleaned_ingred_list))
    #print cleaned_ingred_list
    if len(cleaned_ingred_list) == 0:
        print "length of cleaned_ingred_list is zero"
        print "full_ingred_list:"
        print full_ingred_list
    
    # split on whitespace
    # throw out meas units
    #print
    #print "After..."
    if isinstance(cleaned_ingred_list[0], unicode):
        ingreds_only = [' '.join(w for w in unidecode.unidecode(ingred_item).split() if w not in meas_units and w not in preparatory_descriptions and w not in other_words_to_filter)
             for ingred_item in cleaned_ingred_list
             ]
    else:
        assert isinstance(cleaned_ingred_list[0], str)
        ingreds_only = [' '.join(w for w in ingred_item.split() if w not in meas_units and w not in preparatory_descriptions and w not in other_words_to_filter)
             for ingred_item in cleaned_ingred_list
             ]
    ingreds_only = filter(None, ingreds_only) # remove empty strings at end
    #print ingreds_only
    
    brand_names = [
        'a(r)', # A1 steak sauce
        'absolut(r)',
        'accents(r)', 'accents(tm)', 'accent(r)',
        'act ii(r)',
        'adams(r)',
        'al fresco(r)',
        'amafil(r)',
        'annies lemon chive dressing(r)',
        'archer farms(tm)',
        'arrowhead mills(r)',
        'artisan blends(r)',
        'aylmer(r) accents(r)',
        'babybel(r)',
        'bacardi (r)', 'bacardi(r)',
        'backdraft fire sauce(r)',
        'ball(r) kerr(r)',
        'ball park(r)',
        'barilla(r)',
        'barill pronto(tm)',
        'bay(r)', 'bay (tm)',
        'beau monde (tm)',
        'beechers(r) flagship', 'beechers flagship', 'beechers(r)',
        'beer shiner(r)',
        'belgioioso',
        'benedictine(r)',
        'bertolli(r)',
        'best foods(r)',
        'better than bouillon(r)',
        'betty crocker(r)',
        'birds eye(r)',
        'bisquick(r)',
        'bisquick (r)',
        'bobs red mill(r)',
        'borden(r)',
        'bragg(r)',
        'breeze(r)',
        'briannas home style(r)', 'briannas(r)',
        'brooks(r)',
        'bruces(r)',
        'budweiser(r)',
        'buitoni(r)',
        'bullseye(r)',
        'bumble bee(r)',
        'burnetts(r)',
        'bushs best(r)', 'bushs homestyle(r)', 'bushs chili magic(r)', 'bushs(r)',
        'butterball(r)',
        'butter buds(r)',
        'campari(r)',
        'campbells(r) healthy request', 'campbells(r)', 'campbells (r)',
        'carnation (r)',
        'carolina pride(r)',
        'carroll shelbys(r)',
        'cascadian farm(r)',
        'catelli bistro(r)',
        'cattlemens(r)',
        'cavenders(r)',
        'chambord(r)',
        'chef paul prudhommes salmon magic(r)',
        'cheese whiz(r)', 'cheez whiz(r)',
        'chobani(r)',
        'cholula(r)',
        'ciroc(r)',
        'clamato(r)',
        'classicmac(tm)',
        'classico (tm)',
        'classico (r)',
        'classico(r)',
        'claussen(r)',
        'coke(r)',
        'cointreau(r)',
        'college inn(r)',
        'contadina(r)',
        'cool whip(r)',
        'coors(r)',
        'cornnuts (r)',
        'corona(r)',
        'cortland',
        'country time(r)',
        'craisins(r)',
        'crazy steves(tm) cajun cukes', 'crazy steves(r)',
        'cremovo(r)',
        'crisco(r)',
        'daisy(r)',
        'dei fratelli(r)',
        'del monte(r)',
        'diamond crystal(r)',
        'dickinsons(r)',
        'dole(r)',
        'dole asian island crunch(tm)',
        'dole(r) veggie(tm)',
        'doritos(r)',
        'dubliner(r)',
        'dukes(r)',
        'duncan hines(r)',
        'durkee(r)', 'famous sauce(r)',
        'eagle brand(r)',
        'earth balance(r)',
        'eckrich(r)',
        'edward sons(r)',
        'egg beaters(r)',
        'el paso(r) ',
        'el pato(r)',
        'embasa(r)',
        'emerald(r)',
        'equal(r)',
        'farmland(r)',
        'fat tire(r)',
        'fiesta blend(r)',
        'fleischmanns(r)', 'fleischmanns ',
        'folgers(r)',
        'follow your heart(r) veganaise(r)', 'follow your heart(r)',
        'fosters (r)',
        'foster farms(r)',
        'franks(r) redhot', 'franks red hot (r)', 'franks redhot (r)', 'franks redhot(r)', 'redhot(r)', 'franks(r)',
        'frenchs(r)',
        'frichik(r)',
        'fritos(r)',
        'frontera(r)',
        'goya (r)', 'goya(r)',
        'gebhardt(r)',
        'ghirardelli(r)',
        'girards(r) olde venice',
        'good seasons(r)',
        'goodness(r)',
        'gourmet garden(tm)',
        'grapenuts(tm)',
        'great american spice co(tm)',
        'green giant(r)', 'green giant(tm) crumbles(r)', 'green giant(tm)',
        'grill mates(r)',
        'guinness(r)',
        'hangar one(r)',
        'harp(r)',
        'hatch(r)',
        'healthy choice(r)',
        'heineken(r)', 
        'heinz (r)', 'heinz(r)',
        "hellmann's(r) best foods(r)", 'hellmanns(r) best foods(r)', 'hellmanns(r)best foods(r)',
        'hellmanns light(r)', 'hellmans(r) light', 'hellmanns(r) low fat',
        'hellmans(r)', "hellmann's(r)", 'hellmanns(r)',
        'hershey(r)s', 'hershey(r)s kisses(r)', 'hersheys(r) chipits(r)', 'hersheys(r)',
        'hidden valley ranch(r)', 'hidden valley(r) original ranch(r)', 'hidden valley(r) ranch(r)', 'hidden valley(r)', 'hidden valley (r)',
        'hillshire farm(r)',
        'holland house(r)',
        'horizon(r)',
        'hormel(r)', 'hormel(tm)',
        'house tsang(r)',
        'hunts(r)',
        'idahoan(r)','baby reds(r)', 'buttery homestyle(r)',
        'imagine(r)', 'imagine(tm)',
        'isalys(r)',
        'jack daniels(r)',
        'jello(r)',
        'jennieo(r)',
        'jif(r)',
        'jiffy(r)',
        'jim beam(r)',
        'jimmy dean(r)',
        'johnsonville(r)',
        'johnnys seasoning salt(r)',
        'jose cuervo(r)', 'cuervo(r)',
        'kame(r)',
        'karo(r)',
        'karys(r)',
        'kc masterpiece(r)',
        'keebler club(r)', 'keebler(r)',
        'keens(r)',
        'kens steak house(r)',
        'kens steak house lite northern italian dressing(r)',
        'kerrygold(r)',
        'kewpie(r)',
        'kikkoman(r)',
        'king arthur(r)',
        'king oscar(r)',
        'kirkland(r)',
        'kitchen basics(r)',
        'kitchen bouquet(r)',
        'kitchen secrets(r)',
        'klondike gourmet(r)', 'klondike goldust(r)', 'klondike(r)',
        'knorr(r) rice sides(tm)', 'knorr(r) fiesta sides(tm)', 'knorr(r)', 'knorrs(r)',
        'knox (r)',
        'kraft(r)',
        'land o lakes(r)', 'land olakes(r)',
        'laphroiag(r)',
        'laughing cow(r)',
        'lawrys(r)',
        'libbys(r)',
        'lightlife(r) organic smoky tempeh strips(r)', 'lightlife(r)',
        'lipton(r) recipe secrets(r)', 'lipton(r)',
        'locatelli(r)',
        'loma linda(r)',
        'lotus(r)',
        'louisiana(r)',
        'maggi(r)',
        'magic(r)',
        'mahatma(r)',
        'maifun(r)',
        'maille(r)',
        'maldon(r)',
        'malibu(r)',
        'marie callenders(r)',
        'maries(r)',
        'market pantry(tm)',
        'martha white(r)',
        'mary kitchen(r)',
        'marzettis(r)',
        'maui(r)',
        'mazola(r)',
        'mccormick chili seasoning mix sodium(r)', 'mccormick(r)',
        'mexene(r)',
        'mexenes(r)',
        'mexicorn(r)',
        'miller lite(r)',
        'mimiccreme(r)',
        'minute(r)',
        'miracle whip(r)',
        'miracle whip(tm)',
        'miracle whip (tm)',
        'miracle whip free(r)',
        'miracle whip light(r)', 'light miracle whip(r)',
        'mission(r)',
        'montreal steak seasoning(r)',
        'morningstar farms(r) grillers recipe crumbles(r)', 'morningstar farms(r) recipe crumbles(r)', 'morningstar farms(r)',
        'morton(r) natures seasons(r)', 'mortons(r)', 'morton(r)',
        'motts(r)',
        'mountain dew(r)',
        'mrs butterworths(r)',
        'mrs dash(r)',
        'muir glen(r)',
        'nabisco(r)',
        'natural goodness(tm)',
        'negra modelo(r)',
        'nescafe(r)',
        'nestle(r) toll house(r)', 'nestle(r) carnation(r)', 'nestle(r)',
        'newcastle(r)',
        'newmans own(r)',
        'newmans own(r) lighten up(r)', 'lighten up(r)',
        'niblets(r)',
        'nissin(r) top ramen',
        'no yolks(r)',
        'ocean spray(r)',
        'oikos(r)',
        'old bay(r)', 'old bay (tm)', 'old bay(tm)',
        'old el paso(r)',
        'olive garden(r)',
        'oreida(r) steam n mash(r)', 'oreida(r)', 'ore ida(r)',
        'oreo(r)',
        'ortega(r)',
        'ottogi ramyonsari(r)', 'ottogi(r)',
        'pace(r)',
        'pam(r)',
        'parkay(r)',
        'pataks(r)',
        'penzeys(r)', 'penzys(r)',
        'pepperidge farm(r)',
        'perdue(r)',
        'pernod(r)',
        'perrins(r)',
        'pet(r)',
        'pillsbury grands(r)', 'pillsbury(r) grands(r)', 'pillsbury(r)',
        'planters(r)',
        'plochmans(r)',
        'prego(r)',
        'progresso(tm) recipe starters(tm)', 'progresso(r)',
        'quaker(r)',        
        'quorn(tm)',
        'ragu(r) old world style(r)', 'ragu(r)',
        'rapunzel(r)',
        'ready rice(r)',
        'realime(r)',
        'red gold(r)',
        'reddiwip(r)',
        'reeses(r)',
        'renees(r)',
        'reynolds wrap(r)', 'reynolds(r)',
        'rice sides(tm)',
        'ronzoni(r)',
        'rotel(r)',
        'rumford(r)',
        'saco(r)',
        'sambazon',
        'samuel smith(r)',
        'sargento(r)', 'sargento (r)',
        'sason accent(r)',
        'sb(r) golden curry', 'sb(r)',
        'seagrams(r)',
        'seasonall(r)',
        'shiner bock(r)',
        'silk(r)',
        'simple crisp(tm)',
        'simply balanced(tm)',
        'simply potatoes(r)',
        'skippy(r)',
        'skyline chili(r)',
        'slap ya mama(r)',
        'smart balance(r)',
        'smirnoff(r)',
        'smithfield(r)',
        'smuckers(r)',
        'snack factory(r)',
        'snickers(r)',
        'so delicious(r)',
        'southern comfort (r)',
        #'spam(r)',
        'spectrum naturals(r)', 'spectrum(r)',
        'spice islands(r)',
        'spike(r)',
        'splenda(r)',
        'sprite(r)',
        'sriracha(r)',
        'steam whistle pilsner(r)',
        'stovetop(r)',
        'sunmaid(r)',
        'sunsweet(r) dnoir(tm)',
        'sutton dodge(tm)',
        'swanson(r) vegetable flavor boost(r)', 'swanson(r)', 'natural goodness(r)',
        'sweet baby rays(r)',
        'sweetn low(r)',
        'tabasco(r)', 'tabasco(tm)',
        'taco bell(r)',
        'tajin(r)',
        'tapatio(r)',
        'tater tots(r)',
        'tenderflake(r)',
        'texas pete(r)', 'texas petess(r)',
        'thai kitchen(r)',
        'three bridges(r)',
        'tillamook(r)',
        'tony chacheres(r)',
        'top ramen(r)',
        'tostitos(r)',
        'trader joes(r)',
        'truroots(r)',
        'truvia(r)',
        'tyson(r) ready(r)', 'tyson(r)',
        'uncle bens (r)', 'uncle bens(r)',
        'up(r)', # 7-up
        'v(r)', # what remains from "V8"
        'valentina(r)',
        'vegenaise(r)',
        'veggie(tm)',
        #'velveeta(r)',
        'vernors(r)',
        'vidalia(r)',
        'vietti(r)',
        'vindaloo sauce maya kaimal(r)', 'vindaloo(r)',
        'voskos(r)',
        'watkins(r)',
        'wesson(r)',
        'white lily(r)',
        'wishbone(r)',
        'wolf brand (r)', 'wolf(r)',
        'wondra(r)',
        'yoplait(r)',
        'yuengling(r)',
        'yves veggie cuisine(r)',
        'zatarains(r)',
        'zing zang(r)',
    ]
    
    def remove_brands(ingred_str):
        str_without_brand = ingred_str
        for bn in brand_names:
            str_without_brand = str_without_brand.replace(bn, '')
        return str_without_brand
    
    ingreds_without_brands = [' '.join(remove_brands(ingred_str).strip().split()) for ingred_str in ingreds_only]
    
    def remove_duplicate_words(ingred_str):
        ingred_words = ingred_str.split()
        no_dupes_ingred_str = " ".join(sorted(set(ingred_words), key=ingred_words.index))
        return no_dupes_ingred_str
    
    ingreds_duplicates_removed = [remove_duplicate_words(ingred_str) for ingred_str in ingreds_without_brands]
    
    return ingreds_duplicates_removed


In [3]:
def syntax_text(text):
    """Detects syntax in the text."""
    client = language.LanguageServiceClient()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    # Instantiates a plain text document.
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

    # Detects syntax in the document. You can also analyze HTML with:
    #   document.type == enums.Document.Type.HTML
    tokens = client.analyze_syntax(document).tokens

    # part-of-speech tags from enums.PartOfSpeech.Tag
    pos_tag = ('UNKNOWN', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM',
               'PRON', 'PRT', 'PUNCT', 'VERB', 'X', 'AFFIX')

    token_pos_dict = OrderedDict()
    for token in tokens:
        #print(u'{}: {}'.format(pos_tag[token.part_of_speech.tag], token.text.content))
        if token.text.content not in token_pos_dict:
            token_pos_dict[token.text.content] = pos_tag[token.part_of_speech.tag]
        else:
            #print "{} token already in dict".format(token.text.content)
            pass
    time.sleep(.3)   # avoid Google API quota
    return token_pos_dict

In [4]:
def pos_dict_to_str(d):
    # Takes an OrderedDict that maps each ingredient to a POS
    # Returns a string containing only ingredient words that are nouns or adjectives
    pos_tags_to_remove = ('UNKNOWN', 'ADP', 'ADV', 'CONJ', 'DET', 'NUM', 'PRON', 'PRT', 'PUNCT', 'VERB', 'X', 'AFFIX')
    # keep only nouns and adjectives
    #d_keep = {k:v for k, v in d.iteritems() if v not in pos_tags_to_remove}
    d_keep = OrderedDict((k, v) for k, v in d.iteritems() if v not in pos_tags_to_remove)

    #     for k, v in d_keep.iteritems():
    #         print "word: {} -> tag: {}".format(k, v)

    keep_str = ' '.join('{}'.format(key) for key, val in d_keep.iteritems())
    #print keep_str
    return keep_str

In [5]:
def do_google_api_processing(ingredient_string):
    # takes one ingredient string
    d = syntax_text(ingredient_string)
    noun_adj_str = pos_dict_to_str(d)
    return noun_adj_str
    
    
def reduce_ingreds_list(ingreds_list):
    reduced_ingreds_list = [do_google_api_processing(unidecode.unidecode(ingred_str)) for ingred_str in ingreds_list]
    print reduced_ingreds_list
    return reduced_ingreds_list



In [6]:
df = pd.read_pickle('CleanedIngredients.pkl')
print len(df)

89061


In [7]:
pd.set_option('display.max_colwidth', -1)

In [8]:
df.head(10)

,categories,cookingTime,description,ingredients,instructionSteps,name,rating,ratingCount,url,cookingTimeMinutes,cleanedIngredients
0,"[Tomato Salsa, Salsa, Dips and Spreads, Appetizers and Snacks, Mexican Recipes, Everyday Cooking, Recipes, Summer Appetizers, Mexican Appetizers, Cilantro]",PT15M,NaN,"[6 roma (plum) tomatoes, diced, 1 sweet onion, diced, 1 medium red bell pepper, diced, 1 medium yellow bell pepper, diced, 1 bunch cilantro, finely minced, 1 lime, juiced, 1 teaspoon salt, or to taste, Add all ingredients to list, Add all ingredients to list]","[In a bowl, mix the tomatoes, onion, red bell pepper, yellow bell pepper, cilantro, lime juice, and salt. Cover and refrigerate until ready to serve.]",Fresco Salsa,4.61,61,http://allrecipes.com/recipe/88328/fresco-salsa/,15.0,"[roma plum tomatoes, sweet onion, red bell pepper, yellow bell pepper, cilantro, lime, salt]"
1,"[Kids Healthy Snacks, Kids' Snacks, Snack Recipes, Salsa, Dips and Spreads, Appetizers and Snacks, Mexican Recipes, Everyday Cooking, Recipes, Kids' Nut-Free Snacks]",PT45M,"Easy to make, tasty fruit salsa and cinnamon tortilla chips. Great as an appetizer or a snack. Great for anytime!","[2 kiwis, peeled and diced, 2 Golden Delicious apples - peeled, cored and diced, 8 ounces raspberries, 1 pound strawberries, 2 tablespoons white sugar, 1 tablespoon brown sugar, 3 tablespoons fruit preserves, any flavor, 10 (10 inch) flour tortillas, butter flavored cooking spray, 2 tablespoons cinnamon sugar, Add all ingredients to list, Add all ingredients to list]","[In a large bowl, thoroughly mix kiwis, Golden Delicious apples, raspberries, strawberries, white sugar, brown sugar and fruit preserves. Cover and chill in the refrigerator at least 15 minutes., Preheat oven to 350 degrees F (175 degrees C)., Coat one side of each flour tortilla with butter flavored cooking spray. Cut into wedges and arrange in a single layer on a large baking sheet. Sprinkle wedges with desired amount of cinnamon sugar. Spray again with cooking spray., Bake in the preheated oven 8 to 10 minutes. Repeat with any remaining tortilla wedges. Allow to cool approximately 15 minutes. Serve with chilled fruit mixture.]",Annie's Fruit Salsa and Cinnamon Chips,4.78,3239,http://allrecipes.com/recipe/26692/annies-fruit-salsa-and-cinnamon-chips/,45.0,"[kiwis, golden delicious apples, raspberries, strawberries, white sugar, brown sugar, fruit preserves flavor, flour tortillas, butter flavored cooking spray, cinnamon sugar]"
2,"[Potatoes Au Gratin, Potato Side Dishes, Side Dishes, French Recipes, Everyday Cooking, Recipes, Easter Side Dishes, Allrecipes Magazine Side Dishes, Gourmet Side Dishes, Potato Recipes]",PT2H,"This is my husband's favorite dish, and he considers it a special occasion every time I make it. The creamy cheese sauce and the tender potatoes in this classic French dish combine to make a deliciously addictive experience. It's a great side dish with a roast pork loin or beef tenderloin. Add a green salad and French bread, and you have found the magic path to a man's heart. To avoid lumps in your sauce, add the milk just a little at a time as you stir the flour and butter. Experiment with different cheeses for variety.","[4 russet potatoes, sliced into 1/4 inch slices, 1 onion, sliced into rings, salt and pepper to taste, 3 tablespoons butter, 3 tablespoons all-purpose flour, 1/2 teaspoon salt, 2 cups milk, 1 1/2 cups shredded Cheddar cheese, Add all ingredients to list, Add all ingredients to list]","[Preheat oven to 400 degrees F (200 degrees C). Butter a 1 quart casserole dish., Layer 1/2 of the potatoes into bottom of the prepared casserole dish. Top with the onion slices, and add the remaining potatoes. Season with salt and pepper to taste., In a medium-size saucepan, melt butter over medium heat. Mix in the flour and salt, and stir constantly with a whisk for one minute. Stir in milk. Cook until mixture has thickened. Stir in cheese all at once, and continue stirring until melted,

In [34]:
#%%time

df['POS_cleaned_ingredients2'] = df['ingredients'].apply(reduce_ingreds_list)

df.sample(5)

['roma plum tomatoes', 'sweet onion', 'medium red bell pepper', 'medium yellow bell pepper', 'bunch cilantro', 'lime', 'teaspoon salt', 'ingredients', 'ingredients']
['kiwis diced', 'Golden Delicious apples diced', 'ounces raspberries', 'pound strawberries', 'tablespoons white sugar', 'tablespoon brown sugar', 'tablespoons fruit preserves flavor', 'inch flour tortillas', 'butter cooking spray', 'tablespoons cinnamon sugar', 'ingredients', 'ingredients']
['russet potatoes inch slices', 'onion rings', 'salt pepper', 'tablespoons butter', 'tablespoons purpose flour', 'teaspoon salt', 'cups milk', 'cups Cheddar cheese', 'ingredients', 'ingredients']
['cup butter', 'cup butter shortening', 'cup light brown sugar', 'cup white sugar', 'eggs', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon ground cinnamon', 'teaspoon ground cloves', 'teaspoon salt', 'cups oats', 'cup raisins', 'ingredients', 'ingredients']
['cup purpose flour coating', 'teaspoon salt', 'tea

RetryError: RetryError(Exception occurred in retry method that was not classified as transient, caused by <_Rendezvous of RPC that terminated with (StatusCode.RESOURCE_EXHAUSTED, Insufficient tokens for quota 'DefaultGroup' and limit 'CLIENT_PROJECT-100s' of service 'language.googleapis.com' for consumer 'project_number:764086051850'.)>)

In [9]:
ingredients_google_nouns_adjs = [
    ['roma plum tomatoes', 'sweet onion', 'medium red bell pepper', 'medium yellow bell pepper', 'bunch cilantro', 'lime', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['kiwis diced', 'Golden Delicious apples diced', 'ounces raspberries', 'pound strawberries', 'tablespoons white sugar', 'tablespoon brown sugar', 'tablespoons fruit preserves flavor', 'inch flour tortillas', 'butter cooking spray', 'tablespoons cinnamon sugar', 'ingredients', 'ingredients'],
    ['russet potatoes inch slices', 'onion rings', 'salt pepper', 'tablespoons butter', 'tablespoons purpose flour', 'teaspoon salt', 'cups milk', 'cups Cheddar cheese', 'ingredients', 'ingredients'],
    ['cup butter', 'cup butter shortening', 'cup light brown sugar', 'cup white sugar', 'eggs', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon ground cinnamon', 'teaspoon ground cloves', 'teaspoon salt', 'cups oats', 'cup raisins', 'ingredients', 'ingredients'],
    ['cup purpose flour coating', 'teaspoon salt', 'teaspoon ground black pepper', 'teaspoon oregano', 'skinless boneless chicken breast halves inch thick', 'tablespoons butter', 'tablespoons olive oil', 'cup mushrooms', 'cup Marsala wine', 'cup cooking sherry', 'ingredients', 'ingredients'],
    ['pound cooked chicken', 'ounce tomatoes', 'ounce sauce', 'medium onion', 'ounce green chile peppers', 'cloves garlic', 'cups water', 'ounce chicken broth', 'teaspoon cumin', 'teaspoon chili powder', 'teaspoon salt', 'teaspoon black pepper', 'bay leaf', 'ounce frozen corn', 'tablespoon cilantro', 'corn tortillas', 'vegetable oil', 'ingredients', 'ingredients'],
    ['cup white sugar', 'teaspoon ground cinnamon', 'cup butter', 'cups white sugar', 'eggs', 'ripe bananas', 'ounce container sour cream', 'teaspoons vanilla extract', 'teaspoons ground cinnamon', 'teaspoon salt', 'teaspoons soda', 'cups purpose flour', 'cup walnuts optional', 'ingredients', 'ingredients'],
    ['teaspoon vegetable oil', 'onion', 'cloves garlic', 'cup quinoa', 'cups vegetable broth', 'teaspoon ground cumin', 'teaspoon cayenne pepper', 'salt black pepper', 'cup frozen corn kernels', 'ounce cans black beans', 'cup fresh cilantro', 'ingredients', 'ingredients'],
    ['cup olive oil', 'cloves garlic', 'cup Italian bread crumbs', 'cup Parmesan cheese', 'skinless boneless chicken breast halves', 'ingredients', 'ingredients'],
    ['cup butter', 'cup white sugar', 'eggs', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'cup sour cream', 'cup walnuts', 'medium bananas', 'ingredients', 'ingredients'],
    ['avocados mashed', 'lime', 'teaspoon salt', 'cup diced onion', 'tablespoons fresh cilantro', 'roma plum tomatoes', 'teaspoon garlic', 'pinch ground cayenne pepper optional', 'ingredients', 'ingredients'],
    ['cups purpose apples', 'cup white sugar', 'tablespoon purpose flour', 'teaspoon ground cinnamon', 'cup water', 'cup quick cooking oats', 'cup purpose flour', 'cup brown sugar', 'teaspoon baking powder', 'teaspoon baking soda', 'cup butter', 'ingredients', 'ingredients'],
    ['eggs', 'cups vegetable oil', 'cups white sugar', 'teaspoons vanilla extract', 'cups purpose flour', 'teaspoons soda', 'teaspoons powder', 'teaspoon salt', 'teaspoons ground cinnamon', 'cups carrots', 'cup pecans', 'cup butter', 'ounces cream cheese', 'cups confectioners sugar', 'teaspoon vanilla extract', 'cup pecans', 'ingredients', 'ingredients'],
    ['onion', 'ounce chili beans', 'ounce black beans', 'ounce whole kernel corn', 'ounce tomato sauce', 'fluid ounce beer', 'ounce cans tomatoes green chilies undrained', 'ounce package taco seasoning', 'whole skinless boneless chicken breasts', 'ounce package Cheddar cheese optional', 'sour cream optional', 'tortilla chips optional', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoon salt', 'teaspoon baking soda', 'teaspoon baking powder', 'tablespoon ground cinnamon', 'eggs', 'cup vegetable oil', 'cups white sugar', 'teaspoons vanilla extract', 'cups zucchini', 'cup walnuts', 'ingredients', 'ingredients'],
    ['ounce pumpkin puree', 'eggs', 'cup vegetable oil', 'cup water', 'cups white sugar', 'cups purpose flour', 'teaspoons soda', 'teaspoons salt', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'teaspoon ground cloves', 'teaspoon ground ginger', 'ingredients', 'ingredients'],
    ['recipe pastry inch double crust pie', 'cup unsalted butter', 'tablespoons purpose flour', 'cup water', 'cup white sugar', 'cup brown sugar', 'Granny Smith apples', 'ingredients', 'ingredients'],
    ['ounce package cream cheese', 'ounce condensed milk', 'ounce instant vanilla pudding mix', 'cups cold milk', 'teaspoon vanilla extract', 'ounce container topping', 'bananas', 'ounce package vanilla wafers', 'ingredients', 'ingredients'],
    ['cup chocolate cookie crumbs', 'tablespoons white sugar', 'cup butter', 'ounce frozen raspberries', 'tablespoons white sugar', 'teaspoons cornstarch', 'cup water', 'cups white chocolate chips', 'cup half cream', 'ounce cream cheese', 'cup white sugar', 'eggs', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['fresh peaches thin wedges', 'cup white sugar', 'cup brown sugar', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'teaspoon fresh lemon juice', 'teaspoons cornstarch', 'cup purpose flour', 'cup white sugar', 'cup brown sugar', 'teaspoon baking powder', 'teaspoon salt', 'tablespoons unsalted butter chilled small pieces', 'cup water', '', 'tablespoons white sugar', 'teaspoon ground cinnamon', 'ingredients', 'ingredients'],
    ['inch graham cracker crust', 'cups condensed milk', 'cup sour cream', 'cup key lime juice', 'tablespoon lime zest', 'ingredients', 'ingredients'],
    ['large baking potatoes', 'bacon', 'cup sour cream', 'cup milk', 'tablespoons butter', 'teaspoon salt', 'teaspoon pepper', 'cup Cheddar cheese', 'green onions', 'ingredients', 'ingredients'],
    ['cups sweet potato', 'cup white sugar', 'eggs', 'teaspoon salt', 'tablespoons butter', 'cup milk', 'teaspoon vanilla extract', 'cup brown sugar', 'cup purpose flour', 'tablespoons butter', 'cup pecans', 'ingredients', 'ingredients'],
    ['ounce black beans', 'green bell pepper inch pieces', 'onion wedges', 'cloves garlic', 'egg', 'tablespoon chili powder', 'tablespoon cumin', 'teaspoon Thai chili sauce hot', 'cup bread crumbs', 'ingredients', 'ingredients'],
    ['eggplant', 'eggs', 'cups Italian seasoned bread crumbs', 'spaghetti sauce', 'ounce package mozzarella cheese', 'cup Parmesan cheese', 'teaspoon basil', 'ingredients', 'ingredients'],
    ['ounce package bow tie pasta', 'tablespoons olive oil', 'sweet Italian sausage casings', 'teaspoon red pepper flakes', 'cup diced onion', 'cloves garlic', 'ounce Italian style plum tomatoes', 'cups heavy cream', 'teaspoon salt', 'tablespoons fresh parsley', 'ingredients', 'ingredients'],
    ['cup butter', 'eggs', 'ounce dry corn bread mix', 'ounce whole kernel corn', 'ounce corn', 'cup sour cream', 'ingredients', 'ingredients'],
    ['cup soy sauce', 'cup olive oil', 'cup fresh lemon juice', 'cup Worcestershire sauce', 'tablespoons garlic powder', 'tablespoons basil', 'tablespoons parsley flakes', 'teaspoon ground white pepper', 'teaspoon hot pepper sauce optional', 'teaspoon garlic optional', 'ingredients', 'ingredients'],
    ['boneless skinless chicken breasts inch chunks', 'teaspoon salt pepper', 'tablespoons vegetable oil', 'tablespoons curry powder', 'onion', 'cloves garlic', 'ounce coconut milk', 'ounce tomatoes', 'ounce tomato sauce', 'tablespoons sugar', 'ingredients', 'ingredients'],
    ['ounce cream cheese', 'cup white sugar', 'teaspoon vanilla extract', 'eggs', 'inch graham cracker crust', 'cup pumpkin puree', 'teaspoon ground cinnamon', 'pinch ground cloves', 'pinch ground nutmeg', 'cup topping', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoon baking powder', 'teaspoon baking soda', 'teaspoon salt', 'large bananas', 'cup white sugar', 'egg', 'cup butter', 'ingredients', 'ingredients'],
    ['pounds Brussels sprouts yellow leaves', 'tablespoons olive oil', 'teaspoon kosher salt', 'teaspoon black pepper', 'ingredients', 'ingredients'],
    ['ounce cream cheese', 'cup mayonnaise', 'ounce green chilies', 'ounces jalapeno peppers', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['pound ground beef', 'egg', 'cup bread crumbs', 'teaspoon ground black pepper', 'teaspoon seasoned salt', 'teaspoon onion powder', 'teaspoon garlic powder', 'teaspoon Worcestershire sauce', 'tablespoon vegetable oil', 'cup onion', 'tablespoons purpose flour', 'cup beef broth', 'tablespoon cooking sherry', 'teaspoon seasoned salt', 'ingredients', 'ingredients'],
    ['tablespoons garlic', 'tablespoons olive oil', 'large head cauliflower florets', 'cup Parmesan cheese', 'salt black pepper', 'tablespoon fresh parsley', 'ingredients', 'ingredients'],
    ['ounce package cream cheese', 'cup mayonnaise', 'cup Parmesan cheese', 'cup Romano cheese', 'clove garlic', 'teaspoon basil', 'teaspoon garlic salt', 'salt pepper', 'ounce hearts', 'cup frozen spinach', 'cup mozzarella cheese', 'ingredients', 'ingredients'],
    ['pounds beef stew meat', 'tablespoons vegetable oil', 'cubes beef bouillon', 'cups water', 'teaspoon rosemary', 'teaspoon parsley', 'teaspoon ground black pepper', 'large potatoes cubed', 'carrots inch pieces', 'stalks celery inch pieces', 'large onion', 'teaspoons cornstarch', 'teaspoons cold water', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'tablespoons purpose flour', 'teaspoon salt', 'teaspoon white sugar', 'cup onion', 'cup sour cream', 'ounce cans French style green beans', 'cups Cheddar cheese', 'cup buttery round crackers', 'tablespoon butter', 'ingredients', 'ingredients'],
    ['cup butter', 'cloves garlic', 'cup dry bread crumbs', 'cup Parmesan cheese', 'cups Cheddar cheese', 'teaspoon parsley', 'teaspoon oregano', 'teaspoon ground black pepper', 'teaspoon salt', 'skinless boneless chicken breast halves thin', 'ingredients', 'ingredients'],
    ['ounce bag coleslaw mix', 'tablespoons onion', 'cup creamy salad dressing such Miracle Whip tm', 'tablespoons vegetable oil', 'cup white sugar', 'tablespoon white vinegar', 'teaspoon salt', 'teaspoon poppy seeds', 'ingredients', 'ingredients'],
    ['cloves garlic', 'ounce package spinach', 'ounce hearts', 'ounce container Alfredo style pasta sauce', 'cup mozzarella cheese', 'cup Parmesan cheese', 'ounce package cream cheese', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves inch thickness', 'teaspoons salt', 'slices bacon half', 'cup yellow mustard', 'cup honey', 'cup light corn syrup', 'cup mayonnaise', 'tablespoon onion flakes', 'tablespoon vegetable oil', 'cup fresh mushrooms', 'cups Colby Monterey Jack cheese', 'tablespoons fresh parsley', 'ingredients', 'ingredients'],
    ['pound Italian sausage', 'clove garlic', 'ounce cans beef broth', 'ounce Italian style stewed tomatoes', 'cup carrots', 'ounce great Northern beans undrained', 'small zucchini', 'cups spinach', 'teaspoon ground black pepper', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'clove garlic', 'cup dry bread crumbs', 'cup Parmesan cheese', 'teaspoon basil leaves', 'teaspoon ground black pepper', 'skinless boneless chicken breast halves', 'ingredients', 'ingredients'],
    ['pounds ground beef', 'fluid ounce tomato juice', 'ounce tomato sauce', 'ounce kidney beans', 'ounce pinto beans', 'cups onion', 'cup green bell pepper', 'teaspoon ground cayenne pepper', 'teaspoon white sugar', 'teaspoon oregano', 'teaspoon ground black pepper', 'teaspoon salt', 'teaspoons ground cumin', 'cup chili powder', 'ingredients', 'ingredients'],
    ['tablespoons onion', 'tablespoons margarine', 'cups cubed butternut squash', 'cups water', 'cubes chicken bouillon', 'teaspoon marjoram', 'teaspoon ground black pepper', 'teaspoon ground cayenne pepper', 'ounce cream cheese', 'ingredients', 'ingredients'],
    ['cups whole wheat bread', 'cups white bread', 'pound ground turkey sausage', 'cup onion', 'cup celery', 'teaspoons sage', 'teaspoons rosemary', 'teaspoon thyme', 'Golden Delicious apple', 'cup cranberries', 'cup fresh parsley', 'turkey liver', 'cup turkey stock', 'tablespoons unsalted butter', 'ingredients', 'ingredients'],
    ['large Granny Smith apples', 'ounce cans refrigerated crescent roll dough', 'cup butter', 'cups white sugar', 'teaspoon ground cinnamon', 'fluid ounce Mountain Dew tm', 'ingredients', 'ingredients'],
    ['ounce package hash brown potatoes', 'ounces diced ham', 'ounce cans cream potato soup', 'ounce container sour cream', 'cups sharp Cheddar cheese', 'cups Parmesan cheese', 'ingredients', 'ingredients'],
    ['tablespoon butter', 'cloves garlic', 'cup sun tomatoes', 'cup chicken broth', 'cup heavy cream', 'pound skinless boneless chicken breast halves', 'salt pepper', 'tablespoons vegetable oil', 'tablespoons fresh basil', 'ounces dry fettuccini pasta', 'ingredients', 'ingredients'],
    ['cup butter', 'cup white sugar', 'cups purpose flour', 'eggs', 'cups white sugar', 'cup purpose flour', 'lemons', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'onion', 'ounce package spinach', 'eggs', 'cups Muenster cheese', 'teaspoon salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['pounds skinless boneless chicken breast meat', 'cup red wine vinegar', 'lime', 'teaspoon white sugar', 'teaspoon salt', 'teaspoon ground black pepper', 'green onions', 'cloves garlic', 'teaspoon oregano', 'inch flour tortillas', 'tomato', 'cup lettuce', 'cup Monterey Jack cheese', 'cup salsa', 'ingredients', 'ingredients'],
    ['pound sweet potato', 'cup butter', 'cup white sugar', 'cup milk', 'eggs', 'teaspoon ground nutmeg', 'teaspoon ground cinnamon', 'teaspoon vanilla extract', 'inch unbaked pie crust', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'cup butter', 'teaspoons garlic', 'tablespoon lemon juice', 'ounce cream mushroom soup', 'tablespoon Italian seasoning', 'cup half', 'cup Parmesan cheese', 'ounce cans spinach', 'ounces fresh mushrooms', 'cup bacon bits', 'cups mozzarella cheese', 'ingredients', 'ingredients'],
    ['eggs', 'cup vegetable oil', 'teaspoons vanilla extract', 'cups white sugar', 'cups zucchini', 'cups purpose flour', 'teaspoon salt', 'teaspoon baking powder', 'teaspoon baking soda', 'tablespoon ground cinnamon', 'pint fresh blueberries', 'ingredients', 'ingredients'],
    ['ounce package taco seasoning mix', 'ounce beans', 'ounce package cream cheese', 'ounce container sour cream', 'ounce jar salsa', 'large tomato', 'green bell pepper', 'bunch green onions', 'small head iceberg lettuce', 'ounce black olives', 'cups Cheddar cheese', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'onion', 'cloves garlic', 'cups kidney beans', 'cups water', 'tablespoons chili powder', 'teaspoons prepared mustard', 'teaspoons ground cumin', 'cayenne pepper', 'tablespoons soy sauce', 'cups sweet potatoes', 'inch flour tortillas', 'ounces Cheddar cheese', 'ingredients', 'ingredients'],
    ['recipe pastry inch double crust pie', 'cup unsalted butter', 'tablespoons purpose flour', 'cup white sugar', 'cup brown sugar', 'cup water', 'Granny Smith apples', 'ingredients', 'ingredients'],
    ['chicken broth', 'tablespoons olive oil', 'pound portobello mushrooms', 'white mushrooms', 'shallots', 'cups Arborio rice', 'cup dry white wine', 'sea salt', 'black pepper', 'tablespoons chives', 'tablespoons butter', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['apples', 'cup water', 'cup white sugar', 'teaspoon ground cinnamon', 'ingredients', 'ingredients'],
    ['eggs', 'cup buttermilk', 'cup vegetable oil', 'cups white sugar', 'teaspoons vanilla extract', 'teaspoons ground cinnamon', 'teaspoon salt', 'cups purpose flour', 'teaspoons soda', 'cups carrots', 'cup coconut', 'cup walnuts', 'ounce pineapple juice', 'cup raisins', 'ingredients', 'ingredients'],
    ['ounce pumpkin', 'ounce EAGLE BRAND Condensed Milk', 'large eggs', 'teaspoon ground cinnamon', 'teaspoon ground ginger', 'teaspoon ground nutmeg', 'teaspoon salt', 'inch unbaked pie crust', 'ingredients', 'ingredients'],
    ['small butternut squash', 'red bell peppers seeded diced', 'sweet potato cubed', 'Yukon Gold potatoes', 'red onion', 'tablespoon fresh thyme', 'tablespoons fresh rosemary', 'cup olive oil', 'tablespoons balsamic vinegar', 'salt black pepper', 'ingredients', 'ingredients'],
    ['pound bacon', 'stalks celery', 'onion', 'cloves garlic', 'potatoes cubed', 'cups chicken stock enough potatoes', 'tablespoons butter', 'cup purpose flour', 'cup heavy cream', 'teaspoon tarragon', 'teaspoons fresh cilantro', 'salt pepper', 'ingredients', 'ingredients'],
    ['ounce black beans', 'ounce kidney beans', 'ounce beans', 'green bell pepper', 'red bell pepper', 'ounce frozen corn kernels', 'red onion', 'cup olive oil', 'cup red wine vinegar', 'tablespoons fresh lime juice', 'tablespoon lemon juice', 'tablespoons white sugar', 'tablespoon salt', 'garlic', 'cup fresh cilantro', 'tablespoon ground cumin', 'tablespoon ground black pepper', 'hot pepper sauce', 'teaspoon chili powder', 'ingredients', 'ingredients'],
    ['ounces cranberries', 'cup white sugar', 'cup orange juice', 'ingredients', 'ingredients'],
    ['cup white sugar', 'tablespoons purpose flour', 'tablespoons cornstarch', 'teaspoon salt', 'cups water', 'lemons', 'tablespoons butter', 'egg yolks', 'inch pie crust baked', 'egg whites', 'tablespoons white sugar', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'onion', 'cloves garlic', 'ounce jalapeno peppers', 'ounce green chile peppers', 'teaspoons ground cumin', 'teaspoon oregano', 'teaspoon ground cayenne pepper', 'ounce cans chicken broth', 'cups chicken breast', 'ounce cans white beans', 'cup Monterey Jack cheese', 'ingredients', 'ingredients'],
    ['sweet Italian sausage casings', 'cup onion', 'cloves garlic', 'beef broth', 'cup water', 'cup red wine', 'large tomatoes', 'cup carrots', 'tablespoon fresh basil leaves', 'teaspoon oregano', 'ounce tomato sauce', 'cups zucchini', 'ounces fresh tortellini pasta', 'tablespoons fresh parsley', 'ingredients', 'ingredients'],
    ['inch thick pork chops', 'teaspoon vegetable oil', 'tablespoons brown sugar', 'salt pepper', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'tablespoons unsalted butter', 'tart apples', 'tablespoons pecans optional', 'ingredients', 'ingredients'],
    ['onion', 'cup olive oil', 'carrots', 'stalks celery', 'cloves garlic', 'teaspoon oregano', 'bay leaf', 'teaspoon basil', 'ounce tomatoes', 'cups dry lentils', 'cups water', 'cup spinach', 'tablespoons vinegar', 'salt', 'ground black pepper', 'ingredients', 'ingredients'],
    ['bunch kale', 'tablespoon olive oil', 'teaspoon seasoned salt', 'ingredients', 'ingredients'],
    ['quarts water', 'skinless boneless chicken breast halves', 'teaspoon salt', 'teaspoon ground black pepper', 'teaspoon garlic powder', 'tablespoons parsley', 'tablespoon onion powder', 'cubes chicken bouillon', 'tablespoons olive oil', 'onion', 'cloves garlic', 'ounce jar chunky salsa', 'ounce cans tomatoes', 'ounce tomatoes', 'ounce tomato soup', 'tablespoons chili powder', 'ounce whole kernel corn', 'ounce cans chili beans undrained', 'ounce container sour cream', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'cloves garlic', 'onions', 'cups celery', 'carrots', 'chicken broth', 'cups water', 'tomato sauce', 'cup red wine optional', 'cup kidney beans', 'ounce green beans', 'cups baby spinach', 'zucchinis', 'tablespoon fresh oregano', 'tablespoons fresh basil', 'salt pepper', 'cup seashell pasta', 'tablespoons Parmesan cheese', 'tablespoon olive oil', 'ingredients', 'ingredients'],
    ['onion', 'cups dry pinto beans', 'fresh jalapeno pepper seeded', 'tablespoons garlic', 'teaspoons salt', 'teaspoons fresh ground black pepper', 'teaspoon ground cumin optional', 'cups water', 'ingredients', 'ingredients'],
    ['cup butter', 'cup white sugar', 'teaspoon almond extract', 'cups purpose flour', 'cup seedless raspberry jam', 'cup confectioners sugar', 'teaspoon almond extract', 'teaspoon milk', 'ingredients', 'ingredients'],
    ['ounce package cheese tortellini', 'ounce tomatoes garlic onion', 'cup fresh spinach', 'teaspoon salt', 'teaspoon pepper', 'teaspoons basil', 'teaspoon garlic', 'tablespoons purpose flour', 'cup milk', 'cup heavy cream', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['ounces penne pasta', 'cup butter', 'tablespoons extra-virgin olive oil', 'onion', 'cloves garlic', 'red bell pepper', 'pound portobello mushrooms', 'pound medium shrimp deveined', 'ounce jar Alfredo sauce', 'cup Romano cheese', 'cup cream', 'teaspoon cayenne pepper more', 'Salt pepper', 'cup parsley', 'ingredients', 'ingredients'],
    ['pound lean ground beef', 'cups salsa', 'ounce chili beans', 'tortilla chips', 'cups sour cream', 'ounce black olives', 'cup green onion', 'cup fresh tomato', 'cups Cheddar cheese', 'ingredients', 'ingredients'],
    ['slices bacon', 'cup margarine', 'cup purpose flour', 'cups milk', 'large baked potatoes cubed', 'green onions', 'cups Cheddar cheese', 'cup sour cream', 'teaspoon salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['ounce whole kernel corn', 'ounce cans chicken broth', 'ounce chunk chicken', 'ounce black beans', 'ounce tomatoes green chile peppers', 'ingredients', 'ingredients'],
    ['pound ground beef', 'onion', 'ounce chili beans liquid', 'ounce kidney beans liquid', 'ounce whole kernel corn liquid', 'ounce tomato sauce', 'cups water', 'ounce cans tomatoes', 'ounce green chile peppers', 'ounce package taco seasoning mix', 'ingredients', 'ingredients'],
    ['inch flour tortillas', 'tablespoons vegetable oil', 'small onion', 'red bell pepper', 'teaspoon garlic', 'ounce black beans', 'teaspoon jalapeno peppers', 'ounces cream cheese', 'teaspoon salt', 'tablespoons fresh cilantro', 'ingredients', 'ingredients'],
    ['spaghetti squash seeded', 'tablespoons vegetable oil', 'onion', 'clove garlic', 'cups tomatoes', 'cup feta cheese', 'tablespoons black olives', 'tablespoons fresh basil', 'ingredients', 'ingredients'],
    ['pounds fresh broccoli bite size pieces', 'cup butter', 'tablespoon brown sugar', 'tablespoons soy sauce', 'teaspoons white vinegar', 'teaspoon ground black pepper', 'cloves garlic', 'cup cashews', 'ingredients', 'ingredients'],
    ['pound prime rib roast', 'cloves garlic', 'tablespoons olive oil', 'teaspoons salt', 'teaspoons black pepper', 'teaspoons thyme', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'tablespoon white sugar', 'teaspoons powder', 'teaspoon salt', 'egg', 'cup milk', 'tablespoons vegetable oil', 'ripe bananas', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoon salt', 'teaspoons soda', 'cup butter margarine', 'cups white sugar', 'cups overripe bananas', 'eggs', 'cup walnuts', 'ingredients', 'ingredients'],
    ['ounces linguine pasta', 'skinless boneless chicken breast halves', 'teaspoons Cajun seasoning', 'tablespoons butter', 'red bell pepper', 'green bell pepper', 'fresh mushrooms', 'green onion', 'cup heavy cream', 'teaspoon basil', 'teaspoon lemon pepper', 'teaspoon salt', 'teaspoon garlic powder', 'teaspoon ground black pepper', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['cups garbanzo beans', 'cup tahini', 'cup lemon juice', 'teaspoon salt', 'cloves garlic', 'tablespoon olive oil', 'pinch paprika', 'teaspoon fresh parsley', 'ingredients', 'ingredients'],
    ['eggs', 'cup Parmesan cheese', 'ounces bread crumbs', 'skinless boneless chicken breast halves', 'tablespoon vegetable oil', 'ounces pasta sauce', 'Monterey Jack cheese', 'ingredients', 'ingredients'],
    ['Boston Bibb butter lettuce leaves', 'pound lean ground beef', 'tablespoon cooking oil', 'large onion', 'cup hoisin sauce', 'cloves fresh garlic', 'tablespoon soy sauce', 'tablespoon rice wine vinegar', 'teaspoons pickled ginger', 'Asian chile pepper sauce optional', 'ounce water chestnuts', 'bunch green onions', 'teaspoons Asian dark sesame oil', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves cubes', 'tablespoons butter', 'cloves garlic', 'tablespoon Italian seasoning', 'pound fettuccini pasta', 'onion', 'ounce package mushrooms', 'cup purpose flour', 'tablespoon salt', 'teaspoon ground white pepper', 'cups milk', 'cup half', 'cup Parmesan cheese', 'ounces Colby Monterey Jack cheese', 'roma plum tomatoes', 'cup sour cream', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'cloves garlic', 'skinless boneless chicken breast halves strips', 'cups fresh spinach leaves', 'ounce dry Alfredo sauce mix', 'tablespoons pesto', 'ounce dry penne pasta', 'tablespoon Romano cheese', 'ingredients', 'ingredients'],
    ['slices day old bread inch cubes', 'ounce cream cheese inch cubes', 'cup fresh blueberries', 'eggs', 'cups milk', 'teaspoon vanilla extract', 'cup maple syrup', 'cup white sugar', 'tablespoons cornstarch', 'cup water', 'cup fresh blueberries', 'tablespoon butter', 'ingredients', 'ingredients'],
    ['cup butter', 'cup white sugar', 'cup light brown sugar', 'eggs', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon ground cinnamon', 'teaspoon salt', 'cups oats', 'cup raisins', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'cup uncooked white rice', 'onion', 'green bell pepper', 'cups water', 'ounce tomatoes green chiles', 'teaspoons chili powder', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['Sauce', 'cups water', 'tablespoons orange juice', 'cup lemon juice', 'cup rice vinegar', 'tablespoons soy sauce', 'tablespoon orange zest', 'cup brown sugar', 'teaspoon fresh ginger root', 'teaspoon garlic', 'tablespoons green onion', 'teaspoon red pepper flakes', 'tablespoons cornstarch', 'tablespoons water', 'Chicken', 'boneless skinless chicken breasts inch pieces', 'cup purpose flour', 'teaspoon salt', 'teaspoon pepper', 'tablespoons olive oil', 'ingredients', 'ingredients'],
    ['pound lean ground beef', 'egg', 'tablespoons water', 'cup bread crumbs', 'tablespoons onion', 'ounce cranberry sauce', 'cup chili sauce', 'tablespoon brown sugar', 'teaspoons lemon juice', 'ingredients', 'ingredients'],
    ['pork chops', 'salt pepper', 'garlic salt', 'onion', 'pound fresh mushrooms', 'ounce cream mushroom soup', 'ingredients', 'ingredients'],
    ['pound red potatoes', 'tablespoons onion', 'tablespoons butter', 'tablespoon honey', 'teaspoon dry mustard', 'pinch salt', 'pinch ground black pepper', 'ingredients', 'ingredients'],
    ['eggs', 'cup buttermilk', 'cup vegetable oil', 'cup bananas', 'cups white sugar', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'cup pecans optional', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'small onion', 'stalk celery', 'medium carrot', 'medium potatoes', 'medium butternut squash cubed', 'fluid ounce container chicken stock', 'salt black pepper', 'ingredients', 'ingredients'],
    ['pounds unpeeled red potatoes', 'pounds butter room temperature', 'Romano cheese', 'cups garlic', 'cup salt', 'cup oregano', 'ingredients', 'ingredients'],
    ['egg', 'cup purpose flour', 'cup beer', 'teaspoons powder', 'cup purpose flour', 'cups coconut', 'shrimp', 'cups oil frying', 'ingredients', 'ingredients'],
    ['bunch thin asparagus spears', 'tablespoons olive oil', 'tablespoons Parmesan cheese optional', 'clove garlic optional', 'teaspoon sea salt', 'teaspoon ground black pepper', 'tablespoon lemon juice optional', 'ingredients', 'ingredients'],
    ['cup butter', 'tablespoons garlic', 'tablespoons soy sauce', 'teaspoon black pepper', 'tablespoon parsley', 'boneless chicken thighs skin', 'parsley', 'ingredients', 'ingredients'],
    ['ounce cans black beans', 'cups vegetable broth', 'cup chunky salsa', 'teaspoon ground cumin', 'tablespoons sour cream', 'tablespoons green onion', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'skinless boneless chicken breast halves', 'teaspoons garlic powder', 'teaspoon seasoning salt', 'teaspoon onion powder', 'ingredients', 'ingredients'],
    ['ounce frozen corn kernels', 'cup heavy cream', 'teaspoon salt', 'tablespoons granulated sugar', 'teaspoon black pepper', 'tablespoons butter', 'cup whole milk', 'tablespoons purpose flour', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['cloves garlic', 'cup mayonnaise', 'anchovy fillets', 'tablespoons Parmesan cheese', 'teaspoon Worcestershire sauce', 'teaspoon Dijon mustard', 'tablespoon lemon juice more', 'salt', 'ground black pepper', 'cup olive oil', 'cups day old bread', 'head romaine lettuce bite size pieces', 'ingredients', 'ingredients'],
    ['ounce package lemon cake mix', 'eggs', 'cup vegetable oil', 'teaspoon lemon extract', 'cup confectioners sugar decoration', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'medium onion', 'bay leaves', 'teaspoon ground cumin', 'tablespoons oregano', 'tablespoon salt', 'stalks celery', 'green bell peppers', 'jalapeno peppers', 'cloves garlic', 'ounce cans green chile peppers', 'ounce vegetarian burger crumbles', 'ounce cans whole tomatoes', 'cup chili powder', 'tablespoon ground black pepper', 'ounce kidney beans', 'ounce beans', 'ounce black beans', 'ounce whole kernel corn', 'ingredients', 'ingredients'],
    ['carrots', 'onions', 'ounce cans whole tomatoes liquid', 'large head cabbage', 'ounce envelope dry onion soup mix', 'ounce green beans', 'quarts tomato juice', 'green bell peppers', 'stalks celery', 'ounce broth', 'ingredients', 'ingredients'],
    ['large baking potato', 'tablespoon olive oil', 'teaspoon paprika', 'teaspoon garlic powder', 'teaspoon chili powder', 'teaspoon onion powder', 'ingredients', 'ingredients'],
    ['cup light brown sugar', 'cup purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'cup oats', 'cup butter', 'cup seedless raspberry jam', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoon baking powder', 'teaspoon baking soda', 'teaspoon salt', 'cup butter', 'cup white sugar', 'eggs', 'teaspoons vanilla', 'cups apples', 'cup brown sugar', 'tablespoon purpose flour', 'teaspoon ground cinnamon', 'tablespoon butter', 'ingredients', 'ingredients'],
    ['cooking spray', 'uncooked lasagna noodles', 'onion', 'cloves garlic', 'ounce vegetable broth', 'tablespoon fresh rosemary', 'ounce artichoke hearts', 'ounce package spinach dry', 'ounce jar tomato pasta sauce', 'cups mozzarella cheese', 'ounce package herb garlic feta', 'ingredients', 'ingredients'],
    ['ounce package linguine pasta', 'tablespoon olive oil', 'cloves garlic', 'cup chicken broth', 'cup white wine', 'lemon', 'teaspoon lemon zest', 'salt', 'teaspoons black pepper', 'pound fresh shrimp deveined', 'cup butter', 'tablespoons fresh parsley', 'tablespoon fresh basil', 'ingredients', 'ingredients'],
    ['onion', 'cloves garlic', 'tablespoon olive oil', 'teaspoons chili powder', 'teaspoon oregano', 'ounce tomatoes', 'ounce chicken broth', 'cups water', 'cup whole corn kernels', 'cup white hominy', 'ounce green chile peppers', 'ounce black beans', 'cup fresh cilantro', 'boneless chicken breast halves bite sized pieces', 'tortilla chips', 'sliced avocado', 'Monterey Jack cheese', 'green onions', 'ingredients', 'ingredients'],
    ['cups chicken meat', 'cup mayonnaise', 'teaspoon paprika', 'cups cranberries', 'cup celery', 'green onions', 'cup green bell pepper', 'cup pecans', 'teaspoon seasoning salt', 'ground black pepper', 'ingredients', 'ingredients'],
    ['cups mushrooms', 'skinless boneless chicken breast halves', 'eggs', 'cup seasoned bread crumbs', 'tablespoons butter', 'ounces mozzarella cheese', 'cup chicken broth', 'ingredients', 'ingredients'],
    ['pound fresh broccoli', 'cups chicken meat', 'ounce cream broccoli soup', 'cup milk', 'cup Cheddar cheese', 'tablespoon butter', 'tablespoons bread crumbs', 'ingredients', 'ingredients'],
    ['bunches spinach bite size pieces', 'cups strawberries', 'cup vegetable oil', 'cup white wine vinegar', 'cup white sugar', 'teaspoon paprika', 'tablespoons sesame seeds', 'tablespoon poppy seeds', 'ingredients', 'ingredients'],
    ['cup olive oil', 'tablespoon garlic', 'teaspoon basil', 'teaspoon marjoram', 'teaspoon dill weed', 'teaspoon thyme', 'teaspoon oregano', 'teaspoon parsley', 'teaspoon red pepper flakes', 'teaspoon salt', 'large potatoes cubed', 'ingredients', 'ingredients'],
    ['cup white sugar', 'tablespoons cornstarch', 'teaspoon salt', 'teaspoon ground cinnamon', 'cups fresh blueberries', 'recipe pastry inch double crust pie', 'tablespoon butter', 'ingredients', 'ingredients'],
    ['cabbage', 'cup white rice', 'egg', 'cup milk', 'cup onion', 'pound extra-lean ground beef', 'teaspoons salt', 'teaspoons black pepper', 'ounce tomato sauce', 'tablespoon brown sugar', 'tablespoon lemon juice', 'teaspoon Worcestershire sauce', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'onion', 'clove garlic', 'ounce Italian style diced tomatoes undrained', 'tablespoon basil leaves', 'teaspoon white sugar', 'teaspoon oregano', 'teaspoon salt', 'teaspoon ground black pepper', 'cup heavy cream', 'tablespoon butter', 'ingredients', 'ingredients'],
    ['ounce jar salsa', 'ounce dry taco seasoning mix', 'lime', 'tablespoons fresh cilantro', 'pounds skinless boneless chicken breast halves', 'ingredients', 'ingredients'],
    ['red potatoes chunks', 'tablespoon garlic', 'cubes chicken bouillon', 'ounce container sour cream', 'ounce package cream cheese', 'cup butter', 'salt pepper', 'ingredients', 'ingredients'],
    ['pound ground beef', 'cup uncooked long grain white rice', 'cup water', 'green bell peppers', 'ounce cans tomato sauce', 'tablespoon Worcestershire sauce', 'teaspoon garlic powder', 'teaspoon onion powder', 'salt pepper', 'teaspoon Italian seasoning', 'ingredients', 'ingredients'],
    ['pound uncooked penne pasta', 'cup extra virgin olive oil', 'cloves garlic', 'teaspoon red pepper flakes', 'ounce tomatoes', 'teaspoon salt', 'tablespoons vodka', 'cup heavy whipping cream', 'cup fresh parsley', 'ounce sweet Italian sausage', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'cup butter', 'cup brown sugar', 'cup white sugar', 'egg', 'teaspoon vanilla extract', 'cups coconut', 'ingredients', 'ingredients'],
    ['cup butter', 'large tart apples inch thick', 'teaspoons cornstarch', 'cup cold water', 'cup brown sugar', 'teaspoon ground cinnamon', 'ingredients', 'ingredients'],
    ['cup mayonnaise', 'ounce package spinach', 'cup feta cheese', 'cloves garlic', 'skinless boneless chicken breasts', 'bacon', 'ingredients', 'ingredients'],
    ['cup brandy', 'cup lemon juice', 'cup frozen lemonade concentrate', 'cup orange juice', 'milliliter dry red wine', 'cup triple sec', 'lemon rounds', 'orange rounds', 'lime rounds', 'cup white sugar optional', 'maraschino cherries', 'cups carbonated water optional', 'ingredients', 'ingredients'],
    ['cup pumpkin', 'cup white sugar', 'cup vegetable oil', 'egg', 'cups purpose flour', 'teaspoons powder', 'teaspoons ground cinnamon', 'teaspoon salt', 'teaspoon baking soda', 'teaspoon milk', 'tablespoon vanilla extract', 'cups chocolate chips', 'cup walnuts optional', 'ingredients', 'ingredients'],
    ['cup butter', 'tablespoons olive oil', 'teaspoon coarse salt', 'teaspoon ground black pepper', 'cloves garlic', 'pound fresh asparagus spears', 'ingredients', 'ingredients'],
    ['eggs', 'cups white sugar', 'cup vegetable oil', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoon salt', 'teaspoon baking soda', 'cups sour cream', 'cups blueberries', 'ingredients', 'ingredients'],
    ['egg yolks', 'ounce condensed milk', 'cup key lime juice', 'inch graham cracker crust', 'ingredients', 'ingredients'],
    ['cup onion', 'stalks celery', 'carrot', 'cup butter', 'tablespoons purpose flour', 'teaspoons curry powder', 'chicken broth', 'apple', 'cup white rice', 'skinless boneless chicken breast cut cubes', 'salt', 'ground black pepper', 'thyme', 'cup heavy cream heated', 'ingredients', 'ingredients'],
    ['quarts plain popcorn', 'cups dry roasted peanuts optional', 'cups brown sugar', 'cup light corn syrup', 'teaspoon salt', 'cup margarine', 'teaspoon baking soda', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['teaspoon oregano', 'teaspoon salt', 'teaspoon ground black pepper', 'pounds skinless boneless chicken breast halves', 'tablespoons butter', 'cup water', 'tablespoons fresh lemon juice', 'cloves garlic', 'teaspoon chicken bouillon granules', 'teaspoon fresh parsley', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'teaspoon salt', 'large red onions', 'large sweet onions', 'fluid ounce chicken broth', 'ounce broth', 'cup red wine', 'tablespoon Worcestershire sauce', 'sprigs fresh parsley', 'fresh thyme leaves', 'bay leaf', 'tablespoon balsamic vinegar', 'salt black pepper', 'thick slices French Italian bread', 'slices Gruyere Swiss cheese room temperature', 'cup Asiago mozzarella cheese room temperature', 'paprika', 'ingredients', 'ingredients'],
    ['pound seashell pasta', 'pound Genoa salami', 'pound pepperoni sausage', 'pound Asiago cheese', 'ounce black olives', 'red bell pepper', 'green bell pepper', 'tomatoes', 'ounce dry Italian style salad dressing mix', 'cup extra virgin olive oil', 'cup balsamic vinegar', 'tablespoons oregano', 'tablespoon parsley', 'tablespoon Parmesan cheese', 'salt black pepper', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'teaspoon baking powder', 'teaspoon salt', 'teaspoon ground cinnamon', 'cup butter margarine', 'cup brown sugar', 'cup white sugar', 'egg', 'teaspoon vanilla extract', 'cup apple', 'cup walnuts', 'tablespoons white sugar', 'teaspoons ground cinnamon', 'ingredients', 'ingredients'],
    ['cups carrots', 'cup brown sugar', 'cup raisins', 'eggs', 'cups white sugar', 'cup vegetable oil', 'teaspoons vanilla extract', 'cup pineapple', 'cups purpose flour', 'teaspoons soda', 'teaspoon salt', 'teaspoons ground cinnamon', 'cup walnuts', 'ingredients', 'ingredients'],
    ['ounce sweet Italian sausage', 'tablespoons butter', 'yellow onion', 'red onion', 'cloves garlic', 'large red bell pepper', 'green bell pepper', 'teaspoon basil', 'teaspoon oregano', 'cup white wine', 'ingredients', 'ingredients'],
    ['ounce package penne pasta', 'tablespoons olive oil', 'skinless boneless chicken breast halves cubes', 'salt pepper', 'garlic powder', 'cup low sodium chicken broth', 'bunch slender asparagus spears diagonal inch pieces', 'clove garlic', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['ounce package linguine pasta', 'cup red onion', 'tablespoon olive oil', 'cloves garlic', 'pound skinless boneless chicken breast meat cut bite size pieces', 'ounce artichoke hearts', 'large tomato', 'cup feta cheese', 'tablespoons fresh parsley', 'tablespoons lemon juice', 'teaspoons oregano', 'salt pepper', 'lemons garnish', 'ingredients', 'ingredients'],
    ['cup light olive oil', 'cup white sugar', 'teaspoons vanilla extract', 'teaspoon almond extract', 'eggs', 'cups purpose flour', 'teaspoon salt', 'teaspoon baking powder', 'cup cranberries', 'cups pistachio nuts', 'ingredients', 'ingredients'],
    ['cup olive oil', 'cup fresh parsley', 'lemon', 'tablespoons hot pepper sauce', 'cloves garlic', 'tablespoon tomato paste', 'teaspoons oregano', 'teaspoon salt', 'teaspoon ground black pepper', 'large shrimp tails', 'skewers', 'ingredients', 'ingredients'],
    ['cup butter', 'ounce uncooked medium egg noodles', 'medium onion', 'stalk celery', 'clove garlic', 'ounces button mushrooms', 'cup purpose flour', 'cups milk', 'salt pepper', 'ounce cans tuna', 'cup frozen peas', 'tablespoons bread crumbs', 'tablespoons butter', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['cups zucchini', 'egg', 'tablespoons butter', 'cup bread crumbs', 'cup onion', 'teaspoon Old Bay Seasoning TM', 'cup purpose flour', 'cup vegetable oil frying', 'ingredients', 'ingredients'],
    ['bacon', 'fresh broccoli', 'cups sharp Cheddar cheese', 'large red onion', 'cup red wine vinegar', 'cup white sugar', 'teaspoons black pepper', 'teaspoon salt', 'cup mayonnaise', 'teaspoon fresh lemon juice', 'ingredients', 'ingredients'],
    ['cups mashed sweet potatoes', 'cup butter', 'cup milk', 'cup white sugar', 'teaspoon vanilla extract', 'eggs', 'cup light brown sugar', 'cup purpose flour', 'cup butter', 'cup pecans', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'onion', 'stalk celery', 'chicken broth', 'broccoli florets', 'tablespoons butter', 'tablespoons purpose flour', 'cups milk', 'ground black pepper', 'ingredients', 'ingredients'],
    ['cup uncooked long grain white rice', 'cups water', 'Sauce', 'onion', 'tablespoon olive oil', 'cups marinara sauce', 'cup beef broth', 'tablespoon balsamic vinegar', 'teaspoon red pepper flakes', 'Peppers', 'pound lean ground beef', 'pound hot Italian pork sausage', 'ounce tomatoes', 'cup fresh Italian parsley', 'cloves garlic', 'teaspoons salt', 'teaspoon black pepper', 'pinch ground cayenne pepper', 'large green bell peppers seeded', 'cup Parmigiano Reggiano cheese more', 'ingredients', 'ingredients'],
    ['broccoli florets', 'teaspoons extra-virgin olive oil', 'teaspoon sea salt', 'teaspoon ground black pepper', 'clove garlic', 'teaspoon lemon juice', 'ingredients', 'ingredients'],
    ['cup balsamic vinegar', 'cup soy sauce', 'tablespoons garlic', 'tablespoons honey', 'tablespoons olive oil', 'teaspoons black pepper', 'teaspoon Worcestershire sauce', 'teaspoon onion powder', 'teaspoon salt', 'teaspoon liquid smoke flavoring', 'cayenne pepper', 'pound rib eye steaks', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'onion', 'teaspoon garlic', 'cups fresh broccoli', 'inch unbaked pie crust', 'cups mozzarella cheese', 'eggs', 'cups milk', 'teaspoon salt', 'teaspoon black pepper', 'tablespoon butter', 'ingredients', 'ingredients'],
    ['tomatoes seeded diced', 'tomato juice', 'fresh basil', 'cup heavy whipping cream', 'cup butter', 'salt pepper', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'cups coconut', 'teaspoon salt', 'ounce condensed milk', 'teaspoons vanilla extract', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breasts', 'egg', 'tablespoons lemon juice', 'cup purpose flour', 'teaspoon garlic powder', 'teaspoon paprika', 'cup butter', 'teaspoons chicken bouillon powder', 'cup water', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'pound ground turkey', 'ounce cans low sodium tomato soup', 'ounce cans kidney beans', 'ounce black beans', 'medium onion', 'tablespoons chili powder', 'teaspoon red pepper flakes', 'tablespoon garlic powder', 'tablespoon ground cumin', 'pinch ground black pepper', 'pinch ground allspice', 'salt', 'ingredients', 'ingredients'],
    ['large carrots matchstick pieces', 'baby red potatoes', 'onion bite sized pieces', 'cups water', 'pound corned beef brisket spice packet', 'ounces beer', 'head cabbage', 'ingredients', 'ingredients'],
    ['pound package hash brown potatoes', 'cup butter', 'ounce cream chicken soup', 'ounce container sour cream', 'cup onions', 'cups Cheddar cheese', 'teaspoon salt', 'teaspoon ground black pepper', 'cups cornflakes cereal', 'cup butter', 'ingredients', 'ingredients'],
    ['Sauce', 'ounce container plain yogurt', 'cucumber', 'teaspoon dill weed', 'salt pepper', 'tablespoon mayonnaise optional', 'Falafel', 'ounce chickpeas garbanzo beans', 'onion', 'cup fresh parsley', 'cloves garlic', 'egg', 'teaspoons ground cumin', 'teaspoon ground coriander', 'teaspoon salt', 'dash pepper', 'cayenne pepper', 'teaspoon lemon juice', 'teaspoon baking powder', 'tablespoon olive oil', 'cup dry bread crumbs', 'oil frying', 'pita breads half optional', 'cup tomatoes optional', 'ingredients', 'ingredients'],
    ['cup margarine', 'cup white sugar', 'eggs', 'cups banana', 'cups purpose flour', 'teaspoon baking soda', 'ingredients', 'ingredients'],
    ['fresh jalapeno peppers seeded', 'ounce package cream cheese', 'slices bacon', 'ingredients', 'ingredients'],
    ['teaspoons garlic powder', 'teaspoon ground black pepper', 'teaspoon salt', 'teaspoon paprika', 'cup seasoned bread crumbs', 'cup purpose flour', 'cup milk', 'egg', 'skinless boneless chicken breast halves thin', 'cup oil frying', 'ingredients', 'ingredients'],
    ['tablespoon butter', 'tablespoons olive oil', 'medium head garlic', 'ounce cans green beans', 'salt pepper', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['pound lean ground beef', 'cup onion', 'cup green bell pepper', 'ounce tomatoes', 'cup water', 'cup uncooked long grain rice', 'cup chile sauce', 'teaspoon salt', 'teaspoon brown sugar', 'teaspoon ground cumin', 'teaspoon Worcestershire sauce', 'pinch ground black pepper', 'cup Cheddar cheese', 'tablespoons fresh cilantro', 'ingredients', 'ingredients'],
    ['tablespoons soy sauce', 'tablespoons rice wine', 'tablespoon brown sugar', 'teaspoon cornstarch', 'tablespoon vegetable oil', 'tablespoon fresh ginger root', 'tablespoon garlic', 'pound beef round steak thin strips', 'ounces snow peas', 'ingredients', 'ingredients'],
    ['ounce package angel hair pasta', 'teaspoons olive oil', 'cup onion', 'clove garlic', 'cups tomatoes', 'cups boneless chicken breast halves cubed', 'cup fresh basil', 'teaspoon salt', 'teaspoon hot pepper sauce', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'onion', 'clove garlic', 'carrots', 'teaspoons chili powder', 'teaspoon ground cumin', 'cups vegetable stock', 'ounce cans black beans', 'ounce whole kernel corn', 'teaspoon ground black pepper', 'ounce tomatoes', 'ingredients', 'ingredients'],
    ['cup white sugar', 'cup vegetable oil', 'eggs', 'cup pumpkin', 'cup water', 'cups purpose flour', 'teaspoon baking powder', 'teaspoon baking soda', 'teaspoon ground cloves', 'teaspoon ground cinnamon', 'teaspoon salt', 'teaspoon ground nutmeg', 'cup semisweet chocolate chips', 'ingredients', 'ingredients'],
    ['eggs', 'cup vegetable oil', 'cups white sugar', 'teaspoons ground cinnamon', 'teaspoon salt', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoon baking soda', 'cups apples diced', 'ingredients', 'ingredients'],
    ['ounce beans', 'stalk celery', 'onion', 'tablespoon mayonnaise', 'tablespoon lemon juice', 'teaspoon dill weed', 'salt pepper', 'ingredients', 'ingredients'],
    ['cups fresh blackberries', 'cups fresh raspberries', 'cups fresh blueberries', 'tablespoons white sugar', 'cups purpose flour', 'cups oats', 'cups brown sugar', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'cups butter', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'bacon', 'large onion', 'cloves garlic', 'teaspoon salt', 'teaspoon pepper', 'chicken broth', 'red pepper flakes', 'pound fresh collard greens inch pieces', 'ingredients', 'ingredients'],
    ['ounce envelope dry onion soup mix', 'red potatoes', 'cup olive oil', 'ingredients', 'ingredients'],
    ['ounce package spinach', 'cup purpose flour', 'teaspoon salt', 'teaspoon baking powder', 'eggs', 'cup milk', 'cup butter', 'onion', 'ounce package mozzarella cheese', 'ingredients', 'ingredients'],
    ['cup mayonnaise', 'ounce container sour cream', 'ounce dry leek soup mix', 'ounce water chestnuts', 'ounce package spinach', 'pound loaf sourdough bread', 'ingredients', 'ingredients'],
    ['cup sesame seeds', 'ounce package bow tie pasta', 'cup vegetable oil', 'cup light soy sauce', 'cup rice vinegar', 'teaspoon sesame oil', 'tablespoons white sugar', 'teaspoon ground ginger', 'teaspoon ground black pepper', 'cups chicken breast meat', 'cup fresh cilantro', 'cup green onion', 'ingredients', 'ingredients'],
    ['red potatoes', 'cup butter', 'teaspoons garlic', 'teaspoon salt', 'lemon', 'tablespoon Parmesan cheese', 'ingredients', 'ingredients'],
    ['red lentils', 'large onion', 'tablespoon vegetable oil', 'tablespoons curry paste', 'tablespoon curry powder', 'teaspoon ground turmeric', 'teaspoon ground cumin', 'teaspoon chili powder', 'teaspoon salt', 'teaspoon white sugar', 'teaspoon garlic', 'teaspoon fresh ginger', 'ounce tomato puree', 'ingredients', 'ingredients'],
    ['teaspoon onion powder', 'teaspoon garlic powder', 'tablespoons chipotle chile powder', 'teaspoons salt', 'tablespoons brown sugar', 'pound pork tenderloins', 'ingredients', 'ingredients'],
    ['pound chuck roast', 'teaspoon salt', 'teaspoon ground black pepper', 'tablespoons olive oil', 'large onion', 'cups green chile pepper', 'teaspoon chili powder', 'teaspoon ground cayenne pepper', 'ounce hot pepper sauce', 'teaspoon garlic powder', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'cup white sugar', 'teaspoon baking soda', 'teaspoons pumpkin pie spice', 'cup pumpkin puree', 'eggs', 'teaspoon lemon juice', 'tablespoons confectioners sugar', 'ounce package cream cheese', 'cup butter', 'teaspoon vanilla extract', 'cup confectioners sugar', 'ingredients', 'ingredients'],
    ['cup mayonnaise', 'cloves garlic', 'tablespoon lemon juice', 'cup olive oil', 'cup red bell peppers', 'small zucchini', 'red onion', 'small yellow squash', 'x6 inch focaccia bread pieces', 'cup feta cheese', 'ingredients', 'ingredients'],
    ['cup butter', 'cup white sugar', 'eggs', 'ounce cream style corn', 'ounce green chile peppers', 'cup Monterey Jack cheese', 'cup Cheddar cheese', 'cup purpose flour', 'cup yellow cornmeal', 'teaspoons powder', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['tablespoon peanut oil', 'shallot', 'white onion', 'tablespoons butter', 'teaspoons lemon juice', 'tablespoon ginger garlic paste', 'teaspoon garam masala', 'teaspoon chili powder', 'teaspoon ground cumin', 'bay leaf', 'cup plain yogurt', 'cup half', 'cup tomato puree', 'teaspoon cayenne pepper', 'pinch salt', 'black pepper', 'tablespoon peanut oil', 'pound boneless skinless chicken thighs bite size pieces', 'teaspoon garam masala', 'cayenne pepper', 'tablespoon cornstarch', 'cup water', 'ingredients', 'ingredients'],
    ['oil pan', 'boneless skinless chicken breast halves', 'ounce package cream cheese inch slices', 'ounce package Cheddar cheese inch slices', 'cup milk', 'cups Italian seasoned bread crumbs', 'cup Romano cheese', 'tablespoon garlic', 'cup butter', 'tablespoons lemon juice', 'teaspoon garlic salt', 'teaspoon paprika optional', 'ingredients', 'ingredients'],
    ['cups fresh tomatoes', 'slice onion', 'whole cloves', 'chicken broth', 'tablespoons butter', 'tablespoons purpose flour', 'teaspoon salt', 'teaspoons white sugar', 'ingredients', 'ingredients'],
    ['large skinless boneless chicken breast halves cut inch medallions', 'salt pepper', 'cup purpose flour', 'tablespoons vegetable oil', 'clove garlic', 'cup low sodium chicken broth', 'lemon', 'cup fresh lemon juice', 'tablespoons capers', 'tablespoons butter', 'tablespoons Italian flat leaf parsley', 'ingredients', 'ingredients'],
    ['ounce package penne pasta', 'tablespoons olive oil', 'cup onion', 'clove garlic', 'cups tomatoes', 'cup fresh mushrooms', 'cups spinach leaves', 'salt pepper', 'red pepper flakes', 'ounces feta cheese', 'ingredients', 'ingredients'],
    ['ripe bananas', 'cups oats', 'cup dates', 'cup vegetable oil', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['fresh broccoli', 'red onion', 'pound bacon', 'cup raisins', 'cup almonds', 'cup mayonnaise', 'cup white sugar', 'tablespoons white wine vinegar', 'ingredients', 'ingredients'],
    ['cups white rice', 'cups water', 'cup baby carrots', 'cup frozen green peas', 'tablespoons vegetable oil', 'eggs', 'soy sauce', 'sesame oil optional', 'ingredients', 'ingredients'],
    ['pound shallots', 'tablespoons olive oil', 'salt pepper', 'beef broth', 'cup port wine', 'teaspoons tomato paste', 'beef tenderloin roast', 'teaspoon thyme', 'slices bacon', 'tablespoons butter', 'tablespoon purpose flour', 'sprigs watercress garnish', 'ingredients', 'ingredients'],
    ['potatoes cubed', 'ounces cream cheese', 'cup sour cream', 'teaspoons onion powder', 'teaspoon salt', 'teaspoon ground black pepper', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['cups carrots', 'cups purpose flour', 'cups white sugar', 'teaspoons soda', 'teaspoon baking powder', 'teaspoon salt', 'teaspoon ground cinnamon', 'eggs', 'cup applesauce', 'cup vegetable oil', 'teaspoons vanilla extract', 'ounce pineapple juice', 'cup pecans', 'cups confectioners sugar', 'ounce package Neufchatel cheese', 'cup butter', 'teaspoons vanilla extract', 'cup pecans', 'ingredients', 'ingredients'],
    ['ounce fillets tilapia', 'salt pepper', 'tablespoon Cajun seasoning', 'lemon', 'cup mayonnaise', 'cup sour cream', 'teaspoon garlic powder', 'teaspoon fresh lemon juice', 'tablespoons fresh dill', 'ingredients', 'ingredients'],
    ['cup unsalted butter', 'tablespoons olive oil', 'cups onions', 'ounce cans beef broth', 'tablespoons dry sherry optional', 'teaspoon thyme', 'salt pepper', 'French bread', 'provolone cheese', 'Swiss cheese', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['cups apples', 'teaspoon ground cinnamon', 'cup water', 'cup white sugar', 'cup butter', 'cup purpose flour', 'ingredients', 'ingredients'],
    ['eggs', 'cup vegetable oil', 'cups white sugar', 'cups zucchini', 'teaspoons vanilla extract', 'cups purpose flour', 'teaspoons ground cinnamon', 'teaspoon baking soda', 'teaspoon baking powder', 'teaspoon salt', 'cup walnuts', 'ingredients', 'ingredients'],
    ['small red new potatoes', 'cup olive oil', 'teaspoon Salt black pepper', 'ingredients', 'ingredients'],
    ['pounds apples', 'cups white sugar', 'teaspoons ground cinnamon', 'teaspoon ground cloves', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'onion', 'cloves garlic', 'ounce chicken broth', 'ounce tomatillos', 'ounce tomatoes', 'ounce green chiles', 'teaspoon oregano', 'teaspoon ground coriander seed', 'teaspoon ground cumin', 'ears fresh corn', 'pound chicken meat', 'ounce white beans', 'pinch salt black pepper', 'ingredients', 'ingredients'],
    ['cucumbers', 'small white onion', 'cup white vinegar', 'cup water', 'cup white sugar', 'tablespoon dill', 'ingredients', 'ingredients'],
    ['pounds steak', 'cup white vinegar', 'cup soy sauce', 'cloves garlic', 'limes', 'cup olive oil', 'teaspoon salt', 'teaspoon ground black pepper', 'teaspoon ground white pepper', 'teaspoon garlic powder', 'teaspoon chili powder', 'teaspoon oregano', 'teaspoon ground cumin', 'teaspoon paprika', 'white onion', 'cup fresh cilantro', 'lime', 'large tomatoes', 'jalapeno peppers', 'white onion', 'cloves garlic', 'New Mexico chile pods', 'pinch salt pepper', 'ounce package corn tortillas', 'cups cotija cheese optional', 'limes wedges', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'cup white sugar', 'eggs', 'cup butter', 'bananas', 'ingredients', 'ingredients'],
    ['cup bananas', 'cup sour cream', 'cup margarine', 'cups white sugar', 'eggs', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon baking powder', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['fluid ounce limeade concentrate', 'fluid ounces tequila', 'fluid ounces water', 'fluid ounces beer', 'ice', 'lime wedges', 'ingredients', 'ingredients'],
    ['ounces milk chocolate chips', 'tablespoons', 'pound fresh strawberries leaves', 'ingredients', 'ingredients'],
    ['cup water', 'cup uncooked Arborio rice', 'green bell peppers seeded', 'tablespoon olive oil', 'green onions', 'teaspoon basil', 'teaspoon Italian seasoning', 'teaspoon salt', 'pinch ground black pepper', 'tomato', 'cup feta cheese', 'ingredients', 'ingredients'],
    ['ounces cream cheese', 'ounce package Cheddar cheese', 'tablespoon bacon bits', 'ounces jalapeno peppers', 'cup milk', 'cup purpose flour', 'cup dry bread crumbs', 'quarts oil frying', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cups oats', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'cups brown sugar', 'cups butter', 'quarts apples', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'leeks', 'large carrots', 'chicken broth', 'teaspoons dill weed', 'teaspoons salt', 'teaspoon ground black pepper', 'bay leaf', 'potatoes diced', 'pound fresh mushrooms', 'cup half', 'cup purpose flour', 'fresh dill weed garnish optional', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'cup onion', 'cup green bell pepper', 'cup yellow bell pepper', 'clove garlic', 'pound ground turkey', 'ounce tomatoes', 'pumpkin puree', 'tablespoons chili powder', 'teaspoon ground black pepper', 'dash salt', 'cup Cheddar cheese', 'cup sour cream', 'ingredients', 'ingredients'],
    ['ounce cans tomatoes', 'onion', 'teaspoon garlic', 'lime', 'teaspoon salt', 'cup green chiles', 'tablespoons fresh cilantro', 'ingredients', 'ingredients'],
    ['cup white sugar', 'tablespoons purpose flour', 'cups fresh strawberries', 'cups rhubarb', 'cups purpose flour', 'cup brown sugar', 'cup butter', 'cup oats', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoon baking soda', 'cup white sugar', 'egg', 'tablespoons milk', 'teaspoon baking powder', 'teaspoon salt', 'cup margarine', 'cup bananas', 'cup walnuts optional', 'ingredients', 'ingredients'],
    ['tablespoons margarine', 'teaspoon salt', 'teaspoon garlic salt', 'teaspoons Worcestershire sauce', 'cups raw whole pumpkin seeds', 'ingredients', 'ingredients'],
    ['cup soy sauce', 'cup cider vinegar', 'tablespoons honey', 'tablespoons canola oil', 'large mushrooms half', 'green onions', 'skinless boneless chicken breast halves chunks', 'pound thick cut bacon half', 'ounce pineapple chunks', 'skewers', 'ingredients', 'ingredients'],
    ['ounce package vanilla wafers', 'ounce cream cheese', 'cup white sugar', 'eggs', 'teaspoon vanilla extract', 'ounce cherry pie filling', 'ingredients', 'ingredients'],
    ['ounce pumpkin puree', 'ounce condensed milk', 'egg yolks', 'teaspoon ground cinnamon', 'teaspoon ground ginger', 'teaspoon ground nutmeg', 'teaspoon salt', 'egg whites', 'inch unbaked pie shell', 'tablespoons purpose flour', 'cup brown sugar', 'teaspoon ground cinnamon', 'tablespoons butter', 'cup walnuts', 'ingredients', 'ingredients'],
    ['navy beans', 'pound bacon', 'onion', 'tablespoons molasses', 'teaspoons salt', 'teaspoon ground black pepper', 'teaspoon dry mustard', 'cup ketchup', 'tablespoon Worcestershire sauce', 'cup brown sugar', 'ingredients', 'ingredients'],
    ['teaspoon olive oil', 'onion', 'cloves garlic', 'cup uncooked white rice', 'cups low sodium fat vegetable broth', 'teaspoon ground cumin', 'teaspoon cayenne pepper', 'cups black beans', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cups white sugar', 'tablespoon pumpkin pie spice', 'teaspoon baking soda', 'teaspoon salt', 'eggs', 'cup pumpkin puree', 'cup vegetable oil', 'cups apple', 'tablespoons purpose flour', 'cup white sugar', 'teaspoon ground cinnamon', 'teaspoons butter', 'ingredients', 'ingredients'],
    ['cup soy milk', 'cup oats', 'banana chunks', 'frozen strawberries', 'teaspoon vanilla extract', 'teaspoons white sugar', 'ingredients', 'ingredients'],
    ['cup butter', 'cups white sugar', 'eggs', 'cup sour cream', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoon baking powder', 'teaspoon salt', 'cup fresh frozen blueberries', 'cup brown sugar', 'teaspoon ground cinnamon', 'cup pecans', 'tablespoon confectioners sugar', 'ingredients', 'ingredients'],
    ['Marinade', 'cup extra virgin olive oil', 'tablespoons white vinegar', 'tablespoons fresh lime juice', 'teaspoons lime zest', 'teaspoons honey', 'cloves garlic', 'teaspoon cumin', 'teaspoon chili powder', 'teaspoon seafood seasoning such Old Bay tm', 'teaspoon ground black pepper', 'teaspoon hot pepper sauce', 'pound tilapia fillets chunks', 'Dressing', 'ounce container light sour cream', 'cup adobo sauce chipotle peppers', 'tablespoons fresh lime juice', 'teaspoons lime zest', 'teaspoon cumin', 'teaspoon chili powder', 'teaspoon seafood seasoning such Old Bay tm', 'salt pepper', 'Toppings', 'ounce package tortillas', 'ripe tomatoes seeded diced', 'bunch cilantro', 'small head cabbage', 'limes wedges', 'ingredients', 'ingredients'],
    ['inch deep dish pie crust', 'cups apples', 'cup white sugar', 'teaspoon ground cinnamon', 'cup white sugar', 'cup purpose flour', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['pound dry kidney beans', 'cup olive oil', 'large onion', 'green bell pepper', 'tablespoons garlic', 'stalks celery', 'cups water', 'bay leaves', 'teaspoon cayenne pepper', 'teaspoon thyme', 'teaspoon sage', 'tablespoon parsley', 'teaspoon Cajun seasoning', 'pound andouille sausage', 'cups water', 'cups long grain white rice', 'ingredients', 'ingredients'],
    ['ounce package penne pasta', 'tablespoons olive oil', 'slices bacon', 'tablespoons garlic', 'ounce tomatoes', 'bunch fresh spinach bite size pieces', 'ingredients', 'ingredients'],
    ['ounce package angel hair pasta', 'cup olive oil', 'onion', 'cloves garlic', 'cups roma plum tomatoes', 'tablespoons balsamic vinegar', 'ounce sodium chicken broth', 'crushed red pepper', 'black pepper', 'tablespoons fresh basil', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['ounce fresh spinach leaves', 'cup sour cream', 'cup pepperjack cheese', 'cloves garlic', 'skinless boneless chicken breast halves inch thickness', 'pinch ground black pepper', 'bacon', 'ingredients', 'ingredients'],
    ['cup margarine', 'ounce package cream cheese', 'cups white sugar', 'eggs', 'cup bananas', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoons powder', 'teaspoon baking soda', 'cup pecans', 'tablespoons brown sugar', 'teaspoons ground cinnamon', 'ingredients', 'ingredients'],
    ['ground beef', 'onions', 'cloves garlic', 'tablespoons chili powder', 'teaspoons salt', 'teaspoons oregano', 'ounce cans tomatoes', 'ounce tomato sauce', 'ounce kidney beans liquid', 'ingredients', 'ingredients'],
    ['mango seeded diced', 'avocado diced', 'medium tomatoes', 'jalapeno pepper seeded', 'cup fresh cilantro', 'cloves garlic', 'teaspoon salt', 'tablespoons fresh lime juice', 'cup red onion', 'tablespoons olive oil', 'ingredients', 'ingredients'],
    ['inch pie crust baked', 'quart fresh strawberries', 'cup white sugar', 'tablespoons cornstarch', 'cup water', 'cup heavy whipping cream', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'cloves garlic', 'teaspoons parsley', 'eggplant inch cubes', 'salt', 'cup Parmesan cheese', 'zucchini', 'large onion rings', 'cups fresh mushrooms', 'green bell pepper', 'large tomatoes', 'ingredients', 'ingredients'],
    ['cups fresh strawberries', 'cups purpose flour', 'cups white sugar', 'tablespoon ground cinnamon', 'teaspoon salt', 'teaspoon baking soda', 'cups vegetable oil', 'eggs', 'cups pecans', 'ingredients', 'ingredients'],
    ['pound bacon', 'cup mayonnaise', 'cup sour cream', 'tomato seeded diced', 'ingredients', 'ingredients'],
    ['corned beef brisket spice packet', 'small red potatoes', 'carrots inch pieces', 'large head cabbage small wedges', 'ingredients', 'ingredients'],
    ['cup butter', 'large sweet potatoes', 'large carrots', 'apple', 'onion', 'cup red lentils', 'teaspoon fresh ginger', 'teaspoon ground black pepper', 'teaspoon salt', 'teaspoon ground cumin', 'teaspoon chili powder', 'teaspoon paprika', 'vegetable broth', 'plain yogurt', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'pound skinless boneless chicken breast meat', 'ounce package taco seasoning mix', 'ounce black beans', 'ounce sweet corn', 'cup salsa', 'water', 'cup Mexican style cheese', 'cups plain tortilla chips', 'ingredients', 'ingredients'],
    ['pound baby carrots', 'cup margarine', 'cup brown sugar', 'ingredients', 'ingredients'],
    ['inch unbaked deep dish pie crust', 'cup white sugar', 'teaspoon ground cinnamon', 'teaspoon salt', 'teaspoon ground ginger', 'teaspoon ground cloves', 'eggs', "ounce LIBBY 'S % Pure Pumpkin", 'fluid ounce NESTLE CARNATION Milk', 'ingredients', 'ingredients'],
    ['clove garlic', 'ounce beans liquid', 'tablespoons lemon juice', 'tablespoons', 'clove garlic', 'teaspoon salt', 'black pepper', 'tablespoons olive oil', 'ingredients', 'ingredients'],
    ['ounce container topping', 'ounce instant vanilla pudding mix', 'ounce pumpkin', 'teaspoon pumpkin pie spice', 'ingredients', 'ingredients'],
    ['recipe pastry inch double crust pie', 'tablespoons butter', 'small onion', 'stalks celery', 'carrots', 'tablespoons parsley', 'teaspoon oregano', 'salt pepper', 'cubes chicken bouillon', 'cups water', 'potatoes cubed', 'cups turkey', 'tablespoons purpose flour', 'cup milk', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'cup butter', 'lemons', 'tablespoons Dijon mustard such Grey Poupon Country Mustard tm', 'cup fresh basil leaves', 'cloves garlic', 'salt', 'white pepper', 'fresh shrimp deveined', 'skewers', 'ingredients', 'ingredients'],
    ['potatoes', 'eggs', 'cup celery', 'cup onion', 'cup sweet pickle relish', 'teaspoon garlic salt', 'teaspoon celery salt', 'tablespoon prepared mustard', 'ground black pepper', 'cup mayonnaise', 'ingredients', 'ingredients'],
    ['cups wheat bran', 'cup nonfat milk', 'cup unsweetened applesauce', 'egg', 'cup brown sugar', 'teaspoon vanilla extract', 'cup purpose flour', 'cup whole wheat flour', 'teaspoon baking soda', 'teaspoon baking powder', 'teaspoon salt', 'cup blueberries', 'ingredients', 'ingredients'],
    ['Cornish game hens', 'salt pepper', 'lemon', 'sprigs fresh rosemary', 'tablespoons olive oil', 'cloves garlic', 'cup white wine', 'cup low sodium chicken broth', 'sprigs fresh rosemary garnish', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'ounce broth', 'tablespoons white sugar', 'tablespoons soy sauce', 'pound boneless round steak bite size pieces', 'teaspoon fresh ginger root', 'clove garlic', 'cups fresh broccoli', 'ingredients', 'ingredients'],
    ['bacon', 'tomato', 'onion', 'ounces Swiss cheese', 'cup mayonnaise', 'teaspoon basil', 'ounce buttermilk biscuit dough', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'cayenne pepper', 'salt black pepper', 'purpose flour dredging', 'tablespoons olive oil', 'tablespoon capers', 'cup white wine', 'cup fresh lemon juice', 'cup water', 'tablespoons cold unsalted butter inch slices', 'tablespoons fresh Italian parsley', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'onion', 'cloves garlic', 'quarts water', 'teaspoons chicken bouillon granules', 'teaspoon salt', 'teaspoon black pepper', 'head cabbage', 'ounce Italian style stewed tomatoes diced', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'tablespoons red wine vinegar', 'lime', 'teaspoon chili powder', 'teaspoon paprika', 'teaspoon onion powder', 'teaspoon garlic powder', 'cayenne pepper', 'salt black pepper', 'pound skinless boneless chicken breast halves cut inch pieces', 'skewers', 'ingredients', 'ingredients'],
    ['tablespoons margarine', 'cup celery', 'cup onion', 'ounce cans chicken broth', 'cups cubed potatoes', 'ounce whole kernel corn', 'ounce green chiles', 'ounce package country style gravy mix', 'cups milk', 'cup Mexican style cheese food', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoon salt', 'teaspoon baking soda', 'teaspoons ground cinnamon', 'teaspoon baking powder', 'eggs', 'cups white sugar', 'teaspoons vanilla extract', 'cup vegetable oil', 'cups zucchini', 'cup walnuts optional', 'ingredients', 'ingredients'],
    ['tomatoes', 'onion', 'limes', 'tablespoons fresh cilantro', 'jalapeno pepper seeded', 'salt pepper', 'tablespoons olive oil', 'skinless boneless chicken breast halves strips', 'onion', 'green bell pepper', 'cloves garlic', 'inch flour tortillas', 'cup Monterey Jack cheese', 'cup sour cream', 'ingredients', 'ingredients'],
    ['strawberries', 'cup skim milk', 'cup plain yogurt', 'tablespoons white sugar', 'teaspoons vanilla extract', 'cubes ice', 'ingredients', 'ingredients'],
    ['cup olive oil', 'cup red wine vinegar', 'teaspoons garlic powder', 'teaspoons basil', 'teaspoons oregano', 'teaspoon ground black pepper', 'teaspoon white sugar', 'cups elbow macaroni', 'cups fresh mushrooms', 'cherry tomatoes', 'cup red bell peppers', 'cup feta cheese', 'cup green onions', 'ounce whole black olives', 'cup pepperoni sausage strips', 'ingredients', 'ingredients'],
    ['cup butter', 'teaspoons garlic powder', 'cloves garlic', 'pounds beef top sirloin steaks', 'salt pepper', 'ingredients', 'ingredients'],
    ['cup uncooked long grain white rice', 'cup water', 'tablespoons vegetable oil', 'pounds zucchini', 'cup green onions', 'clove garlic', 'teaspoons garlic salt', 'teaspoon basil', 'teaspoon sweet paprika', 'teaspoon oregano', 'cups tomatoes', 'cups sharp Cheddar cheese', 'ingredients', 'ingredients'],
    ['cup canola oil', 'cup milk', 'egg', 'teaspoon salt', 'teaspoons powder', 'cup white sugar', 'cups purpose flour', 'cup strawberries', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cups white sugar', 'teaspoons soda', 'teaspoons ground cinnamon', 'teaspoon salt', 'cups carrots', 'cup raisins', 'cup walnuts', 'cup unsweetened coconut', 'apple', 'eggs', 'cup vegetable oil', 'teaspoons vanilla extract', 'ingredients', 'ingredients'],
    ['cup warm water', 'cup brown sugar', 'tablespoons orange juice', 'tablespoons soy sauce', 'tablespoons ketchup', 'tablespoon white vinegar', 'cloves garlic', 'teaspoon red pepper flakes', 'teaspoon Chinese spice powder', 'teaspoon orange peel', 'tablespoons olive oil', 'pounds skinless boneless chicken breast halves', 'cups water', 'cup uncooked white rice', 'teaspoons cornstarch', 'tablespoons cold water', 'green onions garnish', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'onion', 'cloves garlic', 'pound ground turkey', 'ounce cans black beans undrained', 'ounce tomatoes', 'tablespoons chili powder', 'tablespoon oregano', 'tablespoon basil leaves', 'tablespoon red wine vinegar', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cup whole wheat flour', 'cups white sugar', 'tablespoon ground cinnamon', 'teaspoons powder', 'teaspoon baking soda', 'teaspoon salt', 'cups carrots', 'apple', 'cup raisins', 'egg', 'egg whites', 'cup apple butter', 'cup vegetable oil', 'tablespoon vanilla extract', 'tablespoons walnuts', 'tablespoons wheat germ', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'pound shrimp deveined', 'salt', 'cloves garlic', 'teaspoon red pepper flakes', 'tablespoons lemon juice', 'tablespoon caper brine', 'teaspoons cold butter', 'cup Italian flat leaf parsley', 'tablespoons cold butter', 'water', 'ingredients', 'ingredients'],
    ['ounce cans whole green chile peppers', 'ounces Monterey Jack cheese', 'ounces Longhorn Cheddar cheese', 'eggs', 'ounce milk', 'tablespoons purpose flour', 'cup milk', 'ounce tomato sauce', 'ingredients', 'ingredients'],
    ['lemon', 'lime', 'orange', 'cups rum', 'cup white sugar', 'milliliter dry red wine', 'cup orange juice', 'ingredients', 'ingredients'],
    ['cup butter', 'tablespoon garlic', 'cup Parmesan cheese', 'tablespoon garlic salt', 'teaspoon Italian seasoning', 'teaspoon ground black pepper', 'teaspoon ground paprika', 'ingredients', 'ingredients'],
    ['cup butter', 'cups white sugar', 'eggs', 'cup sour cream', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'cup ripe bananas', 'ounce container cream cheese frosting', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'potato paper thin peel optional', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['cup graham cracker crumbs', 'cup pecans', 'tablespoons white sugar', 'teaspoon ground cinnamon', 'cup unsalted butter', 'ounce cream cheese', 'cup white sugar', 'eggs', 'teaspoon vanilla extract', 'cups apples', 'cup white sugar', 'teaspoon ground cinnamon', 'cup pecans', 'ingredients', 'ingredients'],
    ['pound ground beef', 'cups chunky salsa', 'ounce whole kernel corn', 'cup creamy salad dressing Miracle Whip', 'teaspoon chili powder', 'cups tortilla chips', 'cups Colby cheese', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'pounds ground turkey', 'ounce package taco seasoning mix', 'teaspoon ground coriander', 'teaspoon oregano', 'teaspoon chili pepper flakes', 'tablespoons tomato paste', 'ounce broth', 'ounce salsa', 'ounce tomatoes puree', 'ounce green chile peppers', 'medium onion', 'green bell pepper', 'medium zucchini', 'bunch green onions', 'cup sour cream', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['tablespoon olive oil canola', 'large garlic cloves', 'medium yellow onion', 'cups raw kale', 'cups low fat sodium chicken vegetable broth', 'ounce cans white beans such cannellini navy undrained', 'plum tomatoes', 'teaspoons Italian herb seasoning', 'Salt pepper', 'cup parsley', 'ingredients', 'ingredients'],
    ['russet potatoes inch cubes', 'teaspoons chili powder', 'teaspoon salt', 'teaspoon ground black pepper', 'tablespoons butter', 'ounce package colby Monterey Jack cheese blend', 'ounce bottle Ranch dressing', 'ingredients', 'ingredients'],
    ['ounce squares unsweetened chocolate', 'eggs', 'cups white sugar', 'cup vegetable oil', 'cups zucchini', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'teaspoon ground cinnamon', 'cup semisweet chocolate chips', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'cups white sugar', 'teaspoon baking powder', 'teaspoon salt', 'tablespoons cold butter', 'cup water', 'tablespoons cornstarch', 'cup cold water', 'tablespoon lemon juice', 'cups fresh blackberries', 'ingredients', 'ingredients'],
    ['eggs', 'cup raisins', 'teaspoon vanilla extract', 'cup butter', 'cup brown sugar', 'cup white sugar', 'cups purpose flour', 'teaspoon ground cinnamon', 'teaspoons soda', 'cups oats', 'cup pecans', 'ingredients', 'ingredients'],
    ['ounce cans pinto beans', 'cup salsa', 'cloves garlic', 'tablespoons fresh cilantro', 'ounce black beans', 'cup tomatoes', 'inch flour tortillas', 'cups fat Cheddar cheese', 'cup salsa', 'cup sour cream', 'ingredients', 'ingredients'],
    ['cup mayonnaise', 'cloves garlic', 'tablespoon olive oil', 'pound beef round steak thin strips', 'green bell peppers inch strips', 'onions rings', 'salt pepper', 'hoagie rolls toasted', 'ounce package mozzarella cheese', 'teaspoon oregano', 'ingredients', 'ingredients'],
    ['pound ground pork sausage', 'ounce package cream cheese', 'cup Parmesan cheese', 'large fresh jalapeno peppers seeded', 'ounce bottle Ranch dressing optional', 'ingredients', 'ingredients'],
    ['cups fresh frozen blueberries', 'cup water', 'cup orange juice', 'cup white sugar', 'cup cold water', 'tablespoons cornstarch', 'teaspoon almond extract', 'teaspoon ground cinnamon', 'ingredients', 'ingredients'],
    ['avocados', 'small onion', 'clove garlic', 'ripe tomato', 'lime', 'salt pepper', 'ingredients', 'ingredients'],
    ['cups chocolate chips', 'ounce beans', 'eggs', 'cup white sugar', 'teaspoon baking powder', 'tablespoon confectioners sugar', 'ingredients', 'ingredients'],
    ['portobello mushroom caps', 'cup balsamic vinegar', 'tablespoons olive oil', 'teaspoon basil', 'teaspoon oregano', 'tablespoon garlic', 'salt pepper', 'ounce provolone cheese', 'ingredients', 'ingredients'],
    ['pounds green beans inch pieces', 'cups water', 'cup butter', 'tablespoon sugar', 'teaspoon garlic salt', 'teaspoon pepper', 'teaspoons fresh basil', 'cherry tomato halves', 'ingredients', 'ingredients'],
    ['slices bacon', 'cup onion', 'cups cabbage thin wedges', 'tablespoons water', 'white sugar', 'salt pepper', 'tablespoon cider vinegar', 'ingredients', 'ingredients'],
    ['quarts vegetable broth', 'cup uncooked barley', 'large carrots', 'stalks celery', 'ounce tomatoes juice', 'zucchini', 'ounce beans', 'onion', 'bay leaves', 'teaspoon garlic powder', 'teaspoon white sugar', 'teaspoon salt', 'teaspoon ground black pepper', 'teaspoon parsley', 'teaspoon curry powder', 'teaspoon paprika', 'teaspoon Worcestershire sauce', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'lemon', 'teaspoons garlic powder', 'teaspoon ground black pepper', 'ounce cans chicken broth', 'cup fresh lemon juice', 'ounce package rotelle pasta', 'cup heavy cream', 'teaspoon lemon zest', 'ingredients', 'ingredients'],
    ['cups water', 'cup quinoa', 'pinch salt', 'cup olive oil', 'teaspoon sea salt', 'cup lemon juice', 'tomatoes', 'cucumber', 'green onions', 'carrots', 'cup fresh parsley', 'ingredients', 'ingredients'],
    ['cups white sugar', 'cups vegetable oil', 'teaspoon vanilla extract', 'cups pumpkin', 'eggs', 'cups purpose flour', 'teaspoons powder', 'teaspoons soda', 'teaspoon salt', 'teaspoons ground cinnamon', 'cup walnuts optional', 'ingredients', 'ingredients'],
    ['ounce packages broccoli', 'instant rice', 'ounce cream mushroom soup', 'ounce cream chicken soup', 'cups water', 'ounce package American cheese', 'tablespoon butter', 'bunch celery', 'large onion', 'salt pepper', 'ingredients', 'ingredients'],
    ['pound bacon', 'cup mayonnaise', 'cup milk', 'teaspoon garlic powder', 'teaspoon ground black pepper', 'salt', 'head romaine lettuce', 'large tomatoes', 'cups croutons', 'ingredients', 'ingredients'],
    ['cups corn kernels', 'cup red onion', 'cup diced jicama', 'cup red bell pepper', 'cup fresh cilantro leaves', 'lime', 'tablespoons cayenne pepper', 'tablespoon ground black pepper', 'tablespoons salt', 'ounce fillets tilapia', 'tablespoons olive oil', 'corn tortillas', 'tablespoons sour cream', 'ingredients', 'ingredients'],
    ['ounce black beans', 'cup quinoa', 'cup water', 'cup bread crumbs', 'cup yellow bell pepper', 'tablespoons onion', 'large clove garlic', 'teaspoons ground cumin', 'teaspoon salt', 'teaspoon hot pepper sauce such Frank RedHot', 'egg', 'tablespoons olive oil', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'cup whole wheat flour', 'cup white sugar', 'cup oat bran', 'cup quick cooking oats', 'cup wheat germ', 'teaspoon baking powder', 'teaspoon baking soda', 'teaspoon salt', 'cup blueberries', 'cup walnuts', 'banana', 'cup buttermilk', 'egg', 'tablespoon vegetable oil', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['sweet Italian sausage', 'pound lean ground beef', 'cup onion', 'cloves garlic', 'ounce tomatoes', 'ounce cans tomato paste', 'ounce cans tomato sauce', 'cup water', 'tablespoons white sugar', 'teaspoons basil', 'teaspoon fennel seed', 'teaspoon Italian seasoning', 'teaspoon salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['ounce black bean soup', 'ounce kidney beans', 'ounce beans', 'ounce vegetarian baked beans', 'ounce tomatoes puree', 'ounce whole kernel corn', 'onion', 'green bell pepper', 'stalks celery', 'cloves garlic', 'tablespoon chili powder', 'tablespoon parsley', 'tablespoon oregano', 'tablespoon basil', 'ingredients', 'ingredients'],
    ['cup brown sugar', 'cup oats', 'cup purpose flour', 'cup butter', 'cups apples', 'cup white sugar', 'teaspoons ground cinnamon', 'ingredients', 'ingredients'],
    ['medium acorn squash seeded', 'tablespoon butter', 'tablespoons brown sugar', 'ingredients', 'ingredients'],
    ['cup butter', 'leeks', 'salt pepper', 'quart chicken broth', 'tablespoon cornstarch', 'cups Yukon Gold potatoes diced', 'cups heavy cream', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cup white sugar', 'teaspoons powder', 'teaspoon baking soda', 'teaspoon ground cinnamon', 'egg whites', 'cup banana', 'cup applesauce', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'ounce jar spaghetti sauce', 'green bell pepper seeded cubed', 'ounces fresh mushrooms', 'onion', 'tablespoons garlic', 'ingredients', 'ingredients'],
    ['roma plum tomatoes', 'cup fresh basil', 'cup Parmesan cheese', 'cloves garlic', 'tablespoon balsamic vinegar', 'teaspoon olive oil', 'teaspoon kosher salt', 'teaspoon black pepper', 'loaf French bread', 'ingredients', 'ingredients'],
    ['ounce package lasagna noodles', 'pound fresh mushrooms', 'cup green bell pepper', 'cup onion', 'cloves garlic', 'tablespoons vegetable oil', 'ounce jars pasta sauce', 'teaspoon basil', 'ounce container part skim ricotta cheese', 'cups mozzarella cheese', 'eggs', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['cups chicken breast meat', 'teaspoon ground cumin', 'teaspoon ground coriander', 'tablespoons fresh cilantro', 'ounce black beans', 'ounce green chile peppers', 'ounce red enchilada sauce', 'inch corn tortillas', 'cups Mexican blend cheese', 'ounce container sour cream', 'ingredients', 'ingredients'],
    ['pounds Yukon Gold potatoes', 'tablespoons butter', 'pint heavy cream', 'cloves garlic', 'salt pepper', 'cups Cheddar cheese', 'provolone cheese', 'cup Parmesan Romano cheese', 'ingredients', 'ingredients'],
    ['cup butter', 'cups brown sugar', 'ounce pineapple', 'maraschino cherries', 'ounce white cake mix', 'ingredients', 'ingredients'],
    ['pint fresh strawberries', 'cup white sugar', 'teaspoon vanilla', 'ingredients', 'ingredients'],
    ['pound ground beef', 'onion', 'cloves garlic', 'small green bell pepper', 'ounce tomatoes', 'ounce tomato sauce', 'ounce tomato paste', 'teaspoons oregano', 'teaspoons basil', 'teaspoon salt', 'teaspoon black pepper', 'ingredients', 'ingredients'],
    ['pound boneless pork loin roast', 'salt', 'cups apple juice', 'cup apple butter', 'cup brown sugar', 'tablespoons water', 'teaspoon ground cinnamon', 'teaspoon ground cloves', 'ingredients', 'ingredients'],
    ['tablespoon canola oil', 'tablespoon unsalted butter', 'cup diced onion', 'cup carrots', 'cups cubed butternut squash', 'cups vegetable stock', 'salt black pepper', 'ground nutmeg', 'cup heavy cream optional', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'ounce package white mushrooms', 'cloves garlic', 'teaspoons soy sauce', 'ground black pepper', 'ingredients', 'ingredients'],
    ['pounds russet potatoes', 'cup purpose flour', 'teaspoon garlic salt', 'teaspoon onion salt', 'teaspoon salt', 'teaspoon paprika', 'cup water', 'cup vegetable oil frying', 'ingredients', 'ingredients'],
    ['inch pie shell', 'cups apples', 'tablespoon lemon juice optional', 'cup white sugar', 'tablespoons purpose flour', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'cup raisins optional', 'cup walnuts optional', 'cup purpose flour', 'cup brown sugar', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['cups fresh blueberries', 'tablespoons white sugar', 'cup orange juice', 'cup purpose flour', 'teaspoon baking powder', 'pinch salt', 'cup butter', 'cup white sugar', 'egg', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['cup shortening', 'cups white sugar', 'eggs', 'tablespoons mayonnaise', 'ripe bananas', 'cups purpose flour', 'teaspoon salt', 'teaspoon baking powder', 'teaspoons soda', 'cup semi-sweet chocolate chips', 'cup walnuts', 'ingredients', 'ingredients'],
    ['cup lemon juice', 'tablespoons vegetable oil', 'cloves garlic', 'teaspoon salt', 'teaspoon oregano', 'teaspoon pepper', 'ounce boneless pork loin chops', 'ingredients', 'ingredients'],
    ['bacon', 'cup onion', 'clove garlic', 'ounce pinto beans', 'ounce great Northern beans', 'ounce baked beans', 'ounce red kidney beans', 'ounce beans', 'cup ketchup', 'cup molasses', 'cup brown sugar', 'tablespoons Worcestershire sauce', 'tablespoon yellow mustard', 'teaspoon pepper', 'ingredients', 'ingredients'],
    ['ounce package spaghetti pieces', 'cup butter', 'cup purpose flour', 'teaspoon salt', 'teaspoon ground black pepper', 'cup chicken broth', 'cup heavy cream', 'tablespoons sherry', 'ounce mushrooms', 'cups cooked chicken', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'large onion', 'stalk celery', 'carrots', 'cloves garlic', 'tablespoons chili powder', 'tablespoon ground cumin', 'black pepper', 'vegetable broth', 'ounce cans black beans', 'ounce whole kernel corn', 'ounce tomatoes', 'ingredients', 'ingredients'],
    ['cup white sugar', 'cup lemon juice', 'teaspoons onion', 'teaspoon Dijon style prepared mustard', 'teaspoon salt', 'cup vegetable oil', 'tablespoon poppy seeds', 'head romaine lettuce bite size pieces', 'ounces Swiss cheese', 'cup cashews', 'cup cranberries', 'apple diced', 'pear', 'ingredients', 'ingredients'],
    ['potatoes cubed', 'tablespoons vegetable oil', 'yellow onion', 'cloves garlic', 'teaspoons ground cumin', 'teaspoons cayenne pepper', 'teaspoons curry powder', 'teaspoons garam masala', 'inch piece fresh ginger root', 'teaspoons salt', 'ounce tomatoes', 'ounce beans chickpeas', 'ounce peas', 'ounce coconut milk', 'ingredients', 'ingredients'],
    ['cups peas carrots', 'cups frozen green beans', 'cup celery', 'cup butter', 'cup onion', 'cup purpose flour', 'teaspoon salt', 'teaspoon ground black pepper', 'teaspoon celery seed', 'teaspoon onion powder', 'teaspoon Italian seasoning', 'cups chicken broth', 'cups milk', 'cups turkey meat light dark mixed', 'inch unbaked pie crusts', 'ingredients', 'ingredients'],
    ['cups water', 'cups uncooked instant rice', 'ounce cans chunk chicken', 'ounce cream mushroom soup', 'ounce cream chicken soup', 'cup butter', 'cup milk', 'ounce package broccoli', 'small white onion', 'cheese food', 'ingredients', 'ingredients'],
    ['cup mayonnaise', 'tablespoons Dijon mustard', 'lemon', 'teaspoons tarragon', 'teaspoon ground black pepper', 'teaspoon salt', 'spears fresh asparagus', 'skinless boneless chicken breast halves', 'provolone cheese', 'cup panko bread crumbs', 'ingredients', 'ingredients'],
    ['ounce yellow cake mix', 'ounce pumpkin puree', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'teaspoon ground cloves', 'ingredients', 'ingredients'],
    ['pound fresh asparagus spears', 'tablespoon olive oil', 'salt pepper', 'ingredients', 'ingredients'],
    ['ounce black beans', 'eggs', 'tablespoons vegetable oil', 'cup cocoa powder', 'pinch salt', 'teaspoon vanilla extract', 'cup white sugar', 'teaspoon instant coffee optional', 'cup milk chocolate chips optional', 'ingredients', 'ingredients'],
    ['inch angel food cake', 'ounce cream cheese', 'cup white sugar', 'ounce container topping', 'quart fresh strawberries', 'ounce jar strawberry glaze', 'ingredients', 'ingredients'],
    ['pound ground beef', 'cup diced onion', 'cup diced celery', 'cup green bell pepper', 'cloves garlic', 'ounce cans tomato puree', 'ounce kidney beans liquid', 'ounce kidney beans', 'ounce beans liquid', 'tablespoon chili powder', 'teaspoon parsley', 'teaspoon salt', 'teaspoon basil', 'teaspoon oregano', 'teaspoon ground black pepper', 'teaspoon hot pepper sauce', 'ingredients', 'ingredients'],
    ['cups milk', 'cup warm water degrees F C', 'ounce packages active dry yeast', 'eggs', 'cup white sugar', 'teaspoon salt', 'cup margarine', 'cup raisins', 'cups purpose flour', 'tablespoons milk', 'cup white sugar', 'tablespoons ground cinnamon', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['butternut squash inch cubes', 'tablespoons olive oil', 'cloves garlic', 'salt black pepper', 'ingredients', 'ingredients'],
    ['ounce uncooked orzo pasta', 'ounce package baby spinach leaves', 'pound feta cheese', 'red onion', 'cup pine nuts', 'teaspoon basil', 'teaspoon ground white pepper', 'cup olive oil', 'cup balsamic vinegar', 'ingredients', 'ingredients'],
    ['red potatoes', 'teaspoons garlic', 'cup butter', 'cup half cream', 'teaspoon steak seasoning', 'teaspoon garlic powder', 'teaspoon white sugar', 'ingredients', 'ingredients'],
    ['ounce hearts', 'ounce package spinach', 'cup sour cream', 'cup mayonnaise', 'cup cream cheese', 'cup Romano cheese', 'teaspoon garlic', 'ingredients', 'ingredients'],
    ['baking potatoes inch slices', 'cup butter', 'slices bacon', 'ounces Cheddar cheese', 'cup green onions', 'ingredients', 'ingredients'],
    ['ounce uncooked farfalle pasta', 'hot Italian turkey sausage inch slices', 'cup olive oil', 'cloves garlic', 'onion', 'small zucchini', 'small yellow squash', 'roma plum tomatoes', 'green bell pepper', 'fresh basil', 'teaspoons chicken bouillon granules', 'teaspoon red pepper flakes', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'large white onion rings', 'cups uncooked green lentils', 'cup uncooked long grain white rice', 'salt pepper', 'cup plain yogurt sour cream optional', 'ingredients', 'ingredients'],
    ['pound uncooked linguine', 'tablespoon butter', 'tablespoons white wine', 'teaspoons Parmesan cheese', 'cloves garlic', 'teaspoon fresh parsley', 'pinch salt pepper', 'pound medium shrimp deveined', 'ingredients', 'ingredients'],
    ['pounds skinless boneless chicken breast halves', 'teaspoons salt', 'cup cooking oil', 'cups onion', 'tablespoon garlic', 'teaspoons fresh ginger root', 'tablespoon curry powder', 'teaspoon ground cumin', 'teaspoon ground turmeric', 'teaspoon ground coriander', 'teaspoon cayenne pepper', 'tablespoon water', 'ounce tomatoes', 'cup plain yogurt', 'tablespoon fresh cilantro', 'teaspoon salt', 'cup water', 'teaspoon garam masala', 'tablespoon fresh cilantro', 'tablespoon fresh lemon juice', 'ingredients', 'ingredients'],
    ['sweet potatoes', 'cups margarine', 'cups brown sugar', 'miniature marshmallows', 'ground cinnamon', 'ground nutmeg', 'ingredients', 'ingredients'],
    ['cup unsalted butter', 'cups white sugar', 'teaspoon salt', 'eggs', 'cups purpose flour', 'teaspoons powder', 'cup buttermilk', 'pint fresh blueberries dry', 'tablespoons white sugar', 'ingredients', 'ingredients'],
    ['cup white sugar', 'cup purpose flour', 'pound fresh rhubarb', 'pints fresh strawberries', 'recipe pastry inch double crust pie', 'tablespoons butter', 'egg yolk', 'tablespoons white sugar', 'ingredients', 'ingredients'],
    ['medium zucchini inch slices', 'cup dry bread crumbs', 'teaspoon ground black pepper', 'tablespoons Parmesan cheese', 'egg whites', 'ingredients', 'ingredients'],
    ['cups gingersnap cookies', 'cup pecans', 'cup butter', 'ounce cream cheese', 'cup white sugar', 'teaspoon vanilla extract', 'eggs', 'cup pumpkin', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'ingredients', 'ingredients'],
    ['cups white sugar', 'ounce pumpkin puree', 'cup vegetable oil', 'cup water', 'eggs', 'cups purpose flour', 'tablespoon ground cinnamon', 'tablespoon ground nutmeg', 'teaspoons soda', 'teaspoons salt', 'cup miniature semisweet chocolate chips', 'cup walnuts optional', 'ingredients', 'ingredients'],
    ['tablespoons soy sauce', 'tablespoons brown sugar', 'tablespoons sherry', 'tablespoon sesame oil', 'teaspoon ground ginger', 'teaspoon garlic powder', 'skinless boneless chicken breast halves cut inch pieces', 'ounce pineapple chunks', 'skewers', 'ingredients', 'ingredients'],
    ['ounce package crescent rolls', 'ounce package cream cheese', 'ounce package Ranch style dressing mix', 'carrots', 'cup red bell peppers', 'cup green bell pepper', 'cup fresh broccoli', 'cup green onions', 'ingredients', 'ingredients'],
    ['large green tomatoes', 'eggs', 'cup milk', 'cup purpose flour', 'cup cornmeal', 'cup bread crumbs', 'teaspoons coarse kosher salt', 'teaspoon ground black pepper', 'quart vegetable oil frying', 'ingredients', 'ingredients'],
    ['medium baking potato', 'teaspoon olive oil', 'teaspoon salt', 'teaspoons butter', 'black pepper', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['cups oats', 'cup brown sugar', 'teaspoons ground cinnamon', 'teaspoons powder', 'teaspoon salt', 'cup milk', 'eggs', 'cup butter', 'teaspoons vanilla extract', 'cup cranberries', 'ingredients', 'ingredients'],
    ['pork tenderloins', 'tablespoons Dijon mustard', 'tablespoon oil', 'tablespoons butter', 'shallots', 'cup Marsala wine', 'tablespoon Dijon mustard', 'cup heavy cream', 'ingredients', 'ingredients'],
    ['cups whole wheat flour', 'teaspoons powder', 'egg', 'cup milk more necessary', 'teaspoon salt', 'tablespoon artificial sweetener', 'cup blueberries', 'ingredients', 'ingredients'],
    ['small red potatoes', 'cup butter', 'cup milk', 'cup sour cream', 'salt pepper', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'onion', 'cloves garlic', 'ounce tomato sauce', 'cups water', 'tablespoon parsley', 'teaspoons basil', 'teaspoons oregano', 'teaspoon salt', 'ounce beans', 'ounce navy beans', 'cup Parmesan cheese', 'pound ditalini pasta', 'ingredients', 'ingredients'],
    ['ounce cranberry sauce', 'cup French salad dressing', 'onion', 'pound boneless pork loin roast', 'ingredients', 'ingredients'],
    ['eggs', 'tablespoons purpose flour', 'teaspoon baking powder', 'teaspoon salt', 'teaspoon pepper', 'medium potatoes', 'cup onion', 'cup vegetable oil', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'ounce jar salsa', 'teaspoons garlic powder', 'teaspoon ground cumin', 'teaspoon chili powder', 'salt', 'ground black pepper', 'ounce Mexican style corn', 'ounce pinto beans', 'ingredients', 'ingredients'],
    ['ounce salmon', 'eggs', 'cup garlic herb dry bread crumbs', 'cup dry potato flakes', 'medium onion', 'clove garlic', 'teaspoon dill weed', 'teaspoon celery salt', 'salt pepper', 'tablespoons olive oil', 'ingredients', 'ingredients'],
    ['pound carrots inch pieces', 'tablespoons butter', 'cup brown sugar', 'pinch salt', 'pinch ground black pepper', 'ingredients', 'ingredients'],
    ['green onions', 'ounce coconut milk', 'cup soy sauce', 'teaspoon brown sugar', 'teaspoons curry powder', 'teaspoon fresh ginger', 'teaspoons chile paste', 'pound firm tofu inch cubes', 'roma plum tomatoes', 'yellow bell pepper', 'ounces fresh mushrooms', 'cup fresh basil', 'cups bok choy', 'salt', 'ingredients', 'ingredients'],
    ['pound ground beef', 'egg', 'tablespoons buttery round cracker crumbs', 'tablespoons onion', 'teaspoon salt', 'teaspoon pepper', 'teaspoon poultry seasoning', 'ounce cans mushrooms juice', 'tablespoons butter', 'tablespoons purpose flour', 'cups milk', 'cubes beef bouillon', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'cup skinless boneless chicken breast halves', 'pound pork sausage links', 'cup olive oil', 'cup purpose flour', 'tablespoons garlic', 'quarts chicken broth', 'fluid ounce beer', 'stalks celery', 'roma plum tomatoes', 'sweet onion', 'ounce tomatoes green chile peppers liquid', 'tablespoons fresh red chile peppers', 'bunch fresh parsley', 'cup Cajun seasoning', 'pound shrimp deveined', 'ingredients', 'ingredients'],
    ['pound ground turkey', 'cup onion', 'tablespoons fresh cilantro', 'chipotle chile adobo sauce', 'teaspoon garlic powder', 'teaspoon onion powder', 'teaspoon seasoned salt', 'teaspoon black pepper', 'mozzarella cheese', 'hamburger buns split toasted', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'onion', 'skinless boneless chicken breast halves', 'tablespoons ketchup', 'tablespoons soy sauce', 'tablespoons white sugar', 'tablespoons lemon juice', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['cup uncooked orzo pasta', 'tablespoons olive oil', 'cloves garlic', 'teaspoon red pepper', 'skinless boneless chicken breast halves bite size pieces', 'salt', 'tablespoon fresh parsley', 'cups fresh spinach leaves', 'grated Parmesan cheese', 'ingredients', 'ingredients'],
    ['ounce package linguine pasta', 'cloves garlic', 'tablespoons sugar', 'tablespoons safflower oil', 'tablespoons rice vinegar', 'tablespoons soy sauce', 'tablespoons sesame oil', 'teaspoons chili sauce', 'green onions', 'teaspoon sesame seeds', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'ounce pork loin chops inch thick', 'teaspoons salt', 'teaspoons black pepper', 'onion strips', 'cup water', 'ingredients', 'ingredients'],
    ['cup butter', 'onion', 'cup celery', 'cup carrots', 'pound fresh mushrooms', 'cup purpose flour', 'chicken broth', 'cups wild rice', 'pound boneless skinless chicken breasts cubed', 'teaspoon salt', 'teaspoon curry powder', 'teaspoon mustard powder', 'teaspoon parsley', 'teaspoon ground black pepper', 'cup almonds', 'tablespoons dry sherry', 'cups half', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'teaspoon garlic salt', 'teaspoon ground black pepper', 'teaspoon oregano', 'boneless skinless chicken breast halves', 'tablespoon olive oil', 'tablespoon butter', 'cup fresh mushrooms', 'cup Marsala wine', 'ingredients', 'ingredients'],
    ['potatoes', 'cup Cheddar cheese', 'cup onion', 'cup ham', 'ounce cream mushroom soup', 'cup water', 'teaspoon garlic powder', 'teaspoon salt', 'teaspoon black pepper', 'ingredients', 'ingredients'],
    ['cup unsalted butter margarine', 'ounces chocolate chips bars bite size chunks', 'large eggs', 'cup sugar', 'Pinch salt', 'teaspoons flour matzo meal ground blender fine powder', 'extra-large paper muffin cups regular cakes', 'Garnish', 'ounce container raspberries', 'cup sugar', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoons powder', 'teaspoon salt', 'tablespoons purpose flour', 'cups fresh blueberries', 'cup butter', 'cup white sugar', 'eggs', 'teaspoon vanilla extract', 'teaspoon lemon zest', 'cup milk', 'tablespoons purpose flour', 'tablespoons white sugar', 'teaspoon ground cinnamon', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cups white sugar', 'cup unsweetened cocoa powder', 'teaspoons soda', 'teaspoon baking powder', 'teaspoon salt', 'teaspoon ground cinnamon', 'eggs', 'cups vegetable oil', 'cups zucchini', 'cup walnuts', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cup confectioners sugar', 'cup butter', 'eggs', 'cups white sugar', 'cup lemon juice', 'tablespoon lemon zest', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoon salt', 'teaspoon baking powder', 'tablespoons butter', 'cup milk', 'cup Cheddar cheese', 'cup butter', 'clove garlic', 'ingredients', 'ingredients'],
    ['cup basmati rice', 'cups water', 'cup peanuts', 'tablespoon margarine', 'onion', 'teaspoon fresh ginger root', 'cup carrots', 'salt', 'cayenne pepper', 'fresh cilantro', 'ingredients', 'ingredients'],
    ['cup margarine', 'cups white sugar', 'egg', 'ripe bananas', 'tablespoon instant coffee granules', 'tablespoon water', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoon salt', 'teaspoon baking powder', 'teaspoon baking soda', 'cup semisweet chocolate chips', 'ingredients', 'ingredients'],
    ['cups oil frying', 'cup purpose flour', 'teaspoon baking powder', 'teaspoon salt', 'teaspoon white sugar', 'egg', 'cup milk', 'tablespoon shortening', 'ounce whole kernel corn', 'ingredients', 'ingredients'],
    ['ground beef', 'large onion', 'small head cabbage', 'ounce cans tomatoes', 'ounce tomato sauce', 'cup water', 'cloves garlic', 'teaspoons salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['small sugar pumpkin seeded', 'cups purpose flour', 'cups white sugar', 'teaspoons soda', 'teaspoon baking powder', 'teaspoons cloves', 'teaspoons ground cinnamon', 'teaspoons ground nutmeg', 'teaspoon ground allspice', 'teaspoon salt', 'cup vegetable oil', 'eggs', 'ingredients', 'ingredients'],
    ['fresh strawberries', 'cups white sugar', 'cup lemon juice', 'ingredients', 'ingredients'],
    ['cup soy sauce', 'tablespoon Worcestershire sauce', 'cloves garlic', 'tablespoons lemon juice', 'teaspoon ground black pepper', 'teaspoon vegetable oil', 'pork chops', 'ingredients', 'ingredients'],
    ['pound salmon fillets', 'cup butter', 'tablespoons lemon juice', 'tablespoon dill weed', 'teaspoon garlic powder', 'sea salt', 'black pepper', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'cup sesame seeds', 'teaspoon black pepper', 'teaspoon Chinese spice powder', 'teaspoon red pepper flakes optional', 'ounce skinless boneless chicken breast halves chunks', 'tablespoons vegetable oil', 'yellow onion wedges', 'cup green bell pepper seeded', 'tablespoons teriyaki sauce', 'tablespoons honey', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'pork chops inch thick', 'tablespoons purpose flour', 'cup Parmesan cheese', 'teaspoon salt', 'teaspoon pepper', 'Yukon Gold potatoes', 'medium onions', 'cubes beef bouillon', 'cup hot water', 'tablespoon lemon juice', 'ingredients', 'ingredients'],
    ['tomatoes', 'cup onion', 'serrano chiles', 'cup fresh cilantro', 'teaspoon salt', 'teaspoons lime juice', 'ingredients', 'ingredients'],
    ['ounce cans Campbell Condensed Cream Mushroom Soup % Fat Free', 'cup milk', 'teaspoons soy sauce', 'teaspoon ground black pepper', 'cups green beans', 'cups French Fried Onions', 'ingredients', 'ingredients'],
    ['eggs', 'cup milk', 'cup seasoned bread crumbs', 'teaspoons minced onion', 'teaspoon salt', 'teaspoon sage', 'cup fresh mushrooms', 'pounds ground beef', 'cup ketchup', 'tablespoons brown sugar', 'teaspoon ground mustard', 'teaspoon Worcestershire sauce', 'ingredients', 'ingredients'],
    ['cup butter', 'harina', 'cup water', 'cups frozen whole kernel corn', 'cup cornmeal', 'cup white sugar', 'tablespoons heavy whipping cream', 'teaspoon salt', 'teaspoon baking powder', 'ingredients', 'ingredients'],
    ['ounce black beans', 'small onion', 'clove garlic', 'tablespoon fresh cilantro', 'teaspoon cayenne pepper', 'salt', 'ingredients', 'ingredients'],
    ['cups carrots', 'tablespoons butter', 'tablespoons brown sugar', 'tablespoons fresh dill', 'teaspoon salt', 'teaspoon black pepper', 'ingredients', 'ingredients'],
    ['bacon', 'cup water', 'tablespoons white sugar', 'onion', 'teaspoons garlic', 'teaspoon red pepper flakes', 'teaspoon seasoning salt', 'teaspoons caraway seed', 'large head cabbage small wedges', 'pound Polish kielbasa', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'large onion', 'bunch green onions', 'cloves garlic', 'pounds spinach', 'cup fresh parsley', 'eggs', 'cup ricotta cheese', 'cup feta cheese', 'sheets dough', 'cup olive oil', 'ingredients', 'ingredients'],
    ['cups cabbage', 'unpeeled red apple', 'unpeeled Granny Smith apple', 'carrot', 'cup red bell pepper', 'green onions', 'cup mayonnaise', 'cup brown sugar', 'tablespoon lemon juice', 'ingredients', 'ingredients'],
    ['clove garlic', 'ounce black beans drain reserve liquid', 'tablespoons lemon juice', 'tablespoons', 'teaspoon ground cumin', 'teaspoon salt', 'teaspoon cayenne pepper', 'teaspoon paprika', 'Greek olives', 'ingredients', 'ingredients'],
    ['pound lean ground beef', 'cup dry bread crumbs', 'cup onions', 'egg', 'teaspoon salt', 'teaspoon ground black pepper', 'beef broth', 'large onion', 'cup mushrooms', 'tablespoons cornstarch', 'tablespoons water', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'onion', 'cloves garlic', 'small green bell peppers', 'habanero pepper seeded', 'pounds ground turkey', 'tablespoons chili powder', 'teaspoons red pepper flakes', 'tablespoon paprika', 'tablespoon ground cumin', 'teaspoons oregano', 'teaspoon ground black pepper', 'ounce envelope instant hot chocolate mix', 'teaspoons seasoned salt', 'tablespoon Worcestershire sauce', 'teaspoon liquid smoke flavoring', 'ounce cans tomatoes green chile peppers', 'ounce tomato sauce', 'ounce kidney beans', 'cup cheap beer', 'cup whole kernel corn', 'tablespoon hot pepper sauce', 'ingredients', 'ingredients'],
    ['ounce beans', 'ounce jar red peppers', 'tablespoons lemon juice', 'tablespoons', 'clove garlic', 'teaspoon ground cumin', 'teaspoon cayenne pepper', 'teaspoon salt', 'tablespoon fresh parsley', 'ingredients', 'ingredients'],
    ['cups fresh blueberries', 'inch deep dish pie crust', 'Custard', 'cup white sugar', 'cup purpose flour', 'teaspoon salt', 'eggs', 'cup sour cream', 'Streusel', 'cup white sugar', 'cup purpose flour', 'cup butter', 'ingredients', 'ingredients'],
    ['teaspoons Italian seasoning', 'teaspoon seasoning salt', 'teaspoon mustard powder', 'teaspoon garlic powder', 'teaspoon ground black pepper', 'pound whole chicken', 'lemons', 'tablespoons olive oil', 'ingredients', 'ingredients'],
    ['cups zucchini', 'cups onions', 'red bell pepper', 'pound extra lean ground turkey', 'cup uncooked couscous', 'egg', 'tablespoons Worcestershire sauce', 'tablespoon Dijon mustard', 'cup barbecue sauce', 'ingredients', 'ingredients'],
    ['bacon', 'ounces fresh cheese tortellini', 'cup mayonnaise', 'cup white sugar', 'teaspoons cider vinegar', 'fresh broccoli florets', 'cup raisins', 'cup sunflower seeds', 'red onion', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'cup onion', 'cup celery', 'teaspoon garlic powder', 'cups potatoes', 'carrots', 'chicken broth', 'teaspoon salt', 'teaspoon ground black pepper', 'teaspoon dill weed', 'ounce cans salmon', 'fluid ounce milk', 'ounce corn', 'pound Cheddar cheese', 'ingredients', 'ingredients'],
    ['ripe bananas', 'eggs', 'cup shortening', 'cups white sugar', 'tablespoon vanilla extract', 'cups purpose flour', 'teaspoons soda', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['shortbread crust', 'cup purpose flour', 'cup unsalted butter room temperature', 'cup confectioners sugar', 'teaspoon vanilla extract', 'teaspoon salt', 'lemon layer', 'large eggs', 'large egg yolk', 'cup white sugar', 'tablespoons purpose flour', 'cup lemon juice', 'tablespoon lemon zest', 'teaspoon confectioners sugar', 'ingredients', 'ingredients'],
    ['ounce uncooked penne pasta', 'tablespoons vegetable oil', 'pound portobello mushrooms', 'cup margarine', 'cup purpose flour', 'large clove garlic', 'teaspoon basil', 'cups milk', 'cups mozzarella cheese', 'ounce package spinach', 'cup soy sauce', 'ingredients', 'ingredients'],
    ['pounds boneless skinless chicken breasts bite size pieces', 'cup onion', 'clove garlic', 'tablespoons butter', 'cubes chicken bouillon', 'cup hot water', 'teaspoon ground cumin', 'cups half cream', 'cups Monterey Jack cheese', 'ounce cream style corn', 'ounce green chiles', 'hot pepper sauce', 'tomato', 'fresh cilantro sprigs garnish optional', 'ingredients', 'ingredients'],
    ['cup balsamic vinegar', 'tablespoons vegetable oil', 'teaspoon salt', 'teaspoon white sugar', 'teaspoon ground black pepper', 'teaspoon ground cumin', 'teaspoon chili powder', 'tablespoons fresh cilantro', 'ounce black beans', 'ounce sweet corn', 'ingredients', 'ingredients'],
    ['apple', 'cup water', 'ounce white cake mix', 'cup brown sugar', 'teaspoon ground cinnamon', 'cup butter', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'tablespoon sesame seeds', 'pound fresh green beans inch pieces', 'cup chicken broth', 'teaspoon salt', 'black pepper', 'ingredients', 'ingredients'],
    ['tablespoon butter', 'cup uncooked quinoa', 'vegetable broth', 'teaspoons garlic', 'tablespoons fresh parsley', 'tablespoon fresh thyme', 'teaspoon salt', 'small onion', 'fresh lemon juice optional', 'ingredients', 'ingredients'],
    ['ounce beans chickpeas mashed', 'tablespoons mayonnaise', 'teaspoons spicy brown mustard', 'tablespoon sweet pickle relish', 'green onions', 'salt pepper', 'ingredients', 'ingredients'],
    ['ounce hearts', 'cup Romano cheese', 'cup Parmesan cheese', 'teaspoon garlic', 'ounce package spinach', 'cup heavy cream', 'cup sour cream', 'cup mozzarella cheese', 'ingredients', 'ingredients'],
    ['teaspoons vegetable oil', 'tablespoon garlic', 'teaspoon fresh ginger root', 'cup soy sauce', 'cup water', 'cup dark brown sugar', 'pound beef flank steak inch thick diagonal', 'cup cornstarch', 'cup vegetable oil frying', 'green onions inch lengths', 'ingredients', 'ingredients'],
    ['cups brown rice', 'cups water', 'ounce cans black beans', 'ounce cans pinto beans', 'ounce whole kernel corn', 'ounce tomatoes green chiles', 'inch flour tortillas', 'pepperjack cheese', 'ingredients', 'ingredients'],
    ['red potatoes', 'ounce container sour cream', 'cup butter', 'ounce cream chicken soup', 'cups Cheddar cheese', 'cup green onion', 'cups ham', 'salt pepper', 'cups Parmesan cheese bread crumbs', 'cup butter', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'ounce packet dry fajita seasoning', 'tablespoon vegetable oil', 'ounce black beans', 'ounce Mexican style corn', 'cup salsa', 'ounce mixed salad greens', 'onion', 'tomato wedges', 'ingredients', 'ingredients'],
    ['cup olive oil', 'carrots', 'onion', 'teaspoon sea salt', 'teaspoon ground black pepper', 'teaspoon ground cumin', 'teaspoon ground turmeric', 'head cabbage', 'potatoes inch cubes', 'ingredients', 'ingredients'],
    ['ounce cans tuna', 'cup mayonnaise', 'cup celery', 'tablespoons onion', 'tablespoon parsley', 'teaspoon red wine vinegar', 'pinch salt', 'black pepper', 'seedless rye bread', 'ripe tomato', 'Swiss cheese', 'paprika garnish', 'ingredients', 'ingredients'],
    ['cup vegetable oil', 'cup honey', 'teaspoon vanilla extract', 'eggs', 'cup bananas', 'cups whole wheat flour', 'teaspoon salt', 'teaspoon baking soda', 'cup hot water', 'cup walnuts', 'ingredients', 'ingredients'],
    ['slices bacon inch pieces', 'onion', 'ounce cans chicken broth', 'cups water', 'large potatoes', 'teaspoon salt', 'teaspoon dill weed', 'teaspoon ground white pepper', 'cup purpose flour', 'cups half cream', 'fluid ounce milk', 'ingredients', 'ingredients'],
    ['ounce sweet potatoes', 'cup butter pieces', 'cup brown sugar', 'cups miniature marshmallows', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'tablespoons olive oil', 'tablespoon garlic', 'small red onion', 'bunch Swiss chard center ribs', 'cup dry white wine', 'tablespoon fresh lemon juice', 'tablespoons Parmesan cheese', 'salt optional', 'ingredients', 'ingredients'],
    ['avocados', 'lemon', 'tablespoons onion', 'teaspoon salt', 'tablespoons olive oil', 'ingredients', 'ingredients'],
    ['potatoes', 'red onion', 'teaspoon salt', 'teaspoon ground black pepper', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['tablespoon parsley', 'tablespoon basil', 'skinless boneless chicken breast halves', 'cloves garlic', 'teaspoon salt', 'teaspoon red pepper flakes', 'tomatoes', 'ingredients', 'ingredients'],
    ['pound ground beef', 'tablespoon olive oil', 'green bell pepper', 'onion', 'carrot', 'celery stalks', 'tablespoon paprika', 'teaspoon salt', 'teaspoon black pepper', 'teaspoon red pepper', 'pinch ground cinnamon', 'pinch ground cloves', 'cup water', 'cup red wine', 'cube beef bouillon', 'tablespoons half', 'potatoes', 'ingredients', 'ingredients'],
    ['medium beets chunks', 'tablespoons olive oil', 'teaspoon garlic powder', 'teaspoon kosher salt', 'teaspoon ground black pepper', 'teaspoon sugar', 'medium sweet potatoes chunks', 'large sweet onion', 'ingredients', 'ingredients'],
    ['cup butter', 'ounces uncooked spaghetti', 'cup onion', 'teaspoon garlic', 'cups uncooked jasmine rice', 'cups vegetable broth', 'salt pepper', 'pound fresh asparagus inch pieces', 'cup cashew halves', 'ingredients', 'ingredients'],
    ['fresh peaches', 'cup purpose flour', 'cup white sugar', 'cup butter', 'recipe pastry inch double crust pie', 'ingredients', 'ingredients'],
    ['tablespoons lemon juice', 'cups water', 'Granny Smith apples', 'tablespoons butter', 'cup brown sugar', 'teaspoon ground cinnamon', 'tablespoon cornstarch', 'tablespoon water', 'ounce package frozen puff pastry sheets', 'cup confectioners sugar', 'tablespoon milk', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['cloves garlic', 'cup olive oil', 'baking potatoes cubed', 'cup milk', 'cup Parmesan cheese', 'tablespoons butter', 'teaspoon salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['pound fresh asparagus', 'cup onion', 'cup vegetable broth', 'tablespoon butter', 'tablespoons purpose flour', 'teaspoon salt', 'pinch ground black pepper', 'cups vegetable broth', 'cup soy milk', 'cup yogurt', 'teaspoon lemon juice', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['ounce coconut milk', 'cups water', 'teaspoon sugar', 'pinch salt', 'cups uncooked jasmine rice', 'ingredients', 'ingredients'],
    ['potatoes', 'eggs', 'stalk celery', 'cup sweet relish', 'clove garlic', 'tablespoons prepared mustard', 'cup mayonnaise', 'salt pepper', 'ingredients', 'ingredients'],
    ['ounce frozen corn kernels', 'ounce package cream cheese', 'cup butter', 'cup milk', 'tablespoon white sugar', 'salt pepper', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'tablespoons white sugar', 'teaspoon salt', 'cup shortening', 'egg', 'cup cold water', 'fresh peaches', 'cup lemon juice', 'cup orange juice', 'cup butter', 'cups white sugar', 'teaspoon ground nutmeg', 'teaspoon ground cinnamon', 'tablespoon cornstarch', 'tablespoon white sugar', 'tablespoon butter', 'ingredients', 'ingredients'],
    ['pound sliced bacon', 'cup butter', 'cup pine nuts', 'Brussels sprouts', 'green onions', 'teaspoon seasoning salt', 'pepper', 'ingredients', 'ingredients'],
    ['pounds ground beef', 'ounce beans', 'cups Cheddar Monterey Jack cheese blend', 'ounce container sour cream', 'cup guacamole', 'cup salsa', 'ounce black olives', 'cup tomatoes', 'cup green onions', 'ingredients', 'ingredients'],
    ['cups water', 'cube chicken bouillon', 'black eyed peas', 'onion', 'cloves garlic', 'red bell pepper diced', 'jalapeno chile', 'ounces ham', 'slices bacon', 'teaspoon cayenne pepper', 'teaspoons cumin', 'salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['ounce beans liquid', 'clove garlic', 'teaspoons ground cumin', 'teaspoon salt', 'tablespoon olive oil', 'ingredients', 'ingredients'],
    ['ounce flaky biscuit dough', 'apples', 'cup white sugar', 'cup water', 'cup butter', 'teaspoons vanilla extract', 'teaspoon ground cinnamon', 'ingredients', 'ingredients'],
    ['ounce pumpkin puree', 'fluid ounce milk', 'eggs', 'cup white sugar', 'teaspoons pumpkin pie spice', 'ounce yellow cake mix', 'cup butter', 'cups walnuts', 'ingredients', 'ingredients'],
    ['ounce cream mushroom soup', 'cup mayonnaise', 'egg', 'cup onion', 'ounce packages broccoli', 'ounces sharp Cheddar cheese', 'salt', 'ground black pepper', 'paprika', 'ingredients', 'ingredients'],
    ['eggs', 'cloves garlic', 'ounces feta cheese', 'ounce box frozen spinach dry', 'ground turkey', 'ingredients', 'ingredients'],
    ['pound red potatoes', 'pound Yukon Gold yellow potatoes', 'fresh jalapeno pepper', 'ounces baby carrots', 'cloves garlic', 'ounce package white corn', 'cup butter', 'cup Cheddar cheese', 'salt pepper', 'ingredients', 'ingredients'],
    ['cup vegetable broth', 'dehydrated sun tomatoes', 'ounce uncooked penne pasta', 'tablespoons pine nuts', 'tablespoon olive oil', 'teaspoon red pepper flakes', 'clove garlic', 'bunch fresh spinach bite size pieces', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['cup Parmesan cheese', 'tablespoon olive oil', 'teaspoon onion powder', 'teaspoon garlic powder', 'teaspoon ground black pepper', 'teaspoon salt', 'russet potatoes eighths', 'ingredients', 'ingredients'],
    ['Lemon Butter Sauce', 'cup white wine', 'tablespoons fresh lemon juice', 'tablespoons heavy cream', 'cup butter', 'salt pepper', 'Chicken Pasta', 'pound dry farfalle bow tie pasta', 'skinless boneless chicken breast halves inch thickness', 'tablespoon olive oil', 'tablespoon butter', 'cup purpose flour', 'salt pepper', 'ounces bacon', 'ounces mushrooms', 'ounces artichoke hearts', 'teaspoons capers', 'fresh parsley garnish', 'ingredients', 'ingredients'],
    ['tablespoons brown sugar', 'teaspoons soy sauce', 'tablespoon sesame oil optional', 'cup vegetable oil', 'tablespoons rice vinegar', 'ounce package rice noodles', 'head iceberg lettuce', 'boneless chicken breast halves', 'green onions', 'tablespoon sesame seeds', 'ingredients', 'ingredients'],
    ['cup white sugar', 'cup lemon juice', 'teaspoons onion', 'teaspoon Dijon style prepared mustard', 'teaspoon salt', 'cup vegetable oil', 'tablespoon poppy seeds', 'head romaine lettuce bite size pieces', 'ounces Swiss cheese', 'cup cashews', 'cup cranberries', 'apple diced', 'pears diced', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'small red potatoes', 'cup fresh spinach', 'tablespoons green onions', 'teaspoon garlic', 'salt pepper', 'eggs', 'cup milk', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['ounce package cream cheese', 'cup brown sugar', 'tablespoon vanilla extract', 'ingredients', 'ingredients'],
    ['large russet potatoes half', 'cup butter', 'cup whole milk', 'salt black pepper', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'teaspoon salt', 'teaspoon paprika', 'egg', 'tablespoons milk', 'skinless boneless chicken breast halves', 'tablespoons butter', 'pound fresh mushrooms', 'cup onion', 'cup chicken broth', 'cup white wine', 'tablespoons lemon juice', 'tablespoon cornstarch', 'tablespoon fresh parsley garnish', 'ingredients', 'ingredients'],
    ['seedless green grapes', 'ounce package cream cheese', 'ounce container sour cream', 'cup white sugar', 'teaspoon vanilla extract', 'ounces pecans', 'tablespoons brown sugar', 'ingredients', 'ingredients'],
    ['Cinnamon Croutons', 'tablespoons butter', 'tablespoon brown sugar', 'teaspoon ground cinnamon', 'slices whole wheat bread', 'Soup', 'cup onion', 'tablespoons butter', 'ounce cans chicken broth', 'ounce pumpkin puree', 'teaspoon salt', 'teaspoon ground cinnamon', 'teaspoon ground ginger', 'teaspoon ground black pepper', 'cup heavy whipping cream', 'ingredients', 'ingredients'],
    ['pound rigatoni pasta', 'pound ground beef', 'pound Italian sausage', 'ounce mushrooms', 'ounce jars spaghetti sauce', 'pounds mozzarella cheese', 'pepperoni', 'ingredients', 'ingredients'],
    ['cups', 'cups white sugar', 'cups pumpkin', 'eggs', 'teaspoons soda', 'teaspoons ground cinnamon', 'teaspoon salt', 'cups purpose flour', 'tablespoons butter', 'tablespoons milk', 'cups confectioners sugar', 'teaspoons vanilla extract', 'cup brown sugar', 'ingredients', 'ingredients'],
    ['ounce package fusilli spiral pasta', 'cherry tomatoes', 'pound provolone cheese', 'pound salami', 'pound pepperoni half', 'large green bell pepper inch pieces', 'ounce black olives', 'ounce jar pimentos', 'ounce Italian salad dressing', 'ingredients', 'ingredients'],
    ['cup butter', 'cup white sugar', 'cup honey', 'eggs', 'teaspoon salt', 'cups purpose flour', 'cup cornmeal', 'teaspoon baking powder', 'cup milk', 'cup frozen corn kernels', 'ingredients', 'ingredients'],
    ['cup olive oil', 'cups water', 'cloves garlic', 'cup fresh lemon juice', 'teaspoon thyme', 'teaspoon rosemary', 'cubes chicken bouillon', 'ground black pepper', 'potatoes', 'ingredients', 'ingredients'],
    ['ounce container ricotta cheese', 'ounce package spinach dry', 'cup onion', 'egg', 'teaspoons fresh parsley', 'teaspoon pepper', 'teaspoon garlic powder', 'cups mozzarella cheese', 'cup Parmesan cheese', 'ounce jars spaghetti sauce', 'cups water', 'ounce package manicotti shells', 'ingredients', 'ingredients'],
    ['pound prosciutto', 'ounce package Neufchatel cheese', 'spears fresh asparagus', 'ingredients', 'ingredients'],
    ['large shrimp deveined', 'cup cornstarch', 'teaspoon salt', 'teaspoon cayenne pepper', 'cups coconut', 'egg whites foamy', 'ingredients', 'ingredients'],
    ['cloves garlic', 'green onions white parts tops', 'tablespoons canola oil', 'whole chicken breast', 'salt pepper', 'cup salsa', 'ounce package tortilla chips', 'ounce package Cheddar Monterey Jack cheese blend', 'large tomato', 'ingredients', 'ingredients'],
    ['pound ground beef', 'medium onion', 'clove garlic', 'tablespoon chili powder', 'teaspoon ground cumin', 'teaspoon paprika', 'teaspoon black pepper', 'teaspoon salt', 'ounce beans', 'inch flour tortillas', 'cup salsa', 'cup Cheddar cheese', 'cup Monterey Jack cheese', 'green onions', 'roma plum tomatoes', 'cup jalapeno pepper', 'cup sour cream optional', 'ingredients', 'ingredients'],
    ['large ripe tomatoes', 'cucumbers', 'small red onion', 'cup olive oil', 'teaspoons lemon juice', 'teaspoons oregano', 'salt pepper', 'cup feta cheese', 'black Greek olives', 'ingredients', 'ingredients'],
    ['ounce envelope dry onion soup mix', 'pound beef chuck roast', 'ounce cranberry sauce', 'tablespoons butter', 'tablespoons purpose flour', 'ingredients', 'ingredients'],
    ['ounce fillets tilapia', 'salt pepper', 'tablespoons butter', 'cloves garlic', 'fresh basil leaves', 'large tomato', 'cup white wine', 'ingredients', 'ingredients'],
    ['ounce tomatoes green chile peppers such RO TEL', 'pound loaf cheese food such Velveeta', 'large skinless boneless chicken breast halves', 'cup sour cream', 'cup green onion', 'tablespoons taco seasoning mix', 'tablespoons jalapeno pepper optional', 'cup black beans', 'ingredients', 'ingredients'],
    ['large tomatoes', 'sweet onion', 'tablespoons olive oil', 'tablespoon fresh oregano', 'teaspoon fresh basil', 'teaspoons fresh parsley', 'pound loaf Italian bread inch slices', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['thick slices bacon', 'cup onions', 'teaspoon garlic', 'pound fresh green beans', 'cup water', 'teaspoon salt', 'pinch ground black pepper', 'ingredients', 'ingredients'],
    ['zucchini', 'pound pork sausage', 'cup dry bread crumbs', 'clove garlic', 'ounce jar spaghetti sauce', 'cup Parmesan cheese', 'cup mozzarella cheese', 'ingredients', 'ingredients'],
    ['lasagna noodles', 'tablespoons olive oil', 'cup fresh mushrooms', 'cup onion', 'tablespoon garlic', 'cups fresh spinach', 'cups cheese', 'cup Romano cheese', 'teaspoon salt', 'teaspoon oregano', 'teaspoon basil leaves', 'teaspoon ground black pepper', 'egg', 'cups mozzarella cheese', 'tomato pasta sauce', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['bone chicken breast halves', 'chicken broth', 'white onion', 'clove garlic', 'teaspoons salt', 'pound fresh tomatillos husks', 'serrano peppers', 'white onion', 'clove garlic', 'pinch salt', 'corn tortillas', 'cup vegetable oil', 'cup queso fresco', 'white onion', 'bunch fresh cilantro', 'ingredients', 'ingredients'],
    ['green bell peppers seeded', 'pound ground turkey', 'tablespoons olive oil', 'onion', 'cup mushrooms', 'zucchini', 'red bell pepper', 'yellow bell pepper', 'cup fresh spinach', 'ounce tomatoes', 'tablespoon tomato paste', 'Italian seasoning', 'garlic powder', 'salt pepper', 'ingredients', 'ingredients'],
    ['ounce package strawberries', 'cup sugar', 'cup heavy cream', 'ingredients', 'ingredients'],
    ['cup orange juice', 'cup dry white wine', 'cup maple syrup', 'teaspoons fresh rosemary', 'teaspoon salt', 'teaspoon black pepper', 'skinless boneless chicken breast halves', 'tablespoons butter', 'tablespoons olive oil', 'ingredients', 'ingredients'],
    ['thin asparagus spears', 'tablespoon extra virgin olive oil', 'ounce Parmesan cheese', 'black pepper', 'cup balsamic vinegar', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cup vegetable oil', 'tablespoons cold milk', 'teaspoons white sugar', 'teaspoon salt', 'Fuji apples', 'cup white sugar', 'tablespoons purpose flour', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'cup purpose flour', 'cup white sugar', 'cup butter', 'ingredients', 'ingredients'],
    ['cups fresh blackberries', 'cup white sugar', 'cup purpose flour', 'recipe pastry inch double crust pie', 'tablespoons milk', 'cup white sugar', 'ingredients', 'ingredients'],
    ['pound fresh asparagus inch pieces', 'cup onion', 'ounce chicken broth', 'tablespoons butter', 'tablespoons purpose flour', 'teaspoon salt', 'pinch ground black pepper', 'cup milk', 'cup sour cream', 'teaspoon fresh lemon juice', 'ingredients', 'ingredients'],
    ['PUDDING', 'cups soft bread cubes', 'cup raisins', 'cups apples', 'cup brown sugar', 'cups milk', 'cup margarine', 'teaspoon ground cinnamon', 'teaspoon vanilla extract', 'eggs', 'VANILLA SAUCE', 'cup white sugar', 'cup brown sugar', 'cup milk', 'cup margarine', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['tomatoes', 'teaspoon basil', 'tablespoons Parmesan cheese', 'tablespoons olive oil', 'clove garlic', 'salt', 'ground black pepper', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cup ground flax seed', 'cup oat bran', 'cup brown sugar', 'teaspoons soda', 'teaspoon baking powder', 'teaspoon salt', 'teaspoons ground cinnamon', 'cup skim milk', 'eggs', 'teaspoon vanilla extract', 'tablespoons vegetable oil', 'cups carrots', 'apples', 'cup raisins', 'cup mixed nuts', 'ingredients', 'ingredients'],
    ['Souvlaki Marinade', 'cup balsamic vinaigrette salad dressing', 'tablespoons lemon juice', 'tablespoon oregano', 'teaspoon black pepper', 'skinless boneless chicken breast halves', 'Tzatziki Sauce cucumber sauce', 'cup shredded cucumber', 'teaspoon kosher salt', 'cup plain yogurt', 'cup sour cream', 'tablespoon lemon juice', 'tablespoon rice vinegar', 'teaspoon olive oil', 'clove garlic', 'tablespoon fresh dill', 'teaspoon Greek seasoning', 'kosher salt', 'black pepper', 'large pita bread rounds', 'heart romaine lettuce inch slices', 'red onion', 'tomato', 'cup kalamata olives', 'cup pepperoncini', 'cup feta cheese', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cup white sugar', 'teaspoon salt', 'teaspoons powder', 'teaspoon ground cinnamon', 'cup vegetable oil', 'egg', 'cup milk', 'apples', 'cup white sugar', 'cup purpose flour', 'cup butter', 'teaspoons ground cinnamon', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'teaspoon salt', 'teaspoon garlic salt', 'teaspoon garlic powder', 'teaspoon oregano', 'pound boneless pork loin chops thin', 'tablespoons butter', 'cup olive oil', 'cups fresh mushrooms', 'teaspoon garlic', 'cup Marsala wine', 'ingredients', 'ingredients'],
    ['cups potatoes', 'tablespoon grated onion', 'eggs', 'tablespoons purpose flour', 'teaspoons salt', 'cup peanut oil frying', 'ingredients', 'ingredients'],
    ['boneless skinless chicken breast halves', 'tablespoon vegetable oil', 'onion', 'ounce tomatoes green chile peppers', 'ounce tomatoes', 'small chile peppers optional', 'ounce cream chicken soup', 'ounce chicken broth', 'bunch green onions', 'pint heavy cream', 'inch corn tortillas', 'cups Cheddar cheese', 'ounce sauce', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'cup onions', 'cup carrots', 'cloves garlic', 'cup green bell pepper', 'cup red bell pepper', 'cup celery', 'tablespoon chili powder', 'cups fresh mushrooms', 'ounce tomatoes liquid', 'ounce kidney beans liquid', 'ounce whole kernel corn undrained', 'tablespoon ground cumin', 'teaspoons oregano', 'teaspoons basil', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'Italian sausage links', 'red bell peppers', 'green bell peppers', 'large red onions', 'cloves garlic', 'fluid ounce bottles beer', 'ounce tomato paste', 'tablespoons fresh oregano', 'tablespoons fresh cilantro', 'tablespoons hot sauce', 'salt pepper', 'ingredients', 'ingredients'],
    ['cups purpose flour coating', 'teaspoon salt', 'teaspoon ground black pepper', 'pound chicken pieces', 'tablespoons vegetable oil', 'onion', 'cloves garlic', 'green bell pepper', 'ounce tomatoes', 'teaspoon oregano', 'cup white wine', 'cups fresh mushrooms', 'salt pepper', 'ingredients', 'ingredients'],
    ['cups sweet potatoes', 'cup white sugar', 'tablespoons vanilla extract', 'eggs', 'pint heavy cream', 'pound butter', 'cup brown sugar', 'cup purpose flour', 'cups pecans', 'ingredients', 'ingredients'],
    ['pounds skinless boneless chicken breast halves', 'cup butter', 'cloves garlic', 'cups dry bread crumbs', 'ingredients', 'ingredients'],
    ['tablespoons purpose flour', 'teaspoon salt', 'teaspoon black pepper', 'pound feta cheese', 'tablespoon fresh lemon juice', 'teaspoon oregano', 'boneless skinless chicken breast halves', 'tablespoons olive oil', 'cups water', 'cube chicken bouillon', 'cups fresh spinach leaves', 'ripe tomato', 'ingredients', 'ingredients'],
    ['tablespoons Dijon mustard', 'teaspoon sesame oil', 'cloves garlic', 'fresh ground black pepper', 'cup maple syrup', 'pounds pork tenderloin', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'tablespoon ground cinnamon', 'teaspoon baking soda', 'teaspoon salt', 'cups vegetable oil', 'eggs', 'cups white sugar', 'cups peaches', 'ingredients', 'ingredients'],
    ['ounces sour cream', 'ounce container topping', 'ounce instant vanilla pudding mix', 'cups whole milk', 'ounce package vanilla wafer cookies', 'bananas', 'ingredients', 'ingredients'],
    ['ounce cans Italian style diced tomatoes', 'ounce beans', 'ounces fresh spinach', 'ounces penne pasta', 'cup feta cheese', 'ingredients', 'ingredients'],
    ['ounce whole kernel corn', 'ounce cream style corn', 'cup sour cream', 'cup butter margarine', 'eggs', 'ounce package corn muffin mix', 'ingredients', 'ingredients'],
    ['cup chicken broth', 'pound boned skinned chicken breast halves', 'salt pepper', 'pound fresh asparagus inch pieces', 'ounce jar red peppers', 'clove garlic', 'cup roma plum tomatoes', 'teaspoon balsamic vinegar', 'cup mozzarella cheese', 'ingredients', 'ingredients'],
    ['cups Cheddar cheese', 'cup Parmesan cheese', 'teaspoon oregano', 'teaspoon basil', 'cloves garlic', 'salt pepper', 'medium zucchinis', 'plum tomatoes', 'cup butter', 'tablespoons onion', 'cup fine bread crumbs', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'pound ground beef', 'large onion', 'cups cabbage', 'ounce cans red kidney beans', 'cups water', 'ounces tomato sauce', 'beef bouillon cubes', 'teaspoons ground cumin', 'teaspoon salt', 'teaspoon pepper', 'ingredients', 'ingredients'],
    ['ounces elbow macaroni', 'carrots', 'large red onion', 'green bell pepper seeded', 'red bell pepper seeded', 'cup celery', 'cups mayonnaise', 'ounce condensed milk', 'cup white sugar', 'cup white vinegar', 'salt pepper', 'ingredients', 'ingredients'],
    ['cup brown sugar', 'cup purpose flour', 'teaspoon ground cinnamon', 'cup butter', 'cups purpose flour', 'teaspoons powder', 'teaspoon salt', 'cup butter', 'cup white sugar', 'egg', 'teaspoon vanilla extract', 'cup milk', 'cup fresh blueberries', 'cup confectioners sugar', 'ingredients', 'ingredients'],
    ['ounce bottles barbeque sauce', 'ounce pineapple chunks', 'green bell pepper', 'onion', 'cloves garlic', 'boneless skinless chicken breast halves', 'ingredients', 'ingredients'],
    ['teaspoons butter', 'mushrooms', 'clove garlic', 'green onion', 'chicken broth', 'cup uncooked white rice', 'teaspoon fresh parsley', 'salt pepper', 'ingredients', 'ingredients'],
    ['tablespoon butter', 'onion', 'cup butter', 'cup flour', 'cups milk', 'cups chicken stock', 'cups broccoli florets', 'cup matchstick cut carrots', 'stalk celery', 'cups sharp Cheddar cheese', 'salt black pepper', 'ingredients', 'ingredients'],
    ['Marinara Sauce', 'tablespoon olive oil', 'cloves garlic', 'pinch red pepper flakes', 'tablespoon fresh basil', 'teaspoon red wine vinegar', 'teaspoon white sugar', 'ounce tomatoes', 'salt pepper', 'Grinders', 'tablespoon butter', 'medium zucchini', 'red pepper flakes', 'salt pepper', 'cups mozzarella cheese', 'inch French Italian sandwich rolls split', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'pound shrimp deveined', 'cloves garlic', 'tablespoon white wine', 'pound linguine pasta', 'tomatoes', 'teaspoon oregano', 'teaspoon basil', 'ounce package feta cheese', 'ingredients', 'ingredients'],
    ['ounce containers plain yogurt', 'cucumbers seeded diced', 'tablespoons olive oil', 'lemon', 'salt pepper', 'tablespoon fresh dill', 'cloves garlic', 'ingredients', 'ingredients'],
    ['cups water', 'cup uncooked white rice', 'large green bell peppers seeded', 'pounds ground beef', 'onion', 'garlic powder', 'salt', 'ground black pepper', 'ounce tomato sauce', 'cups mozzarella cheese', 'ingredients', 'ingredients'],
    ['cup uncooked white rice', 'ounces broccoli florets', 'ounces cauliflower florets', 'cup butter', 'onion', 'cheese food', 'ounce cream chicken soup', 'fluid ounces milk', 'cups buttery round crackers', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breasts', 'cup bread crumbs', 'cup Parmesan cheese', 'cup butter', 'ounce package garlic cheese spread', 'ingredients', 'ingredients'],
    ['ounce container sour cream', 'cup unsalted butter', 'ounce corn', 'ounce whole kernel corn', 'ounce package corn bread mix', 'ingredients', 'ingredients'],
    ['ounce jar maraschino cherries', 'cup butter', 'cup brown sugar', 'cup coconut', 'ounce pineapple juice', 'ounce pineapple juice', 'ounce yellow cake mix', 'ingredients', 'ingredients'],
    ['pound ground beef', 'ounce green beans', 'ounce cream mushroom soup', 'onion', 'cup Cheddar cheese', 'cups potatoes', 'ingredients', 'ingredients'],
    ['ounce package penne pasta', 'yellow squash', 'zucchini', 'carrot matchsticks', 'red bell peppers matchsticks', 'pint grape tomatoes', 'cup fresh green beans inch pieces', 'spears asparagus inch pieces', 'cup olive oil', 'tablespoon Italian seasoning', 'tablespoon lemon juice', 'teaspoon salt', 'teaspoon black pepper', 'tablespoon butter', 'large yellow onion', 'cloves garlic', 'teaspoons lemon zest', 'cup fresh basil leaves', 'cup fresh parsley', 'tablespoons balsamic vinegar', 'cup Romano cheese', 'ingredients', 'ingredients'],
    ['ounce tuna', 'egg', 'cup Italian seasoned bread crumbs', 'cup onion', 'cup celery', 'cup red bell pepper', 'cup mayonnaise', 'tablespoons chili sauce', 'teaspoon dill weed', 'teaspoon salt', 'teaspoon ground black pepper', 'hot pepper sauce', 'Worcestershire sauce', 'hamburger buns', 'tomato', 'leaves lettuce optional', 'ingredients', 'ingredients'],
    ['cups extra-sharp Cheddar cheese', 'ounces cream cheese', 'cup mayonnaise', 'teaspoon garlic powder', 'teaspoon ground cayenne pepper optional', 'teaspoon onion powder', 'jalapeno pepper seeded optional', 'ounce jar pimento', 'salt black pepper', 'ingredients', 'ingredients'],
    ['pound beef top sirloin steak', 'cup soy sauce', 'tablespoons white sugar', 'tablespoons cornstarch', 'teaspoon ground ginger', 'tablespoons vegetable oil', 'red onion inch squares', 'green bell pepper inch squares', 'tomatoes wedges', 'ingredients', 'ingredients'],
    ['cup Parmesan cheese', 'teaspoon Italian seasoning', 'skinless boneless chicken breasts', 'cup green onions', 'tablespoon butter', 'tablespoon purpose flour', 'cup skim milk', 'ounce package spinach', 'tablespoon pimento peppers', 'ingredients', 'ingredients'],
    ['ounce cans green beans', 'ounce cream mushroom soup', 'ounce French onions', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['pound sweet Italian sausage', 'large onion', 'stalk celery', 'tablespoon garlic', 'ounce dry lentils', 'cup carrot', 'cups water', 'ounce cans chicken broth', 'ounce tomatoes', 'tablespoon garlic powder', 'tablespoon fresh parsley', 'bay leaves', 'teaspoon oregano', 'teaspoon thyme', 'teaspoon basil', 'tablespoon salt', 'teaspoon black pepper', 'pound ditalini pasta optional', 'ingredients', 'ingredients'],
    ['fresh mint leaves', 'lime wedges', 'tablespoons white sugar', 'cup ice cubes', 'fluid ounces white rum', 'cup club soda', 'ingredients', 'ingredients'],
    ['Toppings', 'ounce pinto beans liquid', 'cup picante sauce', 'pound ground beef', 'ounce package taco seasoning mix', 'Fry Bread', 'cups purpose flour more', 'tablespoon baking powder', 'teaspoon salt', 'cup milk', 'oil frying', 'cups iceberg lettuce', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'medium onion', 'medium zucchinis thin strips', 'tablespoons teriyaki sauce', 'tablespoon soy sauce', 'tablespoon sesame seeds', 'ground black pepper', 'ingredients', 'ingredients'],
    ['pounds Yukon Gold potatoes diced', 'medium onion', 'medium carrot', 'rib celery', 'ounces ham diced', 'cloves garlic', 'tablespoons butter', 'cup flour', 'chicken broth', 'cups water', 'cup heavy cream', 'salt pepper', 'cayenne pepper optional', 'fresh chives garnish optional', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoons soda', 'teaspoon baking powder', 'teaspoon salt', 'teaspoons ground cinnamon', 'cups white sugar', 'cup vegetable oil', 'eggs', 'teaspoon vanilla extract', 'cups carrots', 'cup coconut', 'cup walnuts', 'ounce pineapple', 'ounce package cream cheese', 'cup butter', 'cups confectioners sugar', 'ingredients', 'ingredients'],
    ['cups pretzels', 'cup butter', 'tablespoons white sugar', 'ounce package cream cheese', 'cup white sugar', 'ounce container topping', 'ounce strawberry Jell O r', 'cups water', 'ounce frozen strawberries', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'tablespoons balsamic vinegar', 'ripe tomatoes', 'tablespoons mayonnaise', 'teaspoon parsley', 'teaspoon oregano', 'teaspoon black pepper', 'tablespoons Parmesan cheese', 'slices bread', 'ingredients', 'ingredients'],
    ['pound skinless boneless chicken breast halves', 'ounce package cream cheese', 'ounce jar mayonnaise', 'ounce package Cheddar cheese', 'ounce green chile peppers', 'jalapeno pepper', 'ingredients', 'ingredients'],
    ['pound ground beef', 'small onion', 'clove garlic', 'ounce packet taco seasoning mix', 'ounce bag frozen Mexican style corn', 'ounce black beans', 'ounce bag Mexican cheese blend', 'ounce frozen tater tots', 'fluid ounce sauce', 'ingredients', 'ingredients'],
    ['cup butter margarine', 'cup brown sugar', 'egg', 'cup applesauce', 'cup purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'teaspoon pumpkin pie spice', 'cups confectioners sugar', 'tablespoons margarine', 'tablespoon milk', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['potatoes', 'pound lean ground beef', 'ounce mushrooms', 'ounce mixed vegetables', 'ounce cream mushroom soup', 'ounce cream celery soup', 'salt pepper', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['Pastry Single Crust Pie', 'cups purpose flour', 'teaspoon salt', 'cup shortening', 'tablespoons cold water', 'Filling', 'cups pumpkin', 'fluid ounce milk', 'eggs', 'cup brown sugar', 'teaspoon ground cinnamon', 'teaspoon ground ginger', 'teaspoon ground nutmeg', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['pound fresh green beans', 'cup water', 'cup Italian style seasoned bread crumbs', 'cup olive oil', 'salt pepper', 'teaspoon garlic powder', 'teaspoon oregano', 'teaspoon basil', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['quarts water', 'cup Old Bay Seasoning TM', 'new red potatoes', 'hot sausage links inch pieces', 'ears corn', 'large fresh shrimp unpeeled', 'ingredients', 'ingredients'],
    ['cups apples', 'cup white sugar', 'tablespoon butter', 'teaspoon ground cinnamon', 'recipe pastry inch double crust pie', 'ingredients', 'ingredients'],
    ['ounce yellow cake mix', 'eggs', 'cup vegetable oil', 'ounce mandarin orange segments', 'ounce container topping', 'ounce pineapple juice', 'ounce instant vanilla pudding mix', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoons soda', 'teaspoon baking powder', 'teaspoons pumpkin pie spice', 'teaspoon salt', 'cups white sugar', 'cup vegetable oil', 'eggs', 'ounce pumpkin puree', 'cup water', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'cup onion', 'inch thick pork chops', 'teaspoon salt', 'ground black pepper', 'apples', 'tablespoons brown sugar', 'teaspoon ground mustard', 'teaspoon ground cloves', 'cup hot water', 'ingredients', 'ingredients'],
    ['ounce package pastry inch double crust pie', 'egg', 'cups peaches', 'tablespoons lemon juice', 'cup purpose flour', 'cup white sugar', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'teaspoon salt', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['cup white sugar', 'eggs', 'cup fresh lemon juice', 'lemons', 'cup unsalted butter', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'medium onion', 'medium green bell pepper', 'stalks celery', 'chicken broth', 'cup brown rice', 'ounce cans black eyed peas liquid', 'ounce tomatoes green chiles', 'ounce tomatoes', 'cloves garlic', 'ingredients', 'ingredients'],
    ['ounce cans green beans', 'cup milk', 'ounce cream mushroom soup', 'ounce French onions', 'salt pepper', 'ingredients', 'ingredients'],
    ['red potatoes wedges', 'tablespoons butter', 'tablespoons vegetable oil', 'tablespoon fresh rosemary', 'salt pepper', 'ingredients', 'ingredients'],
    ['cup plain yogurt', 'tablespoons vegetable oil', 'tablespoon lemon juice', 'egg whites', 'teaspoon lemon extract optional', 'cups purpose flour', 'cup white sugar', 'teaspoons powder', 'teaspoon salt', 'teaspoon lemon zest', 'cup frozen raspberries', 'tablespoons white sugar decoration optional', 'ingredients', 'ingredients'],
    ['tablespoon cornstarch', 'cloves garlic', 'teaspoons fresh ginger root', 'cup vegetable oil', 'small head broccoli florets', 'cup snow peas', 'cup carrots', 'cup green beans', 'tablespoons soy sauce', 'tablespoons water', 'cup onion', 'tablespoon salt', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'cup onion', 'cloves garlic', 'teaspoons chili powder', 'teaspoon cumin', 'teaspoon oregano', 'cups water', 'ounce tomato soup', 'ounce tomatoes', 'cup salsa', 'cups cooked turkey', 'tablespoon parsley', 'chicken bouillon cubes', 'ounce black beans', 'cups frozen corn', 'cup sour cream', 'cup fresh cilantro', 'Toppings', 'corn tortilla chips', 'cup green onion', 'cup Cheddar Monterey Jack cheese blend', 'cup fresh cilantro', 'cup sour cream', 'ingredients', 'ingredients'],
    ['onions', 'cloves garlic', 'teaspoon fresh ginger', 'cups water', 'cup red lentils', 'ounce beans', 'ounce beans', 'ounce tomatoes', 'cup carrots', 'cup celery', 'teaspoon garam masala', 'teaspoons ground cardamom', 'teaspoon ground cayenne pepper', 'teaspoon ground cumin', 'tablespoon olive oil', 'ingredients', 'ingredients'],
    ['cup butter', 'cup brown sugar', 'cup white sugar', 'eggs', 'teaspoon vanilla extract', 'cups quick cooking oats', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon baking powder', 'teaspoon salt', 'cup raisins', 'cup cranberries', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'ounce package kielbasa sausage', 'medium red potatoes', 'red bell pepper', 'yellow bell pepper', 'ingredients', 'ingredients'],
    ['cup butter', 'ounce package spinach', 'ounce hearts', 'ounce cream cheese', 'ounce containers sour cream', 'cup Parmesan cheese', 'garlic salt', 'ingredients', 'ingredients'],
    ['pound fresh asparagus', 'ounce package egg noodles', 'cloves garlic', 'cup extra virgin olive oil', 'cup butter', 'tablespoon lemon juice', 'pound medium shrimp deveined', 'pound fresh mushrooms', 'cup Parmesan cheese', 'salt pepper', 'ingredients', 'ingredients'],
    ['pound ground beef', 'cup onion', 'cup celery', 'cup carrots', 'cloves garlic', 'ounce tomatoes', 'ounce tomato sauce', 'ounce cans kidney beans', 'cups water', 'teaspoons beef bouillon granules', 'tablespoon parsley', 'teaspoon oregano', 'teaspoon basil', 'cups cabbage', 'ounce whole kernel corn', 'ounce green beans', 'cup macaroni', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'onion', 'cloves garlic', 'ounces green peas', 'tablespoon chicken stock', 'salt pepper', 'ingredients', 'ingredients'],
    ['eggplant inch slices', 'tablespoons', 'tablespoons olive oil', 'ounces ricotta cheese', 'ounces mozzarella cheese', 'cup Parmesan cheese', 'egg', 'cup fresh basil', 'pasta sauce', 'ingredients', 'ingredients'],
    ['unpeeled red potatoes', 'eggs', 'cups mayonnaise', 'tablespoons milk', 'tablespoons white vinegar', 'cup green onions', 'teaspoon salt', 'teaspoon ground black pepper', 'cup celery', 'ingredients', 'ingredients'],
    ['tablespoon extra virgin olive oil', 'ounce packages spinach', 'onion', 'teaspoon oregano', 'teaspoon basil', 'cloves garlic', 'ounce jar spaghetti sauce', 'cups water', 'cups non-fat cottage cheese', 'ounce package part mozzarella cheese', 'cup Parmesan cheese', 'cup fresh parsley', 'teaspoon salt', 'teaspoon black pepper', 'egg', 'ounces lasagna noodles', 'ingredients', 'ingredients'],
    ['ounce instant vanilla pudding mix', 'cups cold milk', 'inch angel food cake cubes', 'bananas', 'ounce frozen strawberries', 'ounce container topping', 'ingredients', 'ingredients'],
    ['cup bread crumbs', 'tablespoon purpose flour', 'salt pepper', 'tablespoons vegetable oil', 'pork steaks cutlets thin', 'egg', 'medium onion', 'ounce mushrooms', 'cups water', 'cube beef bouillon', 'tablespoon cornstarch', 'cup sour cream', 'ingredients', 'ingredients'],
    ['cup vegetable oil', 'cup purpose flour', 'small green bell pepper', 'medium onion', 'cloves garlic', 'stalks celery', 'fresh tomatoes', 'tablespoons Louisiana style hot sauce', 'teaspoon ground cayenne pepper optional', 'tablespoons seafood seasoning', 'teaspoon ground black pepper', 'cup fish stock', 'pound crawfish tails', 'pound medium shrimp deveined', 'ingredients', 'ingredients'],
    ['eggs', 'cup unsweetened applesauce', 'cup vegetable oil', 'cup brown sugar', 'teaspoon vanilla extract', 'bananas', 'cups whole wheat flour', 'teaspoon baking soda', 'tablespoon ground cinnamon', 'cup frozen strawberries', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cup brown sugar', 'tablespoon baking powder', 'teaspoon salt', 'cup butter', 'cup fresh blueberries', 'cup half cream', 'egg', 'ingredients', 'ingredients'],
    ['ounce jar salsa', 'ounce package spinach', 'cups Monterey Jack cheese', 'ounce package cream cheese diced', 'cup milk', 'ounce black olives', 'tablespoon red wine vinegar', 'salt pepper', 'ingredients', 'ingredients'],
    ['pound baby carrots', 'cup orange juice', 'tablespoons brown sugar', 'tablespoons butter', 'pinch salt', 'ingredients', 'ingredients'],
    ['ounce package angel hair pasta', 'tablespoons butter', 'tablespoons purpose flour', 'cups milk', 'cup heavy cream', 'tablespoons pesto', 'tablespoons fresh parsley', 'cloves garlic', 'tablespoons Parmesan cheese', 'teaspoons salt', 'teaspoon ground white pepper', 'Worcestershire sauce', 'hot sauce', 'ounce frozen broccoli florets', 'jumbo shrimp deveined', 'cloves garlic', 'ingredients', 'ingredients'],
    ['ounce package angel hair pasta', 'cup olive oil', 'cloves garlic', 'ounces sun tomatoes', 'ounce package tomato basil feta cheese', 'cup Parmesan cheese', 'bunch fresh cilantro', 'salt pepper', 'ingredients', 'ingredients'],
    ['butternut squash', 'cup white sugar', 'cups milk', 'teaspoon vanilla extract', 'pinch salt', 'tablespoons purpose flour', 'eggs', 'cup margarine', 'TOPPING', 'ounce package vanilla wafers', 'cup margarine', 'cup brown sugar', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'salt pepper', 'tablespoon olive oil', 'tablespoon butter', 'ounce artichoke hearts liquid', 'cup fresh mushrooms', 'cup white wine', 'tablespoon capers', 'ingredients', 'ingredients'],
    ['dry black beans', 'teaspoons jalapeno peppers', 'chicken broth', 'teaspoon garlic powder', 'tablespoon chili powder', 'teaspoon ground cumin', 'teaspoon cayenne pepper', 'teaspoon ground black pepper', 'teaspoon hot pepper sauce', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'cup diced onion', 'cups diced potatoes', 'chicken broth', 'cup heavy cream', 'cups sharp Cheddar cheese', 'teaspoon dill weed', 'teaspoon ground black pepper', 'teaspoon salt', 'teaspoon ground cayenne pepper', 'ingredients', 'ingredients'],
    ['ounce package spinach', 'bunch green onions white parts', 'eggs', 'ounce package cottage cheese', 'cups Cheddar cheese', 'cup croutons', 'ingredients', 'ingredients'],
    ['ounce uncooked tri-color spiral pasta', 'cup carrots', 'stalks celery', 'cup green bell pepper', 'cup cucumber', 'large tomatoes', 'cup onion', 'ounce bottles Italian style salad dressing', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['Cornbread Crust', 'cup butter', 'onion', 'clove garlic', 'ounce whole kernel corn', 'ounce cream style corn', 'teaspoon salt', 'cup egg substitute', 'ounce package corn bread mix', 'Chicken Topping', 'cups chicken breast', 'tablespoons green chile peppers', 'ounces fresh mushrooms', 'cups fat sour cream', 'teaspoon salt', 'teaspoon ground black pepper', 'ounce package Monterey Jack cheese', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'pound skinless boneless chicken breast halves inch cubes', 'onion', 'cloves garlic', 'ounce cans great Northern beans', 'ounce chicken broth', 'ounce cans green chiles', 'teaspoon salt', 'teaspoon ground cumin', 'teaspoon oregano', 'teaspoon ground black pepper', 'teaspoon cayenne pepper', 'cup sour cream', 'cup heavy whipping cream', 'ingredients', 'ingredients'],
    ['cups zucchini', 'cup white sugar', 'egg', 'cup vegetable oil', 'cups purpose flour', 'teaspoon salt', 'teaspoon baking soda', 'teaspoon baking powder', 'teaspoon ground cinnamon', 'teaspoons lemon zest', 'ingredients', 'ingredients'],
    ['cup walnuts', 'bunch spinach bite size pieces', 'cup cranberries', 'cup blue cheese', 'tomatoes', 'avocado diced', 'red onion', 'tablespoons red raspberry jam seeds', 'tablespoons red wine vinegar', 'cup walnut oil', 'black pepper', 'salt', 'ingredients', 'ingredients'],
    ['medium eggplants cubed', 'cup sharp Cheddar cheese', 'cup Italian seasoned bread crumbs', 'eggs', 'tablespoons parsley', 'tablespoons onion', 'clove garlic', 'cup vegetable oil frying', 'teaspoon salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['garlic', 'tablespoons olive oil', 'pound loaf Italian bread', 'cup butter', 'tablespoon fresh parsley optional', 'tablespoons Parmesan cheese optional', 'ingredients', 'ingredients'],
    ['cup brown rice', 'cup vegetable broth', 'tablespoon olive oil', 'cup diced onion', 'medium zucchini', 'skinless boneless chicken breast halves', 'cup mushrooms', 'teaspoon cumin', 'salt', 'ground cayenne pepper', 'ounce black beans', 'ounce green chile peppers', 'cup carrots', 'cups Swiss cheese', 'ingredients', 'ingredients'],
    ['ounce medium seashell pasta', 'pound sliced bacon', 'cups light Ranch style salad dressing', 'small onion', 'tomatoes', 'ingredients', 'ingredients'],
    ['bananas chunks', 'cups milk', 'cup peanut butter', 'tablespoons honey', 'cups ice cubes', 'ingredients', 'ingredients'],
    ['cup butter', 'cup white sugar', 'cup light brown sugar', 'eggs', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'cups chocolate chips', 'cups coconut', 'ingredients', 'ingredients'],
    ['fluid ounce bottle apple juice', 'fluid ounce frozen cranberry juice concentrate', 'cup orange juice', 'liters ginger ale', 'apple', 'ingredients', 'ingredients'],
    ['pound beef stew meat', 'ounce whole kernel corn undrained', 'ounce green beans', 'ounce carrots juice', 'ounce potatoes juice', 'ounce tomatoes', 'ounce package beef onion soup mix', 'salt pepper', 'ingredients', 'ingredients'],
    ['lasagna noodles', 'tablespoon olive oil', 'ounce package spinach', 'tablespoons green onion', 'pint part skim ricotta cheese', 'cup Parmesan cheese', 'teaspoon salt', 'egg', 'ounces Alfredo style pasta sauce', 'cup mozzarella cheese', 'ingredients', 'ingredients'],
    ['ounce package linguine', 'cup fresh frozen broccoli flowerets', 'tablespoons butter', 'pound skinless boneless chicken breast cubes', 'ounce Campbell Condensed Cream Mushroom Soup Regular % Fat Free Less Sodium', 'cup milk', 'cup Parmesan cheese', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['cup whole wheat flour', 'cup purpose flour', 'tablespoons brown sugar', 'tablespoons powder', 'teaspoon salt', 'cups quick cooking oats', 'soy milk', 'eggs', 'cup olive oil', 'cup frozen blueberries', 'ingredients', 'ingredients'],
    ['recipe pastry inch double crust pie', 'tablespoons butter', 'small onion', 'stalks celery', 'carrots', 'tablespoons parsley', 'teaspoon oregano', 'salt pepper', 'cubes chicken bouillon', 'cups water', 'potatoes cubed', 'cups turkey', 'tablespoons purpose flour', 'cup milk', 'ingredients', 'ingredients'],
    ['ounce container topping', 'ounce container sour cream', 'inch angel food cake', 'ounce instant vanilla pudding mix', 'kiwis', 'pint fresh strawberries', 'bananas', 'ounce pineapple', 'fresh mint', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'cloves garlic', 'pound ground beef', 'small head cabbage', 'red bell pepper strips', 'tablespoons soy sauce', 'teaspoon cornstarch', 'cup cold water', 'teaspoon ground black pepper', 'salt optional', 'ingredients', 'ingredients'],
    ['broccoli florets', 'tablespoon olive oil', 'skinless boneless chicken breast halves cut inch strips', 'cup green onions', 'cloves garlic', 'tablespoon hoisin sauce', 'tablespoon chile paste', 'tablespoon low sodium soy sauce', 'teaspoon ground ginger', 'teaspoon red pepper', 'teaspoon salt', 'teaspoon black pepper', 'cup chicken stock', 'ingredients', 'ingredients'],
    ['pounds skinless boneless chicken breast halves', 'tablespoons purpose flour', 'tablespoons olive oil', 'teaspoons butter', 'tablespoons shallots', 'cloves garlic', 'cup water', 'cup white wine', 'cube chicken bouillon', 'teaspoon rosemary', 'teaspoon salt', 'pinch ground black pepper', 'ingredients', 'ingredients'],
    ['cup butter', 'cup brown sugar', 'cup white sugar', 'eggs', 'teaspoons vanilla extract', 'cups purpose flour', 'teaspoon baking powder', 'teaspoon salt', 'cups yam', 'cup confectioners sugar', 'tablespoons butter margarine', 'tablespoons milk', 'ingredients', 'ingredients'],
    ['sweet potato cubed', 'medium eggplant', 'green bell pepper', 'red bell pepper', 'carrots', 'onion', 'tablespoons olive oil', 'cloves garlic', 'teaspoon ground turmeric', 'tablespoon curry powder', 'teaspoon ground cinnamon', 'tablespoon sea salt', 'teaspoon cayenne pepper', 'ounce beans', 'cup almonds', 'zucchini', 'tablespoons raisins', 'cup orange juice', 'ounces spinach', 'ingredients', 'ingredients'],
    ['jalapeno pepper seeded', 'clove garlic', 'teaspoon fresh ginger root', 'cup lime juice', 'cup honey', 'teaspoons balsamic vinegar', 'teaspoon salt', 'cup cilantro leaves', 'cup olive oil', 'ingredients', 'ingredients'],
    ['pints fresh strawberries', 'cup white sugar', 'cups purpose flour', 'teaspoons powder', 'tablespoons white sugar', 'teaspoon salt', 'cup shortening', 'egg', 'cup milk', 'cups heavy cream', 'ingredients', 'ingredients'],
    ['cup oil frying', 'cup purpose flour', 'green bell pepper', 'onion', 'cups chicken breast meat', 'ounce tomatoes green chile peppers liquid', 'ounce mushrooms', 'tablespoons fresh parsley', 'teaspoons Worcestershire sauce', 'cloves garlic', 'teaspoon soy sauce', 'teaspoon white sugar', 'teaspoon salt', 'teaspoon ground black pepper', 'hot sauce', 'ingredients', 'ingredients'],
    ['eggplant', 'cup lemon juice', 'cup tahini', 'tablespoons sesame seeds', 'cloves garlic', 'salt pepper', 'tablespoons olive oil', 'ingredients', 'ingredients'],
    ['beef roast', 'potatoes', 'cups baby carrots', 'yellow onion', 'celery', 'cubes beef bouillon', 'cup water', 'ingredients', 'ingredients'],
    ['ounce cans black beans', 'ounce tomatoes', 'cup butter', 'cups onion', 'cloves garlic', 'teaspoon salt', 'teaspoon ground black pepper', 'beef broth', 'ounce pumpkin puree', 'pound ham', 'tablespoons sherry vinegar', 'ingredients', 'ingredients'],
    ['lime', 'fluid ounce colored beer', 'teaspoon honey', 'cloves garlic', 'tablespoons fresh cilantro', 'Salt pepper', 'skinless boneless chicken breast halves', 'ingredients', 'ingredients'],
    ['ounce tomatoes', 'ounce tomato paste', 'carrot', 'onion', 'stalks celery', 'cup white wine', 'pinch red pepper flakes', 'cup green bell pepper', 'cup red bell pepper', 'cup steak sauce', 'bacon', 'pounds ground beef', 'ounce package chili seasoning mix', 'teaspoon ground cumin', 'ounce kidney beans', 'tablespoon fresh cilantro', 'tablespoon fresh parsley', 'ingredients', 'ingredients'],
    ['milliliter dry white wine', 'cup peach vodka', 'tablespoons lemonade concentrate', 'cup white sugar', 'white peaches', 'cup seedless red grapes', 'cup seedless green grapes', 'ingredients', 'ingredients'],
    ['medium head cabbage', 'large red onion', 'cup carrots', 'stalks celery', 'cup white sugar', 'cup white vinegar', 'cup vegetable oil', 'tablespoon salt', 'tablespoon dry mustard', 'black pepper', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'onion', 'pound lean ground beef', 'teaspoon basil', 'clove garlic', 'cup green beans', 'cup tomatoes', 'potatoes mashed', 'egg', 'cup water', 'cup Cheddar cheese optional', 'ingredients', 'ingredients'],
    ['chicken leg quarters', 'ounce bag small potatoes', 'cup olive oil', 'lemons', 'tablespoons basil', 'tablespoons oregano', 'tablespoon salt', 'tablespoon ground black pepper', 'tablespoons lemon herb seasoning', 'ounce fresh green beans', 'ingredients', 'ingredients'],
    ['bone pork chops', 'ground black pepper', 'tablespoon olive oil', 'ounce black beans liquid', 'cup salsa', 'tablespoon fresh cilantro', 'ingredients', 'ingredients'],
    ['cooking spray', 'cups purpose flour', 'teaspoon salt', 'teaspoon baking powder', 'teaspoon baking soda', 'cup butter', 'ripe bananas', 'cup white sugar', 'large eggs', 'teaspoon vanilla extract', 'tablespoon milk', 'cup walnuts', 'cup semisweet chocolate chips', 'ingredients', 'ingredients'],
    ['head cauliflower florets', 'tablespoon olive oil', 'clove garlic', 'cup Parmesan cheese', 'tablespoon fat cream cheese', 'teaspoon kosher salt', 'teaspoon black pepper', 'ingredients', 'ingredients'],
    ['tablespoons peanut oil', 'cloves garlic', 'teaspoon fresh ginger', 'bunch green onions', 'teaspoon salt', 'pound boneless skinless chicken breasts strips', 'onions', 'cup cabbage', 'red bell pepper', 'cups sugar snap peas', 'cup chicken broth', 'tablespoons soy sauce', 'tablespoons white sugar', 'tablespoons cornstarch', 'ingredients', 'ingredients'],
    ['cups uncooked jasmine rice', 'quart water', 'cup red curry paste', 'ounce cans coconut milk', 'skinless boneless chicken breast halves cut thin strips', 'tablespoons fish sauce', 'cup white sugar', 'cups bamboo shoots', 'red bell pepper', 'green bell pepper', 'small onion', 'cup pineapple chunks', 'ingredients', 'ingredients'],
    ['ounce skinless boneless chicken breast halves cut inch strips', 'ounce chicken broth', 'ounce frozen spinach', 'ounce cold cream cheese', 'cherry tomatoes', 'ounce dry penne pasta', 'ingredients', 'ingredients'],
    ['pound bacon', 'cup diced onion', 'cup green bell pepper', 'cups Cheddar cheese', 'eggs', 'cups milk', 'ounce package hash brown potatoes', 'ingredients', 'ingredients'],
    ['ounce refrigerated crescent rolls', 'cup sour cream', 'ounce package cream cheese', 'teaspoon dill weed', 'teaspoon garlic salt', 'ounce package ranch dressing mix', 'small onion', 'stalk celery', 'cup radishes', 'red bell pepper', 'cups fresh broccoli', 'carrot', 'ingredients', 'ingredients'],
    ['sweet potatoes large French fries', 'tablespoon water', 'teaspoons Italian seasoning', 'teaspoon lemon pepper', 'pinch salt pepper', 'tablespoons olive oil', 'ingredients', 'ingredients'],
    ['bunch beets', 'greens', 'cup olive oil', 'cloves garlic', 'tablespoons onion optional', 'salt pepper', 'tablespoon red wine vinegar optional', 'ingredients', 'ingredients'],
    ['ounce corn', 'ounce whole kernel corn', 'ounce container sour cream', 'eggs', 'cup butter', 'tablespoons white sugar', 'ounce package corn muffin mix', 'ingredients', 'ingredients'],
    ['cup dry potato flakes', 'cup water', 'egg', 'teaspoon salt', 'teaspoon ground black pepper', 'cups purpose flour', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'teaspoon paprika', 'sweet potatoes quarters', 'ingredients', 'ingredients'],
    ['cup butter margarine', 'cups white sugar', 'eggs', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoon baking powder', 'teaspoon salt', 'cups fresh peaches', 'ingredients', 'ingredients'],
    ['large cucumbers seeded inch slices', 'tablespoon salt', 'cup white sugar', 'cup rice wine vinegar', 'jalapeno peppers', 'cup cilantro', 'cup peanuts', 'ingredients', 'ingredients'],
    ['tablespoons raw horseradish', 'teaspoon dark brown sugar', 'teaspoon fresh lemon juice', 'tablespoons ketchup', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'stalks celery', 'onion', 'cloves garlic', 'teaspoons parsley', 'teaspoon Italian seasoning', 'teaspoon red pepper flakes', 'salt', 'ounce chicken broth', 'medium tomatoes', 'ounce tomato sauce', 'cup uncooked spinach pasta', 'ounce beans liquid', 'ingredients', 'ingredients'],
    ['chicken broth', 'cup fresh lemon juice', 'cup carrots', 'cup onion', 'cup celery', 'tablespoons chicken soup base', 'teaspoon ground white pepper', 'cup margarine', 'cup purpose flour', 'cup white rice', 'cup chicken meat', 'slices lemon', 'egg yolks', 'ingredients', 'ingredients'],
    ['fresh okra pods inch thick', 'tablespoon olive oil', 'teaspoons kosher salt', 'teaspoons black pepper', 'ingredients', 'ingredients'],
    ['teaspoons olive oil', 'shallots', 'large onion thin strips', 'pound bacon strips', 'clove garlic', 'ounce package pasta', 'egg yolks', 'cup heavy cream', 'cup Parmesan cheese', 'salt pepper', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'onion', 'cup balsamic vinegar', 'cups Arborio rice', 'cup dry white wine', 'cups hot chicken broth', 'tablespoons butter', 'cups chicken breast', 'salt pepper', 'tablespoons fresh thyme', 'ingredients', 'ingredients'],
    ['ounce butter beans', 'small onion', 'tablespoon jalapeno pepper', 'saltine crackers', 'egg', 'cup Cheddar cheese', 'teaspoon garlic powder', 'salt pepper', 'cup vegetable oil', 'ingredients', 'ingredients'],
    ['pound fresh asparagus inch pieces', 'slices bacon', 'inch unbaked pie shells', 'egg white', 'eggs', 'cups half cream', 'teaspoon ground nutmeg', 'salt pepper', 'cups Swiss cheese', 'ingredients', 'ingredients'],
    ['ounce instant vanilla pudding mix', 'cups milk', 'cup frozen orange juice concentrate', 'cup sour cream', 'ounce pineapple tidbits', 'ounce peaches', 'ounce mandarin orange segments', 'bananas', 'apple', 'ingredients', 'ingredients'],
    ['teaspoons olive oil', 'leeks bulb', 'cloves garlic', 'ounce cans fat free chicken broth', 'ounce cans beans', 'bay leaves', 'teaspoons ground cumin', 'cup whole wheat couscous', 'cups fresh spinach', 'salt pepper', 'ingredients', 'ingredients'],
    ['spaghetti squash seeded', 'pound ground beef', 'cup green bell pepper', 'cup red bell pepper', 'cup red onion', 'clove garlic', 'ounce Italian style diced tomatoes', 'teaspoon oregano', 'teaspoon basil', 'teaspoon salt', 'teaspoon ground black pepper', 'cups sharp Cheddar cheese', 'ingredients', 'ingredients'],
    ['buttermilk baking mix', 'cup milk', 'cup Cheddar cheese', 'cup butter', 'teaspoon garlic powder', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'teaspoon ground ginger', 'teaspoon ground cloves', 'teaspoon ground allspice', 'teaspoon salt', 'tablespoon baking powder', 'teaspoon baking soda', 'cup butter', 'cup white sugar', 'cup brown sugar', 'eggs room temperature', 'cup milk', 'cup pumpkin puree', 'Cinnamon Cream Cheese Frosting', 'ounce package cream cheese', 'cup butter', 'cups confectioners sugar', 'teaspoon vanilla extract', 'teaspoon ground cinnamon', 'ingredients', 'ingredients'],
    ['large eggplant', 'tablespoons vegetable oil', 'teaspoon cumin seeds', 'medium onion', 'tablespoon ginger garlic paste', 'tablespoon curry powder', 'tomato', 'cup plain yogurt', 'fresh jalapeno chile pepper', 'teaspoon salt', 'bunch cilantro', 'ingredients', 'ingredients'],
    ['ounce package sugar cookie dough', 'ounce package cream cheese', 'ounce container topping', 'cups fresh strawberries', 'cup white sugar', 'pinch salt', 'tablespoon cornstarch', 'cup orange juice', 'tablespoons lemon juice', 'cup water', 'teaspoon orange zest', 'ingredients', 'ingredients'],
    ['lemon', 'teaspoons oregano', 'cloves garlic', 'tablespoon olive oil', 'teaspoon salt', 'teaspoon ground black pepper', 'chicken legs', 'ingredients', 'ingredients'],
    ['tablespoon canola oil', 'pound chorizo sausage', 'pound ham', 'medium onion', 'cloves garlic', 'pound sweet potatoes diced', 'large red bell pepper', 'ounce cans tomatoes juice', 'small hot green chile pepper', 'cups water', 'ounce cans black beans', 'mango seeded diced', 'cup fresh cilantro', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['pound tomatillos', 'cup onion', 'teaspoon garlic', 'serrano chile peppers', 'tablespoons cilantro', 'tablespoon fresh oregano', 'teaspoon ground cumin', 'teaspoons salt', 'cups water', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'ounce dry onion soup mix', 'fluid ounce Russian style salad dressing', 'cup apricot preserves', 'ingredients', 'ingredients'],
    ['pound ground turkey', 'cup medium salsa', 'cup frozen corn kernels', 'cup water', 'ounce cans great Northern beans', 'ounces pepper jack cheese', 'ingredients', 'ingredients'],
    ['medium tomato', 'onion', 'fresh jalapeno pepper seeded', 'sprigs fresh cilantro', 'green onion', 'teaspoon garlic powder', 'teaspoon salt', 'teaspoon pepper', 'ingredients', 'ingredients'],
    ['tablespoon seafood seasoning such Old Bay', 'new potatoes', 'ounce packages kielbasa sausage inch pieces', 'ears fresh corn husks silks', 'whole crab pieces', 'fresh shrimp deveined', 'ingredients', 'ingredients'],
    ['cup white sugar', 'cup margarine', 'eggs', 'cup bananas', 'cup water', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'teaspoon baking powder', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'cloves garlic', 'onion', 'cups diced eggplant', 'cups zucchini', 'green bell pepper', 'ounce tomatoes', 'tablespoon basil', 'tablespoon parsley', 'teaspoon salt', 'teaspoon black pepper', 'ounce frozen cheese ravioli', 'cup mozzarella cheese', 'ingredients', 'ingredients'],
    ['pound loaf frozen bread dough', 'pound bulk Italian sausage', 'cups mozzarella cheese', 'ounces fresh mushrooms', 'small onion', 'teaspoons olive oil', 'ounce tomatoes', 'teaspoon oregano', 'teaspoon salt', 'teaspoon fennel seed', 'teaspoon garlic powder', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['cup orange juice', 'tablespoon soy sauce', 'ounce envelope dry onion soup mix', 'teaspoon garlic powder', 'chicken thighs', 'ingredients', 'ingredients'],
    ['cup mayonnaise', 'teaspoon Cajun seasoning', 'pounds ground beef sirloin', 'jalapeno pepper seeded', 'cup white onion', 'clove garlic', 'tablespoon Cajun seasoning', 'teaspoon Worcestershire sauce', 'pepperjack cheese', 'hamburger buns split', 'lettuce', 'tomato', 'ingredients', 'ingredients'],
    ['cup butter', 'cup sugar', 'eggs', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'teaspoon ground cinnamon', 'teaspoon ground cloves', 'apples', 'ingredients', 'ingredients'],
    ['tart apples', 'cup butter', 'cup purpose flour', 'cup white sugar', 'cup quick cooking oats', 'tablespoons cinnamon', 'cup butter pieces', 'ingredients', 'ingredients'],
    ['teaspoon olive oil', 'cup zucchini', 'cup onion', 'clove garlic', 'cups fresh tomatoes', 'tablespoons fresh basil', 'teaspoon salt', 'teaspoon ground black pepper', 'ounce halibut steaks', 'cup feta cheese', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'teaspoons powder', 'cup white sugar', 'cup milk', 'cup butter', 'cups fresh peaches', 'ingredients', 'ingredients'],
    ['fresh broccoli', 'cups American cheese', 'ounce cream mushroom soup', 'teaspoons salt', 'teaspoons black pepper', 'tablespoons butter', 'cups seasoned croutons', 'ingredients', 'ingredients'],
    ['apples', 'cup maple syrup', 'cup purpose flour', 'cup oats', 'cup brown sugar', 'pinch salt', 'cup butter', 'ingredients', 'ingredients'],
    ['cup orange juice', 'tablespoon orange zest', 'cup soy sauce', 'teaspoon salt', 'cloves garlic', 'tablespoon brown sugar', 'tablespoons vegetable oil', 'skinless boneless chicken breast halves cut inch cubes', 'tablespoons purpose flour', 'cup bean sprouts optional', 'ounce crispy chow mein noodles', 'ingredients', 'ingredients'],
    ['cup butter', 'tablespoons garlic powder', 'tablespoon parsley', 'pound loaf Italian bread inch slices', 'ounce package mozzarella cheese', 'ingredients', 'ingredients'],
    ['cups heavy cream', 'cube chicken bouillon', 'tablespoon Asiago Cheese', 'tablespoon cornstarch equal parts water', 'cup sun tomatoes', 'ounce package bow tie pasta', 'cup bacon', 'cup butter', 'cup red onion', 'cloves garlic', 'cup green onion', 'skinless boneless chicken breast diced', 'cup heavy cream', 'tablespoons fresh parsley', 'ingredients', 'ingredients'],
    ['pound loaf cinnamon raisin bread', 'ounce package cream cheese', 'cup apples', 'eggs', 'cups half cream', 'tablespoons butter', 'cup maple syrup', 'ingredients', 'ingredients'],
    ['pound potatoes', 'tablespoon canola oil', 'pound ground beef', 'large onion', 'red bell pepper', 'cloves garlic', 'ounce broth', 'tablespoons ketchup', 'tablespoon soy sauce', 'tablespoon Worcestershire sauce', 'teaspoon mild curry powder', 'tablespoon cornstarch', 'tomatoes', 'ounce package mixed peas carrots', 'cup milk', 'tablespoon butter', 'cup extra-sharp Cheddar cheese', 'salt pepper', 'ingredients', 'ingredients'],
    ['ounce package spinach', 'potatoes', 'cup butter', 'cup sour cream', 'tablespoon onion', 'teaspoon salt', 'teaspoon dill weed', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['cups water', 'cup uncooked rice', 'ounce package turkey kielbasa inch slices', 'onion', 'green bell pepper', 'garlic', 'ounce cans kidney beans', 'ounce tomatoes', 'teaspoon oregano', 'salt', 'teaspoon pepper', 'ingredients', 'ingredients'],
    ['bacon', 'skinless boneless chicken breast halves inch thickness', 'ounce package spinach dry', 'cup blue cheese', 'tablespoons purpose flour', 'teaspoon ground black pepper', 'teaspoon salt', 'tablespoons olive oil', 'ingredients', 'ingredients'],
    ['ounce chickpeas garbanzo beans', 'tablespoons olive oil', 'salt optional', 'garlic salt optional', 'cayenne pepper optional', 'ingredients', 'ingredients'],
    ['medium russet potatoes', 'tablespoons butter margarine', 'salt pepper', 'ingredients', 'ingredients'],
    ['butternut squash seeded', 'acorn squash seeded', 'tablespoons butter', 'cup sweet onion', 'quart chicken broth', 'cup brown sugar', 'ounce package cream cheese', 'teaspoon ground black pepper', 'ground cinnamon optional', 'fresh parsley garnish', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'cup onions', 'stalks celery', 'pounds ground pork', 'pounds ground beef', 'pound whole cooked chicken', 'ounce cans whole tomatoes liquid', 'cup ketchup', 'cup hickory barbeque sauce', 'salt pepper', 'hot sauce optional', 'green bell pepper', 'ounce cans cream style corn', 'ingredients', 'ingredients'],
    ['sweet potatoes', 'cup brown sugar', 'cup low fat margarine', 'tablespoons orange juice', 'ground cinnamon', 'ounce miniature marshmallows', 'ingredients', 'ingredients'],
    ['ounce package cream cheese', 'ounce package sharp Cheddar cheese', 'cup mayonnaise', 'fresh jalapeno peppers seeded', 'eggs', 'tablespoon milk', 'cups corn flake cereal', 'ingredients', 'ingredients'],
    ['cups water', 'cup uncooked quinoa', 'cup red bell pepper', 'cup yellow bell pepper', 'small red onion', 'teaspoons curry powder', 'cup fresh cilantro', 'lime', 'cup almonds', 'cup carrots', 'cup cranberries', 'salt black pepper', 'ingredients', 'ingredients'],
    ['ounce package angel food cake mix', 'ounce pineapple juice', 'ounce container topping', 'ingredients', 'ingredients'],
    ['ounce package cheese tortellini', 'ounce package spinach', 'cup Parmesan cheese', 'cherry tomatoes', 'ounce black olives', 'ounce Italian style salad dressing', 'ingredients', 'ingredients'],
    ['pound lean ground beef', 'salt pepper', 'ounce cans dark red kidney beans', 'ounce cans Mexican style stewed tomatoes', 'stalks celery', 'red bell pepper', 'cup red wine vinegar', 'tablespoons chili powder', 'teaspoon ground cumin', 'teaspoon parsley', 'teaspoon basil', 'Worcestershire sauce', 'cup red wine', 'ingredients', 'ingredients'],
    ['onion', 'cup vegetable oil', 'pound potatoes cubed', 'teaspoon salt', 'teaspoon cayenne pepper', 'teaspoon ground turmeric', 'teaspoon ground cumin', 'tomatoes', 'ingredients', 'ingredients'],
    ['pound tilapia fillets chunks', 'cup fresh lime juice', 'cup fresh lime juice', 'tablespoons honey', 'tablespoon vegetable oil', 'teaspoon ground cumin', 'cup mayonnaise', 'chipotle chilies adobo sauce', 'tablespoon adobo sauce chipotle peppers', 'teaspoon salt', 'teaspoon cayenne pepper', 'cup purpose flour', 'eggs', 'panko crumbs', 'salt black pepper', 'cup vegetable oil frying', 'cups color coleslaw blend', 'cup fresh cilantro leaves', 'inch flour tortillas', 'ingredients', 'ingredients'],
    ['cups fresh blueberries', 'cup purpose flour', 'cup white sugar', 'teaspoon ground cinnamon', 'cup mayonnaise', 'ingredients', 'ingredients'],
    ['cups baby spinach leaves', 'cup fresh basil leaves', 'cup pine nuts', 'cup Parmesan cheese', 'cloves garlic', 'teaspoon kosher salt', 'teaspoon black pepper', 'tablespoon fresh lemon juice', 'teaspoon lemon zest', 'cup olive oil', 'ingredients', 'ingredients'],
    ['ounce yellow cake mix', 'ripe bananas', 'teaspoon baking soda', 'ingredients', 'ingredients'],
    ['ounce cans garbanzo beans', 'cucumbers', 'cherry tomatoes', 'red onion', 'cloves garlic', 'ounce black olives', 'ounce feta cheese', 'cup Italian style salad dressing', 'lemon', 'teaspoon garlic salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['bacon', 'eggs', 'head iceberg lettuce', 'cups chicken meat', 'tomatoes', 'cup blue cheese', 'avocado diced', 'green onions', 'ounce bottle Ranch style salad dressing', 'ingredients', 'ingredients'],
    ['ounce black beans', 'ounce cans whole kernel corn', 'green onions', 'jalapeno peppers', 'green bell pepper', 'avocado diced', 'ounce jar pimentos', 'tomatoes', 'cup fresh cilantro', 'lime', 'cup Italian salad dressing', 'teaspoon garlic salt', 'ingredients', 'ingredients'],
    ['ounce cans whole green beans', 'pound bacon half', 'cup butter', 'cup brown sugar', 'teaspoon garlic salt', 'ingredients', 'ingredients'],
    ['chicken breast halves boneless skin', 'salt black pepper', 'tablespoons olive oil', 'ounces fresh mushrooms inch thick', 'pinch salt', 'cup water', 'tablespoon butter', 'salt black pepper', 'ingredients', 'ingredients'],
    ['cup almonds', 'cloves garlic', 'teaspoon honey', 'teaspoon Dijon mustard', 'cup raspberry vinegar', 'tablespoons balsamic vinegar', 'tablespoons brown sugar', 'cup vegetable oil', 'head romaine lettuce', 'pint fresh strawberries', 'cup feta cheese', 'ingredients', 'ingredients'],
    ['ham bone meat', 'onion', 'ounce tomatoes juice', 'ounce kidney beans', 'potatoes', 'green bell pepper seeded cubed', 'cups water', 'cubes chicken bouillon', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'fresh mushrooms', 'tablespoons butter', 'tablespoons green onions', 'teaspoon lemon juice', 'cup crabmeat', 'cup soft bread crumbs', 'egg', 'teaspoon dill weed', 'cup Monterey Jack cheese', 'cup dry white wine', 'ingredients', 'ingredients'],
    ['ounce active dry yeast', 'cup warm water', 'white sugar', 'teaspoons kosher salt', 'tablespoon olive oil', 'cups purpose flour', 'cloves garlic', 'tablespoon fresh basil', 'cup olive oil', 'teaspoon garlic', 'cup tomato sauce', 'cup tomatoes', 'cup black olives', 'cup red peppers', 'cups mozzarella cheese', 'tablespoons fresh basil', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'onions', 'cloves garlic', 'boneless chicken breast half', 'ounce cans chicken broth', 'ounce cans green chile peppers', 'teaspoons ground cumin', 'teaspoons oregano', 'teaspoons cayenne pepper', 'ounce cans great Northern beans undrained', 'cup Monterey Jack cheese', 'ingredients', 'ingredients'],
    ['cup mayonnaise', 'cup white sugar', 'teaspoons white vinegar', 'teaspoon dill weed', 'teaspoon seasoned salt', 'medium cucumbers', 'ingredients', 'ingredients'],
    ['ounce uncooked egg noodles', 'tablespoons vegetable oil', 'cloves garlic', 'cups zucchini', 'cup milk', 'ounces cream cheese', 'cup fresh basil', 'salt pepper', 'grated Parmesan cheese optional', 'ingredients', 'ingredients'],
    ['large head cauliflower', 'cup seasoned bread crumbs', 'tablespoons Parmesan cheese', 'cup margarine', 'teaspoon garlic powder', 'teaspoon salt', 'red pepper flakes', 'oregano', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'skinless boneless chicken breast halves chunks', 'onion', 'cloves garlic', 'small butternut squash', 'ounce beans', 'carrot', 'ounce tomatoes juice', 'ounce vegetable broth', 'tablespoon sugar', 'tablespoon lemon juice', 'teaspoon salt', 'teaspoon ground coriander', 'cayenne pepper', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'boneless skinless chicken thighs', 'cup honey', 'cup lite soy sauce', 'tablespoons ketchup', 'cloves garlic', 'tablespoon fresh ginger root', 'ounce pineapple tidbits juice', 'tablespoons cornstarch', 'cup water', 'ingredients', 'ingredients'],
    ['ear corn', 'ingredients', 'ingredients'],
    ['cup milk', 'cups white sugar', 'cup pumpkin', 'teaspoon ground cinnamon', 'cup white chocolate chips', 'ounces marshmallow creme', 'tablespoons butter room temperature', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['cups water', 'cup uncooked white rice', 'pound medium shrimp deveined', 'teaspoon ground ginger', 'teaspoon cayenne pepper', 'clove garlic', 'tablespoon sesame seeds', 'teaspoon ground black pepper', 'tablespoons sesame oil', 'red bell pepper thin strips', 'green onions', 'tablespoons teriyaki sauce', 'pound sugar snap peas', 'cup cornstarch', 'cup chicken broth', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['medium red potatoes peel', 'medium onion', 'tablespoon garlic powder', 'tablespoon kosher salt', 'teaspoons chili powder', 'cup extra virgin olive oil', 'cup Cheddar cheese optional', 'ingredients', 'ingredients'],
    ['mango', 'cup red bell pepper', 'green onion', 'tablespoons cilantro', 'fresh jalapeno chile pepper', 'tablespoons lime juice', 'tablespoon lemon juice', 'ingredients', 'ingredients'],
    ['ounce cream cheese', 'cup mayonnaise', 'ounce green chiles', 'cup Parmesan cheese', 'ounce jar artichoke hearts', 'cup jalapeno peppers', 'ounce box frozen spinach', 'ingredients', 'ingredients'],
    ['eggs', 'cup white sugar', 'cup pumpkin', 'cup purpose flour', 'teaspoon ground cinnamon', 'teaspoon baking soda', 'cup walnuts', 'confectioners sugar', 'cup confectioners sugar', 'teaspoon vanilla extract', 'tablespoons butter', 'ounces cream cheese', 'ingredients', 'ingredients'],
    ['cup dry white wine', 'tablespoons cider vinegar', 'tablespoons shallots', 'fresh thyme', 'bay leaf', 'cup heavy cream', 'tablespoons unsalted butter tablespoon size pieces', 'tablespoons fresh chives', 'teaspoons fresh lemon juice', 'salt pepper', 'ounce fillets halibut', 'tablespoons vegetable oil', 'tablespoon unsalted butter', 'cup fresh bread crumbs', 'cup almonds', 'tablespoon unsalted butter', 'egg', 'ingredients', 'ingredients'],
    ['tablespoons margarine', 'cup onion', 'cup celery', 'cup carrots', 'ounce beans', 'cup cabbage', 'ounce tomatoes', 'tablespoon tomato paste', 'cups potatoes', 'quart chicken broth', 'cloves garlic', 'tablespoons parsley', 'teaspoon salt', 'cup elbow macaroni', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['pound bottom round roast', 'ground black pepper', 'garlic powder', 'tablespoon vegetable oil', 'ounce cans cream mushroom soup', 'ounce dry onion soup mix', 'carrots inch pieces', 'small new potatoes', 'ingredients', 'ingredients'],
    ['fresh peaches', 'teaspoon ground cinnamon', 'cups white sugar', 'cup shortening', 'cup white sugar', 'cups purpose flour', 'teaspoons powder', 'teaspoon salt', 'cup milk', 'cups water', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['thick slices whole wheat bread', 'tablespoons cream cheese', 'cucumber', 'tablespoons alfalfa sprouts', 'teaspoon olive oil', 'teaspoon red wine vinegar', 'tomato', 'leaf lettuce', 'ounce pepperoncini', 'avocado', 'ingredients', 'ingredients'],
    ['cups apples', 'cup white sugar', 'teaspoon ground cinnamon', 'cup pecans', 'cup purpose flour', 'cup white sugar', 'teaspoon baking powder', 'teaspoon salt', 'egg', 'cup milk', 'cup butter', 'cup pecans', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'onion', 'cloves garlic', 'ounce black beans', 'green bell pepper', 'tomatoes', 'ounce frozen corn', 'inch flour tortillas', 'cup Cheddar cheese', 'cup vegetable oil', 'ingredients', 'ingredients'],
    ['pounds ground beef', 'onion', 'garlic powder', 'salt pepper', 'ounce tomatoes', 'ounce package taco seasoning mix', 'cup hot water', 'ounce whole kernel corn', 'ounce package corn muffin mix', 'cup Cheddar cheese optional', 'ounce black olives optional', 'ingredients', 'ingredients'],
    ['tablespoon canola oil', 'teaspoons sesame oil', 'pound fresh green beans', 'tablespoon soy sauce', 'tablespoon sesame seeds', 'ingredients', 'ingredients'],
    ['fresh broccoli bite size pieces', 'cup raisins', 'cup red onion', 'tablespoons white sugar', 'tablespoons white wine vinegar', 'cup mayonnaise', 'cup sunflower seeds', 'slices bacon', 'ingredients', 'ingredients'],
    ['Pastry Double Crust Pie', 'cups purpose flour', 'teaspoon salt', 'cup shortening', 'tablespoons cold water', 'Berry Filling', 'cup fresh strawberries', 'cups fresh raspberries', 'cups fresh blueberries', 'cup white sugar', 'tablespoons cornstarch', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cup white sugar', 'teaspoons powder', 'teaspoon baking soda', 'teaspoon salt', 'cup butter', 'apple', 'cup milk', 'tablespoons milk', 'tablespoons white sugar', 'teaspoon ground cinnamon', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'bone chicken breast halves skin', 'teaspoon seasoning salt', 'clove garlic', 'thick slices bacon', 'cup mushrooms', 'cup heavy cream', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cup brown sugar', 'cup white sugar', 'teaspoon ground cinnamon', 'teaspoon ground ginger', 'teaspoon ground nutmeg', 'teaspoon baking soda', 'teaspoon baking powder', 'teaspoon salt', 'eggs', 'cup butter', 'cup pumpkin', 'ounce package butterscotch chips', 'ingredients', 'ingredients'],
    ['cup uncooked quinoa', 'cups water', 'tablespoon vegetable oil', 'onion', 'cloves garlic', 'tablespoon chili powder', 'tablespoon ground cumin', 'ounce tomatoes', 'ounce cans black beans', 'green bell pepper', 'red bell pepper', 'zucchini', 'jalapeno pepper seeded', 'tablespoon chipotle peppers adobo sauce', 'teaspoon oregano', 'salt black pepper', 'cup frozen corn', 'cup fresh cilantro', 'ingredients', 'ingredients'],
    ['cup walnut halves', 'cup sugar', 'cups greens', 'cup cranberries', 'cup Gorgonzola cheese', 'tablespoon raspberry vinaigrette', 'tablespoon white vinegar', 'tablespoon olive oil', 'ingredients', 'ingredients'],
    ['ground beef', 'onion', 'cloves garlic', 'ounce tomatoes', 'ounce cans Italian style diced tomatoes', 'ounce tomato sauce', 'cup water', 'ounce kidney beans', 'ounce pinto beans', 'tablespoons chili powder', 'tablespoon ground cumin', 'tablespoons white sugar', 'tablespoon salt', 'teaspoon ground black pepper', 'tablespoon hot pepper sauce', 'ingredients', 'ingredients'],
    ['cup brown sugar', 'teaspoon ground cinnamon', 'cup butter', 'apples', 'cup raisins', 'pound loaf French baguette inch slices', 'eggs', 'cups milk', 'tablespoon vanilla extract', 'teaspoons ground cinnamon', 'ingredients', 'ingredients'],
    ['pound ground beef', 'medium onion', 'ounce Campbell Condensed Cream Mushroom Soup Regular % Fat Free', 'tablespoon ketchup', 'tablespoon Worcestershire sauce', 'cups Tater Tots', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'cup shortening', 'cups white sugar', 'eggs', 'teaspoon vanilla extract', 'cup buttermilk', 'cup ripe bananas', 'apples', 'ingredients', 'ingredients'],
    ['bacon', 'crimini mushrooms', 'tablespoon butter', 'tablespoon onion', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['medium potatoes', 'pound sliced bacon', 'pound American cheese', 'onion', 'cup mayonnaise', 'salt pepper', 'cup black olives', 'ingredients', 'ingredients'],
    ['cup margarine', 'onion', 'stalks celery', 'tablespoons purpose flour', 'cups water', 'ounce chicken broth', 'tablespoons chicken bouillon powder', 'potato diced', 'pound fresh asparagus', 'cup half', 'tablespoon soy sauce', 'teaspoon ground black pepper', 'teaspoon ground white pepper', 'ingredients', 'ingredients'],
    ['cup graham cracker crumbs', 'cup ground pecans', 'tablespoons white sugar', 'tablespoons brown sugar', 'cup butter', 'cup white sugar', 'cup pumpkin', 'egg yolks', 'teaspoons ground cinnamon', 'teaspoon ground mace', 'teaspoon ground ginger', 'teaspoon salt', 'ounce cream cheese', 'cup white sugar', 'egg', 'egg yolk', 'tablespoons cream', 'tablespoon cornstarch', 'teaspoon vanilla extract', 'teaspoon lemon extract', 'ingredients', 'ingredients'],
    ['cup butter', 'cups white sugar', 'eggs', 'cups purpose flour', 'teaspoons powder', 'teaspoon salt', 'cup milk', 'cups fresh blueberries', 'cup white sugar', 'ingredients', 'ingredients'],
    ['portobello mushrooms', 'cup canola oil', 'tablespoons onion', 'cloves garlic', 'tablespoons balsamic vinegar', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'salt pepper', 'teaspoons olive oil', 'ounce tomatoes green peppers onions', 'cup sun tomato pesto', 'ounce hearts water', 'ingredients', 'ingredients'],
    ['cups raisins', 'cups purpose flour', 'cups white sugar', 'teaspoons powder', 'teaspoons soda', 'teaspoons salt', 'teaspoons nutmeg', 'teaspoons cinnamon', 'teaspoons cloves', 'eggs', 'ounce pumpkin', 'cup unsweetened applesauce', 'cup walnuts', 'ingredients', 'ingredients'],
    ['cup brown sugar', 'teaspoon ground cinnamon', 'cup white sugar', 'cup butter', 'eggs', 'teaspoons vanilla extract', 'cups purpose flour', 'teaspoons powder', 'cup milk', 'apple', 'ingredients', 'ingredients'],
    ['ounce cream mushroom soup', 'ounces cheese Velveeta', 'tablespoon real bacon bits', 'ounce stems pieces', 'ounce cans green beans', 'ounce French onions', 'ingredients', 'ingredients'],
    ['pound spicy Italian sausage', 'cup olive oil', 'cloves garlic', 'ounce cavatelli pasta', 'ounce frozen broccoli', 'teaspoon red pepper flakes', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['cups frozen green peas', 'cup water', 'teaspoon salt', 'tablespoons butter', 'cup heavy cream', 'tablespoons purpose flour', 'tablespoon white sugar', 'ingredients', 'ingredients'],
    ['Yukon Gold potatoes', 'ounce cream cheese', 'ounces sour cream', 'cup milk', 'teaspoons onion salt', 'ground black pepper', 'ingredients', 'ingredients'],
    ['thick slices bacon', 'teaspoons olive oil', 'cup onion', 'cup carrots', 'stalk celery', 'cups low fat sodium chicken broth', 'cups potatoes', 'teaspoon cayenne pepper', 'cup Cheddar cheese', 'teaspoon kosher salt', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves cut thin strips', 'teaspoons white sugar', 'tablespoons rice wine vinegar', 'cup soy sauce', 'cups chicken broth', 'cup water', 'tablespoon sesame oil', 'teaspoon ground black pepper', 'tablespoons cornstarch', 'ounce uncooked linguine pasta', 'tablespoons vegetable oil', 'tablespoons fresh ginger root', 'tablespoon garlic', 'pound fresh shiitake mushrooms', 'green onions inch pieces', 'ingredients', 'ingredients'],
    ['cups tomatoes', 'cup green bell pepper', 'cup onion', 'cup fresh cilantro', 'tablespoons fresh lime juice', 'teaspoons fresh jalapeno pepper seeds', 'teaspoon ground cumin', 'teaspoon kosher salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['ripe bananas', 'eggs', 'cup vegetable oil', 'cups pumpkin puree', 'cup honey', 'cup white sugar', 'cups purpose flour', 'teaspoon baking powder', 'teaspoon baking soda', 'teaspoon salt', 'teaspoons pumpkin pie spice', 'teaspoon ground cinnamon', 'cup raisins optional', 'cup walnut pieces optional', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves thin', 'ounce Italian style salad dressing', 'slices stale wheat bread', 'cup Parmesan cheese', 'teaspoon fresh thyme', 'teaspoon pepper', 'cups feta cheese', 'cup sour cream', 'tablespoon vegetable oil', 'cloves garlic', 'cups fresh spinach', 'bunch green onions', 'cup mushrooms', 'ounce jar oil sun tomatoes', 'ingredients', 'ingredients'],
    ['cup oats', 'cup whole wheat flour', 'cup purpose flour', 'cup brown sugar', 'tablespoons dry milk powder', 'teaspoons powder', 'teaspoon baking soda', 'teaspoon salt', 'egg', 'cups milk', 'tablespoons vegetable oil', 'teaspoon vanilla extract', 'banana', 'ingredients', 'ingredients'],
    ['recipe pastry inch double crust pie', 'cup white sugar', 'tablespoons purpose flour', 'teaspoon salt', 'teaspoon ground cinnamon', 'teaspoon lemon zest', 'cups pears', 'tablespoon butter', 'tablespoon lemon juice', 'ingredients', 'ingredients'],
    ['ounce sweet potatoes undrained', 'cup white sugar', 'eggs', 'cup butter', 'cup milk', 'teaspoon vanilla extract', 'cup brown sugar', 'cup pecans', 'cup purpose flour', 'cup butter', 'ingredients', 'ingredients'],
    ['ounce cans cream chicken soup', 'cups sour cream', 'tablespoons butter', 'tablespoons onion flakes', 'ground black pepper', 'pound package hash brown potatoes', 'ounces extra sharp Cheddar cheese', 'cup cornflakes cereal', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'tablespoon thyme', 'tablespoons basil', 'tablespoon parsley', 'teaspoon paprika', 'teaspoon salt', 'teaspoon ground black pepper', 'teaspoon garlic powder', 'boneless skinless chicken breast halves', 'cup butter', 'ounce cream mushroom soup', 'ounce chicken broth', 'cup dry white wine', 'lemon', 'tablespoon fresh parsley', 'tablespoons capers', 'tablespoon lemon zest', 'ingredients', 'ingredients'],
    ['cup butter', 'cup white sugar', 'teaspoon vanilla extract', 'egg', 'cup solid pack pumpkin puree', 'cups purpose flour', 'teaspoons powder', 'teaspoon baking soda', 'teaspoon salt', 'cup fresh cranberries', 'teaspoon ground cinnamon', 'tablespoon orange zest', 'cup walnuts', 'ingredients', 'ingredients'],
    ['roma plum tomatoes', 'red onion', 'tablespoons fresh cilantro', 'jalapeno pepper seeded', 'lime', 'clove garlic', 'garlic powder', 'pinch ground cumin', 'salt black pepper', 'ingredients', 'ingredients'],
    ['teaspoon onion', 'tablespoons brown sugar', 'tablespoons soy sauce', 'tablespoons red wine vinegar', 'teaspoons fresh thyme', 'teaspoon sesame oil', 'cloves garlic', 'teaspoon ground allspice', 'habanero pepper', 'skinless boneless chicken breast halves cut inch strips', 'ingredients', 'ingredients'],
    ['ounce package lasagna noodles', 'cups heavy cream', 'ounce cans cream mushroom soup', 'cup Parmesan cheese', 'cup butter', 'tablespoon olive oil', 'large onion', 'cloves garlic', 'mushrooms', 'roasted chicken', 'salt black pepper', 'cup ricotta cheese', 'bunch fresh spinach', 'cups mozzarella cheese', 'ingredients', 'ingredients'],
    ['tablespoons margarine', 'onion', 'small head cabbage', 'cup water', 'large potatoes', 'seasoning salt', 'dash paprika', 'cups ham', 'ingredients', 'ingredients'],
    ['cup mayonnaise', 'tablespoon red wine vinegar', 'teaspoon salt', 'teaspoon ground black pepper', 'clove garlic', 'tablespoons fresh basil', 'cups rotini pasta', 'cup tomatoes', 'cup yellow bell pepper', 'cup black olives', 'ingredients', 'ingredients'],
    ['tablespoons unsalted butter', 'tablespoon olive oil', 'skinless boneless chicken breast halves thin', 'salt pepper', 'shallots', 'clove garlic', 'cup dry white wine', 'cup chicken broth', 'ingredients', 'ingredients'],
    ['ounce medium seashell pasta', 'ounce package spinach', 'eggs', 'cup olive oil', 'cup bread crumbs', 'ounce jars tomato basil pasta sauce', 'ounce package Cheddar cheese', 'ounce package mozzarella cheese', 'ingredients', 'ingredients'],
    ['cup butter', 'cups purpose flour', 'cups white sugar', 'eggs', 'cup milk', 'teaspoons powder', 'teaspoon salt', 'cups fresh blueberries', 'ingredients', 'ingredients'],
    ['ounce package Hillshire Farm Sausage', 'medium onion', 'cup butter margarine', 'cup purpose flour', 'teaspoon salt', 'teaspoon ground black pepper', 'ounce whole kernel corn', 'cups milk', 'cups potatoes', 'parsley optional', 'Shredded Cheddar cheese optional', 'ingredients', 'ingredients'],
    ['ounce chicken broth', 'ounce tomato vegetable juice cocktail', 'cup water', 'large potato', 'carrots', 'stalks celery', 'ounce tomatoes', 'cup fresh green beans', 'cup fresh corn kernels', 'salt pepper', 'Creole seasoning', 'ingredients', 'ingredients'],
    ['pounds ground beef', 'cup onion', 'cup green chile pepper', 'ounce tomato sauce', 'teaspoons chili powder', 'teaspoon garlic salt', 'ounce buttermilk biscuit dough', 'cups Monterey Jack cheese', 'cup sour cream', 'egg', 'ingredients', 'ingredients'],
    ['ounce yellow corn', 'ounce white corn', 'ounce cans black beans', 'ounce Italian style diced tomatoes', 'bunch cilantro', 'green onions', 'small red onion', 'red bell pepper seeded', 'tablespoon garlic', 'cup lime juice', 'avocado diced', 'tablespoons olive oil', 'ingredients', 'ingredients'],
    ['ounce package cream cheese', 'cup mayonnaise', 'ounce dry Italian salad dressing mix', 'loaves French bread inch slices', 'medium cucumbers', 'dill weed', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves bite size pieces', 'tablespoon chili powder', 'salt pepper', 'tablespoon olive oil', 'cup green bell pepper', 'cup onion', 'jalapeno peppers', 'large tomato chunks', 'hot pepper sauce', 'ingredients', 'ingredients'],
    ['ounce cream cheese', 'cup white sugar', 'teaspoon vanilla extract', 'eggs', 'cup pumpkin puree', 'teaspoon ground cinnamon', 'pinch ground cloves', 'pinch ground nutmeg', 'inch graham cracker crust', 'cup topping', 'ingredients', 'ingredients'],
    ['large sweet potatoes', 'cup butter', 'cups white sugar', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'tablespoon vanilla extract', 'salt', 'ingredients', 'ingredients'],
    ['ounce cream mushroom soup', 'cup mayonnaise', 'egg', 'cup onions', 'ounce packages broccoli', 'ounces sharp Cheddar cheese', 'salt pepper', 'dash paprika', 'ingredients', 'ingredients'],
    ['ounce pineapple chunks juice', 'apples', 'ounce peach pie filling', 'bananas diced', 'kiwis', 'pint strawberries', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'white onions rings', 'cloves garlic', 'cup lentils', 'cups water', 'ounce frozen spinach', 'teaspoon salt', 'teaspoon ground cumin', 'black pepper', 'cloves garlic', 'ingredients', 'ingredients'],
    ['teaspoons', 'teaspoon brown sugar', 'teaspoon black pepper', 'teaspoon onion powder', 'teaspoon thyme', 'teaspoon rosemary', 'teaspoon garlic powder', 'teaspoon cayenne pepper', 'large sweet potatoes', 'teaspoons olive oil', 'ingredients', 'ingredients'],
    ['Salad', 'cup quinoa', 'cups water', 'bunch kale bite sized pieces', 'avocado diced', 'cup cucumber', 'cup red bell pepper', 'tablespoons red onion', 'tablespoon feta cheese', 'Dressing', 'cup olive oil', 'tablespoons lemon juice', 'tablespoons Dijon mustard', 'teaspoon sea salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['cup butter', 'cups white sugar', 'egg', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'cup buttermilk', 'cups rhubarb', 'tablespoon purpose flour', 'cup butter', 'teaspoons ground cinnamon', 'cup brown sugar', 'ingredients', 'ingredients'],
    ['ounce cans tomato sauce', 'ounce cans tomatoes', 'cups onion', 'tablespoon Italian seasoning', 'pound bacon', 'spicy sausage', 'pounds ground beef', 'ounce bottle hickory smoke barbeque sauce', 'cup chili powder', 'ounce cans kidney beans undrained', 'ounce squares unsweetened chocolate', 'ingredients', 'ingredients'],
    ['ounce package egg noodles', 'tablespoons butter', 'ounce mushrooms', 'teaspoon salt', 'teaspoon pepper', 'cups turkey', 'ounce cream celery soup', 'cup sour cream', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'onions', 'cloves garlic', 'pound ground beef', 'pound spicy Italian sausage', 'ounce tomatoes juice', 'fluid ounce dark beer', 'cup strong coffee', 'ounce cans tomato paste', 'ounce broth', 'cup chili powder', 'tablespoon ground cumin', 'cup brown sugar', 'teaspoon oregano', 'teaspoon cayenne pepper', 'teaspoon ground coriander', 'teaspoon salt', 'tablespoon wasabi paste', 'ounce cans kidney beans', 'Anaheim chile peppers', 'serrano pepper', 'habanero pepper', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'onion', 'bay leaf', 'cloves garlic', 'cups split peas', 'cup barley', 'teaspoons salt', 'cups water', 'carrots', 'stalks celery', 'potatoes', 'cup parsley', 'teaspoon basil', 'teaspoon thyme', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'large onions', 'teaspoon garlic', 'carrots', 'stalks celery', 'cups tomatoes', 'cups', 'teaspoon salt', 'teaspoon ground black pepper', 'cup white wine', 'bay leaves', 'cups chicken stock', 'fresh parsley', 'teaspoon paprika', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'cup white sugar', 'cup light brown sugar', 'cup butter', 'eggs', 'tablespoon vanilla extract', 'cup vanilla baking chips', 'cup macadamia nuts', 'cup cranberries', 'ingredients', 'ingredients'],
    ['cups brown sugar', 'cup vegetable oil', 'cups solid pack pumpkin puree', 'eggs', 'cups purpose flour', 'teaspoon salt', 'teaspoon baking powder', 'teaspoon baking soda', 'teaspoon vanilla extract', 'tablespoons ground cinnamon', 'tablespoon ground ginger', 'tablespoon ground cloves', 'egg white', 'tablespoons milk', 'teaspoon vanilla extract', 'cups confectioners sugar', 'cup shortening', 'ingredients', 'ingredients'],
    ['ounce package farfalle pasta', 'cup butter', 'cloves garlic', 'ounce cream chicken soup', 'cup milk', 'tablespoon parsley', 'teaspoon salt', 'teaspoon ground black pepper', 'skinless boneless chicken breast halves bite size pieces', 'cup sun tomatoes water', 'tablespoons Romano cheese', 'ingredients', 'ingredients'],
    ['cup nonfat milk', 'cup fat free plain yogurt', 'frozen banana', 'tablespoons powdered protein supplement', 'tablespoons flax seed', 'teaspoon honey', 'cup frozen strawberries', 'ingredients', 'ingredients'],
    ['boneless skinless chicken breast halves', 'ounce package farfalle bow tie pasta', 'ounce chicken broth', 'head broccoli florets', 'medium red bell pepper', 'cloves garlic', 'salt pepper', 'ounce containers onion cream cheese', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['cups fresh frozen blueberries', 'teaspoon vanilla extract', 'lemon', 'cup white sugar', 'teaspoon purpose flour', 'tablespoon butter', 'cups purpose flour', 'teaspoons powder', 'tablespoons white sugar', 'tablespoons butter', 'cup milk', 'teaspoons sugar', 'pinch ground cinnamon', 'ingredients', 'ingredients'],
    ['small red potatoes', 'tablespoons butter', 'cloves garlic', 'tablespoon fresh rosemary', 'sea salt black pepper', 'cup fresh spinach leaves', 'ingredients', 'ingredients'],
    ['cup raisins', 'cups whole wheat flour', 'cup brown sugar', 'teaspoon pumpkin pie spice', 'teaspoon baking powder', 'teaspoon baking soda', 'teaspoon salt', 'eggs', 'cup pumpkin puree', 'cup vegetable oil', 'cup honey', 'cup walnuts', 'ingredients', 'ingredients'],
    ['pound bacon', 'onions', 'cups fresh mushrooms', 'tablespoon butter', 'cups frozen hash brown potatoes', 'teaspoon salt', 'teaspoon garlic salt', 'teaspoon ground black pepper', 'eggs', 'cups milk', 'parsley', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'teaspoon garlic salt', 'teaspoon black pepper', 'pork tenderloin inch pieces', 'tablespoons olive oil', 'cup balsamic vinegar', 'cup chicken broth', 'teaspoons lemon zest', 'tablespoon capers', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'onion', 'cloves garlic', 'sprigs fresh thyme', 'butternut squash inch cubes', 'cups chicken stock', 'cube chicken bouillon', 'pinch ground cumin', 'pinch ground allspice', 'salt black pepper', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoons soda', 'teaspoons salt', 'teaspoon baking powder', 'teaspoon ground nutmeg', 'teaspoon ground allspice', 'teaspoon ground cinnamon', 'teaspoon ground cloves', 'cups white sugar', 'cup canola oil', 'eggs', 'cups solid pack pumpkin puree', 'cup water', 'ingredients', 'ingredients'],
    ['ears fresh corn', 'tablespoons butter', 'small sweet onion', 'salt pepper', 'ingredients', 'ingredients'],
    ['ounce package orange cake mix', 'ounce instant vanilla pudding mix', 'eggs', 'cup vegetable oil', 'cup water', 'cup lemon juice', 'cup tequila', 'tablespoons triple sec liqueur', 'cup confectioners sugar', 'tablespoon tequila', 'tablespoons triple sec liqueur', 'tablespoons lime juice', 'ingredients', 'ingredients'],
    ['limes', 'cup sugar', 'tablespoons condensed milk', 'cups water', 'ice', 'ingredients', 'ingredients'],
    ['cup light sour cream', 'ounce green salsa', 'bunch fresh spinach', 'cups Monterey Jack cheese', 'ounce package corn tortillas', 'ingredients', 'ingredients'],
    ['russet potatoes inch strips', 'cooking spray', 'teaspoon basil', 'cup Parmesan cheese', 'salt pepper', 'ingredients', 'ingredients'],
    ['ounce jar artichoke hearts', 'cup Romano cheese', 'cup red onion', 'tablespoons mayonnaise', 'French baguette inch thick slices', 'ingredients', 'ingredients'],
    ['ounce yellow cake mix', 'ounce instant lemon pudding mix', 'cups water', 'egg whites', 'cup nonfat milk', 'teaspoon lemon extract', 'ounce instant sugar free vanilla pudding mix', 'ounce container frozen light topping', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'boneless pork chops', 'ounce cream mushroom soup', 'cup milk', 'potatoes', 'cup onion', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['ounce water chestnuts', 'pound bacon half', 'cup mayonnaise', 'cup light brown sugar', 'cup tomato chili sauce', 'ingredients', 'ingredients'],
    ['pounds pork tenderloin inch strips', 'small onion', 'small red bell pepper', 'ounce cans black beans', 'ounce jar salsa', 'cup chicken broth', 'teaspoon oregano', 'teaspoon ground cumin', 'teaspoons chili powder', 'ingredients', 'ingredients'],
    ['pounds lean beef', 'ounce tomatoes', 'ounce tomato sauce', 'cubes beef bouillon', 'medium carrots', 'onion', 'tablespoons white vinegar', 'cup white sugar', 'teaspoons salt', 'teaspoon ground black pepper', 'quarts water', 'cloves garlic', 'head cabbage wedges', 'ingredients', 'ingredients'],
    ['pound whole turkey neck giblets', 'cup butter', 'apples', 'tablespoon garlic powder', 'salt pepper', 'milliliter bottle Champagne', 'ingredients', 'ingredients'],
    ['cups white sugar', 'teaspoon baking soda', 'teaspoon salt', 'cups purpose flour', 'eggs', 'cup sour cream', 'cups rhubarb', 'cup white sugar', 'cup butter', 'cup purpose flour', 'ground cinnamon', 'ingredients', 'ingredients'],
    ['ounces dry fettuccine pasta', 'ounce package cream cheese', 'tablespoons butter', 'cup milk', 'teaspoon garlic powder', 'salt pepper', 'skinless boneless chicken breast halves cubed', 'cups fresh broccoli', 'small zucchini', 'cup red bell pepper', 'ingredients', 'ingredients'],
    ['ounce package angel hair pasta', 'cup butter', 'cloves garlic', 'pounds shrimp deveined', 'cup white wine', 'cup dry bread crumbs', 'tablespoons Parmesan cheese', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'cloves garlic', 'pounds beef sirloin', 'ounce mushrooms liquid', 'ounce tomato sauce', 'salt', 'pepper', 'cup red wine', 'ingredients', 'ingredients'],
    ['pound dry great Northern beans', 'pound ham diced', 'small onion', 'cup brown sugar', 'salt pepper', 'teaspoon cayenne pepper', 'tablespoon parsley', 'ingredients', 'ingredients'],
    ['ounces pasta', 'cup olive oil', 'cloves garlic', 'cups tomatoes sieve', 'anchovy filets', 'tablespoons tomato paste', 'tablespoons capers', 'Greek olives', 'teaspoon red pepper flakes', 'ingredients', 'ingredients'],
    ['tilapia fillets', 'tablespoons olive oil', 'tablespoon Creole seasoning', 'tablespoons butter', 'cloves garlic', 'cup Alfredo sauce', 'ingredients', 'ingredients'],
    ['ounce package rice noodles', 'teaspoon vegetable oil', 'onion', 'cloves garlic', 'cups chicken breast meat', 'small head cabbage', 'carrot', 'cup soy sauce', 'lemons wedges garnish', 'ingredients', 'ingredients'],
    ['eggs', 'cup butter', 'cup light brown sugar', 'ounce pineapple', 'maraschino cherries', 'cup cake flour', 'teaspoon baking powder', 'teaspoon salt', 'cup white sugar', 'tablespoon butter', 'teaspoon almond extract', 'ingredients', 'ingredients'],
    ['ounce yellow cake mix', 'ounce instant banana pudding mix', 'eggs', 'cup water', 'cup vegetable oil', 'cup bananas', 'cups confectioners sugar', 'tablespoons milk', 'dash vanilla extract', 'cup walnuts optional', 'ingredients', 'ingredients'],
    ['cups sweet potatoes', 'cup white sugar', 'eggs', 'cup milk', 'teaspoon salt', 'cup butter', 'teaspoon vanilla extract', 'cup brown sugar', 'cup purpose flour', 'cup butter', 'cup pecans', 'ingredients', 'ingredients'],
    ['lemon', 'sprigs fresh rosemary', 'salmon fillets bones skin', 'coarse salt', 'tablespoon olive oil', 'ingredients', 'ingredients'],
    ['cup brown sugar', 'cup butter', 'tablespoons corn syrup', 'cup pecans', 'inch thick French bread', 'teaspoon orange zest', 'cup fresh orange juice', 'cup % milk', 'tablespoons white sugar', 'teaspoon ground cinnamon', 'teaspoon vanilla extract', 'egg whites', 'eggs', 'tablespoon confectioners sugar', 'ingredients', 'ingredients'],
    ['cup water', 'cup cauliflower', 'cup potatoes', 'cup celery', 'cup carrots', 'cup onion', 'cup butter', 'cup purpose flour', 'cups milk', 'salt pepper', 'ounces Cheddar cheese', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'tablespoon olive oil', 'tablespoon balsamic vinegar', 'clove garlic', 'teaspoon oregano', 'pound button mushrooms', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'tablespoons butter', 'pound button mushrooms', 'clove garlic', 'tablespoon red cooking wine', 'tablespoon teriyaki sauce more', 'teaspoon garlic salt', 'black pepper', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'small onion', 'clove garlic', 'cup quinoa', 'teaspoons curry powder', 'teaspoon salt', 'teaspoon black pepper', 'teaspoon cumin', 'teaspoon cinnamon', 'cups chicken stock', 'ounce beans', 'cup pine nuts', 'cup raisins hot water optional', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'cloves garlic', 'onion', 'ounce box frozen spinach', 'ounce beans', 'teaspoon cumin', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['medium head garlic', 'tablespoon olive oil', 'russet potatoes', 'tablespoons butter', 'cup milk', 'salt pepper', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoon salt', 'teaspoon baking soda', 'cups white sugar', 'teaspoon ground cinnamon', 'cup walnuts', 'eggs', 'cup vegetable oil', 'cups ripe banana', 'ounce pineapple', 'teaspoons vanilla extract', 'cup coconut', 'cup maraschino cherries', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'teaspoon garlic powder', 'teaspoon salt', 'teaspoon black pepper', 'teaspoon Italian seasoning', 'skinless boneless chicken breast halves inch thick thirds', 'tablespoons olive oil', 'clove garlic', 'onion', 'cup white wine', 'ounce chicken broth', 'tablespoons lemon juice', 'ounce hearts liquid', 'cup capers liquid', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cup shortening', 'cup cold water', 'pinch salt', 'cups sour cherries', 'cups white sugar', 'teaspoons cornstarch', 'tablespoon butter', 'teaspoon almond extract', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breasts', 'teaspoon paprika', 'salt pepper', 'garlic powder', 'cup butter', 'onion thin rings', 'pound fresh mushrooms', 'ingredients', 'ingredients'],
    ['cups uncooked jasmine rice', 'quart water', 'cup coconut milk', 'tablespoons soy sauce', 'tablespoons rice wine vinegar', 'tablespoons fish sauce', 'teaspoon red pepper flakes', 'tablespoon olive oil', 'medium onion', 'tablespoons fresh ginger root', 'cloves garlic', 'pounds skinless boneless chicken breast halves cut inch strips', 'shiitake mushrooms', 'green onions', 'cups fresh basil leaves', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'cloves garlic', 'cups tomatoes', 'cup dry white wine', 'tablespoons butter', 'salt black pepper', 'ounce package linguine pasta', 'pound deveined medium shrimp', 'teaspoon Cajun seasoning', 'tablespoons olive oil', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'teaspoon baking powder', 'teaspoon salt', 'cup butter', 'cups white sugar', 'teaspoons vanilla extract', 'eggs', 'cup cocoa powder', 'cup semi-sweet chocolate chips', 'cup pumpkin puree', 'cup walnuts', 'teaspoon ground cinnamon', 'teaspoon ground cloves', 'teaspoon ground nutmeg', 'ingredients', 'ingredients'],
    ['cup peach preserves', 'tablespoons Worcestershire sauce', 'teaspoon chile paste', 'boneless pork chops', 'teaspoon ground ginger', 'pinch ground cinnamon', 'salt pepper', 'tablespoons vegetable oil', 'cup white wine', 'ingredients', 'ingredients'],
    ['cup rice', 'tablespoons olive oil', 'cup carrots', 'cup celery', 'bell peppers color seeds half', 'pound ground beef', 'pound pancetta bacon', 'cups marinara sauce', 'cup red wine', 'teaspoon red pepper flakes', 'cup heavy cream', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoons salt', 'tablespoons butter', 'cups sour cream', 'eggs', 'egg yolk', 'tablespoons vegetable oil', 'baking potatoes cubed', 'cup Cheddar cheese', 'tablespoons cheese sauce', 'onion salt optional', 'salt pepper', 'ingredients', 'ingredients'],
    ['banana chunks', 'cup grapes', 'ounce tub vanilla yogurt', 'apple', 'cups fresh spinach leaves', 'ingredients', 'ingredients'],
    ['ounce package linguine pasta', 'tablespoons olive oil', 'small onion', 'cloves garlic', 'cup butter', 'tablespoons purpose flour', 'cups milk', 'pinch salt', 'pinch pepper', 'cups Romano cheese', 'cup basil pesto', 'pound shrimp deveined', 'mushrooms', 'roma plum tomato', 'ingredients', 'ingredients'],
    ['pound Italian sausage inch pieces', 'green bell peppers', 'ounces farfalle pasta', 'cup beef broth', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['ounce hearts', 'ounce package spinach', 'cup mayonnaise', 'cup Parmesan cheese', 'cups Monterey Jack cheese', 'ingredients', 'ingredients'],
    ['pound beef stew meat', 'tablespoons purpose flour', 'tablespoons', 'teaspoon salt', 'teaspoon ground black pepper', 'cups water', 'onion', 'teaspoon paprika', 'clove garlic', 'teaspoon Worcestershire sauce', 'bay leaf', 'carrots', 'potatoes', 'cup purpose flour', 'cup frozen corn kernels', 'ingredients', 'ingredients'],
    ['cups quick cooking oats', 'cup purpose flour', 'cup white sugar', 'tablespoon baking powder', 'teaspoon salt', 'cup milk', 'egg', 'cup vegetable oil', 'cup blueberries', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cup white sugar', 'teaspoons powder', 'teaspoon salt', 'cup apple juice', 'cup vegetable oil', 'egg', 'teaspoon ground cinnamon', 'cup apples', 'ingredients', 'ingredients'],
    ['cup mayonnaise', 'cloves garlic', 'tablespoons lemon juice', 'teaspoon salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['cup butter', 'lemon', 'teaspoon salt', 'teaspoon ground black pepper', 'teaspoon parsley', 'cloves garlic', 'ounce fillets cod', 'tablespoons lemon pepper', 'ingredients', 'ingredients'],
    ['cup butter', 'egg', 'cup milk', 'cup tart apple', 'cups purpose flour', 'teaspoons powder', 'teaspoon ground cinnamon', 'tablespoon white sugar', 'ingredients', 'ingredients'],
    ['carrots', 'cup butter', 'cup white sugar', 'tablespoons purpose flour', 'teaspoon baking powder', 'teaspoon vanilla extract', 'eggs', 'teaspoon confectioners sugar', 'ingredients', 'ingredients'],
    ['pound ground beef', 'ounce package taco seasoning mix', 'cup water', 'teaspoons chili powder', 'cup rice', 'teaspoon salt', 'teaspoon garlic salt', 'teaspoon ground black pepper', 'ounce cans tomato sauce', 'large red bell peppers', 'inch cubes Colby Jack cheese', 'ingredients', 'ingredients'],
    ['cup unsalted butter', 'pound feta cheese', 'ounce package cream cheese', 'cloves garlic', 'shallot', 'tablespoons dry vermouth', 'white pepper', 'cup pine nuts', 'cup sun tomatoes', 'cup pesto sauce', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'tablespoons butter', 'leeks inch pieces', 'large head cauliflower', 'cloves garlic', 'vegetable broth', 'salt black pepper', 'cup heavy cream optional', 'ingredients', 'ingredients'],
    ['cup quinoa', 'teaspoon salt', 'cups water', 'cup garbanzo beans chickpeas', 'tomato', 'clove garlic', 'tablespoons lime juice', 'teaspoons olive oil', 'teaspoon ground cumin', 'pinch salt pepper', 'teaspoon fresh parsley', 'ingredients', 'ingredients'],
    ['large zucchini', 'tablespoon salt', 'pound ground beef', 'teaspoons black pepper', 'small green bell pepper', 'onion', 'cup tomato paste', 'ounce tomato sauce', 'cup red wine', 'tablespoons fresh basil', 'tablespoon fresh oregano', 'hot water', 'egg', 'ounce container low fat ricotta cheese', 'tablespoons fresh parsley', 'ounce package spinach', 'pound fresh mushrooms', 'ounces mozzarella cheese', 'ounces Parmesan cheese', 'ingredients', 'ingredients'],
    ['potatoes', 'bacon', 'tablespoon purpose flour', 'tablespoons white sugar', 'cup water', 'cup white wine vinegar', 'cup green onions', 'salt pepper', 'ingredients', 'ingredients'],
    ['ounce yellow cake mix', 'cups quick cooking oats', 'cup margarine', 'cup raspberry jam', 'tablespoon water', 'ingredients', 'ingredients'],
    ['cups white sugar', 'cup butter', 'bananas', 'eggs', 'cups purpose flour', 'teaspoon baking soda', 'cup sour milk', 'teaspoon salt', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['beets inch stems', 'cup shallot', 'tablespoons fresh parsley', 'tablespoons extra-virgin olive oil', 'tablespoon balsamic vinegar', 'tablespoon red wine vinegar', 'salt pepper', 'cup feta cheese', 'ingredients', 'ingredients'],
    ['pound sugar snap peas', 'tablespoon olive oil', 'tablespoon shallots', 'teaspoon fresh thyme', 'kosher salt', 'ingredients', 'ingredients'],
    ['sweet potatoes', 'cup fat margarine', 'cup brown sugar', 'tablespoons orange juice', 'pinch ground cinnamon', 'ounce miniature marshmallows', 'ingredients', 'ingredients'],
    ['ounce cream mushroom soup', 'teaspoon minced onion', 'teaspoon parsley', 'cup white wine', 'teaspoon garlic powder', 'tablespoon milk', 'ounce pieces', 'salt pepper', 'boneless skinless chicken breast halves', 'ingredients', 'ingredients'],
    ['ounce frozen potato rounds', 'ounce container sour cream', 'cup Cheddar cheese', 'ounce cream mushroom soup', 'ounce French onions', 'ingredients', 'ingredients'],
    ['pounds shrimp deveined', 'tablespoon garlic', 'cup red onion', 'cup fresh cilantro', 'cups tomato clam juice cocktail', 'cup ketchup', 'cup fresh lime juice', 'teaspoon hot pepper sauce', 'cup prepared horseradish', 'salt', 'ripe avocado', 'ingredients', 'ingredients'],
    ['apples', 'cup water', 'cup brown sugar', 'teaspoon pumpkin pie spice', 'ingredients', 'ingredients'],
    ['cup butter', 'cup purpose flour', 'cup white sugar', 'teaspoon baking powder', 'cup milk', 'cups sour cherries', 'cup white sugar', 'tablespoon purpose flour', 'ingredients', 'ingredients'],
    ['tablespoon peanut oil', 'small head broccoli', 'small red bell pepper', 'fresh mushrooms', 'pound firm tofu', 'cup peanut butter', 'cup hot water', 'tablespoons vinegar', 'tablespoons soy sauce', 'tablespoons molasses', 'ground cayenne pepper', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'large leeks white pale green parts', 'large onion', 'large potato cubed', 'cups butternut squash', 'cup carrots', 'Granny Smith apple inch thick', 'quart chicken stock', 'cup dry white wine optional', 'cup light cream', 'teaspoon ground nutmeg', 'salt pepper', 'tablespoons chives', 'ingredients', 'ingredients'],
    ['inch pie crust baked', 'cups white sugar', 'teaspoon salt', 'cups water', 'cup cornstarch', 'cup water', 'eggs', 'cup lemon juice', 'teaspoons lemon zest', 'tablespoons butter', 'teaspoon salt', 'cup white sugar', 'ingredients', 'ingredients'],
    ['cup butter', 'cup white sugar', 'teaspoon salt', 'teaspoon vanilla extract', 'egg yolks', 'cups purpose flour', 'teaspoon baking powder', 'cup milk', 'egg whites', 'cup white sugar', 'cups fresh blueberries', 'tablespoon purpose flour', 'tablespoon white sugar', 'ingredients', 'ingredients'],
    ['cups uncooked elbow macaroni', 'ounce cans chunk chicken', 'cups Cheddar cheese', 'cups milk', 'ounce cans cream chicken soup', 'ounce mushrooms', 'cup onion', 'ingredients', 'ingredients'],
    ['ounce dry onion soup mix', 'sweet potatoes diced', 'cup vegetable oil', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'onion', 'cloves garlic', 'bell pepper', 'ounce cans black beans mashed', 'tablespoons yellow cornmeal', 'tablespoons cumin', 'teaspoon paprika', 'teaspoon cayenne pepper', 'teaspoon chili powder', 'cup salsa', 'ingredients', 'ingredients'],
    ['ounce bags clean fresh spinach', 'cups heavy cream', 'cup butter', 'tablespoons garlic', 'tablespoons white onion', 'slices provolone cheese', 'cup Parmesan cheese', 'salt pepper', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'large onion', 'cloves garlic', 'ounce packages frozen broccoli', 'potato', 'chicken broth', 'teaspoon ground nutmeg', 'salt pepper', 'ingredients', 'ingredients'],
    ['pound fresh green beans', 'tablespoons butter', 'cup almonds', 'teaspoons lemon pepper', 'ingredients', 'ingredients'],
    ['pound asparagus', 'teaspoon Cajun seasoning', 'ingredients', 'ingredients'],
    ['large skinless boneless chicken breast halves', 'salt black pepper', 'asparagus spears', 'cup mozzarella cheese', 'cup Italian seasoned bread crumbs', 'ingredients', 'ingredients'],
    ['pounds sweet Italian sausage', 'cups inch pieces celery', 'zucchini inch slices', 'ounce cans tomatoes', 'green bell peppers inch slices', 'cup onion', 'teaspoons salt', 'teaspoon white sugar', 'teaspoon oregano', 'teaspoon Italian seasoning', 'teaspoon basil', 'teaspoon garlic powder', 'tablespoons Parmesan cheese', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breasts', 'teaspoon olive oil', 'teaspoon onion powder', 'pinch salt', 'pinch ground black pepper', 'avocados', 'ripe tomatoes', 'ounce package Monterey Jack cheese slices', 'ingredients', 'ingredients'],
    ['cup red bell pepper', 'clove garlic', 'cup olive oil', 'teaspoon onion powder', 'teaspoon salt', 'teaspoon ground black pepper', 'portobello mushroom caps', 'ingredients', 'ingredients'],
    ['tablespoon peanut oil', 'small onion', 'tablespoon fresh ginger root', 'clove garlic', 'fenugreek seeds', 'cup dry red lentils', 'cup butternut squash cubed', 'cup fresh cilantro', 'cups water', 'ounce coconut milk', 'tablespoons tomato paste', 'teaspoon curry powder', 'cayenne pepper', 'pinch ground nutmeg', 'salt pepper', 'ingredients', 'ingredients'],
    ['pound bacon', 'sweet onion', 'cups hash brown potatoes', 'eggs', 'cups Cheddar cheese', 'cups small curd cottage cheese', 'cups Swiss cheese', 'ingredients', 'ingredients'],
    ['grape cherry tomatoes', 'fresh basil leaves', 'small balls fresh mozzarella cheese bocconcini', 'salt pepper', 'cup balsamic vinegar', 'cup extra virgin olive oil', 'toothpicks', 'ingredients', 'ingredients'],
    ['quart vegetable oil deep frying', 'cups purpose flour', 'tablespoon white sugar', 'teaspoons powder', 'teaspoon salt', 'cup milk', 'eggs', 'tablespoon vegetable oil', 'cups apples', 'cup cinnamon sugar', 'ingredients', 'ingredients'],
    ['tablespoon sesame oil', 'onion', 'pounds cubed chicken meat', 'tablespoons soy sauce', 'large carrots', 'stalks celery', 'large red bell pepper', 'cup fresh pea pods', 'large green bell pepper', 'cups white rice', 'eggs', 'cup soy sauce', 'ingredients', 'ingredients'],
    ['ounce package cream cheese', 'ounce container nonfat sour cream', 'ounce package taco seasoning mix', 'head iceberg lettuce', 'cup Cheddar cheese', 'tomatoes', 'green bell pepper', 'ounce black olives', 'ingredients', 'ingredients'],
    ['cups sweet potatoes', 'pound butter', 'eggs', 'cup brown sugar', 'teaspoon salt', 'teaspoon ground ginger', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'cup milk', 'cup white sugar', 'inch unbaked pie crust', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'cup brown sugar', 'fresh peaches', 'teaspoon ground ginger', 'teaspoon ground cloves', 'tablespoons fresh lemon juice', 'ingredients', 'ingredients'],
    ['ounce white cake mix', 'cups strawberries', 'ounces cream cheese', 'ounce squares white chocolate', 'cup heavy whipping cream', 'ounce container topping', 'pint fresh strawberries', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'small onions', 'clove garlic', 'teaspoon fresh parsley', 'teaspoon celery', 'salt pepper', 'cups fresh mushrooms', 'cup whole milk', 'cup heavy cream', 'cup rice', 'cups vegetable stock', 'teaspoon butter', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['ounce yellow cake mix', 'egg', 'cup butter', 'ounce pumpkin puree', 'eggs', 'cup white sugar', 'cup brown sugar', 'teaspoons ground cinnamon', 'cup white sugar', 'tablespoons butter', 'cup nuts optional', 'ingredients', 'ingredients'],
    ['cup milk', 'tablespoon lemon juice', 'teaspoon vanilla extract', 'cups brown sugar', 'cup vegetable oil', 'egg', 'cups purpose flour', 'teaspoon salt', 'teaspoon baking soda', 'cups rhubarb', 'cup walnuts', 'cup brown sugar', 'teaspoon ground cinnamon', 'tablespoon butter', 'ingredients', 'ingredients'],
    ['cup butter', 'cup white sugar', 'eggs', 'pinch ground cinnamon optional', 'pinch ground nutmeg optional', 'white bread', 'ounce pineapple juice', 'ingredients', 'ingredients'],
    ['ounce jar red peppers diced', 'pound Monterey Jack cheese', 'ounce package cream cheese', 'cup mayonnaise', 'tablespoon onion', 'clove garlic', 'tablespoons Dijon style mustard', 'ingredients', 'ingredients'],
    ['Yukon Gold potatoes', 'cup butter', 'cups Parmesan cheese', 'cup fresh chives', 'cups cream cheese', 'medium head garlic', 'pinch salt pepper', 'ingredients', 'ingredients'],
    ['cup buttermilk', 'teaspoon baking soda', 'cup butter', 'cup shortening', 'cups white sugar', 'eggs', 'teaspoon vanilla extract', 'cup coconut', 'teaspoon baking powder', 'cups purpose flour', 'ounces cream cheese', 'cup butter', 'teaspoon vanilla extract', 'cups confectioners sugar', 'tablespoons light cream', 'cup walnuts', 'cup coconut', 'ingredients', 'ingredients'],
    ['ounce package baby carrots', 'cup butter', 'tablespoons honey', 'cup brown sugar', 'ingredients', 'ingredients'],
    ['pound sweet potatoes', 'cups purpose flour', 'teaspoons powder', 'teaspoon salt', 'teaspoon ground nutmeg', 'eggs', 'cups milk', 'cup butter', 'ingredients', 'ingredients'],
    ['eggs', 'ounce cream style corn', 'ounce container sour cream', 'cup butter', 'ounce whole kernel corn', 'cups Cheddar cheese', 'cup onion', 'ounce green chilies', 'ounce dry corn muffin mix', 'ingredients', 'ingredients'],
    ['cup dry bread crumbs', 'tablespoons Parmesan cheese', 'pork chops', 'tablespoon vegetable oil', 'ounce cream mushroom soup', 'cup milk', 'ingredients', 'ingredients'],
    ['ounce tomato sauce', 'cup water', 'envelope taco seasoning mix', 'tablespoons chili powder', 'tablespoon vegetable oil', 'pound chicken breast tenderloins', 'ounce black beans', 'cup cream cheese', 'cup Mexican style cheese blend more', 'ounce package corn bread mix', 'egg', 'cup milk', 'ingredients', 'ingredients'],
    ['recipe pastry inch double crust deep dish pie', 'cup brown sugar', 'cup butter', 'cup purpose flour', 'cups apples', 'cup white sugar', 'tablespoons purpose flour', 'teaspoons ground cinnamon', 'teaspoon lemon juice', 'caramels', 'tablespoons milk', 'ingredients', 'ingredients'],
    ['kielbasa sausage inch pieces', 'cups ketchup', 'cups brown sugar', 'ounce pineapple chunks undrained', 'ingredients', 'ingredients'],
    ['ounce package pork sausage', 'medium beets', 'carrots', 'medium baking potatoes cubed', 'tablespoon vegetable oil', 'medium onion', 'ounce tomato paste', 'cup water', 'medium head cabbage', 'ounce tomatoes', 'cloves garlic', 'salt pepper', 'teaspoon white sugar', 'cup sour cream', 'tablespoon fresh parsley garnish', 'ingredients', 'ingredients'],
    ['pounds red potatoes chunks', 'tablespoons extra virgin olive oil', 'cloves garlic', 'teaspoons rosemary', 'teaspoons thyme', 'teaspoons kosher salt', 'bunch fresh asparagus inch pieces', 'ground black pepper', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'beef chuck roast inch thick strips', 'salt pepper', 'tablespoon butter', 'medium onion sliced diced', 'ounces mushrooms', 'cloves garlic', 'tablespoons purpose flour', 'cup white wine', 'beef broth', 'cup creme fraiche', 'tablespoon fresh chives', 'salt pepper', 'ingredients', 'ingredients'],
    ['tablespoons lime juice', 'tablespoon ground cumin', 'tablespoon paprika', 'teaspoons garlic', 'teaspoon salt', 'teaspoon ground black pepper', 'pounds tilapia fillets chunks', 'tablespoons olive oil', 'onions', 'large bell peppers', 'ounce tomatoes', 'ounce coconut milk', 'bunch fresh cilantro optional', 'ingredients', 'ingredients'],
    ['ounce jumbo pasta shells', 'ounces mushrooms', 'cloves garlic', 'onion', 'pound ground turkey', 'salt pepper', 'ounce package spinach', 'tablespoons fresh parsley', 'cups cottage cheese', 'cup Parmesan cheese', 'ounce jar spaghetti sauce', 'ounce package mozzarella cheese', 'ingredients', 'ingredients'],
    ['teaspoon ground cumin', 'teaspoon ground cayenne pepper', 'teaspoon ground turmeric', 'teaspoon ground coriander', 'skinless boneless chicken breast halves', 'salt pepper', 'tablespoons olive oil', 'onion', 'tablespoon fresh ginger', 'jalapeno peppers', 'cloves garlic', 'tomatoes', 'ounce coconut milk', 'bunch fresh parsley', 'ingredients', 'ingredients'],
    ['butternut squash seeded cubed', 'cup mayonnaise', 'cup onion', 'egg', 'teaspoon brown sugar', 'salt pepper', 'cup saltine crackers', 'tablespoons Parmesan cheese', 'tablespoon butter', 'ingredients', 'ingredients'],
    ['cups fresh blueberries', 'cup purpose flour', 'cups purpose flour', 'teaspoons powder', 'teaspoon salt', 'cup margarine', 'cups white sugar', 'eggs', 'teaspoon vanilla extract', 'cup milk', 'ingredients', 'ingredients'],
    ['cup olive oil', 'tablespoons garlic', 'cups chicken stock', 'turkey drumstick', 'bunches collard greens', 'salt black pepper', 'tablespoon red pepper flakes optional', 'ingredients', 'ingredients'],
    ['teaspoons olive oil', 'tablespoons white wine', 'skinless boneless chicken breast halves', 'cloves garlic', 'cup diced onion', 'cups tomatoes', 'cup white wine', 'teaspoons fresh thyme', 'tablespoon fresh basil', 'cup kalamata olives', 'cup fresh parsley', 'salt pepper', 'ingredients', 'ingredients'],
    ['pound lean ground beef', 'cup carrots', 'cup celery', 'cup onion', 'cubes beef bouillon', 'cups water', 'ounce tomato sauce', 'ounce tomatoes', 'teaspoon salt', 'ounce frozen corn kernels', 'teaspoon basil', 'tablespoons ketchup', 'ingredients', 'ingredients'],
    ['pinch salt', 'cup purpose flour', 'ounce skinless boneless chicken breast halves', 'tablespoons olive oil', 'cup dry white wine', 'cup lemon juice', 'cup cold unsalted butter pieces', 'tablespoons capers', 'lemon wedges', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'cups red cabbage', 'cup green apples', 'cup apple cider vinegar', 'tablespoons water', 'cup white sugar', 'teaspoons salt', 'teaspoon black pepper', 'teaspoon ground cloves', 'ingredients', 'ingredients'],
    ['ounce package egg noodles', 'tablespoons butter', 'pound bacon', 'onion', 'small head cabbage', 'garlic salt', 'ingredients', 'ingredients'],
    ['medium potatoes', 'tablespoons olive oil', 'teaspoon thyme', 'teaspoon garlic powder', 'teaspoons fresh chives', 'salt pepper', 'pound boneless pork loin roast', 'teaspoon thyme', 'teaspoon garlic powder', 'teaspoon onion powder', 'salt pepper', 'ingredients', 'ingredients'],
    ['tablespoons red wine vinegar', 'tablespoons grapeseed oil', 'cup fresh cilantro', 'limes', 'teaspoon white sugar', 'teaspoon salt', 'cloves garlic', 'pound package frozen edamame green soybeans', 'cups frozen corn kernels', 'pint cherry tomatoes', 'green onions', 'ounce black beans', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves inch thickness', 'ounce Italian dressing', 'fresh jalapeno peppers seeded', 'ounce package cream cheese', 'bacon', 'toothpicks', 'ingredients', 'ingredients'],
    ['cup uncooked long grain rice', 'ounce French onion soup', 'ounce broth', 'ounce mushrooms', 'cup butter', 'ingredients', 'ingredients'],
    ['inch graham cracker crusts', 'tablespoons butter', 'ounce package coconut', 'cup pecans', 'ounce condensed milk', 'ounce package cream cheese', 'ounce container topping', 'ounce jar caramel ice cream topping', 'ingredients', 'ingredients'],
    ['ounces boneless skinless chicken breast halves bite sized pieces', 'tablespoon oyster sauce', 'tablespoons dark soy sauce', 'tablespoons vegetable oil', 'cloves garlic', 'large onion rings', 'cup water', 'teaspoon ground black pepper', 'teaspoon white sugar', 'medium head bok choy', 'small head broccoli', 'tablespoon cornstarch equal parts water', 'ingredients', 'ingredients'],
    ['eggplant', 'tablespoon olive oil more', 'large zucchini', 'potatoes', 'onion', 'clove garlic', 'tablespoon white vinegar', 'ounce tomatoes', 'ounce lentils liquid', 'teaspoon oregano', 'tablespoons fresh parsley', 'salt black pepper', 'cup feta cheese', 'tablespoons butter', 'tablespoons purpose flour', 'cups milk', 'ground black pepper', 'pinch ground nutmeg', 'egg', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['dry fettuccine pasta', 'cup olive oil', 'cloves garlic', 'zucchinis', 'teaspoon salt', 'teaspoon red pepper flakes', 'cup fresh basil', 'roma plum tomatoes', 'pound bay scallops', 'tablespoons Parmesan cheese', 'ingredients', 'ingredients'],
    ['pork chops', 'large apples', 'cup brown sugar', 'teaspoon ground cinnamon', 'tablespoons butter', 'salt pepper', 'cup dry sherry', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'pound skinless boneless chicken breasts strips', 'ounce packet fajita seasoning', 'red bell pepper thin strips', 'green bell pepper thin strips', 'poblano pepper thin strips', 'large onion thin strips', 'ounce roasted tomatoes', 'ounce seasoned black beans', 'ounce chicken broth', 'hot sauce', 'salt pepper', 'ingredients', 'ingredients'],
    ['russet potatoes', 'cup mayonnaise', 'teaspoons sweet pickle relish', 'teaspoons white sugar', 'teaspoons white onion', 'teaspoons prepared mustard', 'teaspoon white wine vinegar', 'tablespoon celery', 'teaspoon pimento', 'teaspoon carrot', 'teaspoon parsley', 'teaspoon ground black pepper', 'salt', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cup white sugar', 'teaspoon baking soda', 'teaspoon baking powder', 'ounce container blueberry yogurt', 'egg', 'teaspoon vanilla extract', 'tablespoons butter', 'cups blueberries', 'ingredients', 'ingredients'],
    ['quart vegetable oil frying', 'cup purpose flour', 'cup beer', 'pinch salt', 'pinch ground black pepper', 'onions rings', 'ingredients', 'ingredients'],
    ['olive oil cooking spray', 'pound fresh asparagus tough ends', 'cup Parmesan cheese', 'teaspoon sea salt', 'teaspoon garlic powder', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'onion', 'teaspoons fresh ginger', 'jalapeno pepper seeded diced', 'pound butternut squash cubed', 'ounce chicken broth', 'fluid ounce milk', 'cup coconut milk', 'tablespoon white sugar', 'salt black pepper', 'cup sour cream', 'tablespoon fresh thyme', 'ingredients', 'ingredients'],
    ['cup oats', 'cup buttermilk', 'cup whole wheat flour', 'teaspoon baking powder', 'teaspoon baking soda', 'cup brown sugar', 'cup applesauce', 'egg', 'ingredients', 'ingredients'],
    ['cups zucchini', 'cup lemon juice', 'cup white sugar', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'cups purpose flour', 'cups white sugar', 'cups butter', 'teaspoon ground cinnamon', 'ingredients', 'ingredients'],
    ['cups apples', 'cup orange juice', 'cup purpose flour', 'cup white sugar', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'pinch salt', 'cup butter', 'ingredients', 'ingredients'],
    ['cup fresh blueberries', 'cups purpose flour', 'cup quick cooking oats', 'cup pecans', 'teaspoon baking soda', 'teaspoon salt', 'cup shortening', 'cup white sugar', 'eggs', 'cup bananas', 'ingredients', 'ingredients'],
    ['boneless skinless chicken breast halves', 'cup purpose flour', 'salt pepper', 'tablespoon oregano', 'tablespoons olive oil', 'cup butter', 'cups portobello mushrooms', 'cup sun tomatoes', 'cup fresh spinach', 'cup Marsala wine', 'ingredients', 'ingredients'],
    ['cup butter', 'cup white sugar', 'egg', 'tablespoons corn syrup', 'teaspoon lemon extract', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon baking powder', 'cup white sugar', 'ingredients', 'ingredients'],
    ['cup cider vinegar', 'teaspoon white sugar', 'teaspoon salt', 'teaspoon fresh dill weed', 'teaspoon ground black pepper', 'tablespoons vegetable oil', 'cucumbers', 'cup red onion', 'ripe tomatoes wedges', 'ingredients', 'ingredients'],
    ['chicken thighs', 'ounce cream mushroom soup', 'ounces milk', 'teaspoon parsley', 'teaspoon onion powder', 'cup dry bread crumbs', 'tablespoons butter', 'teaspoon cornstarch', 'ingredients', 'ingredients'],
    ['fresh jalapeno peppers seeded', 'ounce package cream cheese', 'slices bacon half', 'ingredients', 'ingredients'],
    ['apple', 'cups purpose flour', 'teaspoon salt', 'teaspoons ground cinnamon', 'teaspoons powder', 'eggs', 'cups white sugar', 'cup vegetable oil', 'teaspoons vanilla extract', 'cup walnuts', 'teaspoons white sugar', 'teaspoon ground cinnamon', 'ingredients', 'ingredients'],
    ['cup white vinegar', 'tablespoon salt', 'cups white sugar', 'cups cucumbers', 'cup onions', 'cup green bell peppers', 'ingredients', 'ingredients'],
    ['boneless skinless chicken thighs', 'lime', 'cup purpose flour', 'teaspoons salt', 'tablespoon paprika', 'cup vegetable oil', 'tablespoons brown sugar', 'cup chicken broth', 'cup white wine', 'sprigs fresh mint', 'ingredients', 'ingredients'],
    ['ounce cans whole kernel corn', 'ounce package cream cheese', 'cup butter', 'jalapeno peppers', 'teaspoon garlic salt', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'onion', 'cloves garlic', 'bay leaf', 'ounce tomatoes', 'teaspoons curry powder', 'teaspoon salt', 'pound whole chicken bones skin pieces', 'ounce coconut milk', 'lemon', 'ingredients', 'ingredients'],
    ['ground beef', 'large onion', 'green bell pepper', 'ounce cans kidney beans', 'fluid ounce tomato juice', 'ounce tomatoes juice', 'cup pumpkin puree', 'tablespoon pumpkin pie spice', 'tablespoon chili powder', 'cup white sugar', 'ingredients', 'ingredients'],
    ['cups whole wheat flour', 'cup white sugar', 'teaspoon salt', 'teaspoons powder', 'cup vegetable oil', 'egg', 'cup milk', 'cup unsweetened applesauce', 'pint fresh blueberries', 'ingredients', 'ingredients'],
    ['Brussels sprouts', 'cup unsalted butter', 'small red onions strips', 'cup red wine vinegar', 'tablespoons white sugar', 'salt pepper', 'cup pistachios', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'onion', 'tablespoon curry powder', 'carrots', 'vegetable broth', 'cups water', 'ingredients', 'ingredients'],
    ['ounce package linguine pasta', 'cup butter', 'cups fresh mushrooms', 'cup onion', 'cup green bell pepper', 'ounce cans cream mushroom soup', 'chicken broth', 'cups sharp Cheddar cheese', 'ounce frozen green peas', 'cup cooking sherry', 'teaspoon Worcestershire sauce', 'teaspoon salt', 'teaspoon ground black pepper', 'cups chicken breast', 'cup Parmesan cheese', 'paprika', 'ingredients', 'ingredients'],
    ['fresh green beans', 'large onion', 'ham hocks', 'pounds new potatoes', 'teaspoon garlic powder', 'teaspoon onion powder', 'teaspoon seasoning salt', 'tablespoon chicken bouillon granules', 'ground black pepper', 'ingredients', 'ingredients'],
    ['ounces cream cheese', 'tablespoon milk', 'tablespoon white sugar', 'cups topping', 'inch graham cracker crust', 'cup cold milk', 'ounce instant vanilla pudding mix', 'ounce pumpkin puree', 'teaspoon ground cinnamon', 'teaspoon ground ginger', 'teaspoon ground cloves', 'ingredients', 'ingredients'],
    ['pound pork shoulder roast', 'pinto beans', 'ounce cans green chile peppers', 'tablespoons chili powder', 'tablespoons ground cumin', 'tablespoons', 'tablespoons oregano', 'tablespoons garlic powder', 'flour tortillas', 'ingredients', 'ingredients'],
    ['ounce cans green beans', 'bacon', 'cup brown sugar', 'cup butter', 'teaspoons soy sauce', 'teaspoons garlic powder', 'ingredients', 'ingredients'],
    ['cup white sugar', 'cups warm water', 'large russet potatoes inch strips', 'cups vegetable oil frying', 'salt', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'cup white sugar', 'cups purpose flour', 'teaspoon baking powder', 'teaspoon salt', 'cup butter', 'cups white sugar', 'eggs', 'teaspoon vanilla extract', 'cups fresh blueberries', 'cup purpose flour', 'ingredients', 'ingredients'],
    ['skin boneless chicken breast halves', 'teaspoon salt black pepper', 'tablespoons butter', 'tablespoons olive oil', 'white mushrooms', 'shallot', 'tablespoon purpose flour', 'cup Marsala wine', 'cups chicken stock', 'tablespoons fresh parsley', 'teaspoon cold butter', 'ingredients', 'ingredients'],
    ['pound ground turkey', 'ounces fresh mushrooms', 'onion', 'tablespoons soy sauce', 'teaspoon kosher salt', 'teaspoon black pepper', 'cup blue cheese', 'ingredients', 'ingredients'],
    ['ears corn', 'large tomatoes', 'large onion', 'cup fresh basil', 'cup olive oil', 'tablespoons white vinegar', 'salt pepper', 'ingredients', 'ingredients'],
    ['pounds ground beef', 'cup olive oil', 'cups onion', 'cups celery', 'ounce jars garlic', 'teaspoon black pepper', 'ounce cans beef broth', 'ounce tomatoes', 'ounce tomato paste', 'teaspoons thyme', 'teaspoons basil', 'teaspoons oregano', 'tablespoons parsley', 'cups pasta', 'ounce cans kidney beans', 'ingredients', 'ingredients'],
    ['ounces penne pasta', 'tablespoons olive oil', 'cups tomatoes', 'ounces fresh spinach', 'clove garlic', 'ounces tomato basil feta cheese', 'salt', 'ground black pepper', 'ingredients', 'ingredients'],
    ['onion', 'clove garlic', 'teaspoons vegetable oil', 'cups milk', 'cup chicken broth', 'tablespoons purpose flour', 'teaspoon salt', 'teaspoon ground black pepper', 'cup Parmesan cheese', 'ounces dry fettuccine pasta', 'ounce frozen broccoli florets', 'ingredients', 'ingredients'],
    ['cup butter', 'cup brown sugar', 'bananas', 'cup milk', 'teaspoon vanilla extract', 'eggs', 'cups purpose flour', 'cup wheat bran', 'teaspoon baking powder', 'teaspoon baking soda', 'teaspoon salt', 'cup walnuts', 'ingredients', 'ingredients'],
    ['teaspoons lime juice', 'cup milk', 'cup butter', 'cup white sugar', 'egg', 'teaspoons lime zest', 'cups purpose flour', 'teaspoon baking powder', 'teaspoon baking soda', 'tablespoons lime juice', 'cup white sugar', 'ingredients', 'ingredients'],
    ['cups honey', 'tablespoons soy sauce', 'cloves garlic', 'chicken wings', 'ingredients', 'ingredients'],
    ['whole chicken wings split', 'cup honey', 'tablespoons fresh lime juice', 'tablespoon lime zest', 'clove garlic', 'teaspoon salt', 'teaspoon ground black pepper', 'cup purpose flour', 'quarts vegetable oil frying', 'ingredients', 'ingredients'],
    ['cups chicken meat', 'cup onion', 'cups Cheddar cheese', 'cup sour cream', 'inch flour tortillas', 'cups tomatoes', 'cup black olives', 'ingredients', 'ingredients'],
    ['cup milk', 'ounce cream coconut', 'cups heavy cream', 'cups coconut optional', 'ingredients', 'ingredients'],
    ['ounce cheese tortellini', 'ounce jar marinara sauce', 'ounce jar Alfredo sauce', 'ounce box frozen spinach dry', 'tablespoon Italian seasoning', 'ounce package Mozzarella cheese', 'cup Parmesan cheese', 'teaspoon Italian seasoning', 'ingredients', 'ingredients'],
    ['corn tortillas', 'vegetable oil', 'boneless skinless chicken breast halves', 'ounces Monterey Jack cheese', 'cup onion', 'cup butter', 'cup purpose flour', 'chicken broth', 'cup sour cream', 'ounce green chiles', 'ounces Monterey Jack cheese', 'cup green onions', 'cup fresh cilantro', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'ground beef', 'stalks celery', 'onion', 'green bell pepper', 'ounce cans tomatoes', 'ounce tomatoes green chiles such RO TEL', 'ounce tomato sauce', 'cup water', 'ounce chili seasoning such McCormick Mild Chili Seasoning Mix', 'ounce kidney beans undrained', 'ounce pinto beans undrained', 'salt black pepper', 'tablespoon white vinegar more', 'ingredients', 'ingredients'],
    ['pint fresh strawberries', 'ounce package cream cheese', 'cup confectioners sugar', 'tablespoons orange liqueur', 'ingredients', 'ingredients'],
    ['cup shortening', 'cup white sugar', 'eggs', 'teaspoon vanilla extract', 'cup purpose flour', 'cup quick cooking oats', 'teaspoon baking soda', 'teaspoon salt', 'cup milk', 'teaspoon ground cinnamon', 'cups bananas', 'cup raisins optional', 'ingredients', 'ingredients'],
    ['cup butter', 'cups onion', 'cups celery', 'cup fresh parsley', 'ounces fresh mushrooms', 'cups dry bread cubes', 'teaspoon poultry seasoning', 'teaspoons sage', 'teaspoon thyme', 'teaspoon marjoram', 'teaspoons salt', 'teaspoon ground black pepper', 'cups chicken broth', 'eggs', 'ingredients', 'ingredients'],
    ['ounce fillets salmon', 'ounce dry Italian style salad dressing mix', 'cup water', 'tablespoons lemon juice', 'cup fresh mushrooms', 'ingredients', 'ingredients'],
    ['inch pie shell baked', 'ounce instant vanilla pudding mix', 'cups milk', 'cups coconut', 'ounce container topping', 'ingredients', 'ingredients'],
    ['ounce beans liquid', 'ounces fresh jalapeno pepper', 'teaspoon ground cumin', 'tablespoons lemon juice', 'cloves garlic', 'ingredients', 'ingredients'],
    ['cups watermelon', 'cup green bell pepper', 'tablespoons lime juice', 'tablespoons fresh cilantro', 'tablespoon green onions', 'tablespoon jalapeno pepper', 'teaspoon garlic salt', 'ingredients', 'ingredients'],
    ['cup butter', 'cup dark brown sugar', 'cup unsweetened pineapple juice', 'cups purpose flour', 'teaspoons powder', 'teaspoon salt', 'cup white sugar', 'eggs', 'teaspoon vanilla extract', 'fresh pineapple rings', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'cup confectioners sugar', 'eggs', 'cup milk', 'tablespoons butter', 'teaspoon vanilla extract', 'teaspoon salt', 'cup butter', 'cup brown sugar', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'cup half cream', 'bananas', 'cups heavy cream', 'pinch ground cinnamon', 'ingredients', 'ingredients'],
    ['fresh mussels', 'tablespoons extra virgin olive oil', 'cloves garlic', 'cup white wine', 'tablespoons margarine butter', 'green onions', 'bunch fresh parsley', 'roma plum tomatoes', 'salt pepper', 'ingredients', 'ingredients'],
    ['eggs', 'cup vegetable oil', 'cup brown sugar', 'cup white sugar', 'cup zucchini', 'bananas', 'teaspoons vanilla extract', 'cups purpose flour', 'tablespoon ground cinnamon', 'teaspoons powder', 'teaspoon baking soda', 'teaspoon salt', 'cup cranberries', 'cup walnuts', 'ingredients', 'ingredients'],
    ['mangos', 'pound pork shoulder roast', 'tablespoons black pepper', 'teaspoon kosher salt', 'teaspoon chipotle chile powder', 'cup balsamic vinegar', 'cups water', 'teaspoon chipotle chile powder', 'teaspoons honey', 'fluid ounce bourbon whiskey', 'ounce bottles barbeque sauce', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'teaspoons salt', 'teaspoon ground black pepper', 'tablespoons purpose flour', 'ounce chicken broth', 'pork chops', 'cups potatoes', 'dash paprika', 'ingredients', 'ingredients'],
    ['cups pears', 'cups white sugar', 'cups purpose flour', 'teaspoon salt', 'teaspoons soda', 'teaspoon ground nutmeg', 'teaspoon ground cinnamon', 'teaspoon ground cloves', 'egg whites', 'cup canola oil', 'cup pecans', 'ingredients', 'ingredients'],
    ['large ripe tomatoes inch thick', 'pound fresh mozzarella cheese inch thick', 'cup fresh basil leaves', 'tablespoons extra virgin olive oil', 'fine sea salt', 'black pepper', 'ingredients', 'ingredients'],
    ['medium red potatoes', 'tablespoons butter', 'tablespoon fresh dill', 'teaspoons garlic', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['tablespoons butter margarine', 'chicken thighs', 'salt pepper', 'slices prosciutto thin', 'tablespoons garlic', 'cup fresh mushrooms', 'cup dry white wine optional', 'cup sour cream', 'ingredients', 'ingredients'],
    ['potatoes cubed', 'cup milk', 'cup butter', 'pound bacon', 'onion', 'cloves garlic', 'ounce package lasagna noodles', 'cups Cheddar cheese', 'salt pepper', 'ounce container sour cream', 'tablespoons fresh chives', 'ingredients', 'ingredients'],
    ['ounce yellow cake mix', 'cups fresh blueberries', 'cup white sugar', 'cup butter', 'teaspoon ground cinnamon', 'ingredients', 'ingredients'],
    ['tablespoon unsalted butter', 'cloves garlic', 'ounce bags fresh spinach', 'teaspoon garlic salt', 'lemon', 'ingredients', 'ingredients'],
    ['cup butter', 'cups white sugar', 'teaspoon vanilla extract', 'teaspoon salt', 'eggs', 'cups purpose flour', 'teaspoons powder', 'teaspoon ground cinnamon', 'cup milk', 'cups apples', 'cup walnuts', 'cup white sugar', 'teaspoon ground cinnamon', 'tablespoons purpose flour', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'salt pepper', 'paprika', 'cup dry white wine', 'ounce cream mushroom soup', 'ounce mushrooms', 'cup sour cream', 'cup purpose flour', 'ingredients', 'ingredients'],
    ['pumpkin puree', 'cup milk', 'cup white sugar', 'cup self flour', 'eggs', 'teaspoon vanilla extract', 'cup butter', 'ground cinnamon', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cup white sugar', 'teaspoon salt', 'teaspoons powder', 'teaspoon baking soda', 'cup cranberries', 'cup walnuts', 'egg', 'tablespoons vegetable oil', 'cup orange juice', 'tablespoon orange zest', 'ingredients', 'ingredients'],
    ['teaspoon olive oil', 'onion', 'cloves garlic', 'red bell peppers', 'large tomatoes', 'teaspoons thyme', 'teaspoons', 'teaspoon white sugar', 'chicken broth', 'salt pepper', 'pinch ground cayenne pepper', 'hot pepper sauce', 'tablespoons butter', 'tablespoons purpose flour', 'tablespoons sour cream', 'ingredients', 'ingredients'],
    ['Dipping Sauce', 'cup mayonnaise', 'tablespoon ketchup', 'tablespoons cream style horseradish sauce', 'teaspoon paprika', 'teaspoon salt', 'teaspoon oregano', 'pinch ground black pepper', 'teaspoon cayenne pepper', 'Onion', 'egg', 'cup milk', 'cup purpose flour', 'teaspoons salt', 'teaspoons cayenne pepper', 'teaspoon paprika', 'teaspoon ground black pepper', 'teaspoon oregano', 'teaspoon thyme', 'teaspoon ground cumin', 'large sweet onion', 'cup vegetable oil frying', 'ingredients', 'ingredients'],
    ['tablespoons low sodium soy sauce', 'tablespoons fat free Italian dressing', 'teaspoon cornstarch', 'tablespoon garlic', 'teaspoon ground ginger', 'pound round steak strips', 'cups water', 'cubes beef bouillon', 'ounces linguine pasta uncooked', 'cup fat free beef broth', 'cup fresh mushrooms', 'cup green onion', 'pound broccoli florets', 'ingredients', 'ingredients'],
    ['sweet potatoes cubed', 'cup milk', 'cup butter', 'cup maple syrup', 'ingredients', 'ingredients'],
    ['head napa cabbage', 'bunch green onions', 'cup butter', 'ounce package ramen noodles', 'tablespoons sesame seeds', 'cup almonds', 'cup cider vinegar', 'cup vegetable oil', 'cup white sugar', 'tablespoons soy sauce', 'ingredients', 'ingredients'],
    ['pounds skinless boneless chicken breast meat', 'ounce cream chicken soup', 'ounce cream celery soup', 'ounce', 'cups chicken broth', 'ounce seasoned cornbread stuffing mix', 'ingredients', 'ingredients'],
    ['cup vegetable oil', 'large onions', 'ounce packages spinach dry', 'tablespoons fresh dill', 'tablespoons purpose flour', 'ounce feta cheese', 'eggs', 'salt pepper', 'ounce phyllo dough', 'pound butter', 'ingredients', 'ingredients'],
    ['cups fresh basil', 'cup extra virgin olive oil', 'cup pine nuts', 'cup Brazil nuts', 'cup Parmesan cheese', 'tablespoons garlic', 'teaspoon chili powder', 'ingredients', 'ingredients'],
    ['pound bok choy', 'tablespoon vegetable oil', 'tablespoon sesame oil', 'cup water', 'teaspoon fresh ginger root', 'cloves garlic', 'tablespoon oyster sauce', 'tablespoon light soy sauce', 'tablespoon brown sugar', 'teaspoon red pepper flakes optional', 'ingredients', 'ingredients'],
    ['tablespoons chili oil', 'cloves garlic', 'hot chile peppers', 'pound skinless boneless chicken breast halves bite size pieces', 'teaspoons white sugar', 'teaspoon garlic salt', 'teaspoon black pepper', 'tablespoons oyster sauce', 'cup fresh mushrooms', 'cup onions', 'bunch fresh basil leaves', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'lamb meat inch cubes', 'teaspoons', 'teaspoon ground turmeric', 'teaspoon ground cumin', 'teaspoon cayenne pepper', 'teaspoon ground cinnamon', 'teaspoon ground cloves', 'teaspoon ground cardamom', 'teaspoon kosher salt', 'teaspoon ground ginger', 'pinch saffron', 'teaspoon garlic powder', 'teaspoon ground coriander', 'medium onions inch cubes', 'carrots fourths thin strips', 'cloves garlic', 'tablespoon ginger', 'lemon', 'ounce homemade chicken broth low sodium canned', 'tablespoon sun tomato paste', 'tablespoon honey', 'tablespoon cornstarch optional', 'tablespoon water optional', 'ingredients', 'ingredients'],
    ['teaspoon rosemary', 'teaspoon basil', 'teaspoon garlic', 'black pepper', 'tablespoons olive oil', 'pork chops', 'cup fresh lemon juice', 'cup feta cheese basil sun tomatoes', 'ingredients', 'ingredients'],
    ['pounds Brussels sprouts', 'teaspoon salt', 'tablespoons butter', 'tablespoons Parmesan cheese', 'tablespoons bread crumbs', 'teaspoon garlic powder', 'teaspoon ground black pepper', 'teaspoon seasoning salt', 'ingredients', 'ingredients'],
    ['ounce corn', 'ounce whole kernel corn', 'pound butter', 'ounce package corn bread mix', 'ounce container sour cream optional', 'ingredients', 'ingredients'],
    ['cups day old Italian bread bite size pieces', 'cup olive oil', 'salt pepper', 'cloves garlic', 'cup olive oil', 'tablespoons balsamic vinegar', 'medium ripe tomatoes wedges', 'cup red onion', 'basil leaves', 'cup green olives', 'cup fresh mozzarella bite size pieces', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'cup extra virgin olive oil', 'lemon', 'teaspoons garlic', 'teaspoon salt', 'teaspoons black pepper', 'teaspoon paprika', 'ingredients', 'ingredients'],
    ['cup brown sugar', 'cup butter', 'tablespoons water', 'ounce peaches', 'inch thick day old French bread', 'eggs', 'tablespoon vanilla extract', 'pinch ground cinnamon', 'ingredients', 'ingredients'],
    ['cup uncooked long grain white rice', 'ounce chicken broth', 'cup fat sour cream', 'ounce green chile peppers', 'cup Monterey Jack cheese', 'ounce whole kernel corn', 'cup fresh cilantro', 'salt black pepper', 'ingredients', 'ingredients'],
    ['cups potatoes', 'cups ham', 'onion', 'cups milk', 'cup frozen peas', 'tablespoons purpose flour', 'tablespoons butter', 'teaspoon salt', 'teaspoon ground black pepper', 'teaspoon garlic powder', 'teaspoon onion powder', 'teaspoon parsley', 'pound Cheddar cheese', 'pinch paprika', 'ingredients', 'ingredients'],
    ['inch unbaked pie crust', 'cup white sugar', 'cup purpose flour', 'teaspoons lemon zest', 'tablespoon lemon juice', 'cups fresh frozen blueberries', 'cup brown sugar', 'cup oats', 'cup purpose flour', 'teaspoon ground cinnamon', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['ounce frozen corn kernels', 'ounce cans ripe olives', 'red bell pepper', 'small onion', 'cloves garlic', 'cup olive oil', 'cup lemon juice', 'tablespoons cider vinegar', 'teaspoon oregano', 'teaspoon salt', 'teaspoon ground black pepper', 'avocados diced', 'ingredients', 'ingredients'],
    ['ounce package spice cake mix', 'ounce pumpkin', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'large onion', 'cloves garlic', 'large potatoes cubed', 'carrots', 'ounce cans chicken broth', 'head cauliflower', 'cup milk', 'teaspoon salt', 'teaspoon ground black pepper', 'teaspoon ground nutmeg', 'tablespoon dry sherry', 'tablespoon fresh parsley', 'ingredients', 'ingredients'],
    ['cup brown sugar', 'cup butter', 'cups cranberries', 'cup pecans', 'cup butter room temperature', 'cup white sugar', 'eggs', 'teaspoon vanilla extract', 'cup sour cream', 'cups purpose flour', 'teaspoons powder', 'teaspoon baking soda', 'teaspoon ground cinnamon', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['potatoes cubed', 'carrots inch pieces', 'fresh broccoli florets', 'zucchini', 'salt', 'cup olive oil', 'ounce dry onion soup mix', 'ingredients', 'ingredients'],
    ['bunch spinach', 'large strawberries', 'cup white sugar', 'teaspoon salt', 'cup white wine vinegar', 'cup vegetable oil', 'tablespoon poppy seeds', 'ingredients', 'ingredients'],
    ['ounce package strawberry Jell O r', 'cup cold water', 'ounce white cake mix', 'eggs', 'cup vegetable oil', 'tablespoons purpose flour', 'ounce frozen strawberries', 'tablespoons butter', 'cups confectioners sugar', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast half', 'tablespoons butter margarine', 'cloves garlic', 'tablespoons green onion', 'teaspoon basil', 'ounce pizza crust dough', 'roma plum tomatoes', 'cup fresh cilantro', 'cup ricotta cheese', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['young arugula leaves', 'cup cherry tomatoes', 'cup pine nuts', 'tablespoons grapeseed oil olive', 'tablespoon rice vinegar', 'salt', 'black pepper', 'cup Parmesan cheese', 'large avocado', 'ingredients', 'ingredients'],
    ['cups kale', 'tablespoons lemon juice', 'tablespoon olive oil', 'tablespoon garlic', 'teaspoon soy sauce', 'salt', 'ground black pepper', 'ingredients', 'ingredients'],
    ['cups baby spinach leaves', 'medium red onion rings', 'ounce mandarin oranges', 'cups cranberries', 'cup honey sliced almonds', 'cup feta cheese', 'cup balsamic vinaigrette salad dressing', 'ingredients', 'ingredients'],
    ['pound bacon', 'pound ground', 'pound ground pork', 'green bell pepper', 'yellow onion', 'jalapeno peppers', 'habanero peppers', 'Anaheim peppers seeded diced', 'cloves garlic', 'tablespoons ground cumin', 'tablespoon red pepper flakes', 'tablespoons chili powder', 'tablespoons beef bouillon granules', 'ounce tomatoes', 'ounce cans whole tomatoes', 'ounce cans chili beans', 'fluid ounce beer', 'ounces tomato paste', 'ounce chile paste', 'cups water', 'ingredients', 'ingredients'],
    ['pound fresh mushrooms', 'pound beef rib roast', 'ounce envelope onion soup mix', 'fluid ounce bottle beer', 'ground black pepper', 'ingredients', 'ingredients'],
    ['lemons', 'cup white sugar', 'cups cold water', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'teaspoon salt', 'teaspoon black pepper', 'teaspoon cayenne pepper', 'pounds fresh shrimp tails', 'cup olive oil', 'cloves garlic', 'shallot', 'cup fresh parsley', 'teaspoon oregano', 'tablespoons white wine', 'tablespoons brandy', 'ingredients', 'ingredients'],
    ['ounce yellow cake mix', 'cup white sugar', 'eggs', 'ounce package cream cheese room temperature', 'cup vegetable oil', 'teaspoon vanilla extract', 'cups blueberries', 'ingredients', 'ingredients'],
    ['ounce peaches', 'tablespoons butter', 'pinch ground cinnamon', 'pinch ground nutmeg', 'tablespoon cornstarch', 'cup water', 'cup milk', 'cup white sugar', 'cup purpose flour', 'teaspoons powder', 'pinch salt', 'cup butter', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'ingredients', 'ingredients'],
    ['cup olive oil', 'onions', 'green bell peppers', 'red bell peppers', 'cloves garlic', 'ounce firm tofu cubed', 'ounce cans black beans', 'ounce cans tomatoes', 'teaspoons salt', 'teaspoon ground black pepper', 'teaspoons ground cumin', 'tablespoons chili powder', 'tablespoons oregano', 'tablespoons white vinegar', 'tablespoon liquid hot pepper sauce such Tabasco tm', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'red leaf lettuce', 'green leaf lettuce', 'fresh tomato', 'bunch cilantro', 'ounce whole kernel corn', 'ounce black beans', 'ounce French onions', 'cup Ranch dressing', 'cup barbeque sauce', 'ingredients', 'ingredients'],
    ['long Chinese eggplants', 'tablespoons soy sauce', 'tablespoon red wine vinegar', 'tablespoon white sugar', 'green chile pepper', 'teaspoon cornstarch', 'teaspoon chili oil', 'teaspoons salt', 'tablespoons vegetable oil', 'ingredients', 'ingredients'],
    ['eggs', 'cup white sugar', 'teaspoon ground cinnamon', 'cup pumpkin puree', 'cup purpose flour', 'teaspoon baking soda', 'tablespoons butter', 'ounces cream cheese', 'cup confectioners sugar', 'teaspoon vanilla extract', 'confectioners sugar', 'ingredients', 'ingredients'],
    ['head romaine lettuce', 'fresh spinach', 'pint fresh strawberries', 'Bermuda onion', 'cup mayonnaise', 'tablespoons white wine vinegar', 'cup white sugar', 'cup milk', 'tablespoons poppy seeds', 'ingredients', 'ingredients'],
    ['pounds potatoes', 'teaspoon vegetable oil', 'tablespoon white sugar', 'teaspoon salt', 'pinch ground cayenne pepper', 'ingredients', 'ingredients'],
    ['ounce containers plain yogurt', 'teaspoons kosher salt', 'teaspoon black pepper', 'teaspoon ground cloves', 'tablespoons ginger', 'cloves garlic', 'teaspoons', 'teaspoons ground cumin', 'teaspoons ground cinnamon', 'teaspoons ground coriander', 'chicken thighs', 'olive oil spray', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'tablespoon butter', 'onion', 'pounds ground lamb', 'cup purpose flour', 'salt black pepper', 'teaspoons fresh rosemary', 'teaspoon paprika', 'teaspoon ground cinnamon', 'tablespoon ketchup', 'cloves garlic', 'cups water', 'ounce frozen peas carrots', 'pounds Yukon Gold potatoes', 'tablespoon butter', 'pinch ground cayenne pepper', 'cup cream cheese', 'pound Irish cheese such Dubliner', 'salt black pepper', 'egg yolk', 'tablespoons milk', 'ingredients', 'ingredients'],
    ['ounce container ricotta cheese', 'eggs', 'tablespoons Italian seasoning', 'cups mozzarella cheese', 'cup Parmesan cheese', 'ounce package spinach dry', 'salt pepper', 'ounce package white bread dough', 'ingredients', 'ingredients'],
    ['cups cookie crumbs', 'cup ground hazelnuts', 'tablespoons brown sugar', 'tablespoons unsalted butter', 'ounce cream cheese', 'cup brown sugar', 'cups solid pack pumpkin', 'cup heavy cream', 'cup maple syrup', 'tablespoon vanilla extract', 'teaspoon ground cinnamon', 'teaspoon ground allspice', 'eggs', 'ingredients', 'ingredients'],
    ['recipe pastry inch double crust pie', 'tablespoons cooking tapioca', 'teaspoon salt', 'cup white sugar', 'cups cherries', 'teaspoon almond extract', 'teaspoon vanilla extract', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves thin', 'cup purpose flour', 'salt black pepper', 'teaspoons butter', 'teaspoons vegetable oil', 'ounce mushrooms', 'cup water', 'packet chicken bouillon granules', 'teaspoon cornstarch', 'teaspoon parsley', 'teaspoon garlic powder', 'ground black pepper', 'ingredients', 'ingredients'],
    ['cup butter', 'large onion', 'potatoes diced', 'carrots', 'cups water', 'tablespoons chicken bouillon powder', 'ground black pepper', 'tablespoons purpose flour', 'cups milk', 'tablespoon parsley', 'teaspoon thyme', 'ingredients', 'ingredients'],
    ['cucumbers', 'onions', 'green bell peppers', 'cloves garlic', 'cup salt', 'cider vinegar', 'cups white sugar', 'tablespoons mustard seed', 'teaspoons celery seed', 'teaspoon whole cloves', 'tablespoon ground turmeric', 'ingredients', 'ingredients'],
    ['recipe pastry double crust pie', 'large Granny Smith apples', 'cup butter', 'cup brown sugar', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'cups water', 'cups white sugar', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['tablespoon oil peanut sesame', 'cloves garlic', 'pound fresh green beans', 'tablespoon white sugar', 'tablespoons oyster sauce', 'teaspoons soy sauce', 'ingredients', 'ingredients'],
    ['pound butternut squash seeded', 'salt', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'pound ham steak', 'red apple', 'green apple', 'cup maple pancake syrup', 'teaspoon ground cinnamon', 'ingredients', 'ingredients'],
    ['large zucchini', 'cup Italian style salad dressing', 'ingredients', 'ingredients'],
    ['Rome Beauty apples cubed', 'Comice pears cubed', 'cup cranberries', 'tablespoon purpose flour', 'tablespoons honey', 'tablespoons lemon juice', 'cup purpose flour', 'cup brown sugar', 'cup quick cooking oats', 'cup ground walnuts', 'cup butter', 'ingredients', 'ingredients'],
    ['large sweet potatoes inch chunks', 'medium Vidalia other sweet onions inch chunks', 'tablespoons olive oil', 'cup amaretto liqueur', 'teaspoon thyme', 'Salt black pepper', 'cup almonds', 'ingredients', 'ingredients'],
    ['slices bacon inch pieces', 'small onion', 'leek', 'large carrot', 'cloves garlic', 'ounce cans chicken broth', 'cups green split peas', 'bay leaves', 'teaspoon fresh rosemary', 'ingredients', 'ingredients'],
    ['cloves garlic', 'tablespoons olive oil', 'teaspoon thyme', 'tablespoons balsamic vinegar', 'salt pepper', 'large portobello mushroom caps', 'hamburger buns', 'tablespoon capers', 'cup mayonnaise', 'tablespoon capers', 'large tomato', 'lettuce', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'cup onion', 'cup celery', 'cup carrots', 'cup green bell pepper', 'clove garlic', 'ounce tomatoes', 'tablespoons chili powder', 'tablespoon tomato paste', 'tablespoon white vinegar', 'teaspoon ground black pepper', 'ounce kidney beans', 'kaiser rolls', 'ingredients', 'ingredients'],
    ['cups white sugar', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'pinch salt', 'cup butter', 'sweet potatoes', 'tablespoon vanilla extract', 'ingredients', 'ingredients'],
    ['dry fettuccine pasta', 'cup butter', 'cup onion', 'cloves garlic', 'pound fresh mushrooms', 'teaspoons oregano', 'teaspoons basil', 'teaspoons parsley', 'slices ham', 'cups heavy whipping cream', 'cup spaghetti sauce', 'teaspoon red pepper optional', 'ingredients', 'ingredients'],
    ['recipe pastry inch single crust pie', 'eggs', 'egg yolk', 'cup white sugar', 'cup brown sugar', 'teaspoon salt', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'teaspoon ground ginger', 'teaspoon ground cloves', 'cups milk', 'cup heavy whipping cream', 'pumpkin puree', 'ingredients', 'ingredients'],
    ['large baking potatoes', 'pound bacon', 'tablespoons butter', 'large onion', 'cup fresh mushrooms', 'teaspoon red pepper', 'teaspoon garlic powder', 'teaspoon ground black pepper', 'teaspoon fresh chives', 'teaspoon salt', 'ounce container sour cream', 'ounce package Cheddar cheese', 'teaspoon dry bread crumbs', 'ingredients', 'ingredients'],
    ['onion', 'green bell pepper', 'bunch green onions', 'jalapeno peppers', 'tablespoon garlic', 'pint cherry tomatoes', 'ounce zesty Italian dressing', 'ounce black beans', 'ounce black eyed peas', 'teaspoon ground coriander', 'bunch fresh cilantro', 'ingredients', 'ingredients'],
    ['head cauliflower small florets', 'tablespoons garlic extra-virgin olive oil', 'teaspoon ground nutmeg', 'teaspoons garlic powder', 'teaspoons salt', 'teaspoon ground black pepper', 'tablespoon butter', 'onion', 'tablespoons purpose flour', 'ounce chicken broth', 'cup milk', 'tablespoon dry sherry', 'ingredients', 'ingredients'],
    ['ounce packages spinach', 'cups Italian style seasoned bread crumbs', 'cup Parmesan cheese', 'cup butter', 'small green onion', 'eggs', 'salt pepper', 'ingredients', 'ingredients'],
    ['potatoes', 'pound lean ground beef', 'onion', 'ounce cans tomato soup', 'ounce green beans', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['large artichokes', 'lemon', 'cup olive oil', 'cloves garlic', 'teaspoon salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['cups white sugar', 'cup cornstarch', 'teaspoons ground cinnamon', 'teaspoon ground nutmeg', 'teaspoons salt', 'cups water', 'tablespoons lemon juice', 'yellow food coloring', 'apples', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'large portobello mushrooms', 'ounce package spinach', 'teaspoon basil', 'teaspoon salt', 'teaspoon black pepper', 'clove garlic', 'tablespoons dry red wine', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['cup shortening', 'cups white sugar', 'eggs', 'teaspoons vanilla extract', 'ounce pumpkin puree', 'cups purpose flour', 'teaspoons soda', 'teaspoons powder', 'teaspoon salt', 'teaspoon ground cinnamon', 'pinch ground nutmeg', 'cup semisweet chocolate chips', 'cup walnuts optional', 'ingredients', 'ingredients'],
    ['pound ground beef', 'onion', 'ounce tomatoes', 'ounce tomato sauce', 'ounce kidney beans', 'cups water', 'chili powder', 'garlic powder', 'salt pepper', 'ingredients', 'ingredients'],
    ['cup fresh orange juice', 'cup fresh lemon juice', 'cup brown sugar', 'teaspoon orange zest', 'teaspoon lemon zest', 'teaspoon vanilla extract', 'cups fresh pineapple', 'cups strawberries', 'kiwi fruit', 'bananas', 'oranges', 'cup seedless grapes', 'cups blueberries', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'pound round steak thin strips diagonal', 'cup orange peel slivers', 'clove garlic', 'teaspoon ground ginger', 'tablespoons cornstarch', 'cup beef broth', 'cup soy sauce', 'cup sherry', 'cup orange marmalade', 'teaspoon red pepper flakes', 'ingredients', 'ingredients'],
    ['inch graham cracker crust', 'ounce cream cheese', 'ounce condensed milk', 'lemons', 'teaspoon lemon zest', 'ingredients', 'ingredients'],
    ['cup butter', 'cup brown sugar', 'teaspoon ground cinnamon', 'ounce cans apple pie filling', 'slices white bread', 'eggs', 'cups milk', 'teaspoon vanilla extract', 'cup maple syrup', 'ingredients', 'ingredients'],
    ['cups fresh cranberries', 'cups water', 'whole cloves', 'whole allspice berries', 'cinnamon sticks', 'cups white sugar', 'ingredients', 'ingredients'],
    ['Yukon Gold potatoes cubed', 'cup butter', 'cup milk', 'ounce package cream cheese', 'onion', 'egg', 'salt pepper', 'ingredients', 'ingredients'],
    ['jalapeno chile peppers', 'cloves garlic', 'onion', 'tablespoon white sugar', 'teaspoon salt', 'teaspoon ground cumin', 'ounce tomatoes green chile peppers', 'ounce tomatoes', 'ingredients', 'ingredients'],
    ['cooking spray', 'tablespoons peanut butter', 'slices whole wheat bread', 'banana', 'ingredients', 'ingredients'],
    ['cups cake flour', 'teaspoons powder', 'cups white sugar', 'ounce package strawberry Jell O r', 'cup butter', 'eggs', 'cup milk', 'teaspoon vanilla extract', 'cup strawberries', 'cups heavy cream', 'tablespoons sugar', 'teaspoon vanilla extract', 'cups fresh strawberries', 'cup margarine', 'ounce package cream cheese', 'cups confectioners sugar', 'teaspoons vanilla extract', 'cups strawberries', 'ingredients', 'ingredients'],
    ['pounds carrots', 'cup white sugar', 'teaspoons powder', 'teaspoons vanilla extract', 'tablespoons purpose flour', 'eggs', 'cup margarine', 'teaspoons confectioners sugar', 'ingredients', 'ingredients'],
    ['ounce pumpkin puree', 'fluid ounce milk', 'eggs', 'cups white sugar', 'teaspoons pumpkin pie spice', 'teaspoon salt', 'ounce yellow cake mix', 'cup pecans', 'cup margarine', 'ounce container topping', 'ingredients', 'ingredients'],
    ['cup butter', 'onions', 'stalk celery', 'cloves garlic', 'medium head cabbage squares', 'ounce tomatoes liquid', 'salt pepper', 'ingredients', 'ingredients'],
    ['cup butter', 'cup white sugar', 'cup self flour', 'cup milk', 'ounce peaches light syrup', 'ingredients', 'ingredients'],
    ['ounce kidney beans undrained', 'ounce baked beans pork', 'ounce butter beans undrained', 'cup ketchup', 'teaspoons white vinegar', 'tablespoon dry mustard', 'cup brown sugar', 'pound lean ground beef', 'ounces bacon', 'cup onion', 'salt', 'ground black pepper', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'cloves garlic', 'green chilies', 'cups skinless boneless chicken breast meat', 'cups jasmine rice', 'tablespoon white sugar', 'tablespoon fish sauce', 'tablespoon soy sauce', 'teaspoons green onion', 'tablespoons fresh basil leaves', 'tablespoons fresh cilantro', 'ingredients', 'ingredients'],
    ['tomatoes', 'small yellow onion', 'cup white sugar', 'cup white vinegar', 'cup vegetable oil', 'salt pepper', 'ingredients', 'ingredients'],
    ['eggs', 'cups white sugar', 'ounce pumpkin', 'cups vegetable oil', 'cups purpose flour', 'teaspoons soda', 'teaspoons powder', 'teaspoon ground cinnamon', 'teaspoon salt', 'cups chocolate chips', 'ingredients', 'ingredients'],
    ['CRAB STUFFING', 'tablespoons olive oil', 'teaspoon onion', 'teaspoon green onion', 'teaspoon garlic', 'teaspoon celery', 'teaspoon green bell pepper', 'tablespoons shrimp stock water', 'ounce crab meat', 'tablespoons bread crumbs', 'teaspoon Cajun seasoning', 'PEPPERCORN SAUCE', 'cups beef broth', 'teaspoon black pepper', 'fluid ounce whiskey', 'cup heavy cream', 'STEAKS', 'ounce filet mignon steaks', 'slices bacon', 'salt black pepper', 'tablespoon olive oil', 'clove garlic', 'teaspoon shallot', 'cup crimini mushrooms', 'fluid ounce whiskey', 'teaspoon Dijon mustard', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'shallot', 'green onions', 'cup garbanzo beans', 'salt pepper', 'bunch red Swiss chard', 'tomato', 'lemon', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoons powder', 'teaspoon baking soda', 'teaspoon salt', 'cup white sugar', 'cup butter', 'eggs', 'cup mashed banana', 'cup walnuts', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoons soda', 'teaspoon salt', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'cup solid pack pumpkin puree', 'cup brown sugar', 'cup buttermilk', 'egg', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'cloves garlic', 'cup onion', 'cup carrots', 'cup green bell pepper', 'cup red bell pepper', 'tablespoons chili powder', 'cups fresh mushrooms', 'ounce tomatoes liquid', 'ounce black beans undrained', 'ounce kidney beans undrained', 'ounce pinto beans undrained', 'ounce whole kernel corn', 'tablespoon cumin', 'tablespoons oregano', 'tablespoons basil', 'tablespoon garlic powder', 'ingredients', 'ingredients'],
    ['cup uncooked white rice', 'cup water', 'pound pork sausage', 'cup onion', 'cup fresh tomato', 'cups zucchini squash', 'ounce cans mushrooms', 'ounce package cheese food', 'oregano', 'salt pepper', 'ingredients', 'ingredients'],
    ['roma plum tomatoes', 'small cucumber', 'green onions', 'cup fresh basil leaves thin strips', 'tablespoons olive oil', 'tablespoons balsamic vinegar', 'tablespoons feta cheese', 'salt black pepper', 'ingredients', 'ingredients'],
    ['pound Brussels sprouts', 'slices bacon inch pieces', 'salt black pepper', 'cup heavy cream', 'cup bread crumbs', 'cup Parmesan cheese', 'tablespoons butter tiny pieces', 'ingredients', 'ingredients'],
    ['ounce skinless boneless chicken breast halves', 'tablespoons garlic', 'medium onion', 'cups water', 'tablespoon chicken bouillon granules', 'fresh poblano chilies', 'ounce container sour cream', 'tablespoons olive oil', 'inch corn tortillas', 'cup mozzarella cheese', 'ingredients', 'ingredients'],
    ['cup butter', 'cups fresh peaches', 'tablespoons white sugar', 'cup white sugar', 'teaspoon salt', 'teaspoon baking powder', 'cup purpose flour', 'teaspoon ground cinnamon', 'cup milk', 'ingredients', 'ingredients'],
    ['green bell peppers', 'red bell pepper', 'onion', 'cup fresh mushrooms', 'cups chicken meat', 'ounce dry Italian style salad dressing mix', 'inch flour tortillas', 'ingredients', 'ingredients'],
    ['vanilla wafer crumbs', 'bananas inch slices', 'cups white sugar', 'cup purpose flour', 'cups milk', 'egg yolks', 'teaspoons butter', 'teaspoons vanilla extract', 'egg whites', 'cup white sugar', 'ingredients', 'ingredients'],
    ['ounce dry penne pasta', 'tablespoons olive oil', 'pound skinless boneless chicken breast meat cut bite size pieces', 'cloves garlic', 'ounces asparagus inch pieces', 'teaspoon red pepper flakes', 'salt pepper', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['cup butter', 'cups white sugar', 'eggs', 'teaspoon vanilla extract', 'cups buttermilk', 'cups purpose flour', 'teaspoons powder', 'teaspoon baking soda', 'teaspoon salt', 'cups blueberries', 'cup purpose flour', 'cup white sugar', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['cup shortening', 'cup white sugar', 'eggs', 'cup bananas', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'cup quick cooking oats', 'cup walnuts', 'ingredients', 'ingredients'],
    ['pound fresh mushrooms', 'ounces crabmeat', 'green onions', 'teaspoon thyme', 'teaspoon oregano', 'teaspoon ground savory', 'ground black pepper', 'cup Parmesan cheese', 'cup mayonnaise', 'tablespoons Parmesan cheese', 'teaspoon paprika', 'ingredients', 'ingredients'],
    ['pound ground beef', 'pound sausage inch slices', 'onion', 'stalks celery', 'clove garlic', 'medium head cabbage', 'ounce tomatoes', 'fluid ounces water', 'cup uncooked rice', 'garlic salt', 'ingredients', 'ingredients'],
    ['pound ground pork sausage', 'ounce package cream cheese', 'ounce fresh mushrooms', 'ingredients', 'ingredients'],
    ['cup canola oil', 'cup brown sugar', 'eggs', 'teaspoon vanilla extract', 'cup purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'cups oats', 'cup raisins', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'onion', 'cloves garlic', 'cup apricots', 'cups red lentils', 'cups chicken stock', 'roma plum tomatoes', 'teaspoon ground cumin', 'teaspoon thyme', 'salt', 'ground black pepper', 'tablespoons fresh lemon juice', 'ingredients', 'ingredients'],
    ['large cucumbers', 'cup rice vinegar', 'teaspoons white sugar', 'teaspoon salt', 'teaspoons fresh ginger root', 'ingredients', 'ingredients'],
    ['medium cucumbers', 'avocados', 'tablespoons fresh cilantro', 'clove garlic', 'tablespoons green onions optional', 'teaspoon salt', 'black pepper', 'large lemon', 'lime', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'medium onion', 'pound ground beef', 'clove garlic', 'pound eggplant', 'cup carrots', 'cup celery', 'ounce cans Italian tomatoes', 'ounce cans beef broth', 'teaspoon sugar', 'teaspoon ground nutmeg', 'teaspoon salt', 'teaspoon ground black pepper', 'cup dry macaroni', 'teaspoons fresh parsley', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['tablespoons peanut oil', 'pound firm tofu', 'red onion', 'red bell pepper', 'green chile pepper', 'cloves garlic', 'cup hot water', 'tablespoons white vinegar', 'tablespoons soy sauce', 'tablespoon brown sugar', 'teaspoon cornstarch', 'teaspoon red pepper flakes', 'ingredients', 'ingredients'],
    ['whole salmon', 'cup soy sauce', 'tablespoon chile sauce', 'tablespoon fresh ginger root', 'clove garlic', 'lime', 'lime', 'tablespoon brown sugar', 'green onions', 'ingredients', 'ingredients'],
    ['ounce fresh cranberries', 'orange', 'cinnamon sticks', 'cups orange juice', 'cups brown sugar', 'ingredients', 'ingredients'],
    ['ripe tomatoes', 'tablespoons olive oil', 'tablespoons butter', 'onion', 'green bell pepper', 'carrots', 'cloves garlic', 'cup fresh basil', 'teaspoon Italian seasoning', 'cup Burgundy wine', 'bay leaf', 'celery', 'tablespoons tomato paste', 'ingredients', 'ingredients'],
    ['ounce white corn', 'ounce yellow corn', 'ounce tomatoes green chile peppers', 'ounce package cream cheese diced', 'teaspoon chili powder', 'teaspoon garlic powder', 'fresh cilantro', 'ingredients', 'ingredients'],
    ['red potatoes', 'tablespoon olive oil', 'onion', 'green bell pepper seeded', 'tablespoons olive oil', 'teaspoon salt', 'teaspoon paprika', 'teaspoon ground black pepper', 'cup fresh parsley', 'ingredients', 'ingredients'],
    ['bunch collard greens', 'ham hocks', 'ounce cans chicken broth', 'fluid ounces water', 'tablespoon white vinegar', 'salt pepper', 'ingredients', 'ingredients'],
    ['Lime Sour Cream', 'cup fat sour cream', 'teaspoons fresh lime juice', 'small fresh jalapeno pepper', 'salt', 'Bean Cakes', 'tablespoons olive oil', 'green onions', 'cloves garlic', 'fresh jalapeno peppers', 'tablespoon ground cumin', 'ounce cans black beans', 'salt black pepper', 'cups raw sweet potato', 'egg', 'cup plain bread crumbs', 'ingredients', 'ingredients'],
    ['ounce uncooked angel hair pasta', 'tablespoons olive oil', 'skinless boneless chicken breast halves strips', 'tablespoons garlic', 'tablespoons basil', 'tablespoons Cajun style seasoning', 'salt pepper', 'roma plum tomatoes', 'cup feta cheese', 'ingredients', 'ingredients'],
    ['pound bacon', 'tablespoons green onion', 'teaspoons garlic', 'ounce tomatoes', 'teaspoon basil', 'teaspoon salt', 'ground black pepper', 'ounce package linguine pasta', 'tablespoons Parmesan cheese', 'ingredients', 'ingredients'],
    ['cup mayonnaise', 'cup Parmesan cheese', 'ounce jars artichoke hearts', 'cups mozzarella cheese', 'teaspoons garlic powder', 'teaspoon paprika', 'ingredients', 'ingredients'],
    ['pounds fresh asparagus inch pieces', 'cup chicken broth', 'pound fresh mushrooms', 'ounces angel hair pasta', 'tablespoon olive oil', 'teaspoon red pepper', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['cup butter', 'cup white sugar', 'lemon', 'eggs', 'cups purpose flour', 'teaspoon baking powder', 'teaspoon salt', 'cup milk', 'tablespoons lemon zest', 'cup fresh blueberries', 'tablespoons lemon juice', 'cup white sugar', 'ingredients', 'ingredients'],
    ['ounces goat cheese', 'cups arugula', 'cup cherry tomatoes', 'cup olive oil', 'teaspoons garlic', 'teaspoon ground black pepper', 'teaspoon salt', 'ounces penne pasta', 'ingredients', 'ingredients'],
    ['ounce cans beer', 'large onion', 'bratwurst', 'teaspoons red pepper flakes', 'teaspoon garlic powder', 'teaspoon salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['cup chicken broth', 'cup tomato sauce', 'bacon', 'onions', 'cup uncooked white rice', 'tomatoes', 'green bell peppers', 'teaspoon chili powder', 'teaspoon salt black pepper', 'ounce black olives optional', 'ounce whole kernel corn optional', 'ingredients', 'ingredients'],
    ['eggs', 'bacon', 'tablespoons Dijon mustard', 'cup mayonnaise', 'stalks celery', 'small potatoes', 'tablespoons fresh parsley', 'salt pepper', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'tablespoons lime juice', 'cloves garlic', 'tuna steaks', 'fresh mango', 'cup red bell pepper', 'Spanish onion', 'green onion', 'tablespoons fresh cilantro', 'jalapeno pepper seeded', 'tablespoons lime juice', 'teaspoons olive oil', 'tablespoons', 'tablespoon cayenne pepper', 'tablespoon onion powder', 'teaspoons salt', 'teaspoon ground black pepper', 'teaspoon thyme', 'teaspoon basil', 'teaspoon oregano', 'tablespoon garlic powder', 'tablespoons olive oil', 'ingredients', 'ingredients'],
    ['ounce package farfalle bow tie pasta', 'tablespoons vegetable oil', 'pound skinless boneless chicken breast halves', 'cups heavy cream', 'cube chicken bouillon', 'cup Asiago cheese', 'tablespoon cornstarch', 'tablespoons butter', 'cup prosciutto', 'tablespoon fresh garlic', 'cup mushrooms', 'tablespoon parsley flakes', 'ingredients', 'ingredients'],
    ['pound lean ground beef', 'ounce package taco seasoning mix', 'ounce package nacho flavor tortilla chips', 'cups Cheddar cheese', 'ounce kidney beans', 'cup ranch style salad dressing', 'tomato', 'lettuce', 'green onion', 'ingredients', 'ingredients'],
    ['pound seashell pasta', 'tablespoons olive oil', 'onions', 'green bell pepper', 'cup sweet corn kernels', 'ounce black beans', 'ounce tomatoes', 'cup salsa', 'cup black olives', 'tablespoons taco seasoning mix', 'salt pepper', 'ingredients', 'ingredients'],
    ['pound carrots', 'cup butter', 'tablespoons honey', 'pinch ground ginger', 'tablespoon lemon juice', 'ingredients', 'ingredients'],
    ['pound spaghetti pieces', 'ounce cans cream mushroom soup', 'ounces Cheddar cheese', 'cups chicken breast meat', 'pound sauteed mushrooms', 'ounce jar pimento peppers', 'cups chicken broth', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'small onion', 'cloves garlic', 'tablespoon Dijon mustard', 'teaspoons white sugar', 'tablespoon cider vinegar', 'cups chicken broth', 'cups kale', 'cup cranberries', 'salt pepper', 'cup almonds', 'ingredients', 'ingredients'],
    ['inch pie crust', 'tablespoon lime zest', 'cup fresh lime juice', 'ounce condensed milk', 'ounce instant vanilla pudding mix', 'ounce container topping', 'ingredients', 'ingredients'],
    ['tablespoons extra virgin olive oil', 'cups white onions', 'tablespoons garlic', 'cups plum tomatoes', 'cups dry white wine', 'cup green olives', 'cup capers', 'teaspoon red pepper flakes', 'ounce fillets sea bass', 'tablespoons butter', 'cup fresh cilantro', 'ingredients', 'ingredients'],
    ['lemons', 'cups white sugar', 'quarts water', 'ingredients', 'ingredients'],
    ['ounces almond paste', 'cup butter', 'cup white sugar', 'eggs', 'cups purpose flour', 'red food coloring', 'green food coloring', 'cup seedless red raspberry jam', 'cup apricot jam', 'cup semisweet chocolate chips', 'ingredients', 'ingredients'],
    ['cup warm water degrees F', 'tablespoon butter', 'tablespoon dry milk powder', 'tablespoon white sugar', 'teaspoons salt', 'tablespoons parsley', 'teaspoons garlic powder', 'bread flour', 'teaspoons active dry yeast', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'lemon', 'salt pepper', 'tablespoon olive oil', 'oregano', 'sprigs fresh parsley garnish', 'ingredients', 'ingredients'],
    ['cup shortening', 'cup white sugar', 'cup brown sugar', 'egg', 'teaspoon vanilla extract', 'cup purpose flour', 'teaspoon baking powder', 'teaspoon baking soda', 'teaspoon salt', 'cup oats', 'cup coconut optional', 'cup miniature semi-sweet chocolate chips optional', 'ingredients', 'ingredients'],
    ['ounces garbanzo beans', 'tablespoons red onion', 'cloves garlic', 'tomato', 'cup parsley', 'tablespoons olive oil', 'tablespoon lemon juice', 'salt pepper', 'ingredients', 'ingredients'],
    ['cup broccoli florets', 'cup cauliflower florets', 'cups eggs optional', 'cup Cheddar cheese', 'bacon', 'cup mayonnaise', 'cup white sugar', 'tablespoons white wine vinegar', 'ingredients', 'ingredients'],
    ['ounce package spinach', 'ounce cans chicken broth', 'ounce package cheese tortellini', 'tablespoon basil', 'tablespoon garlic powder', 'salt pepper', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'large onion', 'large stalks celery', 'teaspoons Italian seasoning', 'salt pepper', 'ounce Italian style diced tomatoes', 'vegetable broth', 'large sweet potatoes diced', 'large carrots thin', 'ounces green beans inch pieces', 'cloves garlic', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'tablespoons brown sugar', 'tablespoon white sugar', 'teaspoons powder', 'teaspoon baking soda', 'teaspoon salt', 'cup pumpkin puree', 'teaspoon ground cinnamon', 'teaspoon ground ginger', 'teaspoon ground allspice', 'egg', 'cups milk', 'tablespoons vegetable oil', 'tablespoons lemon juice', 'teaspoons lemon zest', 'teaspoon vegetable oil', 'ingredients', 'ingredients'],
    ['ounce package dates', 'ounces almonds', 'pound sliced bacon', 'ingredients', 'ingredients'],
    ['red potatoes', 'eggs', 'cups mayonnaise', 'onion', 'green onions', 'small green bell pepper seeded diced', 'stalks celery', 'teaspoons salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'skinless boneless chicken breast halves', 'salt pepper', 'ounces button mushrooms', 'cup Marsala wine', 'ounce cream chicken soup', 'ounce cream mushroom soup', 'cup heavy cream', 'teaspoon fresh rosemary', 'teaspoon fresh thyme', 'ounce uncooked linguine pasta', 'ingredients', 'ingredients'],
    ['potatoes', 'sweet potato', 'eggs', 'stalks celery', 'onion', 'cup mayonnaise', 'tablespoon prepared mustard', 'teaspoon salt', 'teaspoons black pepper', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'pound fresh white mushrooms', 'pinch salt', 'tablespoon butter', 'cup onion', 'cloves garlic', 'cup quick cooking oats', 'ounce Parmigiano Reggiano cheese', 'cup bread crumbs', 'cup flat leaf Italian parsley', 'eggs', 'teaspoon salt', 'black pepper', 'cayenne pepper', 'oregano', 'pasta sauce', 'tablespoon Parmigiano Reggiano cheese', 'tablespoon flat leaf Italian parsley', 'ingredients', 'ingredients'],
    ['Crust', 'cup butter', 'cup white sugar', 'teaspoon vanilla extract', 'cup purpose flour', 'Cheese Mixture', 'ounce package cream cheese', 'cup white sugar', 'egg', 'teaspoon vanilla extract', 'Apple Mixture', 'apples', 'cup white sugar', 'teaspoon ground cinnamon', 'cup almonds', 'ingredients', 'ingredients'],
    ['head cauliflower', 'cup sour cream', 'cup Cheddar cheese', 'teaspoon dry ranch salad dressing mix such Hidden Valley Ranch', 'teaspoon onion powder', 'teaspoon garlic powder', 'tablespoon butter small pieces more', 'ingredients', 'ingredients'],
    ['bacon', 'pound ground beef', 'onion', 'green bell pepper', 'cups pizza sauce', 'roma plum tomatoes', 'ounces Cheddar cheese', 'eggs', 'cup milk', 'tablespoon vegetable oil', 'cup purpose flour', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['pork chops', 'salt pepper', 'onions rings', 'sweet potatoes', 'apples rings', 'tablespoons brown sugar', 'teaspoons black pepper', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['tablespoons Cajun seasoning such Emeril Essence', 'lime', 'tablespoon vegetable oil', 'pound deveined medium shrimp', 'ingredients', 'ingredients'],
    ['Irish Stew', 'tablespoon vegetable oil', 'pound beef chuck roast fat inch cubes', 'tablespoons purpose flour', 'cup onion', 'cup carrot', 'fluid ounce dark beer', 'bay leaves', 'teaspoon thyme', 'teaspoon salt', 'teaspoon ground black pepper', 'cloves garlic', 'tablespoons Worcestershire sauce', 'Colcannon', 'bacon', 'russet potatoes chunks', 'cups cabbage', 'cup milk', 'tablespoons butter', 'teaspoon salt', 'teaspoon ground black pepper', 'tablespoons fresh parsley', 'ingredients', 'ingredients'],
    ['cup onion', 'butternut squash', 'teaspoon fresh rosemary', 'salt black pepper', 'tablespoons olive oil', 'ounce package pizza crust dough', 'tablespoon cornmeal', 'tablespoons Asiago Parmesan cheese', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'large onions', 'chile pepper', 'cloves garlic', 'pounds ground beef', 'ounce cans whole tomatoes liquid', 'teaspoons salt', 'teaspoon black pepper', 'tablespoons ground cumin', 'tablespoon chili powder', 'tablespoons', 'tablespoons oregano', 'cinnamon sticks', 'whole cloves', 'ounce cans red kidney beans', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'teaspoon salt', 'teaspoon ground black pepper', 'skinless boneless chicken breast halves strips', 'cup butter', 'medium garlic', 'lime', 'lemon pepper', 'ingredients', 'ingredients'],
    ['potato', 'ingredients', 'ingredients'],
    ['ounce fresh cheese ravioli', 'tablespoon olive oil', 'teaspoon olive oil', 'pound roma tomatoes', 'ounce jar artichoke hearts', 'cup green onions', 'cloves garlic', 'teaspoon salt', 'teaspoon black pepper', 'tablespoons Parmesan cheese', 'ingredients', 'ingredients'],
    ['ounces ziti pasta', 'pound ham cubed', 'large red bell pepper inch pieces', 'large green bell pepper inch pieces', 'large red onion', 'small sweet pickles juice', 'cup cherry tomatoes', 'cup mayonnaise', 'cup sour cream', 'teaspoons beef bouillon granules', 'tablespoon white vinegar', 'teaspoon salt', 'teaspoon ground black pepper', 'cloves garlic', 'ingredients', 'ingredients'],
    ['roma plum tomatoes', 'pound roma plum tomatoes', 'tablespoons garlic powder', 'cup lemon juice', 'tablespoons', 'tablespoon ground cayenne pepper', 'teaspoons ground cumin', 'red onion', 'white onion', 'yellow onion', 'pound jalapeno peppers', 'bunch fresh cilantro', 'ingredients', 'ingredients'],
    ['ounce tomato paste', 'onion', 'cup garlic', 'tablespoon olive oil', 'cup water', 'salt black pepper', 'small eggplant', 'zucchini', 'yellow squash', 'red bell pepper', 'yellow bell pepper', 'tablespoons olive oil', 'teaspoon fresh thyme leaves', 'tablespoons mascarpone cheese', 'ingredients', 'ingredients'],
    ['cups hot whole milk', 'teaspoons white sugar', 'teaspoon vanilla extract', 'teaspoon pumpkin pie spice', 'ounces double strength coffee', 'tablespoons cream', 'pumpkin pie spice', 'ingredients', 'ingredients'],
    ['cups uncooked orzo pasta', 'cup fresh basil leaves', 'cup oil sun tomatoes', 'tablespoons olive oil', 'cup Parmesan cheese', 'teaspoon salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['banana peppers', 'tablespoons butter', 'cup onion', 'cup celery', 'ounce tomatoes', 'ounce tomato sauce', 'cloves garlic', 'teaspoon basil', 'teaspoon oregano', 'teaspoons salt', 'teaspoon ground black pepper', 'egg', 'teaspoon Worcestershire sauce', 'cup Parmesan cheese', 'hot Italian sausage', 'pound mild Italian sausage', 'cups bread crumbs', 'ingredients', 'ingredients'],
    ['cups water', 'cups uncooked white rice', 'tablespoon vegetable oil', 'pound ground pork', 'bunch green onions', 'teaspoon garlic', 'ounce firm tofu cubed', 'carrots', 'tablespoons hoisin sauce', 'tablespoons soy sauce', 'teaspoon sesame oil', 'teaspoon hot chile paste', 'head iceberg lettuce leaves', 'ingredients', 'ingredients'],
    ['pounds spinach', 'cup purpose flour', 'eggs', 'cup Cheddar cheese', 'cup bread crumbs', 'tablespoons fresh parsley optional', 'cup butter', 'cup milk', 'teaspoon salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['tablespoons beef bouillon granules', 'tablespoons water', 'cloves garlic', 'teaspoons cayenne pepper', 'teaspoon salt', 'teaspoon black pepper', 'pounds beef sirloin inch cubes', 'wooden skewers water hour', 'tablespoons vegetable oil', 'ingredients', 'ingredients'],
    ['cups sour cherries', 'cups white sugar', 'tablespoons purpose flour', 'cups purpose flour', 'cup old fashioned oats', 'cup brown sugar', 'cup butter', 'cup shortening', 'ingredients', 'ingredients'],
    ['ounce uncooked orzo', 'cup olive oil', 'tablespoons butter', 'teaspoon garlic', 'teaspoon basil', 'teaspoon red pepper flakes', 'cup pine nuts', 'ounce bag baby spinach', 'cup balsamic vinegar', 'ounce package feta cheese', 'fresh tomato', 'salt', 'ingredients', 'ingredients'],
    ['ounce corn', 'eggs', 'tablespoons white sugar', 'tablespoons purpose flour', 'teaspoon salt', 'cup mild Cheddar cheese', 'ounce package broccoli', 'ingredients', 'ingredients'],
    ['quart oil frying', 'cup dry pancake mix', 'large Spanish onion rings', 'ingredients', 'ingredients'],
    ['quarts vegetable broth', 'cups fresh button mushrooms', 'ounce shiitake mushrooms pieces', 'cup uncooked pearl barley', 'cup dry lentils', 'cup onion flakes', 'teaspoons garlic', 'teaspoons summer savory', 'bay leaves', 'teaspoon basil', 'teaspoons black pepper', 'salt', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'pounds skinless boneless chicken breast meat', 'cup onion', 'chicken broth', 'ounce cans green chile peppers', 'ounce cans great Northern beans undrained', 'tablespoon garlic powder', 'tablespoon ground cumin', 'tablespoon oregano', 'teaspoons fresh cilantro', 'teaspoon red pepper', 'ingredients', 'ingredients'],
    ['cup shortening', 'large potatoes cubed', 'teaspoon salt', 'teaspoon ground black pepper', 'teaspoon garlic powder', 'teaspoon paprika', 'ingredients', 'ingredients'],
    ['cup butter', 'ounce package cream cheese', 'teaspoon vanilla extract', 'cups confectioners sugar', 'cups coconut', 'tablespoon ground cinnamon', 'ingredients', 'ingredients'],
    ['shortcakes', 'cups purpose bleached flour', 'teaspoon salt', 'tablespoon baking powder', 'tablespoons sugar', 'cup butter frozen', 'egg', 'egg white optional', 'cup cold half', '', 'ounce unsweetened frozen raspberries blackberries blueberries strawberries', 'ounces fresh raspberries blackberries blueberries dry strawberries', 'tablespoons sugar', 'cup heavy cream', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['ounce yellow cake mix', 'ounce instant vanilla pudding mix', 'cup white sugar', 'cup water', 'cup vegetable oil', 'eggs', 'ounce package cream cheese room temperature', 'ounce blueberries juice', 'cup confectioners sugar', 'ingredients', 'ingredients'],
    ['large onion', 'cloves garlic', 'fresh ginger root', 'tablespoon olive oil', 'teaspoons ground cumin', 'teaspoon ground turmeric', 'teaspoon salt', 'teaspoon ground black pepper', 'teaspoon ground cardamom', 'inch piece cinnamon stick', 'teaspoon ground cloves', 'bay leaves', 'teaspoon ground nutmeg', 'skinless chicken thighs', 'ounce tomatoes', 'ingredients', 'ingredients'],
    ['cup butter', 'cup brown sugar', 'cup purpose flour', 'cup oats', 'eggs', 'cup white sugar', 'ounce pumpkin', 'fluid ounce milk', 'teaspoon salt', 'teaspoon ground cinnamon', 'teaspoon ground ginger', 'teaspoon ground cloves', 'ingredients', 'ingredients'],
    ['ounce containers sour cream', 'ounce bacon bits', 'cups Cheddar cheese', 'bunch green onions', 'ingredients', 'ingredients'],
    ['cup butter', 'cups purpose flour', 'cup white sugar', 'eggs', 'cup white sugar', 'tablespoons purpose flour', 'teaspoon baking powder', 'tablespoons lemon juice', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'tablespoons sesame oil', 'teaspoon fresh ginger root', 'bunch asparagus stalks cut', 'tablespoon soy sauce', 'cup cashews', 'ingredients', 'ingredients'],
    ['large carrots', 'large potatoes', 'large onion', 'medium head green cabbage', 'cloves garlic', 'cups chicken stock', 'tablespoon olive oil', 'teaspoon thyme', 'teaspoon basil', 'teaspoon parsley', 'teaspoon salt', 'ground black pepper', 'ingredients', 'ingredients'],
    ['Cupcakes', 'eggs', 'cup Mazola Vegetable Plus Oil', 'cups sugar', 'ounce pumpkin', 'cups purpose flour', 'cup Argo Kingsford Corn Starch', 'teaspoons Spice Islands Pumpkin Pie', 'teaspoons Argo Baking Powder', 'teaspoon baking soda', 'teaspoon salt', 'Frosting', 'ounce package cream cheese', 'tablespoons butter margarine', 'tablespoon orange juice', 'teaspoons Spice Islands % Pure Bourbon Vanilla Extract', 'teaspoons orange peel', 'cups powdered sugar', 'ingredients', 'ingredients'],
    ['cup olive oil', 'cup white vinegar', 'cup white sugar', 'tablespoons soy sauce', 'bunches baby bok choy', 'bunch green onions', 'cup almonds', 'ounce package chow mein noodles', 'ingredients', 'ingredients'],
    ['ounce salmon fillets', 'tablespoons butter', 'tablespoons soy sauce', 'lemon pepper', 'ingredients', 'ingredients'],
    ['large russet potato', 'tablespoon butter margarine', 'tablespoons Cheddar cheese', 'salt pepper', 'teaspoons sour cream', 'ingredients', 'ingredients'],
    ['eggs', 'cup vegetable oil', 'cups white sugar', 'cups purpose flour', 'teaspoons ground cinnamon', 'teaspoon baking soda', 'teaspoon salt', 'teaspoon vanilla extract', 'cups apple peel', 'ingredients', 'ingredients'],
    ['pound bacon', 'stalks celery', 'small onion', 'tablespoons margarine', 'salt', 'ground black pepper', 'ounce cream style corn', 'ounce whole kernel corn', 'chicken broth', 'cups milk', 'small potatoes', 'carrots', 'tablespoons purpose flour', 'ingredients', 'ingredients'],
    ['pound boneless skinless chicken meat', 'tablespoons vegetable oil', 'ounce cans coconut milk', 'cups water', 'tablespoons fresh ginger root', 'tablespoons fish sauce', 'cup fresh lime juice', 'teaspoon cayenne pepper', 'teaspoon ground turmeric', 'tablespoons green onion', 'tablespoon fresh cilantro', 'ingredients', 'ingredients'],
    ['cups heavy cream', 'cup white sugar', 'ounce cream cheese', 'teaspoons lemon juice', 'teaspoons vanilla extract', 'cup white sugar', 'ounce package pound cake', 'ounce frozen raspberries', 'tablespoons unsweetened cocoa powder', 'ingredients', 'ingredients'],
    ['Chicken Marinade', 'cup water', 'cup teriyaki sauce', 'tablespoons lime juice', 'teaspoons garlic', 'teaspoon liquid smoke flavoring', 'teaspoon salt', 'teaspoon ground ginger', 'teaspoon tequila', 'skinless boneless chicken breast halves', 'Mexi Ranch Dressing', 'cup mayonnaise', 'cup sour cream', 'tablespoon milk', 'teaspoons tomato', 'teaspoons green chile peppers', 'teaspoon onion', 'teaspoon parsley', 'teaspoon hot pepper sauce', 'pinch salt', 'dill weed', 'pinch paprika', 'cayenne pepper', 'pinch ground cumin', 'chili powder', 'pinch ground black pepper', 'cup Cheddar Monterey Jack cheese blend', 'cups corn chips', 'ingredients', 'ingredients'],
    ['cups cubed chicken breast meat', 'ounce cream chicken soup', 'ounce green beans', 'salt pepper', 'ounce package unseasoned dry bread stuffing mix', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['avocados diced', 'teaspoons salt', 'large tomato', 'onion', 'jalapeno peppers', 'tablespoon fresh cilantro', 'tablespoons fresh lime juice', 'ingredients', 'ingredients'],
    ['ounce tomatoes', 'ounce chicken broth undiluted', 'tablespoons butter', 'tablespoons white sugar', 'tablespoon onion', 'teaspoon baking soda', 'cups heavy whipping cream', 'ingredients', 'ingredients'],
    ['ounce whole kernel corn', 'ounce cream style corn', 'cup margarine', 'cup sour cream', 'ounce dry cornbread mix', 'ingredients', 'ingredients'],
    ['large russet potato', 'large sweet potato', 'cup corn', 'teaspoon Dijon style mustard', 'tablespoons fresh lime juice', 'tablespoons fresh cilantro', 'clove garlic', 'tablespoons canola oil', 'teaspoon salt', 'teaspoon ground black pepper', 'cucumber', 'red onion', 'cup peanuts', 'ingredients', 'ingredients'],
    ['egg', 'tablespoon water', 'teaspoon salt', 'teaspoon ground black pepper', 'cup dry bread crumbs', 'tablespoons olive oil', 'skinless boneless chicken breast halves', 'tablespoon garlic', 'tablespoons olive oil', 'zucchinis', 'tomatoes', 'cup mozzarella cheese', 'teaspoons fresh basil', 'cup mozzarella cheese', 'ingredients', 'ingredients'],
    ['pound ground beef', 'pound ground pork', 'tablespoons olive oil', 'large onion', 'green bell pepper', 'habanero peppers', 'jalapeno pepper seeded', 'cloves garlic', 'tablespoons green onion', 'ounce cans chili beans', 'ounce tomatoes', 'ounce tomato paste', 'ounce tomato sauce', 'ounce bottle lager style beer', 'tablespoons cornmeal', 'cup water', 'cup chili powder', 'tablespoon ground cumin', 'teaspoon garlic powder', 'teaspoon cayenne pepper', 'tablespoon salt', 'teaspoons black pepper', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['cups white sugar', 'cups vegetable oil', 'teaspoons vanilla extract', 'eggs', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon ground cinnamon', 'teaspoon salt', 'medium Granny Smith apples', 'cup walnuts', 'cup butter margarine', 'teaspoons milk', 'cup brown sugar', 'ingredients', 'ingredients'],
    ['cup graham cracker crumbs', 'tablespoons white sugar', 'tablespoons butter', 'eggs', 'cup key lime juice', 'ounce condensed milk', 'pinch salt', 'pinch cream tartar', 'cup heavy whipping cream', 'lime', 'ingredients', 'ingredients'],
    ['ounce Cheddar cheese soup', 'cup onion', 'black pepper', 'large baking potatoes', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['cup white sugar', 'cup milk', 'cup heavy cream', 'cup butter', 'tablespoons cornstarch', 'tablespoons milk', 'teaspoon vanilla extract', 'tart apples', 'tablespoon purpose flour', 'teaspoon ground cinnamon', 'ounce package pastry double crust pie', 'cup confectioners sugar', 'tablespoon milk', 'teaspoon vanilla extract', 'tablespoon butter', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'onion', 'stalk celery', 'clove garlic', 'ounce cans white kidney beans', 'ounce chicken broth', 'teaspoon ground black pepper', 'teaspoon thyme', 'cups water', 'bunch fresh spinach', 'tablespoon lemon juice', 'ingredients', 'ingredients'],
    ['tablespoons canola oil', 'pounds skinless boneless chicken breast halves bite size pieces', 'tablespoon fresh ginger root', 'tablespoons rice vinegar', 'tablespoons teriyaki sauce', 'tablespoon honey', 'pound dark sweet cherries', 'cups carrots', 'cup green onion', 'cup almonds', 'lettuce leaves', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'onion', 'ounce corn', 'tablespoon curry paste', 'salt', 'ground black pepper', 'teaspoon garlic powder', 'ounce beans chickpeas', 'ounce firm tofu', 'bunch fresh spinach', 'teaspoon basil', 'ingredients', 'ingredients'],
    ['cup butter', 'teaspoon garlic salt', 'teaspoon rosemary', 'teaspoon basil', 'teaspoon thyme', 'teaspoon garlic powder', 'tablespoon Parmesan cheese', 'pound loaf French bread', 'ingredients', 'ingredients'],
    ['pounds turkey breast', 'cup frozen spinach', 'tablespoons goat cheese', 'ingredients', 'ingredients'],
    ['cup garbanzo beans', 'cup jalapeno pepper slices juice', 'tablespoons', 'cloves garlic', 'tablespoons lemon juice', 'teaspoon ground cumin', 'teaspoon curry powder', 'crushed red pepper', 'ingredients', 'ingredients'],
    ['ounce beans', 'cup guacamole', 'cup mayonnaise', 'ounce container sour cream', 'ounce package taco seasoning mix', 'cups Cheddar cheese', 'tomato', 'cup green onions', 'cup black olives', 'ingredients', 'ingredients'],
    ['bunch rainbow chard leaves', 'large onion', 'tablespoon olive oil', 'salt black pepper', 'tablespoons olive oil', 'ounces feta cheese inch pieces', 'ingredients', 'ingredients'],
    ['ounce tilapia fillets', 'Miami completa', 'olive oil cooking spray such PAM', 'cup Parmesan cheese', 'tablespoons butter margarine blend such Smart Balance', 'tablespoons fat free mayonnaise such Kraft', 'tablespoons lemon juice', 'teaspoons lemon zest', 'teaspoon basil', 'teaspoon ground black pepper', 'teaspoon onion powder', 'teaspoon celery salt', 'ingredients', 'ingredients'],
    ['small new red potatoes', 'cup Ranch style salad dressing', 'ounce package Colby Monterey Jack cheese', 'teaspoon black pepper', 'ingredients', 'ingredients'],
    ['tomatoes seeded diced', 'cloves garlic', 'cup fresh basil', 'cup olive oil', 'salt', 'tablespoons Parmesan cheese', 'pound pasta', 'ingredients', 'ingredients'],
    ['pound cheese tortellini', 'large portobello mushrooms', 'cup white wine', 'tablespoon fresh parsley', 'cloves garlic', 'ounces Alfredo style pasta sauce', 'salt pepper', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'onion', 'ounce black beans undrained', 'ounce tomatoes', 'teaspoon oregano', 'teaspoon garlic powder', 'cups uncooked instant brown rice', 'ingredients', 'ingredients'],
    ['jewel yams inch cubes', 'tablespoons olive oil', 'salt black pepper', 'tablespoon olive oil', 'onion', 'cloves garlic', 'bunch kale bite sized pieces', 'tablespoons red wine vinegar', 'teaspoon fresh thyme', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'leek', 'carrots', 'zucchini', 'ounces green beans inch pieces', 'stalks celery', 'quarts vegetable stock', 'pound tomatoes', 'tablespoon fresh thyme', 'ounce beans liquid', 'cup elbow macaroni', 'salt black pepper', 'ingredients', 'ingredients'],
    ['bacon', 'tablespoon olive oil', 'white onion', 'teaspoons garlic', 'cups beef stock', 'ounce pinto beans', 'ounce Italian style stewed tomatoes', 'stalks celery', 'bay leaf', 'medium apple', 'cup red wine', 'salt pepper', 'ingredients', 'ingredients'],
    ['large head cauliflower small florets', 'cup butter', 'cup Parmesan cheese', 'cup Italian seasoned bread crumbs', 'pinch salt', 'teaspoon red pepper flakes', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['cup butter', 'cup shiitake mushrooms', 'cup portobello mushrooms', 'shallots', 'tablespoons purpose flour', 'ounce chicken broth', 'cup half', 'salt pepper', 'pinch ground cinnamon optional', 'ingredients', 'ingredients'],
    ['cups dry lentils', 'quarts chicken broth', 'onion', 'cup tomato paste', 'cloves garlic', 'tablespoon ground cumin', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'onion', 'red bell pepper seeded', 'jalapeno pepper seeded', 'fresh mushrooms', 'roma plum tomatoes', 'cup fresh corn kernels', 'teaspoon ground black pepper', 'teaspoon ground cumin', 'tablespoon chili powder', 'ounce cans black beans', 'cups chicken broth vegetable', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'onion', 'chicken leg quarters', 'cloves garlic', 'cups chicken stock', 'cup balsamic vinegar', 'cup sun tomatoes', 'salt pepper', 'ingredients', 'ingredients'],
    ['tablespoon butter', 'cloves garlic', 'tablespoon butter', 'Brussels sprouts', 'tablespoon butter', 'tablespoons Parmesan cheese more', 'salt black pepper', 'ingredients', 'ingredients'],
    ['pounds orange sweet potatoes', 'tablespoon vegetable oil', 'onion', 'inch piece fresh ginger root', 'tablespoon red curry paste', 'ounce coconut milk', 'vegetable broth', 'tablespoons lemon juice', 'teaspoon sea salt', 'tablespoon sesame oil', 'cup fresh cilantro', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'large yam inch pieces', 'large parsnip inch pieces', 'cup baby carrots', 'zucchini inch slices', 'bunch fresh asparagus inch pieces', 'cup red peppers inch pieces', 'cloves garlic', 'cup fresh basil', 'teaspoon kosher salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['pounds red potatoes', 'cup sour cream', 'cup mayonnaise', 'cup white sugar', 'bunch green onions', 'cup Cheddar cheese', 'tablespoon real bacon bits', 'ingredients', 'ingredients'],
    ['ounce cheese tortellini', 'cup extra virgin olive oil', 'cup lemon juice', 'cup red wine vinegar', 'tablespoons fresh parsley', 'teaspoon oregano', 'teaspoon salt', 'eggs', 'pound baby spinach leaves', 'cup feta cheese', 'cup red onion', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoon salt', 'cup unsalted butter', 'ounce package cream cheese', 'cup sour cream', 'cup white sugar', 'tablespoon ground cinnamon', 'cup walnuts', 'cup raisins', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'tablespoons olive oil', 'teaspoon parsley', 'teaspoon basil', 'teaspoon oregano', 'cloves garlic', 'teaspoon salt', 'teaspoons lemon juice', 'teaspoons white cooking wine', 'skinless boneless chicken breast halves', 'pound fresh asparagus thirds', 'cup fresh mushrooms', 'ingredients', 'ingredients'],
    ['acorn squash seeded', 'tablespoons butter', 'cup diced celery', 'cup onion', 'cup fresh mushrooms', 'teaspoon salt', 'pinch ground black pepper', 'teaspoon parsley', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['tart green apples', 'cup brown sugar', 'tablespoons butter', 'teaspoons ground cinnamon', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'teaspoon ground allspice', 'teaspoon salt', 'cup butter', 'cups white sugar', 'eggs', 'teaspoon baking soda', 'tablespoon warm water', 'teaspoon vanilla extract', 'apples', 'cup raisins', 'cup confectioners sugar', 'ingredients', 'ingredients'],
    ['cups cabbage', 'carrot', 'cup fresh bean sprouts', 'celery stalk', 'tablespoons onion optional', 'ounce shrimp', 'tablespoons soy sauce', 'teaspoon garlic powder', 'black pepper', 'egg', 'cornstarch', 'egg roll wrappers', 'vegetable oil frying', 'ingredients', 'ingredients'],
    ['cup uncooked orzo pasta', 'cup green olives', 'cup feta cheese', 'tablespoons fresh parsley', 'tablespoons fresh dill', 'ripe tomato', 'cup virgin olive oil', 'cup lemon juice', 'salt pepper', 'ingredients', 'ingredients'],
    ['pounds pork stew meat', 'cup vegetable oil', 'large yellow onions', 'cloves garlic', 'tablespoon sea salt', 'pepper', 'tablespoon ground cumin', 'quarts chicken broth', 'fresh poblano chile peppers', 'fresh jalapeno peppers', 'yellow bell peppers', 'fresh tomatillos husks', 'bunch cilantro leaves', 'ingredients', 'ingredients'],
    ['cup vegetable oil', 'eggs', 'ounce pumpkin puree', 'teaspoon vanilla extract', 'cups white sugar', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon ground nutmeg', 'teaspoon ground allspice', 'teaspoon ground cinnamon', 'teaspoon ground cloves', 'teaspoon salt', 'cup walnuts optional', 'ingredients', 'ingredients'],
    ['pounds ground beef', 'onion', 'cups mixed vegetables', 'ounce cream chicken soup', 'ounce frozen potato rounds', 'ingredients', 'ingredients'],
    ['potatoes cubed', 'large onions', 'pounds cheese food eg Velveeta', 'ham', 'ground black pepper', 'tablespoons purpose flour', 'cup milk', 'ingredients', 'ingredients'],
    ['fresh peaches', 'teaspoon almond extract', 'cup purpose flour', 'cup white sugar', 'cup brown sugar', 'teaspoon ground cinnamon', 'teaspoon salt', 'cup butter', 'ingredients', 'ingredients'],
    ['Apple Filling', 'large Granny Smith apples', 'cup white sugar', 'tablespoon purpose flour', 'teaspoon ground cinnamon', 'tablespoon lemon juice', 'cup water', 'Crumble', 'cups purpose flour', 'cup brown sugar', 'cup quick cooking oats', 'cup butter', 'Caramel Sauce', 'ounce package caramels', 'ounce milk', 'ingredients', 'ingredients'],
    ['cup balsamic vinegar', 'tablespoon olive oil', 'tablespoon Dijon mustard more', 'clove garlic more', 'salt pepper', 'large skinless boneless chicken breast halves', 'pint cherry tomatoes', 'lemon', 'ingredients', 'ingredients'],
    ['pounds chicken drumettes', 'large potatoes chunks', 'carrots inch pieces', 'large onion pieces', 'cloves garlic', 'cup water', 'cup soy sauce', 'tablespoons white sugar', 'tablespoons gochujang Korean hot pepper paste', 'ingredients', 'ingredients'],
    ['inch deep dish pie crust', 'large tomatoes', 'cup fresh basil', 'green onions', 'pound bacon', 'teaspoon garlic powder', 'teaspoon oregano', 'teaspoon red pepper', 'cups Cheddar cheese', 'cup mayonnaise', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'pound round steak inch pieces', 'salt', 'ground black pepper', 'large onion', 'cups fresh broccoli florets', 'ounce cream broccoli soup', 'cup water', 'tablespoons soy sauce', 'ingredients', 'ingredients'],
    ['cup butter', 'cup onion', 'large mushrooms', 'clove garlic', 'tablespoons purpose flour', 'cup chicken stock', 'teaspoon celery salt', 'teaspoon white pepper', 'cup white wine', 'cups Monterey Jack cheese', 'cup purpose flour', 'salt pepper', 'skinless boneless chicken breast halves thin', 'ingredients', 'ingredients'],
    ['eggs', 'cups white sugar', 'cup vegetable oil', 'teaspoons vanilla extract', 'cups purpose flour', 'teaspoons powder', 'teaspoons ground cinnamon', 'teaspoons ground nutmeg', 'cups summer squash', 'ingredients', 'ingredients'],
    ['cup soy sauce', 'cup brown sugar', 'teaspoon ground ginger', 'red pepper flakes', 'tablespoons water', 'tablespoons cornstarch', 'teaspoons vegetable oil', 'skinless boneless chicken breast halves chunks', 'onion', 'broccoli florets', 'ingredients', 'ingredients'],
    ['ounce bag mixed field greens', 'cup red onion optional', 'Bosc pear', 'cup pecans', 'cup blue cheese', 'cup maple syrup', 'cup apple cider vinegar', 'cup mayonnaise', 'tablespoons brown sugar', 'teaspoon salt', 'teaspoon black pepper', 'cup walnut oil', 'ingredients', 'ingredients'],
    ['cup vegetable oil', 'cups granulated sugar', 'eggs', 'cups pears', 'cup pecans', 'teaspoons vanilla extract', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon baking powder', 'teaspoon salt', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'ingredients', 'ingredients'],
    ['ounce cream cheese', 'onion', 'cups Parmesan cheese', 'cup mayonnaise', 'ingredients', 'ingredients'],
    ['pounds corned beef brisket', 'black peppercorns', 'teaspoon garlic powder', 'onion whole', 'bay leaves', 'pinch salt', 'small head cabbage wedges', 'large potatoes', 'large carrots', 'cup fresh parsley', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['sweet potatoes', 'ounce cans tomatoes chili seasonings', 'ounce tomato sauce', 'cup sweet onion', 'cup celery', 'cup water', 'tablespoon chili powder', 'teaspoon ground cumin', 'teaspoon ground cinnamon', 'pinch salt', 'pinch ground black pepper', 'cayenne pepper', 'garlic powder', 'onion powder', 'pound ground turkey', 'pound ground beef', 'ounce black beans', 'cup corn', 'ingredients', 'ingredients'],
    ['ounce beans', 'ounce package taco seasoning mix', 'large tomato', 'cup guacamole', 'cup sour cream room temperature', 'cup sharp Cheddar cheese', 'cup green onions', 'cup black olives', 'ingredients', 'ingredients'],
    ['butternut squash seeded', 'tablespoons butter', 'medium onion', 'leek', 'cloves garlic', 'fluid ounce cans chicken broth', 'large russet potatoes', 'teaspoon cayenne pepper', 'teaspoon ground allspice', 'teaspoon ground nutmeg', 'teaspoon ground ginger', 'salt pepper', 'cup sherry wine', 'cup half cream', 'cup sour cream optional', 'ingredients', 'ingredients'],
    ['inch long pickling cucumbers', 'cups water', 'cups white vinegar', 'cups fresh dill weed', 'cup white sugar', 'cloves garlic', 'tablespoons coarse salt', 'tablespoon pickling spice', 'teaspoons dill seed', 'teaspoon red pepper flakes', 'sprigs fresh dill weed', 'ingredients', 'ingredients'],
    ['ounce bag baby spinach leaves', 'eggs', 'cup mushrooms', 'strips bacon', 'ounces Swiss cheese', 'cup almonds', 'tablespoon olive oil', 'large shallot', 'teaspoon garlic', 'cup white wine vinegar', 'cup Dijon mustard', 'cup honey', 'strips bacon', 'salt pepper', 'ingredients', 'ingredients'],
    ['pound ground beef', 'ounce cream mushroom soup', 'fluid ounces milk', 'teaspoon garlic salt', 'ounce French style green beans', 'ounce package tater tots', 'ingredients', 'ingredients'],
    ['eggs', 'pinch ground black pepper', 'ounce cream chicken soup', 'ounces sour cream', 'pound package hash brown potatoes', 'cups ham cubed', 'onion', 'cup Cheddar cheese', 'cup butter', 'ingredients', 'ingredients'],
    ['white potatoes', 'chicken broth', 'leeks bulb', 'cup butter', 'cup white wine', 'salt', 'teaspoon white pepper', 'ingredients', 'ingredients'],
    ['cup butter', 'cup white sugar', 'tablespoon orange zest', 'teaspoon vanilla extract', 'eggs', 'egg white', 'cups purpose flour', 'teaspoons powder', 'teaspoon salt', 'ounces white chocolate', 'cup cranberries', 'cups pistachio nuts', 'ingredients', 'ingredients'],
    ['cup shortening', 'cup white sugar', 'cup pumpkin puree', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoon salt', 'teaspoon baking soda', 'teaspoon baking powder', 'teaspoon ground cinnamon', 'cups butterscotch chips', 'ingredients', 'ingredients'],
    ['cups green tomatoes', 'tablespoon salt', 'cup butter', 'cups white sugar', 'eggs', 'cups purpose flour', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'teaspoon baking soda', 'teaspoon salt', 'cup raisins', 'cup walnuts', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'small onion', 'apples', 'tablespoons purpose flour', 'tablespoon curry powder', 'skinless boneless chicken breasts', 'cup hot chicken broth', 'cup milk', 'salt pepper', 'ingredients', 'ingredients'],
    ['pound skinless boneless chicken breast halves', 'cup pecans', 'cup red wine vinegar', 'cup white sugar', 'cup vegetable oil', 'onion', 'teaspoon ground mustard', 'teaspoon salt', 'teaspoon ground white pepper', 'Bibb lettuce', 'cup fresh strawberries', 'ingredients', 'ingredients'],
    ['pound linguine pasta', 'tablespoon olive oil', 'cloves garlic', 'medium shrimp deveined', 'ounce roasted tomatoes', 'tablespoon fresh basil', 'salt pepper', 'cup feta cheese', 'ingredients', 'ingredients'],
    ['bratwurst links inch slices', 'medium potatoes cubed', 'tablespoon minced onion', 'ounce green beans', 'small red bell pepper seeded', 'cups Cheddar cheese', 'ounce cream mushroom soup', 'cup water', 'ingredients', 'ingredients'],
    ['tablespoons white wine vinegar', 'fluid ounces white wine', 'cup shallots', 'tablespoon wasabi paste', 'tablespoon soy sauce', 'cup unsalted butter', 'salt black pepper', 'tablespoon olive oil', 'cup cilantro leaves', 'ounce fresh tuna steaks inch thick', 'ingredients', 'ingredients'],
    ['cup onions', 'cup celery', 'cup carrots', 'teaspoons garlic', 'tablespoons vegetable oil', 'cups water', 'tomatoes', 'ounce tomatoes juice', 'ounce chicken broth', 'cup uncooked barley', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'large eggplant', 'egg', 'tablespoon water', 'cup dry bread crumbs', 'tomato', 'cup Parmesan cheese', 'cup Italian style salad dressing', 'ingredients', 'ingredients'],
    ['ounce yellow cake mix', 'cup butter', 'egg', 'ounce pumpkin', 'cup brown sugar', 'cup milk', 'eggs', 'tablespoons pumpkin pie spice', 'cup butter', 'cup white sugar', 'cup walnuts', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'tablespoons olive oil', 'eggs', 'cup Italian style dry bread crumbs', 'ounce chicken broth', 'ounce asparagus', 'provolone cheese', 'ingredients', 'ingredients'],
    ['large cucumbers seeded', 'cup red wine vinegar', 'tablespoon white sugar', 'teaspoon salt', 'large tomatoes seeded', 'cup red onion', 'cup fresh mint leaves', 'tablespoons olive oil', 'salt pepper', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoon baking powder', 'teaspoon baking soda', 'teaspoon salt', 'cup butter', 'cup brown sugar', 'cup white sugar', 'egg', 'teaspoons vanilla extract', 'cup milk chocolate chips', 'cup coconut', 'cup pecans', 'ingredients', 'ingredients'],
    ['fluid ounce frozen cranberry juice concentrate', 'fluid ounce frozen pink lemonade concentrate', 'ounce limeade concentrate', 'milliliter white wine', 'liter carbonated water', 'milliliter bottles champagne', 'lemon garnish', 'cup fresh mint garnish', 'ingredients', 'ingredients'],
    ['cup water', 'cup white sugar', 'cinnamon stick', 'orange', 'whole cloves', 'milliliter red wine', 'ingredients', 'ingredients'],
    ['ounce yellow cake mix', 'cup white sugar', 'cup vegetable oil', 'cup pumpkin', 'cup water', 'teaspoon ground cinnamon', 'eggs', 'ingredients', 'ingredients'],
    ['head romaine lettuce', 'ounce cans mandarin oranges', 'green onions', 'tablespoons white sugar', 'cup almonds', 'cup red wine vinegar', 'cup olive oil', 'tablespoon white sugar', 'teaspoon red pepper flakes', 'ground black pepper', 'ingredients', 'ingredients'],
    ['potatoes cubed', 'cup butter', 'cup purpose flour', 'cups milk', 'cup onion', 'ounce container sour cream', 'cup Cheddar cheese', 'salt pepper', 'ingredients', 'ingredients'],
    ['tablespoon onion', 'cup butter', 'cups fresh bread crumbs', 'cups apples', 'cup celery', 'teaspoons fresh parsley', 'teaspoon salt', 'inch thick pork chops', 'salt pepper', 'tablespoon vegetable oil', 'ingredients', 'ingredients'],
    ['cups apple cider', 'cup real maple syrup', 'cinnamon sticks', 'whole cloves', 'whole allspice berries', 'orange peel strips', 'lemon peel strips', 'ingredients', 'ingredients'],
    ['cup uncooked white rice', 'cup water', 'teaspoon vegetable oil', 'ounces turkey kielbasa', 'onion', 'carrot', 'teaspoon red pepper flakes', 'cups water', 'cups sodium chicken broth', 'cup dry lentils', 'ounce bag fresh spinach', 'ingredients', 'ingredients'],
    ['pound lean ground beef', 'ounce package taco seasoning mix', 'ounce cans beans', 'inch flour tortillas', 'cups Cheddar cheese', 'green onions', 'roma plum tomatoes', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'cup purpose flour', 'teaspoon salt', 'teaspoon baking powder', 'cup milk', 'cup white sugar', 'tablespoon cornstarch', 'cup water', 'cups fresh cherries', 'cup white sugar', 'ingredients', 'ingredients'],
    ['cup butter', 'cups white sugar', 'cups mashed sweet potatoes', 'teaspoon vanilla extract', 'eggs', 'cups purpose flour', 'teaspoons powder', 'teaspoon ground cinnamon', 'teaspoon baking soda', 'teaspoon ground nutmeg', 'teaspoon salt', 'cup confectioners sugar', 'teaspoons orange juice', 'tablespoons orange zest', 'ingredients', 'ingredients'],
    ['cooking spray', 'spaghetti squash seeded', 'cup butter margarine', 'small onion', 'cloves garlic', 'salt pepper', 'ingredients', 'ingredients'],
    ['ounce dry cornbread mix', 'cup butter', 'onions', 'green bell pepper', 'stalks celery', 'pound pork sausage', 'slices white bread', 'teaspoons sage', 'teaspoon thyme', 'teaspoon poultry seasoning', 'teaspoon salt', 'teaspoon ground black pepper', 'cup fresh parsley', 'eggs', 'cups chicken stock', 'ingredients', 'ingredients'],
    ['inch flour tortillas', 'ounces white Cheddar cheese', 'ounces Monterey Jack cheese', 'ounces Cheddar cheese', 'tomato', 'cup red bell pepper', 'cup green onions', 'cup black beans', 'tablespoons hot salsa', 'teaspoon chili powder', 'ingredients', 'ingredients'],
    ['ounce packages cheese tortellini', 'pound bacon', 'cups broccoli', 'pint grape tomatoes', 'green onions', 'cup coleslaw dressing', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'ounce bag spinach leaves', 'teaspoons garlic salt', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['cooking spray', 'tablespoons hot pepper sauce', 'cup olive oil', 'tablespoons garlic powder', 'tablespoon black pepper', 'tablespoon paprika', 'teaspoons salt', 'potatoes inch cubes', 'pounds skinless boneless chicken breast halves inch cubes', 'cups Mexican cheese blend such Great Value Fiesta Blend', 'cup cooked bacon', 'cup green onions', 'ingredients', 'ingredients'],
    ['large sweet potatoes diced', 'tablespoons butter', 'cup white sugar', 'teaspoon ground cinnamon', 'teaspoon ground allspice', 'Granny Smith apple', 'cup milk', 'ingredients', 'ingredients'],
    ['pounds Scotch fillet eye steaks', 'cup soy sauce', 'tablespoons white sugar', 'tablespoons sesame seeds', 'tablespoons sesame oil', 'shallots', 'cloves garlic', 'tablespoons mirin Japanese sweet wine', 'ingredients', 'ingredients'],
    ['bacon', 'diced potatoes', 'teaspoon garlic salt', 'teaspoon seasoning salt', 'teaspoon black pepper', 'eggs', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['ounce tomatoes', 'ounce tomato vegetable juice cocktail', 'ounce chicken broth', 'fresh basil leaves thin strips', 'cup heavy cream', 'cup butter margarine', 'ingredients', 'ingredients'],
    ['cup butter', 'cup white sugar', 'egg', 'cup milk', 'cups purpose flour', 'teaspoons powder', 'teaspoon salt', 'cups blueberries', 'cup brown sugar', 'tablespoons purpose flour', 'teaspoon ground cinnamon', 'cup pecans', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['eggs', 'cups white sugar', 'fresh peaches', 'cups heavy cream', 'cups half cream', 'teaspoons vanilla extract', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['chicken thighs', 'teaspoons oregano', 'salt pepper', 'cup olive oil', 'lemon', 'ingredients', 'ingredients'],
    ['pound package hash brown potatoes', 'cup green onions', 'cups Cheddar cheese', 'ounce cream potato soup', 'cup butter', 'cups sour cream', 'salt pepper', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'teaspoon Spanish paprika', 'teaspoon seasoned salt', 'teaspoon ground black pepper', 'beef chuck inch cubes', 'tablespoon olive oil', 'onion', 'ounce package mushrooms', 'potatoes', 'carrots', 'stalks celery', 'cloves garlic', 'cup Marsala wine', 'tablespoon Worcestershire sauce', 'ounce cans organic beef broth', 'ounce tomatoes', 'ounce dry onion soup mix', 'teaspoon Spanish paprika', 'salt black pepper', 'tablespoons cornstarch optional', 'tablespoons water optional', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'cloves garlic', 'cups eggplant peel', 'tablespoon water', 'cup Parmesan cheese', 'cup fresh parsley', 'eggs', 'cup bread crumbs', 'ingredients', 'ingredients'],
    ['large shrimp', 'cup fresh lime juice', 'roma plum tomatoes', 'white onion', 'cup fresh cilantro', 'tablespoon Worcestershire sauce', 'tablespoon ketchup', 'teaspoon hot pepper sauce', 'salt pepper', 'avocado diced', 'saltine crackers', 'ingredients', 'ingredients'],
    ['small cucumbers', 'small red onion', 'large tomato', 'tablespoons mayonnaise', 'tablespoon white vinegar', 'teaspoon salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['boneless skinless chicken breast halves', 'Italian dressing', 'tablespoon Greek seasoning optional', 'tablespoon butter', 'tablespoon white cooking wine', 'tablespoon Worcestershire sauce', 'teaspoon garlic salt', 'ounce package fresh mushrooms', 'salt', 'ounce cans whole green chili peppers', 'ounces Havarti cheese dill', 'ingredients', 'ingredients'],
    ['cup olive oil', 'skinless boneless chicken breast halves strips', 'salt pepper', 'red bell pepper', 'yellow bell pepper', 'orange bell pepper', 'medium onion', 'large cloves garlic', 'tablespoon basil', 'cup balsamic vinegar', 'ingredients', 'ingredients'],
    ['ounce envelope dry onion soup mix', 'pork chops', 'cup purpose flour', 'cup olive oil frying', 'ingredients', 'ingredients'],
    ['cup butter', 'cup white sugar', 'egg', 'cup purpose flour', 'teaspoon baking soda', 'teaspoon ground cinnamon', 'cup apples', 'cup walnuts', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'spears fresh asparagus', 'slices prosciutto', 'ingredients', 'ingredients'],
    ['cup walnut halves', 'ounces salad greens arugula', 'large navel oranges', 'cup red onion', 'cup olive oil', 'cup vegetable oil', 'cup orange juice', 'cup white sugar', 'tablespoons balsamic vinegar', 'teaspoons Dijon mustard', 'teaspoon oregano', 'teaspoon ground black pepper', 'cup Gorgonzola cheese', 'ingredients', 'ingredients'],
    ['cups apples', 'cup water', 'teaspoon ground cinnamon', 'teaspoon ground cloves', 'cup white sugar', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breasts', 'cloves garlic', 'tablespoons olive oil', 'tablespoons red wine vinegar', 'tablespoon Cajun style seasoning', 'cup green onion', 'ounce package fresh mushrooms', 'Swiss cheese', 'ingredients', 'ingredients'],
    ['ounces ground beef', 'ounce package taco seasoning mix', 'head iceberg lettuce', 'red onion', 'bunch green onions', 'ounce pinto beans', 'ounce kidney beans', 'large tomatoes', 'avocados cubed', 'ounces Cheddar cheese', 'ounce package corn chips', 'ounce bottle Catalina salad dressing', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'pounds fresh mushrooms', 'teaspoon Italian seasoning', 'cup dry white wine', 'cloves garlic', 'salt pepper', 'tablespoons fresh chives', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cup butter', 'ounce cream cheese', 'cup white sugar', 'ounce instant lemon pudding mix', 'cups milk', 'ounce container topping', 'ingredients', 'ingredients'],
    ['cup quick cooking oats', 'cup orange juice', 'cups purpose flour', 'cup white sugar', 'teaspoons powder', 'teaspoon salt', 'teaspoon baking soda', 'cup vegetable oil', 'egg', 'cup fresh blueberries', 'teaspoons white sugar', 'teaspoon ground cinnamon', 'ingredients', 'ingredients'],
    ['apples', 'ounce package caramels', 'tablespoons milk', 'ingredients', 'ingredients'],
    ['head broccoli florets', 'ounces whole button mushrooms', 'small zucchini', 'cups carrots', 'cup olive oil', 'salt pepper', 'cups water', 'cup milk', 'cup butter', 'ounce instant mashed potato flakes', 'ounce package corn tortillas', 'enchilada sauce', 'ounces Cheddar cheese', 'ingredients', 'ingredients'],
    ['cups peas', 'cup water', 'cup sesame oil', 'cups pork loin', 'green onions', 'large carrot', 'cloves garlic', 'ounces cauliflower', 'tablespoons soy sauce', 'eggs', 'ingredients', 'ingredients'],
    ['cups plain yogurt', 'cloves garlic', 'teaspoon salt', 'teaspoon ground black pepper', 'cup fresh mint leaves', 'large cucumber', 'ingredients', 'ingredients'],
    ['carrots', 'tablespoons olive oil', 'cup honey', 'salt black pepper', 'ingredients', 'ingredients'],
    ['cup margarine', 'cup white sugar', 'eggs', 'ounce pineapple', 'white bread', 'ingredients', 'ingredients'],
    ['ounce package spinach', 'cups herb dry bread stuffing mix', 'cup Parmesan cheese', 'teaspoons garlic powder', 'teaspoon ground black pepper', 'teaspoon Italian seasoning', 'cup butter', 'eggs', 'ingredients', 'ingredients'],
    ['ounce package frozen French fries', 'ounce package broccoli', 'cups cubed ham', 'ounce cream mushroom soup', 'ounce', 'cup mayonnaise', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoons powder', 'teaspoons soda', 'teaspoon salt', 'teaspoons ground cinnamon', 'cups white sugar', 'eggs', 'cups vegetable oil', 'cups carrots', 'cup walnuts', 'ingredients', 'ingredients'],
    ['cup figs', 'cup water', 'tablespoon olive oil', 'tablespoons balsamic vinegar', 'teaspoon rosemary', 'teaspoon thyme', 'teaspoon cayenne pepper', 'cup kalamata olives', 'cloves garlic', 'salt pepper', 'cup walnuts', 'ounce package cream cheese', 'ingredients', 'ingredients'],
    ['ounce uncooked pasta shells', 'teaspoon olive oil', 'onion', 'cloves garlic', 'zucchini', 'teaspoon oregano', 'salt black pepper', 'teaspoon red pepper flakes', 'cup chicken broth', 'cup cooked chicken', 'ounce red peppers', 'tablespoons cream cheese', 'cup fresh basil leaves', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['pound ground beef', 'small onion', 'egg', 'tablespoon ketchup', 'tablespoon Worcestershire sauce', 'ounce whole kernel corn', 'inch unbaked pie crust', 'cups mashed potatoes', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['cup dark brown sugar', 'cups oat bran', 'cups purpose flour', 'teaspoons powder', 'teaspoons soda', 'teaspoon salt', 'eggs', 'cup chilled applesauce', 'tablespoons vegetable oil', 'ingredients', 'ingredients'],
    ['ounces cream cheese', 'cup white sugar', 'teaspoons vanilla extract', 'pint heavy whipping cream', 'ounce ladyfinger cookies', 'ounce frozen strawberries', 'tablespoon cornstarch', 'ingredients', 'ingredients'],
    ['eggs', 'cups white sugar', 'teaspoons vanilla extract', 'cup vegetable oil', 'cups peaches', 'cups purpose flour', 'teaspoon baking powder', 'teaspoon salt', 'teaspoon baking soda', 'teaspoons ground cinnamon', 'cup walnuts optional', 'ingredients', 'ingredients'],
    ['ounce package egg noodles', 'stick butter', 'medium head green cabbage', 'salt pepper', 'ingredients', 'ingredients'],
    ['eggplant inch rounds', 'tomato', 'cup fat free Parmesan cheese', 'ingredients', 'ingredients'],
    ['frozen bananas skins chunks', 'cup frozen blueberries', 'cup orange juice', 'tablespoon honey optional', 'teaspoon vanilla extract optional', 'ingredients', 'ingredients'],
    ['eggs', 'cups white sugar', 'cups strawberries', 'cup bananas', 'tablespoon orange zest', 'cup vegetable oil', 'cups purpose flour', 'teaspoon baking soda', 'teaspoons ground cinnamon', 'teaspoon salt', 'teaspoon ground nutmeg', 'cup walnuts', 'ingredients', 'ingredients'],
    ['eggs', 'cups white sugar', 'teaspoons vanilla extract', 'cup vegetable oil', 'teaspoons ground cinnamon', 'teaspoon ground nutmeg', 'ounce pineapple', 'cups zucchini', 'cups purpose flour', 'teaspoon salt', 'teaspoons soda', 'teaspoons powder', 'ingredients', 'ingredients'],
    ['cups corn flakes', 'tablespoon chili powder', 'ounce packet dry fajita seasoning', 'cup red bell pepper', 'cup yellow bell pepper', 'cup orange bell pepper', 'cup fresh mushrooms', 'medium red onion', 'skinless boneless chicken breast halves thin', 'cup Cheddar cheese', 'cup salsa', 'toothpicks', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'eggs', 'cup Italian seasoned dry bread crumbs', 'eggplant inch rounds', 'salt', 'medium onion', 'tomatoes', 'cup Parmesan cheese', 'cup feta cheese', 'slices Muenster cheese strips', 'ingredients', 'ingredients'],
    ['tablespoon butter', 'tablespoon olive oil', 'pound portobello mushrooms', 'teaspoon salt', 'cups low sodium chicken broth', 'ounce package pepper Boursin cheese', 'pound uncooked pasta shells', 'pound fresh asparagus', 'ingredients', 'ingredients'],
    ['ounce cans garbanzo beans', 'cup water', 'cup tahini sesame seed paste', 'cup fresh lemon juice', 'tablespoons olive oil', 'chipotle pepper adobo sauce', 'cloves garlic', 'teaspoons cumin', 'ounce jar red bell peppers', 'oil sun tomatoes', 'cup cilantro', 'teaspoon salt', 'ground black pepper', 'ingredients', 'ingredients'],
    ['ounce beans', 'small onion', 'cloves garlic', 'tablespoons fresh cilantro', 'teaspoon parsley', 'teaspoons ground cumin', 'teaspoon ground turmeric', 'teaspoon baking powder', 'cup dry bread crumbs', 'teaspoon salt', 'teaspoon black peppercorns', 'quart vegetable oil frying', 'ingredients', 'ingredients'],
    ['beef shank bone', 'tablespoon vegetable oil', 'teaspoons salt', 'teaspoons black pepper', 'onion', 'ounce tomatoes', 'beef broth', 'cups water', 'medium carrot', 'cup fresh cilantro', 'potato optional', 'ears corn thirds', 'chayotes optional', 'medium head cabbage wedges', 'cup jalapenos', 'cup onion', 'cup fresh cilantro', 'limes wedges', 'radishes', 'ingredients', 'ingredients'],
    ['cup butter', 'cups purpose flour', 'cup white sugar', 'tablespoon baking powder', 'teaspoon salt', 'cup milk', 'teaspoon vanilla extract', 'cups fresh peaches', 'ingredients', 'ingredients'],
    ['cup dry lentils', 'cup water', 'ounces tomatoes', 'cup onion', 'stalks celery leaves', 'teaspoon curry powder', 'cloves garlic', 'salt', 'ground black pepper', 'ingredients', 'ingredients'],
    ['cup raisins', 'egg', 'cup milk', 'cups bran cereal such Kellogg All Bran Original Cereal', 'cup whole wheat flour', 'teaspoon baking soda', 'teaspoon ground cinnamon', 'tablespoons butter', 'apple', 'ripe banana', 'cup honey', 'ingredients', 'ingredients'],
    ['teaspoons olive oil', 'small onion', 'cloves garlic', 'carrots', 'small summer squash', 'small zucchini', 'cups oats', 'cup Cheddar cheese', 'egg', 'tablespoon soy sauce', 'cups purpose flour', 'ingredients', 'ingredients'],
    ['cups uncooked egg noodles', 'ounce cans tuna', 'cup celery', 'cup green onions', 'cup sour cream', 'teaspoons prepared mustard', 'cup mayonnaise', 'teaspoon thyme', 'teaspoon salt', 'small zucchini', 'cup Monterey Jack cheese', 'tomato', 'ingredients', 'ingredients'],
    ['tablespoons canola oil', 'large eggplant', 'eggs', 'cups dry bread crumbs', 'ingredients', 'ingredients'],
    ['medium zucchini', 'white bread small pieces', 'cup bacon bits', 'tablespoon black olives', 'jalapeno pepper', 'tablespoons green chile peppers', 'cup onion', 'cup tomato', 'tablespoons sharp Cheddar cheese', 'basil', 'seasoned salt', 'ground black pepper', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'onion', 'boneless pork chops', 'ounce jar pasta sauce', 'ounce tomatoes', 'green bell pepper seeded strips', 'ounce fresh mushrooms', 'large cloves garlic', 'teaspoon Italian seasoning', 'teaspoon basil', 'cup dry white wine', 'mozzarella cheese', 'ingredients', 'ingredients'],
    ['medium butternut squash', 'tablespoons olive oil', 'onion', 'pound carrots diced', 'cloves garlic', 'inch piece fresh ginger', 'cups water', 'salt pepper', 'pinch ground cinnamon', 'cup heavy cream optional', 'ingredients', 'ingredients'],
    ['ounce green enchilada sauce', 'cups water', 'cube vegetable bouillon', 'teaspoon garlic powder', 'teaspoon chili powder', 'teaspoon ground cumin', 'ounce pinto beans', 'ounce tomatoes', 'cup frozen corn', 'cup vegetarian chicken substitute diced optional', 'inch corn tortillas strips', 'tablespoon fresh cilantro', 'salt pepper', 'ingredients', 'ingredients'],
    ['tablespoon butter', 'clove garlic', 'pork chops', 'salt pepper', 'ounce mushrooms', 'cup dry sherry', 'ounce broth', 'tablespoons cornstarch', 'tablespoons water', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'tablespoons butter', 'onion', 'cup beer', 'ounce tomato soup', 'teaspoon curry powder', 'teaspoon basil', 'teaspoon ground black pepper', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['apples', 'cup maple syrup', 'cup butter', 'cup brown sugar', 'cup purpose flour', 'pinch salt', 'cup oats', 'ingredients', 'ingredients'],
    ['pound ground beef', 'pound ground pork', 'onion', 'teaspoon salt', 'black pepper', 'teaspoon fresh parsley', 'cup brown rice', 'teaspoons garlic salt', 'ounce cans tomato soup', 'ounces sauerkraut', 'ounce tomatoes', 'medium head cabbage', 'bacon', 'tablespoons white sugar', 'cups water', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'onion', 'mushrooms', 'salt black pepper', 'ounce jar artichoke hearts', 'ounce package cream cheese', 'tablespoons sour cream', 'cup Italian cheese blend', 'tablespoons Parmesan cheese', 'teaspoon garlic salt', 'ingredients', 'ingredients'],
    ['ounce container plain low fat yogurt', 'English cucumber peel', 'clove garlic', 'tablespoons fresh lemon juice', 'tablespoons extra-virgin olive oil', 'teaspoons lemon zest', 'tablespoons fresh dill', 'tablespoon salt', 'tablespoon black pepper', 'ingredients', 'ingredients'],
    ['cup butter', 'cup white sugar', 'cup chilled applesauce', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon ground cinnamon', 'teaspoon ground cloves', 'cup raisins', 'cup walnuts', 'ingredients', 'ingredients'],
    ['teaspoon salt', 'teaspoon black pepper', 'teaspoon paprika', 'pound top sirloin roast', 'cloves garlic', 'Yukon Gold potatoes', 'carrots inch pieces', 'large sweet onions', 'cup water', 'cup beef broth', 'cubes beef bouillon', 'bay leaf', 'large green bell peppers inch pieces', 'ingredients', 'ingredients'],
    ['small eggplant', 'tablespoon olive oil', 'cup mayonnaise', 'cloves garlic', 'inch French sandwich rolls', 'small tomato', 'cup feta cheese', 'cup fresh basil leaves', 'ingredients', 'ingredients'],
    ['large potatoes diced', 'ounce corned beef chunks', 'medium onion', 'cup beef broth', 'ingredients', 'ingredients'],
    ['onion', 'teaspoons garlic', 'red bell pepper', 'cup fresh mushrooms', 'tomato', 'carrots', 'ground chicken', 'egg', 'cup fresh bread crumbs', 'tablespoon Old Bay tm Seasoning', 'kosher salt', 'black pepper', 'ingredients', 'ingredients'],
    ['cup butter', 'tablespoons brown sugar', 'cups pretzels', 'ounce package strawberry Jell O r', 'cups water', 'cups strawberries', 'ounce package cream cheese', 'cup white sugar', 'ounce container topping', 'ingredients', 'ingredients'],
    ['medium sugar pumpkin', 'tablespoon vegetable oil', 'recipe pastry inch single crust pie', 'teaspoon ground ginger', 'teaspoon ground cinnamon', 'teaspoon salt', 'eggs', 'cup honey', 'cup milk', 'cup heavy whipping cream', 'ingredients', 'ingredients'],
    ['ounce active dry yeast', 'tablespoons white sugar', 'cup sweet potato puree', 'cup warm water degrees F C', 'tablespoons margarine', 'teaspoon salt', 'eggs', 'cups purpose flour', 'ingredients', 'ingredients'],
    ['eggs', 'cups white sugar', 'cup vegetable oil', 'cup unsweetened cocoa powder', 'teaspoons vanilla extract', 'cups zucchini', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon baking powder', 'teaspoon salt', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'teaspoon ground cloves', 'teaspoon ground cardamom', 'ingredients', 'ingredients'],
    ['pound broccoli florets', 'tablespoons Parmesan cheese', 'teaspoon brown sugar', 'tablespoons olive oil', 'teaspoon red pepper flakes', 'teaspoon kosher salt', 'teaspoon black pepper', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoons powder', 'teaspoon baking soda', 'teaspoon salt', 'cup white sugar', 'cup pecans', 'eggs', 'cup bananas', 'cup vegetable oil', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['cups diced pumpkin', 'eggplant inch rounds', 'tomatoes', 'pint ricotta cheese', 'ounces feta cheese', 'cup pesto', 'eggs', 'salt pepper', 'ounce tomato sauce', 'fresh pasta sheets', 'cups mozzarella cheese', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['apples', 'cups purpose flour', 'cups white sugar', 'teaspoons powder', 'teaspoon salt', 'eggs', 'cup vegetable oil', 'teaspoon ground cinnamon', 'ingredients', 'ingredients'],
    ['teaspoons ground paprika', 'tablespoons fresh minced garlic', 'teaspoons Italian seasoning blend', 'tablespoons fresh lemon juice', 'cup olive oil', 'teaspoon ground black pepper', 'teaspoons basil leaves', 'tablespoons brown sugar', 'large shrimp pound deveined', 'ingredients', 'ingredients'],
    ['cups brown rice', 'ounce kidney beans', 'ounce black beans', 'ounce whole kernel corn', 'small onion', 'green bell pepper', 'jalapeno peppers seeded diced', 'lime', 'cup cilantro leaves', 'teaspoon garlic', 'teaspoons ground cumin', 'salt', 'ingredients', 'ingredients'],
    ['ounce package angel hair pasta', 'tomatoes', 'cloves garlic', 'tablespoon olive oil', 'tablespoon fresh basil', 'tablespoon tomato paste', 'salt', 'ground black pepper', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['medium head cabbage', 'red onions strips', 'cup butter', 'ounce wide egg noodles', 'salt', 'ground black pepper', 'ingredients', 'ingredients'],
    ['ounce packages spinach', 'ounce cream mushroom soup', 'tablespoon butter', 'garlic salt', 'ingredients', 'ingredients'],
    ['avocado diced', 'lime', 'mango seeded diced', 'small red onion', 'habanero pepper seeded', 'tablespoon fresh cilantro', 'salt', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'tablespoons butter', 'pound Yukon Gold potatoes', 'pound fresh shiitake mushrooms', 'red bell pepper', 'small acorn squash', 'shallot', 'teaspoons garlic powder', 'pinch salt', 'pinch ground black pepper', 'cup kale', 'sprigs fresh sage', 'ingredients', 'ingredients'],
    ['ounce frozen cut green beans', 'cup extra virgin olive oil', 'salt', 'clove garlic', 'cup fresh cilantro', 'ingredients', 'ingredients'],
    ['tablespoons cornstarch', 'cup chicken broth', 'tablespoons soy sauce', 'teaspoon ground ginger', 'teaspoon hot pepper sauce', 'tablespoons vegetable oil', 'pound skinless boneless chicken breast meat strips', 'small onion', 'green bell pepper', 'ounce water chestnuts', 'cup cashews', 'ingredients', 'ingredients'],
    ['cups dry navy beans hour', 'cups ketchup', 'cups water', 'cup molasses', 'large onion', 'tablespoon dry mustard', 'tablespoon salt', 'slices thick cut bacon inch pieces', 'cup brown sugar', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'whole garlic cloves', 'tablespoons garlic cloves', 'pound whole boneless pork loin dry room temperature', 'Salt pepper', 'tablespoons fresh rosemary snip scissors', 'apricots similar amount favorite fruit', 'cup cranberries', 'Heavy kitchen string twine', 'cup apple jelly', 'cup port', 'cup chicken broth', 'tablespoons apple jelly', 'teaspoons cornstarch', 'ingredients', 'ingredients'],
    ['inch unbaked pie crust', 'cups fresh rhubarb', 'egg', 'cups white sugar', 'cup sour cream', 'cup purpose flour', 'cup purpose flour', 'cup brown sugar', 'cup butter', 'ingredients', 'ingredients'],
    ['ounce tri-color pasta', 'celery', 'pound imitation crabmeat', 'cup frozen green peas', 'cup mayonnaise', 'tablespoons white sugar', 'tablespoons white vinegar', 'tablespoons milk', 'teaspoon salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['teaspoon chili powder', 'teaspoon oregano', 'onion powder', 'cayenne pepper', 'garlic powder', 'salt pepper', 'cup butter', 'ears corn', 'ingredients', 'ingredients'],
    ['cups cubed potatoes', 'stalks celery', 'large carrot', 'cups water', 'tablespoons butter', 'cups ham', 'tablespoons green bell pepper', 'teaspoons onion', 'cup butter', 'tablespoons purpose flour', 'cup milk more', 'teaspoon salt', 'teaspoon ground black pepper', 'cup Cheddar cheese', 'cup dry bread crumbs', 'ingredients', 'ingredients'],
    ['teaspoon olive oil', 'onion', 'cloves garlic', 'sweet potato cubed', 'orange bell pepper seeded cubed', 'pound chicken breast', 'ounce tomatoes', 'cups water', 'teaspoon salt', 'tablespoons chili powder', 'teaspoon ground cumin', 'teaspoon oregano', 'teaspoon cocoa powder', 'teaspoon ground cinnamon', 'teaspoon red pepper flakes', 'tablespoons purpose flour', 'tablespoons water', 'cup frozen corn', 'ounce kidney beans', 'cup fresh cilantro', 'ingredients', 'ingredients'],
    ['tilapia fillets', 'tablespoons olive oil', 'tablespoons orange zest', 'cup fresh orange juice', 'salt pepper', 'crushed red pepper flakes', 'Salsa', 'mango seeded diced', 'small red onion', 'avocado diced', 'roma tomatoes', 'lime', 'jalapeno pepper seeded', 'tablespoon fresh ginger root', 'cup fresh cilantro', 'teaspoon kosher salt', 'ingredients', 'ingredients'],
    ['cup graham cracker crumbs', 'cup pecan sandies cookies', 'tablespoons butter', 'tablespoons white sugar', 'ounces cream cheese', 'cup white sugar', 'tablespoons lemon juice', 'cup heavy whipping cream', 'cup fresh strawberries optional', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'cups fresh mushrooms', 'cups spinach leaves', 'green onions', 'ounce package crescent rolls', 'ounce package herb lemon soup mix', 'cup half', 'eggs', 'cup Monterey Jack cheese', 'ingredients', 'ingredients'],
    ['head romaine lettuce', 'red onion', 'ounce black olives', 'green bell pepper', 'red bell pepper', 'large tomatoes', 'cucumber', 'cup feta cheese', 'tablespoons olive oil', 'teaspoon oregano', 'lemon', 'ground black pepper', 'ingredients', 'ingredients'],
    ['pound ground beef', 'medium onion', 'teaspoon red pepper flakes', 'tablespoon ground cumin', 'ounce cans tomato soup', 'ounce cans chicken broth', 'ounce tomatoes', 'tablespoons chili powder', 'teaspoon ground black pepper', 'teaspoon salt', 'ounce cans pinto beans optional', 'cayenne pepper', 'ingredients', 'ingredients'],
    ['ounces dry fettuccine pasta', 'cloves garlic', 'sweet onion wedges', 'tablespoons fresh oregano leaves', 'tablespoons olive oil', 'medium tomatoes', 'tablespoons fresh basil', 'salt pepper', 'cup spinach leaves', 'pound shrimp deveined', 'ounces fresh mozzarella cheese', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'cup onion', 'cloves garlic', 'teaspoon oregano', 'ounce cans chicken broth', 'ounce Mexican style stewed tomatoes', 'medium zucchini inch slices', 'medium yellow squash inch slices', 'ounce whole kernel corn', 'ounce green chile peppers', 'ounces cheese food', 'teaspoon black pepper', 'cup fresh cilantro', 'ingredients', 'ingredients'],
    ['ounce inch thick filet mignon steaks', 'teaspoons olive oil', 'teaspoon onion powder', 'Salt pepper', 'tablespoons shallot', 'bacon', 'ingredients', 'ingredients'],
    ['ounce fillets halibut', 'lime wedges', 'salt pepper', 'cloves garlic', 'cup fresh cilantro', 'tablespoon fresh lime juice', 'tablespoons butter', 'tablespoon olive oil', 'ingredients', 'ingredients'],
    ['ounce whole kernel corn', 'ounce cream style corn', 'ounce container sour cream', 'cup butter', 'eggs', 'ounce dry corn muffin mix', 'ingredients', 'ingredients'],
    ['cup buttermilk', 'teaspoon baking soda', 'cups white sugar', 'cup butter', 'cup vegetable oil', 'cup shortening', 'egg yolks', 'teaspoon vanilla extract', 'egg whites', 'cups purpose flour', 'ounce package coconut', 'cup pecans', 'ounce package cream cheese', 'cup margarine', 'cups confectioners sugar', 'teaspoon vanilla extract', 'cup pecans', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'pounds seeded butternut squash inch chunks', 'large onion dice', 'tablespoon butter', 'pinch sugar', 'large garlic cloves', 'teaspoons cinnamon', 'teaspoon ground ginger', 'teaspoon ground cloves', 'teaspoon cayenne pepper', 'chicken broth homemade carton', 'cups half whole milk', 'Salt pepper', 'Garnish store apple chips', 'ingredients', 'ingredients'],
    ['cup canola oil', 'cup butter', 'tablespoons lemon juice', 'cloves garlic', 'tablespoon dill weed', 'teaspoon salt', 'teaspoon ground black pepper', 'large potatoes cubed', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoon baking soda', 'teaspoon ground cinnamon', 'teaspoon ground cloves', 'teaspoon ground nutmeg', 'teaspoon salt', 'cup softened butter', 'cups brown sugar', 'egg', 'cup walnuts', 'cup apples', 'cup raisins', 'cup confectioners sugar', 'tablespoon milk', 'ingredients', 'ingredients'],
    ['ounce package cream cheese', 'tablespoons chives', 'clove garlic', 'skinless boneless chicken breast halves', 'turkey bacon', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['ounce boneless skinless chicken breasts', 'ounce bottle garlic herb marinade', 'fresh basil leaves', 'large tomato', 'provolone cheese', 'slices bacon', 'cup Parmesan', 'ingredients', 'ingredients'],
    ['ounce black beans', 'ounce black eyed peas', 'ounce tomatoes', 'cups frozen corn kernels', 'medium onion', 'green bell pepper', 'cup jalapeno peppers', 'teaspoon garlic salt', 'cup Italian salad dressing', 'cup cilantro', 'ingredients', 'ingredients'],
    ['cup butter', 'cup peanut butter', 'cups brown sugar', 'teaspoons vanilla extract', 'eggs', 'cup water', 'cup purpose flour', 'cup whole wheat flour', 'cups quick cooking oats', 'cup wheat germ', 'teaspoon salt', 'teaspoon ground cinnamon', 'teaspoons soda', 'cup raisins', 'ingredients', 'ingredients'],
    ['ounce pineapple chunks', 'tablespoons white sugar', 'tablespoons butter', 'tablespoons purpose flour', 'ounces Cheddar cheese', 'buttery round crackers', 'ingredients', 'ingredients'],
    ['cup uncooked white rice', 'cup water', 'green bell peppers', 'onion', 'tablespoons olive oil', 'ounces vegetable protein', 'tablespoons fresh parsley', 'tomato sauce', 'ounces mozzarella cheese', 'salt', 'ground black pepper', 'ingredients', 'ingredients'],
    ['potatoes', 'small carrot', 'stalk celery', 'small onion', 'cups vegetable broth', 'teaspoon salt', 'cups milk', 'tablespoons butter', 'tablespoons purpose flour', 'tablespoon parsley', 'teaspoon ground black pepper', 'cup Swiss cheese', 'ingredients', 'ingredients'],
    ['liter bottles ginger ale', 'fluid ounce pineapple juice', 'fluid ounce white grape juice', 'ingredients', 'ingredients'],
    ['pounds Granny Smith apples', 'tablespoon sugar', 'cup sugar', 'cup flour', 'teaspoon cinnamon', 'cup butter', 'egg', 'ingredients', 'ingredients'],
    ['pound eggplant', 'tablespoon olive oil', 'pound bulk Italian sausage', 'teaspoon garlic powder', 'teaspoon Italian seasoning', 'teaspoon black pepper', 'tablespoons dry bread crumbs', 'spaghetti sauce', 'cup mozzarella cheese', 'egg', 'ingredients', 'ingredients'],
    ['cup olive oil', 'cup sesame oil', 'cup fresh parsley', 'tablespoons hot sauce', 'tablespoons garlic', 'tablespoon ketchup', 'tablespoon Asian chile paste', 'teaspoon salt', 'teaspoon black pepper', 'tablespoons lemon juice', 'large shrimp deveined', 'wooden skewers water', 'ingredients', 'ingredients'],
    ['inch unbaked pie crusts', 'cups white sugar', 'cup purpose flour', 'teaspoon ground cinnamon', 'cups fresh strawberries', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['ounces angel hair pasta', 'tablespoon garlic', 'tablespoon olive oil', 'zucchini', 'salt pepper', 'tomatoes', 'fresh basil', 'ounces mozzarella cheese', 'ingredients', 'ingredients'],
    ['large ripe tomatoes inch thick', 'cup Romano cheese', 'cup fresh bread crumbs', 'clove garlic', 'sprigs fresh parsley', 'salt pepper', 'teaspoon oregano', 'tablespoon olive oil', 'ingredients', 'ingredients'],
    ['sweet potatoes', 'tablespoons salad oil', 'ounce package cream cheese', 'cup brown sugar', 'tablespoons margarine', 'tablespoon vanilla extract', 'teaspoon salt', 'teaspoon pepper', 'cup walnuts', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'teaspoons purpose flour', 'cup milk', 'tablespoons lemon juice', 'teaspoon salt', 'teaspoon cayenne pepper', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'skinless boneless chicken breast halves inch thickness', 'tablespoons olive oil', 'tablespoons butter', 'cup hoisin sauce', 'cup orange juice', 'ounce mandarin oranges', 'tablespoon green onions', 'cup cashews', 'ingredients', 'ingredients'],
    ['ounce package egg noodles', 'tablespoons olive oil', 'onion', 'cup chicken meat', 'red bell pepper', 'stalks celery', 'cup chicken stock', 'cups sour cream', 'teaspoon oregano', 'pound fresh asparagus inch pieces', 'tablespoons Parmesan cheese', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'fresh mushrooms', 'tablespoon purpose flour', 'cup chicken broth', 'cup heavy cream', 'cup turkey', 'cup frozen peas', 'salt pepper', 'ingredients', 'ingredients'],
    ['bacon', 'ounce package spinach', 'ounce artichoke hearts', 'ounce container garlic herb cheese spread', 'cup Parmesan cheese', 'ounce container sour cream', 'cup mayonnaise', 'ingredients', 'ingredients'],
    ['cup red green lentils', 'cup onion', 'stalk celery', 'cups cabbage', 'ounce tomatoes', 'chicken broth', 'carrots', 'clove garlic', 'teaspoon salt', 'teaspoon ground black pepper', 'teaspoon white sugar', 'teaspoon basil', 'teaspoon thyme', 'teaspoon curry powder', 'ingredients', 'ingredients'],
    ['ounce pineapple chunks reserve juice', 'tablespoon cornstarch', 'cup tomato chili sauce', 'cup raisins', 'tablespoon brown sugar', 'teaspoon ground cinnamon', 'pork chops', 'tablespoon vegetable oil', 'ingredients', 'ingredients'],
    ['cup pine nuts', 'cup quinoa', 'cups water', 'sea salt', 'cup fresh lemon juice', 'stalks celery', 'red onion', 'teaspoon cayenne pepper', 'teaspoon ground cumin', 'bunch fresh parsley', 'ingredients', 'ingredients'],
    ['eggs', 'cup milk', 'cup water', 'tablespoons butter', 'cup purpose flour', 'teaspoon salt', 'ounce package cream cheese', 'cups confectioners sugar', 'tablespoon lemon juice', 'teaspoon lemon zest', 'teaspoon vanilla extract', 'cup heavy cream', 'cups strawberries', 'ingredients', 'ingredients'],
    ['teaspoons olive oil', 'small onion', 'green onions', 'teaspoons garlic', 'ounce tomatoes', 'teaspoons basil', 'teaspoons oregano', 'teaspoon white sugar', 'ingredients', 'ingredients'],
    ['cup butter', 'cup white sugar', 'eggs', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoons powder', 'teaspoon salt', 'cup milk', 'cups blueberries', 'tablespoons purpose flour', 'tablespoons brown sugar', 'teaspoon ground cinnamon', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['potatoes', 'cup Colby Monterey Jack cheese', 'teaspoons chili powder', 'teaspoon ground cumin', 'clove garlic', 'salt pepper', 'inch flour tortillas', 'cup red enchilada sauce', 'ingredients', 'ingredients'],
    ['ounce fresh cranberries', 'cup white sugar', 'cup apricot jam', 'cup pecans', 'ounce package cream cheese', 'ingredients', 'ingredients'],
    ['cup vegetable oil', 'tablespoons margarine', 'cup unpopped popcorn', 'teaspoon seasoned salt', 'ingredients', 'ingredients'],
    ['bacon', 'cups chicken', 'cup fresh tomato', 'stalks celery', 'cup mayonnaise', 'tablespoons green onion', 'tablespoon parsley', 'teaspoon lemon juice', 'Worcestershire sauce', 'salt black pepper', 'romaine lettuce', 'large avocado', 'ingredients', 'ingredients'],
    ['ounce black beans', 'ounce pinto beans', 'ounce beans', 'ounce black olives', 'ounce whole kernel corn', 'cup green bell pepper', 'jalapeno pepper seeded', 'tablespoon ground cumin', 'tablespoon chili powder', 'ground black pepper', 'inch whole wheat tortillas', 'cups Cheddar cheese', 'cup salsa optional', 'cup sour cream optional', 'ingredients', 'ingredients'],
    ['pint ricotta cheese', 'eggs', 'ounce package spinach', 'cup mozzarella cheese', 'cup Parmesan cheese', 'tablespoons white sugar', 'teaspoon salt', 'teaspoon ground black pepper', 'manicotti shells', 'ounce jar spaghetti sauce', 'ingredients', 'ingredients'],
    ['GARLIC MAYONNAISE', 'cup mayonnaise', 'clove garlic', 'teaspoon paprika', 'SALMON CAKES', 'cup couscous', 'cup orange juice', 'ounce red salmon', 'ounce package spinach dry', 'egg yolks', 'cloves garlic', 'teaspoon ground cumin', 'teaspoon ground black pepper', 'teaspoon salt', 'tablespoons olive oil', 'ingredients', 'ingredients'],
    ['ounces egg noodles', 'tablespoons butter', 'yellow onion', 'cup purpose flour', 'cups chicken broth', 'cup milk', 'salt pepper', 'cups chicken breast meat', 'ounce package frozen broccoli', 'cup Cheddar cheese', 'cup provolone cheese', 'ingredients', 'ingredients'],
    ['pound thick bacon inch pieces', 'inch flour tortillas', 'cup Cheddar cheese', 'head iceberg lettuce', 'tomato', 'ingredients', 'ingredients'],
    ['ounce package pizza crust dough', 'ounces provolone cheese', 'Bosc pear', 'ounces walnuts', 'ounces Gorgonzola cheese', 'tablespoons fresh chives', 'ingredients', 'ingredients'],
    ['apples', 'ground cinnamon', 'ground nutmeg', 'cup walnuts', 'cup pecans', 'cup brown sugar', 'cup purpose flour', 'tablespoons butter', 'pinch salt', 'ingredients', 'ingredients'],
    ['butternut squash seeded', 'large onions', 'medium head garlic', 'vegetable broth', 'bay leaf', 'teaspoon brown sugar', 'teaspoon mild curry powder', 'teaspoon oregano', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'salt pepper', 'cup plain yogurt', 'cup fresh parsley optional', 'ingredients', 'ingredients'],
    ['eggs', 'cup hickory barbeque sauce', 'cloves garlic', 'chipotle chilies adobo sauce', 'tablespoons adobo sauce chipotle peppers', 'teaspoon kosher salt', 'teaspoon coarse ground black pepper', 'teaspoon celery salt', 'teaspoon ground cumin', 'tablespoon Worcestershire sauce', 'onion', 'cup dry oatmeal', 'pounds ground beef', 'tablespoons hickory barbeque sauce', 'ingredients', 'ingredients'],
    ['ounce yellow cake mix', 'ounce container topping', 'pint blueberries', 'pints fresh strawberries', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'teaspoon salt', 'teaspoon pepper', 'green tomatoes inch thick', 'cup saltine crackers', 'eggs', 'cup butter', 'ingredients', 'ingredients'],
    ['eggplant', 'egg', 'ounce container sour cream', 'cups Italian seasoned bread crumbs', 'tablespoons purpose flour', 'ingredients', 'ingredients'],
    ['inch unbaked pie crust', 'pears half', 'cup white sugar', 'cup butter', 'cup purpose flour', 'tablespoon vanilla extract', 'eggs', 'ingredients', 'ingredients'],
    ['ears corn', 'tablespoons butter', 'cup garlic powder', 'ingredients', 'ingredients'],
    ['grape tomatoes', 'ounces mozzarella cheese', 'tablespoons extra virgin olive oil', 'tablespoons fresh basil leaves', 'pinch salt', 'pinch ground black pepper', 'toothpicks', 'ingredients', 'ingredients'],
    ['medium zucchini inch slices', 'tablespoons olive oil', 'teaspoon sesame oil', 'teaspoons sesame seeds', 'tablespoons Parmesan cheese', 'teaspoon cayenne pepper', 'teaspoon garlic salt', 'ingredients', 'ingredients'],
    ['cup butter', 'garlic', 'tablespoon basil', 'teaspoons oregano', 'ingredients', 'ingredients'],
    ['chicken thighs', 'ounce cranberry sauce', 'ounce Russian style salad dressing', 'packet dry onion soup mix', 'ingredients', 'ingredients'],
    ['ounce peaches', 'cups purpose flour', 'cups white sugar', 'pinch salt', 'teaspoons ground cinnamon', 'cup butter', 'ingredients', 'ingredients'],
    ['ounces pork sausage', 'large zucchini', 'eggs', 'cup Colby cheese', 'cups cottage cheese', 'cups Italian cheese blend', 'cup onion', 'tablespoons Italian seasoning', 'teaspoon salt', 'teaspoon ground black pepper', 'garlic salt', 'cup tomatoes', 'ingredients', 'ingredients'],
    ['large potatoes', 'cups fresh broccoli', 'cup milk', 'teaspoon salt', 'tablespoon olive oil', 'onion', 'cup Cheddar cheese', 'eggs', 'cup milk', 'teaspoon salt', 'teaspoon ground black pepper', 'teaspoon ground nutmeg', 'ingredients', 'ingredients'],
    ['tater tots', 'ounce cream chicken soup', 'cup onion', 'ounce container sour cream', 'ounce sharp Cheddar cheese', 'garlic powder', 'teaspoon seasoning salt', 'cup butter', 'cups cornflakes cereal', 'cup butter', 'tablespoons Parmesan cheese', 'paprika', 'ingredients', 'ingredients'],
    ['cooking spray', 'chicken wings joints tips', 'salt black pepper', 'tablespoons hot sauce such Frank Red Hot', 'tablespoons vegetable oil', 'cup purpose flour', 'glaze', 'garlic cloves', 'tablespoons fresh ginger root', 'tablespoon Asian chile pepper sauce', 'cup rice vinegar', 'cup brown sugar', 'tablespoon soy sauce', 'ingredients', 'ingredients'],
    ['bacon', 'inch flour tortillas', 'tablespoons Ranch style salad dressing', 'avocado diced', 'tomato', 'cup lettuce', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'boneless pork loin chops thin', 'large onion', 'teaspoon sugar', 'teaspoons chili powder', 'teaspoon fennel seeds', 'teaspoon red pepper flakes', 'teaspoon oregano', 'ounce tomato sauce', 'fluid ounces water', 'teaspoon Worcestershire sauce', 'salt pepper', 'ingredients', 'ingredients'],
    ['bunch kale', 'clove garlic', 'tablespoon olive oil', 'tablespoons balsamic vinegar', 'Salt ground black pepper', 'ingredients', 'ingredients'],
    ['ounce boneless salmon fillets', 'tablespoon basil', 'tomato', 'tablespoon olive oil', 'tablespoons Parmesan cheese', 'ingredients', 'ingredients'],
    ['pound corned beef brisket', 'tablespoons spice', 'large orange rounds', 'stalks celery', 'large onion', 'cup cold water', 'tablespoons margarine', 'large head cabbage', 'cup Golden Delicious apples peel', 'cup cold water', 'ingredients', 'ingredients'],
    ['cups uncooked orecchiette pasta', 'cup butter', 'cup shallots', 'cloves garlic', 'cup green onion', 'teaspoons Cajun seasoning', 'teaspoon black pepper', 'cup white wine', 'cup plum tomatoes', 'pound medium shrimp deveined', 'cup baby spinach', 'ingredients', 'ingredients'],
    ['pound butter', 'ounce whole kernel corn', 'ounce corn', 'ounce package corn bread mix', 'ounce sour cream', 'ingredients', 'ingredients'],
    ['cup bulgur', 'tomatoes', 'cucumbers', 'green onions', 'cloves garlic', 'cup fresh parsley', 'cup fresh mint leaves', 'teaspoons salt', 'cup lemon juice', 'cup olive oil', 'ingredients', 'ingredients'],
    ['cups zucchini', 'onion', 'eggs', 'cup buttermilk baking mix', 'cup vegetable oil', 'cup Parmesan cheese', 'teaspoon marjoram', 'teaspoon parsley', 'ground black pepper', 'ingredients', 'ingredients'],
    ['ounce package ramen noodle pasta', 'tablespoons butter', 'cup sunflower seeds', 'cup pine nuts', 'cups bok choy', 'green onions', 'cup chicken breast meat', 'ounce water chestnuts', 'pods snow peas', 'cup vegetable oil', 'cup rice wine vinegar', 'tablespoon soy sauce', 'cup white sugar', 'tablespoon lemon juice', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoon baking powder', 'teaspoon baking soda', 'teaspoon salt', 'tablespoon ground cinnamon', 'cup vegetable oil', 'eggs', 'cups white sugar', 'cups pears', 'cup pecans', 'teaspoons vanilla extract', 'ingredients', 'ingredients'],
    ['cup shortening', 'cups white sugar', 'egg', 'cup pumpkin', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoon baking powder', 'teaspoon baking soda', 'teaspoon salt', 'teaspoon ground nutmeg', 'teaspoon ground cinnamon', 'cup walnuts optional', 'cup semisweet chocolate chips', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoons powder', 'teaspoons soda', 'teaspoons salt', 'teaspoons ground cinnamon', 'cups pumpkin', 'cup vegetable oil', 'eggs', 'cups white sugar', 'cup walnuts', 'ingredients', 'ingredients'],
    ['Granny Smith apples', 'cup cranberries', 'cup white sugar', 'teaspoons ground cinnamon', 'teaspoon ground nutmeg', 'cup quick cooking oats', 'cup purpose flour', 'cup light brown sugar', 'cup butter pieces', 'cup pecans', 'ingredients', 'ingredients'],
    ['ounce beans', 'pound ground beef', 'ounce package taco seasoning mix', 'tablespoon vegetable oil', 'inch corn tortillas', 'ounces Cheddar cheese', 'tablespoons sour cream', 'roma plum tomatoes', 'green onion', 'ounce green chiles', 'avocado', 'tablespoon black olives', 'ingredients', 'ingredients'],
    ['pound package hash browns', 'ounce cream potato soup', 'ounce cream celery soup', 'ounce container sour cream', 'small diced onion', 'green bell pepper', 'cups Cheddar cheese', 'ingredients', 'ingredients'],
    ['cup butter', 'cup brown sugar', 'eggs', 'cups oats', 'cups purpose flour', 'teaspoon salt', 'teaspoon baking soda', 'cups cranberries', 'cup white chocolate', 'ingredients', 'ingredients'],
    ['cup olive oil', 'cloves garlic', 'teaspoon salt', 'teaspoon red pepper flakes', 'tablespoons fresh parsley', 'dry penne pasta', 'cup Romano cheese', 'ingredients', 'ingredients'],
    ['cups chicken stock', 'cup chicken meat', 'cup green peas', 'cup diced celery', 'cup carrots', 'cups Cheddar cheese', 'tablespoons cornstarch', 'cup milk', 'recipe pastry inch double crust pie', 'ingredients', 'ingredients'],
    ['pounds Italian sausage links', 'small zucchini', 'small yellow squash', 'cup onion', 'ounce tomatoes liquid', 'ingredients', 'ingredients'],
    ['ounce jumbo pasta shells', 'pound Italian sausage', 'ounce package spinach dry', 'cup ricotta cheese', 'egg', 'cloves garlic', 'lemon', 'cup Parmesan cheese', 'salt pepper', 'teaspoon oregano', 'spaghetti sauce', 'cups mozzarella cheese', 'ingredients', 'ingredients'],
    ['pound broccoli rabe', 'tablespoons extra virgin olive oil', 'clove garlic', 'tablespoon Parmesan cheese', 'ingredients', 'ingredients'],
    ['cups water', 'cup quinoa', 'kale small pieces', 'tablespoons olive oil', 'tablespoons lemon juice', 'teaspoon Dijon mustard', 'large garlic clove', 'teaspoon fresh black pepper', 'teaspoon ground sea salt', 'cup pecans', 'cup currants', 'cup feta cheese', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'onion', 'green bell pepper', 'red bell pepper', 'pound sausage inch pieces', 'ounce frozen corn kernels', 'teaspoon seasoned salt', 'teaspoon garlic powder', 'teaspoon onion powder', 'teaspoon monosodium glutamate MSG Accent flavor enhancer', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['ounce package frozen edamame green soybeans', 'tablespoon olive oil', 'cup Parmesan cheese', 'salt pepper', 'ingredients', 'ingredients'],
    ['cups tomatoes', 'small onion', 'tablespoons olive oil', 'cloves garlic', 'fresh basil', 'ounces pasta', 'tablespoons Parmesan cheese', 'cups feta cheese', 'salt black pepper', 'ingredients', 'ingredients'],
    ['cup red lentils', 'slice ginger inch piece', 'teaspoon ground turmeric', 'teaspoon salt', 'teaspoon cayenne pepper', 'teaspoons vegetable oil', 'teaspoons minced onion', 'teaspoon cumin seeds', 'ingredients', 'ingredients'],
    ['pound Italian sausage casings', 'cup diced celery', 'cup diced onion', 'ounce tomatoes', 'ounce tomato sauce', 'teaspoon garlic powder', 'teaspoons salt', 'teaspoon oregano', 'dry ziti pasta', 'ounce cans mushrooms', 'ounces mozzarella cheese', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'pound boneless pork chops', 'cup orange juice', 'cup soy sauce', 'tablespoons white sugar', 'teaspoons water', 'teaspoon cornstarch', 'ingredients', 'ingredients'],
    ['cup butter', 'cup celery', 'tablespoons shallots', 'quart half cream', 'ounce containers fresh oysters undrained', 'salt black pepper', 'cayenne pepper', 'ingredients', 'ingredients'],
    ['cup vegetable oil', 'tablespoons red wine vinegar', 'clove garlic', 'teaspoon basil', 'teaspoon red pepper flakes', 'teaspoon salt', 'ounces macaroni', 'cup Parmesan cheese', 'broccoli florets', 'ounces pepperoni sausage', 'cherry tomatoes', 'cup mozzarella cheese', 'ingredients', 'ingredients'],
    ['inch white cake layers', 'pints fresh strawberries', 'cup white sugar', 'pint fresh blueberries', 'bananas', 'cup orange juice', 'ounce instant vanilla pudding mix', 'cups milk', 'cup heavy whipping cream', 'cup almonds', 'maraschino cherries', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breasts', 'cloves garlic', 'tablespoons rosemary', 'tablespoon lemon juice', 'salt pepper', 'ingredients', 'ingredients'],
    ['large orange', 'cup butter', 'cups white sugar', 'eggs', 'cup pumpkin', 'cup water', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon baking powder', 'teaspoon salt', 'teaspoon ground cinnamon', 'teaspoon ground cloves', 'cup walnuts', 'cup raisins', 'ingredients', 'ingredients'],
    ['pound carrots', 'cup margarine', 'teaspoon vanilla extract', 'eggs', 'tablespoons purpose flour', 'teaspoon baking powder', 'teaspoon salt', 'cup white sugar', 'ingredients', 'ingredients'],
    ['large cubed potatoes', 'ounce cream mushroom soup', 'cups milk', 'onion', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'pound skinless boneless chicken breast halves cut inch pieces', 'tablespoon onion', 'cloves garlic', 'ounce cans tomatoes', 'cups fresh broccoli florets', 'salt pepper', 'oregano', 'ounces dry penne pasta', 'cup fresh basil leaves thin strips', 'tablespoons Parmesan cheese', 'ingredients', 'ingredients'],
    ['pound bacon', 'tablespoons white wine vinegar', 'tablespoons honey', 'teaspoon dry hot mustard', 'teaspoon celery salt', 'teaspoon ground paprika', 'cup olive oil', 'red leaf lettuce bite size pieces', 'ounce mandarin oranges', 'bunch green onion', 'cup almonds', 'ingredients', 'ingredients'],
    ['cup butter shortening', 'cup white sugar', 'eggs', 'teaspoon vanilla extract', 'banana mashed', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'tablespoons butter', 'cup confectioners sugar', 'tablespoon milk', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['cup white sugar', 'cup margarine', 'eggs', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon ground cloves', 'teaspoon ground cinnamon', 'ripe bananas', 'cups oats', 'cup semisweet chocolate chips', 'ingredients', 'ingredients'],
    ['fresh broccoli florets', 'cup butter', 'tablespoons lemon juice', 'teaspoon lemon zest', 'cup almonds', 'ingredients', 'ingredients'],
    ['ripe avocado', 'cup milk', 'cup vanilla yogurt', 'tablespoons honey', 'ice cubes', 'ingredients', 'ingredients'],
    ['tablespoons canola oil', 'Chinese eggplants inch half moons', 'cup water', 'tablespoon red pepper flakes', 'tablespoons garlic powder', 'teaspoons white sugar', 'teaspoon cornstarch', 'tablespoons soy sauce', 'tablespoons oyster sauce', 'ingredients', 'ingredients'],
    ['cup balsamic vinaigrette salad dressing', 'teaspoon seasoned pepper', 'teaspoon cilantro', 'teaspoon ground cayenne pepper', 'teaspoon ground cumin', 'ounce cans black beans', 'ounce cans whole kernel corn', 'cup onion', 'cup green onions', 'cup red bell pepper', 'ingredients', 'ingredients'],
    ['dry black beans', 'quarts water', 'carrot', 'stalk celery', 'large red onion', 'cloves garlic', 'green bell peppers', 'jalapeno pepper seeded', 'cup dry lentils', 'ounce tomatoes', 'tablespoons chili powder', 'teaspoons ground cumin', 'teaspoon oregano', 'teaspoon ground black pepper', 'tablespoons red wine vinegar', 'tablespoon salt', 'cup uncooked white rice', 'ingredients', 'ingredients'],
    ['potatoes', 'cup milk', 'cup butter', 'cloves garlic', 'salt', 'pinch white pepper', 'tablespoons sesame seeds', 'ingredients', 'ingredients'],
    ['pounds ground beef', 'ounce jar taco sauce', 'ounce container sour cream', 'ounce package cream cheese', 'ounce package taco seasoning mix', 'ounce package Cheddar cheese', 'cup lettuce', 'tomato', 'green onions', 'ounce black olives', 'ingredients', 'ingredients'],
    ['cup olive oil', 'cups fresh mushrooms', 'cup onion', 'cup carrot', 'cup green bell pepper', 'cup celery', 'tablespoon garlic', 'tablespoon chili powder more', 'tablespoon ground cumin', 'teaspoons salt', 'teaspoon ground black pepper', 'teaspoon basil', 'teaspoon oregano', 'ounce tomatoes juice', 'black beans undrained', 'ounce tomato paste', 'cup red wine', 'teaspoon hot pepper sauce such Tabasco', 'cups water', 'ingredients', 'ingredients'],
    ['cups tomato', 'cup carrots', 'cup onion', 'ounce chicken broth', 'tablespoon white sugar', 'teaspoons sea salt', 'tablespoons butter', 'tablespoons purpose flour', 'cup % milk', 'teaspoons basil', 'teaspoon celery salt', 'teaspoon ground black pepper', 'teaspoon garlic powder', 'ingredients', 'ingredients'],
    ['bacon', 'large head fresh broccoli', 'cup celery', 'cup green onions', 'cup red onion', 'cups seedless grapes', 'cup almonds', 'cup white sugar', 'tablespoons white vinegar', 'cup mayonnaise', 'ingredients', 'ingredients'],
    ['ounce fillets tilapia', 'cloves garlic', 'tablespoons olive oil', 'onion', 'teaspoon cayenne pepper', 'ingredients', 'ingredients'],
    ['cup butter', 'cups brown sugar', 'egg', 'tablespoon vanilla extract', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'cup sour cream', 'cups rhubarb', 'cup white sugar', 'teaspoon ground nutmeg', 'ingredients', 'ingredients'],
    ['salmon fillets', 'lemons', 'cups pesto', 'cup white wine', 'ingredients', 'ingredients'],
    ['small sugar pumpkin', 'cup brown sugar', 'teaspoons ground cinnamon', 'teaspoon ground ginger', 'teaspoon ground nutmeg', 'teaspoon ground cloves', 'teaspoon ground allspice', 'teaspoon salt', 'cup real maple syrup', 'cups half cream', 'teaspoon purpose flour', 'eggs', 'inch unbaked pie shell', 'ingredients', 'ingredients'],
    ['cups strawberries', 'cup jalapeno peppers', 'cup lemon juice', 'ounce powdered fruit pectin', 'cups white sugar', 'half pint jars lids rings', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves cut inch pieces', 'lime', 'lemon', 'tablespoons Dijon mustard', 'black pepper', 'Creole style seasoning', 'ingredients', 'ingredients'],
    ['ounce beans chickpeas', 'ounce kidney beans', 'ounce green beans', 'green onions', 'stalk celery', 'cup cider vinegar', 'cup vegetable oil', 'tablespoon honey', 'teaspoon ground dry mustard', 'teaspoon garlic powder', 'teaspoon ground black pepper', 'teaspoon onion powder optional', 'teaspoon ground cayenne pepper optional', 'ingredients', 'ingredients'],
    ['eggplant inch slices', 'egg', 'cup Italian seasoned bread crumbs', 'tablespoons olive oil', 'cup ricotta cheese', 'slices prosciutto', 'ounce jar spaghetti sauce', 'cups mozzarella cheese', 'pound angel hair pasta', 'ingredients', 'ingredients'],
    ['pound fresh green beans half', 'tablespoons butter', 'cloves garlic', 'lemon pepper', 'salt', 'ingredients', 'ingredients'],
    ['cups fresh broccoli', 'cup butter', 'tablespoons purpose flour', 'small onion', 'cups milk', 'salt pepper', 'cups Swiss cheese', 'eggs', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'large yellow onion', 'teaspoon garlic', 'cups milk', 'ounce tomato puree', 'tablespoon white sugar', 'cups fresh spinach', 'cup fresh basil', 'teaspoon salt', 'teaspoon black pepper', 'tablespoon Parmesan cheese', 'ingredients', 'ingredients'],
    ['cups winter squash', 'cup milk', 'ounce packages active dry yeast', 'cup warm water degrees F C', 'cups purpose flour', 'cup white sugar', 'teaspoons salt', 'cup shortening', 'ingredients', 'ingredients'],
    ['ounce package mixed vegetables', 'small potatoes', 'ounce Polish beef sausage inch slices', 'cup butter pieces', 'tablespoon lemon pepper', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['inch graham cracker crust', 'ounce unflavored gelatin', 'teaspoon ground cinnamon', 'teaspoon ground ginger', 'teaspoon ground nutmeg', 'teaspoon salt', 'ounce condensed milk', 'eggs', 'ounce pumpkin puree', 'ingredients', 'ingredients'],
    ['cups potatoes', 'pound sausage', 'ounce cream mushroom soup', 'ounce vegetable beef soup', 'ingredients', 'ingredients'],
    ['eggs', 'cup buttermilk', 'tablespoon Worcestershire sauce', 'teaspoon vinegar hot pepper sauce', 'teaspoon cayenne pepper', 'teaspoon seasoning salt', 'teaspoon garlic powder', 'cup cornmeal', 'cups purpose flour', 'teaspoon salt', 'teaspoon ground black pepper', 'ounce jar dill pickle slices', 'cup vegetable oil deep frying', 'salt pepper', 'ingredients', 'ingredients'],
    ['ounce black beans', 'cloves garlic', 'onion', 'ounce green chile peppers', 'jalapeno pepper seeded', 'ounce package tempeh', 'inch corn tortillas', 'ounce sauce', 'ounce black olives', 'ounces Cheddar cheese', 'ingredients', 'ingredients'],
    ['pound beef chuck pot roast', 'cloves garlic', 'red bell peppers large wedges', 'green bell peppers large wedges', 'yellow onion wedges', 'sweet onion wedges', 'ounce tomato paste', 'cup red wine vinegar', 'teaspoon garlic powder', 'teaspoon seasoned salt', 'salt black pepper', 'ingredients', 'ingredients'],
    ['bacon', 'quart water', 'ounce frozen green peas', 'cup onions', 'cup Ranch dressing', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['teaspoons vegetable oil', 'eggs', 'medium head cabbage', 'carrot', 'ounce bamboo shoots', 'cup wood ear mushroom', 'pound Chinese barbequed roasted pork matchsticks', 'green onions', 'teaspoons soy sauce', 'teaspoon salt', 'teaspoon sugar', 'teaspoon monosodium glutamate MSG', 'ounce package egg roll wrappers', 'egg white', 'cups oil frying', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'small onion', 'medium tomatoes', 'teaspoon salt', 'teaspoon pepper', 'small zucchini inch slices', 'small yellow summer squash inch slices', 'bay leaf', 'teaspoon basil', 'ingredients', 'ingredients'],
    ['cups potatoes', 'cup beets', 'cups vegetable stock water', 'tablespoons butter', 'cups onions', 'teaspoon caraway seed optional', 'teaspoons salt', 'celery stalk', 'large carrot', 'cups red cabbage', 'black pepper', 'teaspoon fresh dill weed', 'tablespoon cider vinegar', 'tablespoon honey', 'cup tomato puree', 'sour cream', 'tomatoes garnish', 'ingredients', 'ingredients'],
    ['ounce cans black eyed peas', 'ounce petite tomatoes', 'fresh medium jalapenos seeded', 'small onion dice', 'yellow bell pepper seeded small dice', 'cup fresh cilantro', 'tablespoons red wine vinegar', 'tablespoons olive oil extra virgin', 'teaspoon salt', 'teaspoon ground black pepper', 'teaspoon garlic powder', 'teaspoon oregano', 'teaspoons ground cumin', 'ingredients', 'ingredients'],
    ['slices bacon', 'tablespoons olive oil', 'teaspoon garlic', 'shallots', 'ground venison', 'tablespoon Worcestershire sauce', 'tablespoon fresh parsley', 'salt pepper', 'egg', 'hamburger buns', 'ingredients', 'ingredients'],
    ['ground beef', 'onion', 'cup Cheddar cheese', 'cup sour cream', 'tablespoon parsley', 'tablespoon taco seasoning', 'teaspoon oregano', 'teaspoon ground black pepper', 'cups enchilada sauce', 'teaspoons chili powder', 'clove garlic', 'teaspoon salt', 'flour tortillas', 'ounce black beans', 'ounce black olives', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['MARINADE', 'cup fresh cilantro', 'clove garlic', 'jalapeno chile pepper seeded', 'tablespoons fresh lime zest', 'teaspoons salt', 'teaspoon onion powder', 'teaspoon ground black pepper', 'teaspoon chipotle chile powder', 'tablespoon olive oil', 'pound chicken breast tenderloins strips', 'SALSA', 'medium tomato', 'small sweet onion', 'tablespoons fresh cilantro', 'jalapeno chile pepper seeded', 'clove garlic', 'teaspoon ground black pepper', 'teaspoon sea salt', 'teaspoon chipotle chile powder', 'tablespoon fresh lime juice', 'VEGETABLES', 'sweet onion cut inch slices', 'red bell pepper', 'tablespoon olive oil', 'teaspoon salt', 'teaspoon garlic', 'LIME MAYONNAISE', 'cup mayonnaise', 'tablespoons fresh lime juice', 'thick slices French bread', 'mangos', 'Monterey Jack cheese', 'ingredients', 'ingredients'],
    ['tablespoons extra-virgin olive oil', 'onion', 'cup diced carrot', 'cloves garlic', 'cups low sodium chicken broth', 'cups water', 'cup white wine', 'potatoes', 'teaspoon fresh rosemary', 'teaspoon fresh sage', 'teaspoon fresh thyme', 'ounce beans', 'cups kale leaves', 'small red chile pepper seeded fine', 'ground black pepper', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'cup whole wheat flour', 'teaspoons soda', 'teaspoon baking powder', 'teaspoon salt', 'teaspoon cinnamon', 'cup white sugar', 'cup brown sugar', 'cup canola oil', 'eggs', 'teaspoon vanilla extract', 'cup oats', 'cup coconut', 'cup raisins', 'cups carrots', 'ounce pineapple juice', 'cup cream cheese', 'ingredients', 'ingredients'],
    ['eggplant inch rounds', 'tablespoons olive oil', 'tablespoon garlic powder', 'tablespoons olive oil', 'small onion', 'cloves garlic', 'small tomatoes', 'ounce fresh spinach leaves', 'cup ricotta cheese', 'cup mozzarella cheese', 'cup Parmesan cheese', 'cup tomato pasta sauce', 'teaspoons Italian seasoning', 'ingredients', 'ingredients'],
    ['tablespoons butter oil', 'pounds boneless pork inch cubes', 'cloves garlic', 'medium onion', 'chicken broth', 'teaspoon salt', 'teaspoon rosemary', 'teaspoon sage', 'bay leaf', 'cups cubed butternut squash', 'MacIntosh apples cubed', 'large potatoes cubed optional', 'cups carrots diced optional', 'ingredients', 'ingredients'],
    ['eggs', 'cup corn oil', 'cup buttermilk', 'cups Cheddar cheese', 'ounce cream style corn', 'large onion', 'fresh jalapeno peppers', 'cup cornmeal', 'cup purpose flour', 'teaspoons powder', 'teaspoon baking soda', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['pound whole chicken pieces', 'tablespoons butter', 'teaspoon salt', 'teaspoon ground black pepper', 'cup celery', 'onion', 'ounce whole cranberry sauce', 'cup barbecue sauce', 'ingredients', 'ingredients'],
    ['cucumbers', 'cups feta cheese', 'cup black olives', 'cups roma tomatoes', 'cup oil sun tomatoes', 'red onion', 'ingredients', 'ingredients'],
    ['roma plum tomatoes', 'tablespoons extra virgin olive oil', 'Spanish onion', 'cloves garlic', 'Italian seasoning', 'kalamata olives', 'cup white wine', 'cup capers', 'teaspoon fresh lemon juice', 'fresh basil', 'tablespoons Parmesan cheese', 'pound flounder fillets', 'fresh basil', 'ingredients', 'ingredients'],
    ['cups uncooked white rice', 'ounce cream mushroom soup', 'cup vegetable broth', 'cup onion', 'cup fresh mushrooms', 'teaspoon parsley', 'teaspoon oregano', 'cup butter', 'salt pepper', 'ingredients', 'ingredients'],
    ['cup white sugar', 'tablespoon cornstarch', 'cup water', 'cups strawberries', 'tablespoons butter', 'cup purpose flour', 'tablespoon white sugar', 'teaspoons powder', 'teaspoon salt', 'tablespoons butter', 'cup heavy whipping cream', 'ingredients', 'ingredients'],
    ['cup cake flour', 'cup whole wheat flour', 'cup brown sugar', 'cup white sugar', 'teaspoon baking powder', 'teaspoon baking soda', 'teaspoons ground cinnamon', 'eggs', 'cup vegetable oil', 'cup buttermilk', 'cups carrots', 'ingredients', 'ingredients'],
    ['large baking potatoes', 'ounce container sour cream', 'cup Cheddar cheese', 'cup butter', 'salt pepper', 'slices American cheese', 'ingredients', 'ingredients'],
    ['red apple', 'Granny Smith apple', 'nectarine', 'stalks celery', 'cup cranberries', 'cup walnuts', 'ounce container nonfat lemon yogurt', 'ingredients', 'ingredients'],
    ['cups potatoes', 'cup butter', 'eggs', 'teaspoon onion', 'teaspoon salt', 'teaspoon paprika', 'cup milk', 'cup sharp Cheddar cheese', 'ingredients', 'ingredients'],
    ['pound bacon', 'ounce cans water chestnuts', 'cup brown sugar', 'cup mayonnaise', 'cup chili sauce', 'ingredients', 'ingredients'],
    ['pound bone ham', 'cup whole cloves', 'ounce pineapple rings heavy syrup', 'cup brown sugar', 'fluid ounce lemon lime carbonated beverage', 'ounce jar maraschino cherries', 'ingredients', 'ingredients'],
    ['medium head cabbage', 'slices bacon', 'onion', 'cup ham', 'tablespoon butter', 'cups potatoes', 'teaspoon paprika', 'salt pepper', 'ingredients', 'ingredients'],
    ['pounds skinless boneless chicken breast halves', 'tablespoons taco seasoning mix', 'tablespoon vegetable oil', 'cup onions', 'cup celery', 'teaspoons ground cumin', 'teaspoon ground black pepper', 'cup water', 'ounce cans chicken broth', 'cup tomatoes', 'tablespoon fresh cilantro', 'cup Cheddar cheese', 'cup tortilla chips', 'avocado diced', 'ingredients', 'ingredients'],
    ['cup margarine', 'cup white sugar', 'egg', 'cup zucchini', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'teaspoon ground cinnamon', 'teaspoon ground cloves', 'cup raisins', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'cup onion', 'green bell pepper', 'cloves garlic', 'teaspoon salt', 'tablespoons tomato paste', 'ounce kidney beans liquid', 'cup uncooked white rice', 'ingredients', 'ingredients'],
    ['pound ground beef', 'teaspoon red pepper flakes', 'teaspoon garlic', 'large onion', 'green bell pepper', 'red bell pepper', 'ounce kidney beans', 'ounce black beans', 'ounce Great Northern beans', 'ounce tomato sauce', 'ounce tomato sauce garlic onions', 'ounce cans petite tomatoes', 'ounce roasted tomatoes', 'ounce pumpkin puree', 'teaspoons pumpkin pie spice', 'teaspoons chili powder', 'teaspoon ground cumin', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['apples', 'eggs', 'cups white sugar', 'cup vegetable oil', 'teaspoon baking soda', 'teaspoon ground cinnamon', 'tablespoon vanilla extract', 'cups purpose flour', 'cup walnuts', 'ingredients', 'ingredients'],
    ['ounce package lemon cake mix', 'cup hot water', 'cup cold water', 'ounce lemon Jell O mix', 'cup milk', 'ounce instant vanilla pudding mix', 'ounce container topping', 'ingredients', 'ingredients'],
    ['ounce cans sweet potatoes mashed', 'cup butter', 'tablespoons white sugar', 'tablespoons brown sugar', 'tablespoon orange juice', 'eggs', 'cup milk', 'cup pecans', 'cup brown sugar', 'tablespoons purpose flour', 'teaspoons butter', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cup cornmeal', 'cup brown sugar', 'teaspoon salt', 'tablespoon baking powder', 'cup milk', 'cup butter', 'egg', 'cup frozen blueberries', 'ingredients', 'ingredients'],
    ['ounce cans black beans', 'ounce Mexican style corn', 'ounce cans tomatoes green chile peppers', 'tomatoes', 'green onions', 'cilantro leaves garnish', 'ingredients', 'ingredients'],
    ['ounce package frozen broccoli', 'ounce package chicken dry stuffing mix', 'cup Parmesan cheese', 'medium onion', 'eggs', 'cup margarine', 'teaspoon ground black pepper', 'teaspoon garlic salt', 'ingredients', 'ingredients'],
    ['avocados diced', 'cloves garlic', 'tablespoons Greek style yogurt', 'tablespoon fresh lemon juice', 'salt pepper', 'salmon steaks', 'teaspoons dill weed', 'teaspoons lemon pepper', 'salt', 'ingredients', 'ingredients'],
    ['cup walnuts', 'tablespoon butter', 'onion', 'teaspoon garlic', 'cup instant brown rice', 'cup vegetable broth', 'pound fresh broccoli florets', 'teaspoon salt', 'teaspoon ground black pepper', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'teaspoon lemon pepper', 'garlic powder', 'teaspoon onion powder', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'ounce fresh mushrooms', 'large onion', 'slices bacon', 'cup heavy cream', 'cup Swiss cheese', 'salt pepper', 'teaspoon fresh dill', 'ounce package frozen puff pastry', 'egg', 'ingredients', 'ingredients'],
    ['pound whole filet salmon skin pieces', 'tablespoons extra-virgin olive oil', 'large garlic cloves', 'cup fresh dill', 'teaspoons salt', 'teaspoon ground black pepper', 'teaspoon lemon zest wedges', 'ingredients', 'ingredients'],
    ['cup dry fusilli pasta', 'tablespoons olive oil', 'chicken tenderloins chunks', 'tablespoon minced onion', 'salt pepper', 'garlic powder', 'tablespoon basil', 'tablespoon parsley', 'ounce cream chicken soup', 'ounce cream mushroom soup', 'cups mixed vegetables', 'cup dry bread crumbs', 'tablespoons Parmesan cheese', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'teaspoon cumin seeds', 'teaspoon garlic', 'teaspoon ginger paste', 'medium potatoes cubed', 'teaspoon ground turmeric', 'teaspoon paprika', 'teaspoon ground cumin', 'teaspoon garam masala', 'salt', 'pound cauliflower', 'teaspoon fresh cilantro', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'yellow onion', 'red bell pepper', 'Anaheim chile pepper', 'red jalapeno pepper', 'garlic cloves', 'pounds ground beef', 'cup Worcestershire sauce', 'garlic powder', 'beef bouillon cubes', 'fluid ounce light beer such Coors', 'ounce San Marzano tomatoes', 'ounce diced tomatoes', 'ounce tomato paste', 'cup white wine', 'tablespoons chili powder', 'tablespoons ground cumin', 'tablespoon brown sugar', 'tablespoon chipotle pepper sauce', 'teaspoons basil', 'teaspoons', 'teaspoon salt', 'teaspoon oregano', 'teaspoon ground black pepper', 'ounce cans dark red kidney beans such Bush', 'cup sour cream', 'tablespoons fresh cilantro', 'teaspoon ground cumin', 'ingredients', 'ingredients'],
    ['cup long grain white rice', 'cups water', 'teaspoon chicken bouillon granules', 'tablespoons fresh lime juice', 'tablespoons fresh cilantro', 'salt', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'large red onion', 'cup red bell pepper', 'cloves garlic', 'cup Gorgonzola cheese', 'ounces cream cheese', 'cup heavy cream', 'cup milk', 'ounce tomatoes', 'cups tomato juice', 'teaspoons basil', 'teaspoon white sugar', 'teaspoon ground white pepper', 'ingredients', 'ingredients'],
    ['unpeeled red potatoes', 'eggs', 'cup sour cream', 'cup mayonnaise', 'tablespoon apple cider vinegar', 'tablespoon Dijon mustard', 'white onion', 'stalk celery', 'teaspoon celery salt', 'salt black pepper', 'tablespoon dill weed', 'ingredients', 'ingredients'],
    ['teaspoon vegetable oil', 'small onion', 'large sweet potato diced', 'clove garlic', 'chicken broth', 'teaspoon thyme', 'teaspoon ground cumin', 'cup chunky salsa', 'ounce beans', 'cup zucchini', 'cup rice', 'tablespoons creamy peanut butter', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cups white sugar', 'cup butter', 'teaspoons powder', 'teaspoon salt', 'egg yolks', 'cup milk', 'egg whites', 'cup blueberries', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'cup baby carrots', 'onion', 'cloves garlic', 'pound lean ground beef', 'cup whole milk', 'ounce tomatoes', 'ounce tomato paste', 'teaspoon salt', 'teaspoon ground black pepper', 'teaspoon basil', 'teaspoon oregano', 'teaspoon red pepper flakes', 'cup whole milk', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['cups unpeeled zucchini', 'cups onion', 'tablespoons salt', 'red bell pepper', 'green bell pepper', 'cups white sugar', 'cups white vinegar', 'tablespoon cornstarch', 'teaspoon ground nutmeg', 'teaspoon ground turmeric', 'teaspoons celery seed', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['cooking spray', 'ounce package bite size potato nuggets such Tater Tots', 'pound ground beef', 'cup onion', 'teaspoons Worcestershire sauce', 'teaspoons Montreal style steak seasoning optional', 'teaspoon garlic powder', 'teaspoon ground black pepper', 'ounce cream mushroom soup', 'cup milk', 'cup Cheddar cheese', 'teaspoon Worcestershire sauce', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['pounds fillets', 'tablespoon butter', 'salt pepper', 'tablespoon lemon juice', 'cup fresh tomato', 'teaspoons basil', 'teaspoon garlic powder', 'ingredients', 'ingredients'],
    ['avocados mashed', 'cup onions', 'teaspoon garlic salt', 'inch corn tortillas', 'bunch fresh cilantro leaves', 'jalapeno pepper sauce', 'ingredients', 'ingredients'],
    ['pound bacon', 'broccoli florets', 'green onions', 'cup sunflower seeds', 'cup golden raisins', 'cup mayonnaise', 'cup white sugar', 'tablespoons red wine vinegar', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'cup quick cooking oats', 'teaspoon baking powder', 'teaspoon baking soda', 'teaspoon cinnamon', 'cup unsweetened applesauce', 'cup fat free sour cream', 'egg', 'teaspoon vanilla extract', 'cup milk', 'ingredients', 'ingredients'],
    ['cup vegetable oil', 'cloves garlic', 'ounce uncooked linguine pasta', 'salt', 'cup broccoli', 'cup cabbage', 'cup carrots', 'cup cauliflower', 'tablespoons green onions', 'pound chicken tenders bite size pieces', 'tablespoons soy sauce', 'salt pepper', 'cup cilantro', 'lime wedges', 'ingredients', 'ingredients'],
    ['medium yellow squash', 'cup extra virgin olive oil', 'cloves garlic', 'salt pepper', 'ingredients', 'ingredients'],
    ['potatoes', 'onion', 'tablespoons purpose flour', 'tablespoons butter diced', 'salt black pepper', 'cups whole milk', 'ingredients', 'ingredients'],
    ['cup olive oil', 'cup onion', 'cup carrots', 'cup celery', 'teaspoon garlic', 'pound fresh mushrooms', 'chicken broth', 'cup barley', 'salt pepper', 'ingredients', 'ingredients'],
    ['pound acorn squash seeded inch slices', 'cup Parmesan cheese', 'sprigs fresh thyme', 'tablespoons olive oil', 'teaspoon kosher salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['ounces elbow macaroni', 'tablespoons olive oil', 'tilapia fillets', 'tablespoon lemon juice', 'teaspoon oregano', 'basil', 'cilantro', 'pinch salt black pepper', 'pasta', 'tablespoon light olive oil', 'cup Parmesan cheese', 'teaspoon lemon juice', 'garlic powder', 'basil', 'oregano', 'cilantro optional', 'pinch salt black pepper', 'ingredients', 'ingredients'],
    ['boneless beef sirloin top round steaks thick', 'tablespoons cornstarch', 'ounce Campbell Condensed Beef Broth', 'cup soy sauce', 'tablespoons sugar', 'tablespoons vegetable oil', 'cups shiitake mushrooms', 'Chinese cabbage bok choy', 'medium red peppers long strips', 'stalks celery', 'medium green onions pieces', 'Hot regular long grain white rice', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'cup white sugar', 'cup butter', 'cups rhubarb', 'cup white sugar', 'tablespoon purpose flour', 'ounce cream cheese', 'cup white sugar', 'eggs', 'cup sour cream', 'tablespoons white sugar', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['cups cubed butternut squash', 'cup brown sugar', 'tablespoon cornstarch', 'egg', 'cup milk', 'teaspoon ground cinnamon', 'pinch ground allspice', 'pinch ground cloves', 'pinch ground ginger', 'pinch ground nutmeg', 'inch unbaked pie shell', 'ingredients', 'ingredients'],
    ['cups whole wheat flour', 'teaspoon baking powder', 'teaspoon baking soda', 'teaspoon salt', 'teaspoons pumpkin pie spice', 'cup brown sugar', 'cup white sugar', 'cup canola oil', 'cup applesauce', 'cup pumpkin', 'cup buttermilk', 'eggs', 'cup golden raisins optional', 'cup pecans optional', 'ingredients', 'ingredients'],
    ['slices bacon', 'baby bok choy', 'teaspoon olive oil', 'small red onion', 'teaspoon red pepper flakes', 'teaspoon garlic', 'salt', 'ingredients', 'ingredients'],
    ['cup butter', 'cup onion', 'ounce cans chicken broth', 'cloves garlic', 'bay leaves', 'teaspoon cayenne pepper', 'teaspoon Cajun seasoning', 'salt pepper', 'ears corn kernels cob', 'cup half', 'tablespoons purpose flour', 'cup milk', 'ounces fresh lump crabmeat', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'Japanese eggplants inch cubes', 'tablespoons vegetable oil', 'onions', 'tablespoon garlic', 'tablespoons soy sauce', 'tablespoons water', 'tablespoons oyster sauce', 'tablespoon chili garlic sauce', 'teaspoon white sugar', 'ground black pepper', 'teaspoon Asian toasted sesame oil', 'ingredients', 'ingredients'],
    ['baking potatoes', 'tablespoon extra virgin olive oil', 'kosher salt', 'sheets aluminum foil', 'ingredients', 'ingredients'],
    ['bunch kale', 'tablespoons olive oil', 'cloves garlic', 'ingredients', 'ingredients'],
    ['pint cherry tomatoes', 'ounce fresh basil leaves', 'ounce small fresh mozzarella balls', 'toothpicks', 'tablespoons olive oil', 'salt pepper', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'pound asparagus bottoms', 'cup almonds', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['cloves garlic', 'teaspoons salt', 'bunch broccoli florets', 'cup olive oil', 'cup red wine vinegar', 'tablespoon Dijon mustard', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['large potatoes', 'yellow onion', 'egg', 'teaspoon salt', 'tablespoons purpose flour', 'ground black pepper', 'cups vegetable oil frying', 'ingredients', 'ingredients'],
    ['tart apples', 'cup white sugar', 'teaspoons ground cinnamon', 'tablespoons purpose flour', 'teaspoon ground nutmeg', 'salt', 'recipes unbaked pie shells', 'tablespoon butter', 'ingredients', 'ingredients'],
    ['ounce tuna', 'cup celery', 'tablespoons mayonnaise', 'pinch salt', 'English muffins split toasted', 'ripe tomato', 'Cheddar cheese', 'ingredients', 'ingredients'],
    ['Yukon Gold potatoes', 'cloves garlic', 'teaspoon salt', 'cup heavy cream', 'black pepper', 'nutmeg', 'cups Gruyere cheese', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cup brown sugar', 'tablespoon baking powder', 'teaspoon ground nutmeg', 'teaspoon salt', 'cup butter chilled diced', 'cup fresh cranberries', 'cup white sugar', 'grated zest orange', 'cup walnuts', 'cup half cream', 'egg', 'ingredients', 'ingredients'],
    ['cup butter', 'cup uncooked long grain rice', 'cup uncooked orzo pasta', 'cup fresh mushrooms', 'cup onion', 'cup celery', 'chicken broth', 'tablespoons fresh parsley', 'teaspoon marjoram', 'teaspoon ground black pepper', 'cup hazelnuts', 'ingredients', 'ingredients'],
    ['inch corn tortillas half', 'ounce cans sauce', 'cups sour cream', 'cups Cheddar cheese', 'ounce cans green chiles', 'cup fresh corn kernels', 'pound skinless boneless chicken breast halves', 'ingredients', 'ingredients'],
    ['eggs', 'cup white sugar', 'cup milk', 'tablespoons vegetable oil', 'cups purpose flour', 'teaspoon salt', 'teaspoons powder', 'cup fresh blueberries', 'cup walnuts', 'ingredients', 'ingredients'],
    ['cups cherry tomatoes', 'cup vegetable oil', 'tablespoons cider vinegar', 'teaspoon parsley', 'teaspoon basil', 'teaspoon oregano', 'teaspoon salt', 'teaspoons white sugar', 'ingredients', 'ingredients'],
    ['cups orange juice', 'cup olive oil', 'cup fresh chives', 'tablespoons fresh rosemary', 'teaspoon salt', 'teaspoon ground black pepper', 'pound whole chicken pieces', 'ingredients', 'ingredients'],
    ['Mashed potato layer', 'russet potatoes inch cubes', 'cup vegan mayonnaise', 'cup soy milk', 'cup olive oil', 'tablespoons vegan cream cheese substitute such Tofutti', 'teaspoons salt', 'Bottom layer', 'tablespoon vegetable oil', 'large yellow onion', 'carrots', 'stalks celery', 'cup frozen peas', 'tomato', 'teaspoon Italian seasoning', 'clove garlic more', 'pinch ground black pepper', 'ounce vegetarian ground beef substitute', 'cup Cheddar style soy cheese', 'ingredients', 'ingredients'],
    ['ounce package cheese tortellini', 'tablespoons Butter', 'small onion', 'ounce package cream cheese', 'cup Parmesan cheese', 'cup milk', 'fresh mushrooms', 'ounce package spinach', 'cherry tomatoes', 'ingredients', 'ingredients'],
    ['clove garlic', 'pound loaf Italian bread', 'cup tomatoes', 'cup cucumber', 'cup red onion', 'clove garlic', 'cups fresh basil', 'cup fresh thyme', 'cup olive oil', 'tablespoons balsamic vinegar', 'ingredients', 'ingredients'],
    ['jumbo pasta shells', 'cups cheese', 'ounce packages spinach', 'cup Parmesan cheese', 'tablespoons fennel seed', 'teaspoons basil', 'cloves garlic', 'salt pepper', 'cups spaghetti sauce', 'ingredients', 'ingredients'],
    ['small spaghetti squash seeded', 'tablespoon olive oil', 'cup onion', 'cloves garlic', 'green onions', 'ounces white meat turkey', 'cups tomatoes', 'tablespoons red wine', 'teaspoons capers', 'teaspoons fresh oregano', 'teaspoons red pepper flakes', 'tablespoons fresh parsley', 'ingredients', 'ingredients'],
    ['cup margarine', 'cup shortening', 'cups white sugar', 'egg yolks', 'cups purpose flour', 'teaspoon baking soda', 'cup buttermilk', 'teaspoon vanilla extract', 'cups coconut', 'cup pecans', 'egg whites', 'ounces cream cheese', 'cup margarine', 'cups confectioners sugar', 'teaspoon vanilla extract', 'cup pecans', 'ingredients', 'ingredients'],
    ['pounds skinless boneless chicken breast halves', 'bunch green onions', 'red bell pepper', 'yellow bell pepper', 'fresh jalapeno peppers', 'clove garlic', 'teaspoon ground ginger', 'teaspoon salt', 'teaspoon sage', 'teaspoon ground cumin', 'teaspoon ground white pepper', 'tablespoon olive oil', 'tablespoons butter', 'cup purpose flour', 'chicken broth', 'ounce cans great Northern beans undrained', 'ingredients', 'ingredients'],
    ['ounce kidney beans', 'red bell pepper', 'cups cabbage', 'green onions', 'cup feta cheese', 'cup fresh parsley', 'clove garlic', 'tablespoons lemon juice', 'tablespoon olive oil', 'ingredients', 'ingredients'],
    ['pound whole turkey neck giblets', 'cloves garlic', 'tablespoons seasoned salt', 'cup butter', 'fluid ounce cans cola carbonated beverage', 'apple', 'onion', 'tablespoon garlic powder', 'tablespoon salt', 'tablespoon ground black pepper', 'ingredients', 'ingredients'],
    ['cup white sugar', 'cup vegetable oil', 'eggs', 'cup honey', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoon baking powder', 'teaspoon baking soda', 'teaspoon salt', 'teaspoon ground cinnamon', 'teaspoon ground allspice', 'apples', 'cup walnuts', 'ingredients', 'ingredients'],
    ['pinto beans', 'quarts chicken stock', 'tablespoon salt', 'tablespoon ground black pepper', 'fluid ounce dark beer', 'ounce cans tomatoes', 'white onion', 'cup jalapeno peppers', 'cloves garlic', 'bay leaves', 'tablespoons oregano', 'cups fresh cilantro', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'tablespoons butter', 'pounds chicken breast', 'large carrot', 'large onion', 'stalks celery', 'cloves garlic', 'tablespoons chili powder', 'tablespoons ground cumin', 'tablespoon ground paprika', 'salt pepper', 'cup hot buffalo wing sauce such Frank REDHOT Buffalo Wing Sauce', 'ounce cans tomato sauce', 'ounce tomatoes', 'ounce white kidney cannellini beans', 'ounce red kidney beans', 'ingredients', 'ingredients'],
    ['eggplant', 'tablespoons vegetable oil', 'teaspoon cumin seeds', 'medium onion', 'teaspoon fresh ginger', 'large tomato seeded diced', 'clove garlic', 'teaspoon ground turmeric', 'teaspoon ground cumin', 'teaspoon ground coriander', 'teaspoon cayenne pepper', 'teaspoon salt', 'ground black pepper', 'cup fresh cilantro', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'cup purpose flour', 'teaspoon salt', 'teaspoon ground black pepper', 'tablespoon olive oil', 'cup cooking sherry', 'cup chicken broth', 'clove garlic', 'lemon', 'carrots', 'zucchini squashes', 'ingredients', 'ingredients'],
    ['cloves garlic', 'tablespoons olive oil', 'pounds potatoes cubed', 'pounds yams cubed', 'cup milk', 'cup butter', 'teaspoon rosemary', 'cup Parmesan cheese', 'salt pepper', 'ingredients', 'ingredients'],
    ['cups chicken stock', 'teaspoons salt', 'cups fresh pumpkin', 'teaspoon fresh parsley', 'cup onion', 'teaspoon fresh thyme', 'clove garlic', 'cup heavy whipping cream', 'whole black peppercorns', 'ingredients', 'ingredients'],
    ['cloves garlic', 'cup prunes', 'small green olives', 'tablespoons capers liquid', 'tablespoons olive oil', 'tablespoons red wine vinegar', 'bay leaves', 'tablespoon oregano', 'salt pepper', 'pound whole chicken skin pieces', 'cup brown sugar', 'cup dry white wine', 'tablespoon fresh parsley garnish', 'ingredients', 'ingredients'],
    ['pound lean ground beef', 'ounce package taco seasoning mix', 'ounce corn chips', 'cups lettuce', 'fresh tomato', 'cup Cheddar cheese', 'cup salsa', 'cup sour cream', 'ingredients', 'ingredients'],
    ['chicken broth', 'cup water', 'pound chicken breast meat', 'teaspoons oregano', 'teaspoon pepper', 'cup uncooked long grain rice', 'tablespoon olive oil', 'cloves garlic', 'onion', 'carrot', 'pound mushrooms', 'tablespoons purpose flour', 'cup milk', 'ingredients', 'ingredients'],
    ['tablespoon butter', 'cup Italian salad dressing', 'lemon', 'tablespoon Worcestershire sauce', 'chicken tenderloins', 'lemon pepper', 'garlic salt', 'onion powder', 'ingredients', 'ingredients'],
    ['pound chicken drumsticks', 'tablespoon adobo seasoning', 'ounce packet sazon seasoning', 'teaspoon salt', 'large red potatoes', 'large red bell pepper seeded', 'large green bell pepper seeded', 'large onion', 'cloves garlic', 'bunch fresh cilantro', 'tablespoons olive oil', 'cup dry red wine', 'teaspoon ground cumin', 'teaspoon oregano', 'fresh bay leaves', 'ingredients', 'ingredients'],
    ['large head cauliflower small florets', 'ounce cream chicken soup', 'cup mayonnaise', 'cup milk', 'teaspoons curry powder', 'wheat crackers', 'cup butter', 'ingredients', 'ingredients'],
    ['cup vegetable oil', 'cup Worcestershire sauce', 'cup soy sauce', 'cup steak sauce', 'cup water', 'teaspoon salt', 'teaspoon pepper', 'yellow onion', 'cloves garlic', 'ingredients', 'ingredients'],
    ['cup cream cheese', 'cup sharp Cheddar cheese', 'jalapeno peppers seeds membranes', 'slices bacon', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'ounce skinless boneless chicken breast halves cubes', 'teaspoons chili powder', 'teaspoons ground cumin', 'large onion', 'medium green pepper', 'ounce Campbell Condensed Cream Chicken Soup Regular % Fat Free', 'cup water', 'cups frozen whole kernel corn', 'ounce cans white kidney beans cannellini', 'tablespoons Cheddar cheese', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'onions', 'cups cubed potatoes', 'cups celery', 'cups carrots', 'cloves garlic', 'tablespoon Italian seasoning', 'cups milk', 'tablespoon cornstarch', 'ounce tomatoes', 'cups chicken broth', 'tablespoons tomato paste', 'salt pepper', 'ingredients', 'ingredients'],
    ['sweet potatoes', 'ounce package cream cheese', 'green onions', 'teaspoon chili powder', 'teaspoon ground cumin optional', 'teaspoon oregano', 'teaspoon salt', 'teaspoon ground black pepper', 'cup vegetable oil frying', 'inch corn tortillas', 'ounce sauce', 'ounce package Monterey Jack cheese', 'ingredients', 'ingredients'],
    ['sweet potatoes', 'cup white sugar', 'cup milk', 'cup butter', 'teaspoon vanilla extract', 'eggs', 'teaspoon salt', 'cup dark brown sugar', 'cup purpose flour', 'cup butter', 'cup pecans', 'ingredients', 'ingredients'],
    ['ounce squares German sweet chocolate', 'cup water', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'cup butter', 'cups white sugar', 'egg yolks', 'teaspoon vanilla extract', 'cup buttermilk', 'egg whites', 'fluid ounces milk', 'cups white sugar', 'cup butter', 'egg yolks', 'teaspoons vanilla extract', 'ounce package coconut', 'cups pecans', 'ingredients', 'ingredients'],
    ['pound whole chicken pieces', 'tablespoons soy sauce', 'tablespoons honey', 'cup orange juice', 'cloves garlic', 'teaspoon oregano', 'teaspoon ground black pepper', 'teaspoon paprika', 'ingredients', 'ingredients'],
    ['ounce package lasagna noodles', 'tablespoons olive oil', 'onion', 'ounce mushrooms', 'tablespoons garlic', 'zucchini', 'ounce cans tomatoes', 'ounce tomato paste', 'tablespoon oregano', 'brown sugar', 'salt', 'ounce package spinach', 'ounce container nonfat cottage cheese', 'eggs', 'tablespoons basil', 'cup Parmesan cheese', 'mozzarella cheese', 'ingredients', 'ingredients'],
    ['cup olive oil', 'tablespoons butter', 'onions', 'teaspoon basil', 'pinch ground black pepper', 'cup water', 'tablespoon chicken bouillon', 'pound uncooked pasta package directions', 'ingredients', 'ingredients'],
    ['roma plum tomatoes', 'cloves garlic unpeeled', 'small onion', 'jalapeno chile pepper', 'tablespoons olive oil', 'teaspoon ground cumin', 'teaspoon salt', 'tablespoons fresh lime juice', 'cup fresh cilantro', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoon salt', 'teaspoons powder', 'cups white sugar', 'cup vegetable oil', 'eggs', 'cup orange juice', 'teaspoons vanilla extract', 'apples', 'teaspoons ground cinnamon', 'teaspoons white sugar', 'ingredients', 'ingredients'],
    ['pounds beef top sirloin thin bite size slices', 'teaspoon salt', 'teaspoon black pepper', 'crushed red pepper', 'lime', 'ounce tomatillos', 'fresh jalapeno peppers seeded', 'tablespoons canola oil', 'ounce broth', 'inch corn tortillas', 'large onion', 'tomatoes', 'avocado', 'bunch fresh cilantro', 'lemon', 'ingredients', 'ingredients'],
    ['pound ground beef', 'onion', 'red potatoes', 'carrots', 'cup water', 'ounce whole kernel corn', 'cups frozen green beans', 'fluid ounce bottle tomato vegetable juice cocktail', 'cup water', 'teaspoon salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['ounce frozen broccoli', 'tablespoons butter', 'teaspoon salt', 'tablespoons purpose flour', 'cups milk', 'ounce package cream cheese', 'cup buttery round crackers', 'ingredients', 'ingredients'],
    ['pound ground beef', 'garlic powder', 'salt pepper', 'ounce package tater tots', 'ounce cream mushroom soup', 'ounce cream celery soup', 'ounce', 'garlic powder', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'yellow onion', 'cloves garlic', 'pound butternut squash inch chunks', 'teaspoon brown sugar', 'cup chicken broth', 'ounces Gruyere cheese', 'ounces extra-sharp Cheddar cheese', 'cup dry bread crumbs', 'tablespoon fresh thyme', 'tablespoon fresh rosemary', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'pound beef sirloin inch strips', 'cups fresh broccoli florets', 'red bell pepper matchsticks', 'carrots', 'green onion', 'teaspoon garlic', 'tablespoons soy sauce', 'tablespoons sesame seeds', 'ingredients', 'ingredients'],
    ['inch pre-baked pizza crust', 'cup pesto', 'ripe tomato', 'cup green bell pepper', 'ounce black olives', 'small red onion', 'ounce hearts', 'cup feta cheese', 'ingredients', 'ingredients'],
    ['large green tomatoes', 'red bell peppers seeded', 'green bell peppers seeded', 'large onions', 'tablespoons celery seed', 'tablespoons mustard seed', 'tablespoon salt', 'cups white sugar', 'cider vinegar', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'tablespoon pumpkin pie spice', 'teaspoons powder', 'teaspoon salt', 'eggs', 'cups white sugar', 'ounce pumpkin puree', 'cup vegetable oil', 'cup cranberries', 'cup walnuts', 'ingredients', 'ingredients'],
    ['cheese', 'cups mayonnaise', 'pound package hash brown potatoes', 'white onion', 'ounce jar real bacon bits', 'ingredients', 'ingredients'],
    ['eggs', 'cup celery', 'cup onion', 'cup bean sprouts', 'cup fresh mushrooms', 'cup chicken breast', 'cup ground beef', 'cup pork', 'teaspoon salt', 'teaspoon ground black pepper', 'FOO YUNG SAUCE', 'cubes chicken bouillon', 'cups hot water', 'teaspoons white sugar', 'tablespoons soy sauce', 'tablespoons cold water', 'tablespoons cornstarch', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoons soda', 'teaspoon salt', 'teaspoons ground cinnamon', 'teaspoon ground mace', 'teaspoon ground nutmeg', 'teaspoon ground cloves', 'cup butter', 'cups brown sugar', 'cup white sugar', 'eggs', 'tablespoon milk', 'teaspoons vanilla extract', 'cup cornflakes cereal', 'cups oats', 'cup coconut', 'cups chocolate chips', 'cup walnuts', 'ingredients', 'ingredients'],
    ['inch unbaked pie crusts', 'eggs', 'cups sour cream', 'teaspoon vanilla extract', 'cup white sugar', 'pinch salt', 'cup purpose flour', 'cups raspberries', 'cup brown sugar', 'cup purpose flour', 'cup walnuts', 'cup butter', 'ingredients', 'ingredients'],
    ['teaspoon olive oil', 'eggs', 'tablespoons water', 'cup Parmesan cheese', 'cup Italian breadcrumbs', 'salt black pepper', 'large eggplants inch rounds', 'tablespoons olive oil', 'pound ground beef', 'ounces chunky tomato sauce such Prego', 'cups mozzarella cheese', 'ingredients', 'ingredients'],
    ['pound lean ground beef', 'onion', 'ounce ranch style beans', 'teaspoon ground cumin', 'cloves garlic', 'cups cabbage', 'green bell pepper', 'ounce tomatoes liquid', 'stalks celery', 'cup picante sauce', 'cup water', 'salt', 'pepper', 'ingredients', 'ingredients'],
    ['ounce pizza crust dough', 'cup olive oil', 'tablespoons fresh garlic', 'tablespoons Parmesan cheese', 'tablespoons fresh parsley', 'teaspoon red pepper', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['cup water', 'cup white sugar', 'ounce fresh cranberries', 'cup apples diced', 'cup cider vinegar', 'cup raisins', 'teaspoon ground cinnamon', 'teaspoon ground ginger', 'teaspoon ground allspice', 'teaspoon ground cloves', 'ingredients', 'ingredients'],
    ['ounce pumpkin puree', 'cups white sugar', 'cup vegetable oil', 'eggs', 'cups purpose flour', 'teaspoons soda', 'teaspoon ground cinnamon', 'teaspoon salt', 'ounce package cream cheese', 'tablespoons butter', 'teaspoon vanilla extract', 'cups confectioners sugar', 'teaspoons milk', 'cup walnuts', 'ingredients', 'ingredients'],
    ['green bell peppers', 'red bell peppers', 'fresh jalapeno peppers', 'celery stalk', 'medium carrot', 'small onion', 'cup fresh cauliflower florets', 'cup salt', 'water', 'cloves garlic', 'tablespoon oregano', 'teaspoon red pepper flakes', 'teaspoon black pepper', 'ounce jar pimento green olives', 'cup white vinegar', 'cup olive oil', 'ingredients', 'ingredients'],
    ['clove garlic', 'teaspoon chili powder', 'teaspoon cayenne pepper', 'pod cardamom seeds', 'teaspoon water', 'teaspoon vegetable oil', 'cup rice wine vinegar', 'cup sugar', 'mango', 'teaspoon salt', 'teaspoon cilantro', 'teaspoons lemon juice', 'fresh jalapeno pepper', 'cups unsweetened applesauce', 'pineapple rings', 'white pepper', 'cup soy sauce', 'cup rice wine vinegar', 'pork chops', 'ingredients', 'ingredients'],
    ['medium yellow squash', 'onion', 'tablespoons', 'cup purpose flour', 'cup cornmeal', 'egg', 'cup Cheddar cheese', 'ground black pepper', 'tablespoon vegetable oil', 'ingredients', 'ingredients'],
    ['bunch fresh asparagus', 'cup water', 'cup olive oil', 'cup Parmesan cheese', 'grape tomatoes', 'ingredients', 'ingredients'],
    ['cup butter', 'cup shortening', 'cups white sugar', 'eggs', 'cups purpose flour', 'teaspoon baking powder', 'cup milk', 'teaspoon coconut extract', 'teaspoon lemon extract', 'teaspoon rum extract', 'teaspoon butter extract', 'teaspoon vanilla extract', 'cup white sugar', 'cup water', 'teaspoon coconut extract', 'teaspoon rum extract', 'teaspoon butter extract', 'teaspoon lemon extract', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['ounce package lemon cake mix', 'ounce instant lemon pudding mix', 'cup water', 'cup vegetable oil', 'eggs', 'tablespoons key lime juice', 'cups confectioners sugar', 'tablespoons key lime juice', 'ingredients', 'ingredients'],
    ['ounce cream cheese', 'ounce package feta cheese', 'cloves garlic', 'tablespoons fresh dill', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'teaspoon ground cumin', 'tablespoons fresh cilantro', 'lime', 'salt black pepper', 'skinless boneless chicken breast halves cut inch cubes', 'small zucchini inch slices', 'onion wedges', 'red bell pepper inch pieces', 'cherry tomatoes', 'ingredients', 'ingredients'],
    ['cup unsalted butter', 'pounds fresh mushrooms', 'pinch salt', 'yellow onion', 'tablespoons purpose flour', 'sprigs fresh thyme', 'cloves garlic', 'chicken broth', 'cup water', 'cup heavy whipping cream', 'pinch salt black pepper', 'teaspoon fresh thyme leaves garnish', 'ingredients', 'ingredients'],
    ['cup butter', 'cup brown sugar', 'Granny Smith apples', 'inch prerolled pie crusts', 'cup white sugar', 'teaspoons ground cinnamon', 'cup white sugar', 'tablespoon butter small chunks', 'ingredients', 'ingredients'],
    ['ounce package cream cheese', 'ounce jalapeno peppers', 'inch square wonton wrappers', 'cup sweet chili sauce', 'oil deep frying', 'ingredients', 'ingredients'],
    ['cup couscous', 'teaspoon ground cumin', 'teaspoon salt', 'cups water', 'unpeeled garlic', 'ounce black beans', 'cup whole kernel corn', 'cup red onion', 'cup fresh cilantro', 'jalapeno pepper', 'tablespoons olive oil', 'tablespoons fresh lime juice', 'ingredients', 'ingredients'],
    ['ounce package elbow macaroni', 'cup butter', 'cup sour cream', 'egg', 'tablespoons cream cheese', 'cups Asiago cheese', 'cups Vermont Cheddar cheese', 'bacon', 'tablespoons butter', 'large onion thin', 'cloves garlic', 'teaspoon brown sugar', 'cup fresh parsley', 'cup panko bread crumbs', 'tablespoons butter', 'tablespoons purpose flour', 'cups milk', 'teaspoon ground mustard', 'teaspoon paprika', 'teaspoon salt', 'teaspoon pepper', 'tablespoon hot pepper sauce', 'ingredients', 'ingredients'],
    ['cup short grain white rice', 'tablespoons rice vinegar', 'tablespoons white sugar', 'teaspoons salt', 'sheets seaweed', 'cucumber small strips', 'tablespoons pickled ginger', 'avocado', 'pound imitation crabmeat', 'ingredients', 'ingredients'],
    ['pound fresh mushrooms', 'cup margarine', 'green onions', 'cloves garlic', 'teaspoon fresh thyme', 'tablespoons purpose flour', 'vegetable broth', 'cup light cream', 'salt pepper', 'fresh thyme leaves', 'tablespoon fresh chives', 'ingredients', 'ingredients'],
    ['cup margarine', 'cup white sugar', 'eggs', 'cups purpose flour', 'teaspoon ground cinnamon', 'teaspoon ground allspice', 'teaspoons powder', 'teaspoon baking soda', 'cup applesauce', 'ingredients', 'ingredients'],
    ['cup quinoa', 'tablespoons olive oil', 'tablespoons raw sunflower seeds', 'cloves garlic', 'cup fresh spinach leaves', 'teaspoons lemon juice', 'cup goat gouda cheese', 'ingredients', 'ingredients'],
    ['ounce uncooked farfalle bow tie pasta', 'tablespoon olive oil', 'medium red bell pepper', 'medium yellow bell pepper', 'cup kale', 'cloves garlic', 'basil', 'pinch ground cayenne pepper', 'salt black pepper', 'ounces feta cheese', 'ingredients', 'ingredients'],
    ['uncooked lasagna noodles', 'onion', 'cup butter', 'cups dry potato flakes', 'ounce package cream cheese', 'ingredients', 'ingredients'],
    ['cups pasta', 'ounce package imitation crabmeat', 'small green bell pepper', 'sweet onion', 'cup black olives', 'radishes', 'tomato seeded diced', 'cup celery', 'cup mayonnaise', 'tablespoon white vinegar', 'teaspoon lemon juice', 'packet artificial sweetener', 'teaspoon garlic powder', 'teaspoon dill weed', 'teaspoon basil', 'teaspoon oregano', 'ingredients', 'ingredients'],
    ['pound beef flank steak', 'cup soy sauce', 'cup olive oil', 'teaspoons steak seasoning', 'ounces provolone cheese', 'slices thick cut bacon', 'cup fresh spinach leaves', 'cup crimini mushrooms', 'red bell pepper seeded strips', 'ingredients', 'ingredients'],
    ['cups whole milk', 'cups heavy cream', 'cup white sugar', 'teaspoon salt', 'teaspoons vanilla extract', 'cups fresh strawberries', 'red food coloring optional', 'ingredients', 'ingredients'],
    ['ounce jar green salsa verde', 'ounce tomatoes green chile peppers', 'ounce cans white beans', 'ounce chicken broth', 'ounce corn', 'onion', 'teaspoon oregano', 'teaspoon ground cumin', 'salt black pepper', 'skinless boneless chicken breasts', 'ingredients', 'ingredients'],
    ['cooking spray', 'cup bread crumbs', 'cup Parmesan cheese', 'eggs', 'zucchinis ends inch strips', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'tablespoon fresh oregano', 'tablespoon fresh basil', 'clove garlic', 'ground black pepper', 'ounce beans', 'large zucchini', 'cup mushrooms', 'tablespoon fresh cilantro', 'tomato', 'ingredients', 'ingredients'],
    ['eggplant', 'cup olive oil', 'salt pepper', 'medium shrimp', 'cup fresh basil', 'cloves garlic', 'cup white wine', 'cup Italian seasoned bread crumbs', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['turkey carcass', 'cups leftover stuffing', 'celery stalks', 'carrots', 'onion diced', 'bay leaves', 'tablespoon poultry seasoning', 'teaspoon ground sage', 'quarts chicken broth', 'garlic salt', 'ground black pepper', 'cups uncooked regular long grain white rice', 'ounce frozen green peas', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'boneless pork shoulder', 'large onion', 'cloves garlic', 'ounce cans tomatillos', 'ounce green chile peppers', 'fresh jalapeno peppers', 'cup fresh cilantro', 'teaspoon oregano', 'salt pepper', 'quart water', 'cup Monterey Jack cheese', 'cup sour cream', 'sprigs fresh cilantro garnish', 'ingredients', 'ingredients'],
    ['cups water', 'tablespoons butter', 'ounces dry bread stuffing mix', 'large eggplant', 'large zucchini', 'onion', 'tomato', 'teaspoon thyme', 'cups Colby cheese', 'salt', 'ground black pepper', 'ingredients', 'ingredients'],
    ['ounce cans mandarin oranges', 'ounce pineapple', 'pound cottage cheese', 'ounce orange flavored Jell O mix', 'ounce package topping', 'ingredients', 'ingredients'],
    ['medium head garlic', 'pounds steak', 'salt', 'teaspoons black pepper', 'tablespoons butter', 'cup green onions', 'cup dry red wine', 'ingredients', 'ingredients'],
    ['pounds sauerkraut', 'tablespoon caraway seeds optional', 'cup brown sugar', 'apple', 'pound bacon inch pieces', 'large onion', 'pounds kielbasa sausage inch thick slices', 'ingredients', 'ingredients'],
    ['cup pumpkin puree', 'quart milk', 'cup white sugar', 'teaspoon ground cinnamon', 'tablespoon vanilla extract', 'ingredients', 'ingredients'],
    ['cup quinoa', 'cup water', 'pinch salt', 'tablespoon olive oil', 'teaspoon garlic', 'small sweet onion', 'cup crimini mushrooms', 'small sweet potato diced', 'teaspoon cayenne pepper', 'salt pepper', 'cup pecans', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'medium onion', 'green bell pepper', 'large carrot', 'cloves garlic', 'ounce cans black beans', 'ounce chicken broth', 'cups ham', 'teaspoon cumin', 'teaspoon salt', 'ground black pepper', 'cup Cheddar cheese', 'cup sour cream', 'ingredients', 'ingredients'],
    ['mini phyllo tart shells', 'ounces cream cheese', 'cup Cheddar cheese', 'jalapeno peppers', 'tablespoon hot pepper sauce', 'bacon bits', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breasts', 'egg', 'tablespoon water', 'cup bread crumbs', 'teaspoon salt', 'teaspoon ground black pepper', 'teaspoon garlic powder', 'cup purpose flour', 'tablespoons butter margarine', 'tablespoons vegetable oil', 'tablespoons fresh lemon juice', 'tablespoons dry white wine', 'tablespoon fresh parsley', 'lemons wedges garnish', 'ingredients', 'ingredients'],
    ['cup bacon', 'pound lean ground beef', 'ounce pork beans', 'ounce kidney beans', 'ounce butter beans', 'ounce lima beans', 'cup ketchup', 'cup brown sugar', 'cup onion', 'tablespoons white wine vinegar', 'teaspoon mustard powder', 'cup celery', 'ingredients', 'ingredients'],
    ['oranges', 'cups mashed sweet potatoes', 'cup white sugar', 'cup orange juice', 'eggs', 'teaspoon vanilla extract', 'cup butter', 'tablespoon orange peel', 'cup brown sugar', 'teaspoon purpose flour', 'cup pecans', 'ingredients', 'ingredients'],
    ['ounces dry fettuccine pasta', 'cup butter', 'cup milk', 'tablespoon purpose flour', 'cup Parmesan cheese', 'pound salmon', 'cup fresh spinach', 'tablespoons capers', 'cup sun tomatoes', 'cup fresh oregano', 'ingredients', 'ingredients'],
    ['mango seeded diced', 'cup heavy cream', 'teaspoon lemon juice', 'cup macadamia nuts', 'cup seasoned bread crumbs', 'teaspoon olive oil', 'teaspoon black pepper', 'red pepper flakes', 'fresh sea bass', 'salt black pepper', 'cloves garlic', 'tablespoon extra virgin olive oil', 'ingredients', 'ingredients'],
    ['ounces fresh cauliflower', 'ounces fresh broccoli', 'tablespoons butter', 'tablespoons purpose flour', 'cups milk', 'ounce package sharp Cheddar cheese', 'teaspoon English mustard', 'cayenne pepper', 'salt pepper', 'ingredients', 'ingredients'],
    ['carrots', 'bananas', 'zucchini', 'cup vegetable oil', 'cup yogurt', 'eggs', 'cup whole wheat flour', 'teaspoons soda', 'cup brown sugar', 'cup oats', 'cup coconut', 'cup pecans', 'cup cherries', 'teaspoon ground cinnamon', 'teaspoon salt', 'teaspoon ground ginger', 'ingredients', 'ingredients'],
    ['bell peppers color', 'pound ground beef', 'egg', 'slices whole wheat bread', 'small onion', 'small tomato', 'cloves garlic', 'cup chili sauce', 'cup yellow mustard', 'tablespoons Worcestershire sauce', 'teaspoon salt', 'teaspoon pepper', 'ingredients', 'ingredients'],
    ['pound black beans', 'cup olive oil', 'large onion', 'medium green bell pepper', 'cloves garlic', 'cups water', 'ounce tomato paste', 'ounce jar pimentos', 'tablespoon vinegar', 'teaspoons salt', 'teaspoon white sugar', 'teaspoon black pepper', 'ingredients', 'ingredients'],
    ['ounce cans vegetable broth', 'cup small seashell pasta', 'tablespoon olive oil', 'onion', 'cloves garlic', 'ounce beans', 'ounce tomatoes juice', 'teaspoon basil', 'teaspoon thyme', 'salt pepper', 'ingredients', 'ingredients'],
    ['boneless skinless chicken breast halves', 'tablespoon ground cinnamon', 'salt pepper', 'large baking potatoes cubed', 'cup canola oil', 'large yellow onion', 'large clove garlic', 'tablespoon jalapeno peppers', 'lime', 'inch corn tortillas', 'ingredients', 'ingredients'],
    ['cup butter', 'cups fresh mushrooms', 'cup purpose flour', 'teaspoon garlic powder', 'teaspoon ground black pepper', 'teaspoon salt', 'pounds skinless boneless chicken meat cut pieces', 'slices bacon', 'cup cold water', 'teaspoon cornstarch', 'cup dry Marsala wine', 'ingredients', 'ingredients'],
    ['jumbo shrimp deveined', 'cup olive oil', 'teaspoons sesame oil', 'cup lemon juice', 'onion', 'cloves garlic', 'tablespoons fresh ginger root', 'tablespoons fresh cilantro leaves', 'teaspoon paprika', 'teaspoon salt', 'teaspoon ground black pepper', 'skewers', 'ingredients', 'ingredients'],
    ['pounds ground beef', 'onion', 'teaspoon salt', 'pinch ground black pepper', 'ounce package tater tots', 'ounce cream mushroom soup', 'ounce cream celery soup', 'ounce French onion rings', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'leek', 'ounce jars artichoke hearts', 'ounce jar red peppers', 'cup Parmesan cheese', 'tablespoons mayonnaise', 'ingredients', 'ingredients'],
    ['ounce tomatoes', 'ounce cans great Northern beans undrained', 'ounce spinach', 'ounce cans chicken broth', 'ounce tomato sauce', 'cups water', 'tablespoon garlic', 'slices crisp bacon', 'tablespoon parsley', 'teaspoon garlic powder', 'teaspoons salt', 'teaspoon ground black pepper', 'teaspoon basil', 'pound seashell pasta', 'ingredients', 'ingredients'],
    ['inch unbaked pie crust', 'cups mashed sweet potatoes', 'tablespoons butter', 'eggs', 'cup white sugar', 'tablespoon purpose flour', 'teaspoon salt', 'cup buttermilk', 'teaspoon baking soda', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['onion', 'tablespoon vegetable oil', 'ounce tomatoes', 'ounce black beans undrained', 'teaspoon oregano', 'teaspoon garlic powder', 'cup instant white rice', 'ingredients', 'ingredients'],
    ['cup Parmesan cheese', 'cup mayonnaise', 'cup heavy whipping cream', 'ounce hearts', 'tablespoons green onion', 'tablespoon pimento peppers', 'ingredients', 'ingredients'],
    ['cup peanut butter', 'cup brown sugar', 'cup corn syrup', 'cup butter', 'teaspoons vanilla extract', 'cups oats', 'cup coconut', 'cup sunflower seeds', 'cup raisins', 'cup semisweet chocolate chips', 'ingredients', 'ingredients'],
    ['cups white sugar', 'eggs', 'cup water', 'teaspoons soda', 'teaspoon baking powder', 'teaspoon ground allspice', 'cups purpose flour', 'teaspoons salt', 'teaspoon ground nutmeg', 'teaspoon ground cinnamon', 'teaspoon ground cloves', 'ounce pumpkin puree', 'cup walnuts', 'cup raisins', 'cup vegetable oil', 'ingredients', 'ingredients'],
    ['pounds skinless boneless chicken breast halves', 'tablespoons vegetable oil', 'onion', 'cloves garlic', 'quarts chicken broth', 'cups water', 'teaspoon oregano', 'teaspoons salt', 'tablespoons chili powder', 'cups white hominy', 'tostada shells', 'ingredients', 'ingredients'],
    ['ounce cans black eyed peas', 'tomatoes', 'medium sweet onion', 'cup jalapeno peppers', 'fluid ounces Italian dressing', 'cayenne pepper', 'avocados', 'ingredients', 'ingredients'],
    ['French baguettes inch diagonal slices', 'large minced onion', 'cloves garlic', 'cup butter', 'cups mozzarella cheese', 'cup Parmesan cheese', 'cup mayonnaise', 'ingredients', 'ingredients'],
    ['cup onion', 'pint sour cream', 'ounce cream chicken soup', 'cups Cheddar cheese', 'salt pepper', 'pound package hash brown potatoes', 'cups potato chips', 'cup butter', 'ingredients', 'ingredients'],
    ['ounce package spinach', 'ounce container sour cream', 'cup mayonnaise', 'ounce packet dry vegetable soup mix', 'ingredients', 'ingredients'],
    ['cup butter', 'small white onion', 'small zucchinis', 'ears corn husks silk', 'sea salt', 'black pepper', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cup unsweetened cocoa powder', 'teaspoons soda', 'teaspoon salt', 'cup shortening', 'cups white sugar', 'eggs', 'teaspoon vanilla extract', 'cups buttermilk', 'cup kirschwasser', 'cup butter', 'cups confectioners sugar', 'pinch salt', 'teaspoon strong coffee', 'ounce cans Bing cherries', 'cups heavy whipping cream', 'teaspoon vanilla extract', 'tablespoon kirschwasser', 'ounce square semisweet chocolate', 'ingredients', 'ingredients'],
    ['tablespoon butter', 'skinless chicken thighs', 'salt pepper', 'ounce cream celery soup', 'ounce cream mushroom soup', 'ounce jar pimento green olives', 'ounce package fresh mushrooms', 'cups Chablis wine', 'tablespoon purpose flour', 'ingredients', 'ingredients'],
    ['ounce dry fettuccini noodles', 'cup olive oil', 'pound shrimp deveined', 'green onions', 'cloves garlic', 'cup brandy', 'large tomatoes', 'cup dry white wine', 'cup butter', 'cup fresh basil', 'teaspoons salt', 'teaspoon pepper', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['ounce cream style corn', 'ounce style corn', 'eggs', 'cup sour cream', 'ounce dry corn bread mix', 'cup butter', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['onions', 'beef broth', 'tablespoons vegetable oil', 'tablespoons butter margarine', 'pound Swiss cheese', 'teaspoon white sugar', 'cup white wine', 'salt pepper', 'pound loaf French bread', 'ingredients', 'ingredients'],
    ['cup zucchini', 'cup red bell pepper', 'cup yellow bell pepper', 'cup yellow squash', 'cup red onion', 'large fresh button mushrooms', 'cherry tomatoes', 'cup olive oil', 'cup soy sauce', 'cup lemon juice', 'clove garlic', 'ingredients', 'ingredients'],
    ['pounds sweet potatoes inch pieces', 'pound chicken tenders', 'teaspoons steak seasoning such Montreal Steak Seasoning', 'tablespoons vegetable oil', 'cup maple syrup', 'cup green onions', 'ingredients', 'ingredients'],
    ['cup butter', 'large onion', 'ounce package broccoli', 'cup milk', 'ounce Campbell Condensed Cream Chicken Soup Regular % Fat Free', 'fl oz jar process cheese sauce Cheez Whiz', 'cups regular long grain white rice', 'ingredients', 'ingredients'],
    ['cup white sugar', 'cup purpose flour', 'teaspoons powder', 'cup milk', 'cup margarine', 'cups fresh blackberries', 'cup white sugar', 'cup water', 'ingredients', 'ingredients'],
    ['cups seeded watermelon', 'cup fresh strawberries', 'cup fresh lemon juice', 'cup white sugar', 'cups water', 'ingredients', 'ingredients'],
    ['ounce cranberry sauce', 'ounce dark sweet cherries', 'ounces pineapple juice', 'ounce package cherry Jell O r', 'cups water', 'cup pecans optional', 'ingredients', 'ingredients'],
    ['cup dry split peas', 'cup beef bouillon granules', 'cup pearl barley', 'cup dry lentils', 'cup onion flakes', 'teaspoons Italian seasoning', 'cup uncooked long grain white rice', 'bay leaves', 'cup alphabet pasta', 'ingredients', 'ingredients'],
    ['sweet potato mashed', 'cup milk', 'cup orange juice', 'teaspoon vanilla extract', 'cup white sugar', 'teaspoon salt', 'tablespoons butter', 'teaspoon ground nutmeg', 'teaspoon ground cinnamon', 'cup butter', 'cup brown sugar', 'cup purpose flour', 'cup pecans', 'ingredients', 'ingredients'],
    ['ounce yellow cake mix', 'ounce instant vanilla pudding mix', 'eggs', 'cup sour cream', 'cup vegetable oil', 'apples', 'cup white sugar', 'teaspoons ground cinnamon', 'cup walnuts', 'ingredients', 'ingredients'],
    ['cups uncooked long grain rice', 'tablespoon vegetable oil', 'ounce frozen broccoli florets', 'green onions', 'eggs', 'tablespoons soy sauce', 'teaspoon salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['bagel toasted', 'tablespoons cream cheese', 'roma plum tomatoes', 'salt pepper', 'fresh basil', 'ingredients', 'ingredients'],
    ['tablespoons unsalted butter', 'cup olive oil', 'tablespoon garlic', 'tablespoon shallots', 'tablespoons fresh chives', 'salt black pepper', 'teaspoon paprika', 'large shrimp deveined', 'ingredients', 'ingredients'],
    ['ounce green beans', 'pound wax beans', 'ounce kidney beans', 'onion thin rings', 'cup white sugar', 'cup white vinegar', 'cup vegetable oil', 'teaspoon salt', 'teaspoon ground black pepper', 'teaspoon celery seed', 'ingredients', 'ingredients'],
    ['pound eggplant', 'tablespoons soy sauce', 'cup chicken stock', 'teaspoon chili sauce', 'teaspoon white sugar', 'teaspoon ground black pepper', 'tablespoons oyster sauce optional', 'tablespoon cornstarch', 'tablespoons water', 'cloves garlic', 'large green onions', 'tablespoon fresh ginger root', 'pound fresh shrimp diced', 'pound lean ground beef', 'tablespoon sesame oil', 'cups hot rice', 'ingredients', 'ingredients'],
    ['cup shortening', 'cup brown sugar', 'egg', 'cup ripe banana', 'teaspoon vanilla extract', 'cup purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'teaspoon ground cinnamon', 'teaspoon ground cloves', 'cups oats', 'cup walnuts', 'ingredients', 'ingredients'],
    ['carrots inch pieces', 'cup water', 'tablespoons butter', 'teaspoon white sugar', 'teaspoon salt', 'teaspoon dill seed', 'ingredients', 'ingredients'],
    ['medium cucumber', 'ounce whole kernel corn', 'ounce tomatoes', 'green bell pepper', 'red bell pepper', 'tablespoons red wine vinegar', 'tablespoon red pepper flakes', 'teaspoon garlic', 'teaspoon cumin', 'teaspoon cilantro', 'teaspoon salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['ounce cans chick peas garbanzo beans', 'tablespoons olive oil', 'garlic cloves', 'teaspoons curry powder', 'tablespoons fresh lemon juice', 'cup water', 'Salt', 'Hot sauce', 'ingredients', 'ingredients'],
    ['ounce green beans', 'ounce beans', 'ounce beans', 'ounce kidney beans', 'ounce black beans', 'cup green pepper', 'cup onion', 'cup celery', 'cup salad oil', 'cup vinegar', 'teaspoon salt', 'teaspoon ground black pepper', 'cup white sugar', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'pound sirloin steak strips', 'ounces fresh mushrooms', 'green bell pepper seeded strips', 'medium onion', 'slices provolone cheese', 'loaf French bread', 'ounce broth', 'teaspoon salt', 'teaspoon ground black pepper', 'teaspoon garlic powder', 'tablespoons Worcestershire sauce', 'teaspoon red pepper flakes', 'cup Pinot Noir other dry red wine', 'cup horseradish optional', 'cup brown mustard optional', 'ingredients', 'ingredients'],
    ['jalapeno peppers', 'cloves garlic', 'teaspoons black pepper', 'tablespoon coarse salt', 'cup lime juice', 'tablespoon oregano', 'pounds sirloin steak', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'potatoes cubed', 'carrots', 'stalks celery', 'sweet onion', 'cloves garlic', 'ounce tomato paste', 'ounce kidney beans', 'ounce vegetable broth', 'quarts water', 'tablespoons fresh basil', 'tablespoons fresh oregano', 'tablespoon salt', 'cups uncooked elbow macaroni', 'ingredients', 'ingredients'],
    ['fresh jalapeno peppers seeded', 'ounce package cream cheese', 'ounce frozen vegetarian bacon strips', 'toothpicks', 'ingredients', 'ingredients'],
    ['ounce fillets tilapia', 'tablespoons olive oil', 'salt pepper', 'lemon', 'cup white wine', 'tomatoes', 'tablespoons capers', 'cup asparagus spears half', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'tablespoons olive oil', 'ounce salmon steaks', 'teaspoon garlic', 'tablespoon lemon pepper', 'teaspoon salt', 'cup water', 'cup fresh tomatoes', 'cup fresh cilantro', 'cups water', 'cup uncooked couscous', 'ingredients', 'ingredients'],
    ['coconut milk', 'cups milk', 'tablespoons white sugar', 'cup Basmati rice', 'cup raisins', 'teaspoon ground cardamom', 'teaspoon water optional', 'cup almonds', 'cup pistachio nuts', 'ingredients', 'ingredients'],
    ['ounce package macaroni', 'tomatoes seeded diced', 'stalks celery', 'ounce tuna', 'cups light mayonnaise', 'tablespoon Italian style salad dressing', 'tablespoon white sugar', 'pinch ground black pepper', 'ingredients', 'ingredients'],
    ['cup brown sugar', 'tablespoons Worcestershire sauce', 'cups ketchup', 'pound bacon', 'ounce cans water chestnuts', 'ingredients', 'ingredients'],
    ['teaspoons black pepper', 'teaspoon paprika', 'teaspoon cayenne pepper', 'teaspoon garlic', 'tablespoon Dijon mustard', 'tablespoon brown sugar', 'teaspoon onion powder', 'teaspoon salt', 'tablespoon olive oil', 'ounce salmon fillets', 'tablespoons olive oil', 'tablespoons onion', 'ingredients', 'ingredients'],
    ['cup mayonnaise', 'ears corn husk silk', 'cup Parmesan cheese', 'tablespoon chili powder', 'teaspoon salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['cup lemon juice', 'cup olive oil', 'tablespoons Greek seasoning', 'teaspoons lemon zest', 'ounce skinless boneless chicken breast halves', 'ounce package feta cheese', 'sun tomatoes', 'kalamata olives optional', 'tablespoon oil sun tomatoes', 'strips red pepper', 'ingredients', 'ingredients'],
    ['cup plain low fat Greek yogurt', 'lemon', 'tablespoon lemon zest', 'tablespoon olive oil', 'cloves garlic', 'tablespoon paprika', 'teaspoon Provence', 'teaspoon salt', 'teaspoon ground black pepper', 'pound whole chicken pieces', 'cup plain low fat Greek yogurt', 'tablespoon lemon juice', 'teaspoon harissa', 'pinch salt', 'ingredients', 'ingredients'],
    ['large tomatoes', 'onion', 'cup fresh cilantro', 'cloves garlic', 'tablespoon lime juice', 'tomatillo optional', 'salt', 'jalapeno pepper', 'ingredients', 'ingredients'],
    ['ounce yellow cake mix', 'ounce package strawberry Jell O mix', 'tablespoons purpose flour', 'cup water', 'cup vegetable oil', 'eggs', 'ounce frozen strawberries', 'cup butter', 'cups confectioners sugar', 'ingredients', 'ingredients'],
    ['ounces butter', 'cup white sugar', 'cup brown sugar', 'egg', 'teaspoon vanilla extract', 'cup pumpkin puree', 'cup oats', 'teaspoon baking powder', 'teaspoon ground cinnamon', 'teaspoon salt', 'cup raisins', 'cups purpose flour', 'ingredients', 'ingredients'],
    ['large tomatoes', 'large onion', 'cloves garlic', 'cup fresh cilantro', 'jalapeno pepper seeded', 'tablespoons fresh lime juice', 'salt', 'ingredients', 'ingredients'],
    ['chicken thighs', 'large potatoes', 'cup vegetable oil', 'cup wine vinegar', 'lemons', 'cloves garlic', 'tablespoons oregano', 'tablespoons parsley', 'onion', 'salt pepper', 'ingredients', 'ingredients'],
    ['pound linguine pasta', 'cup olive oil', 'cup butter', 'cloves garlic', 'pound bay scallops', 'pound medium shrimp deveined', 'ounce jar clam juice', 'cup sun tomatoes', 'cup fresh parsley', 'teaspoons lemon zest', 'teaspoon salt', 'teaspoon red pepper flakes', 'ingredients', 'ingredients'],
    ['spaghetti squash seeded', 'cup pine nuts', 'cup Pecorino Romano cheese', 'tablespoons fresh sage', 'teaspoons butter', 'salt pepper', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'cup onion', 'cloves garlic', 'cup carrot', 'cup celery', 'ounce cans tomatoes', 'cups vegetable broth', 'tablespoon vegetarian Worcestershire sauce', 'teaspoon salt', 'teaspoon thyme', 'teaspoon ground black pepper', 'hot pepper sauce', 'ingredients', 'ingredients'],
    ['fluid ounce frozen pink lemonade concentrate', 'fluid ounces water', 'fluid ounces vodka', 'fluid ounce beer', 'ingredients', 'ingredients'],
    ['sweet potato cubed', 'tablespoons orange juice', 'cup brown sugar', 'teaspoon ground nutmeg', 'tablespoons butter', 'cup miniature marshmallows', 'ingredients', 'ingredients'],
    ['cup onion', 'tablespoons olive oil', 'cup lentils', 'cups water', 'broccoli florets', 'cup fresh tomatoes', 'eggs', 'cup milk', 'teaspoon salt', 'ground black pepper', 'teaspoons Italian seasoning', 'cup Cheddar cheese optional', 'ingredients', 'ingredients'],
    ['inch refrigerated pie crust', 'teaspoons butter', 'leeks', 'pinch salt black pepper', 'cup light cream', 'cups Gruyere cheese', 'ingredients', 'ingredients'],
    ['red potatoes', 'cup mayonnaise', 'cup sour cream', 'tablespoon onion powder', 'tablespoon garlic powder', 'teaspoon seasoned salt', 'teaspoon ground black pepper', 'cup cooked bacon', 'cup red onion', 'cups American cheese', 'tablespoons Parmesan cheese', 'ingredients', 'ingredients'],
    ['cup butter', 'tablespoons garlic', 'large sea scallops', 'teaspoon salt', 'teaspoon pepper', 'tablespoons fresh lemon juice', 'ingredients', 'ingredients'],
    ['pounds beef stew meat inch pieces', 'cube beef bouillon', 'beef broth', 'large onion', 'teaspoon ground black pepper', 'bay leaf', 'potatoes cubed', 'cups cabbage', 'celery ribs', 'carrot', 'ounce tomato sauce', 'salt', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'tablespoon garlic', 'tablespoon fresh basil', 'tablespoon fresh rosemary', 'tablespoon fresh parsley', 'teaspoon red pepper flakes', 'teaspoon salt', 'large potatoes cubed', 'ingredients', 'ingredients'],
    ['ounce cream cheese', 'cup white sugar', 'ounce pumpkin puree', 'teaspoons ground cinnamon', 'teaspoon ground ginger', 'teaspoon ground nutmeg', 'eggs', 'teaspoon salt', 'inch pastry shells', 'ingredients', 'ingredients'],
    ['ounce jar artichoke hearts', 'roma plum tomatoes', 'tablespoons red onion', 'cup black olives', 'tablespoon garlic', 'tablespoons fresh basil', 'salt pepper', 'ingredients', 'ingredients'],
    ['cups mayonnaise', 'cup buttermilk', 'tablespoons white sugar', 'teaspoon celery seed', 'teaspoon ground black pepper', 'ounce packages coleslaw mix', 'ingredients', 'ingredients'],
    ['ounce sweet Italian sausage', 'green bell pepper', 'onion', 'teaspoons garlic', 'ounce tomatoes liquid', 'cup fresh parsley', 'teaspoons basil', 'chicken broth', 'pound orzo pasta', 'ounce package frozen spinach dry', 'Parmesan cheese', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'sheets dough', 'ounce halibut fillets', 'salt pepper', 'teaspoons fresh dill', 'tablespoons lemon juice', 'cup whipping cream', 'green onions', 'ingredients', 'ingredients'],
    ['ounce packages spinach', 'ounce package ranch dressing mix', 'cup mayonnaise', 'cup sour cream', 'cup bacon bits', 'tablespoons onions', 'inch flour tortillas', 'ingredients', 'ingredients'],
    ['cup margarine', 'cup honey', 'egg', 'teaspoon vanilla extract', 'cup stone ground whole wheat flour', 'teaspoon baking soda', 'teaspoon ground cinnamon', 'cups cooking oats', 'apple', 'ingredients', 'ingredients'],
    ['cup unsweetened applesauce', 'cup vegetable oil', 'eggs', 'cups purpose flour', 'cup white sugar', 'teaspoon baking soda', 'teaspoon ground cinnamon', 'teaspoon salt', 'cup fresh frozen cranberries', 'cup pecans', 'ingredients', 'ingredients'],
    ['small bananas chunks', 'cup frozen unsweetened strawberries', 'ounce container vanilla low fat yogurt', 'cup milk', 'ingredients', 'ingredients'],
    ['ounces bittersweet chocolate', 'pint fresh strawberries leaves', 'ingredients', 'ingredients'],
    ['ounce peaches', 'white bread crusts', 'cups white sugar', 'tablespoons self flour', 'egg', 'cup margarine', 'ingredients', 'ingredients'],
    ['kiwi', 'banana', 'cup blueberries', 'cup strawberries', 'cup ice cubes', 'cup orange juice', 'ounce container peach yogurt', 'ingredients', 'ingredients'],
    ['cup butter', 'large Vidalia onions', 'teaspoons sugar', 'ounce package pizza dough', 'pound Gorgonzola cheese', 'ingredients', 'ingredients'],
    ['tablespoons soy sauce', 'tablespoon brown sugar', 'teaspoons garlic powder', 'teaspoons peanut butter', 'teaspoons olive oil', 'ounce package mixed vegetables', 'ingredients', 'ingredients'],
    ['large head cabbage', 'ounce tomatoes juice', 'onion', 'tablespoon Italian seasoning', 'salt black pepper', 'pound lean ground beef', 'ingredients', 'ingredients'],
    ['russet potatoes inch thick fries', 'tablespoons olive oil', 'tablespoons lime juice', 'cloves garlic', 'teaspoon red pepper flakes', 'teaspoon cayenne pepper', 'teaspoon chili powder', 'tablespoons spicy brown mustard', 'teaspoon ground black pepper', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['boneless pork chops', 'fluid ounce cola carbonated beverage', 'ounce envelope dry onion soup mix', 'ingredients', 'ingredients'],
    ['avocados diced', 'tablespoon sour cream', 'ounce cream cheese', 'tablespoons salsa', 'pinch salt', 'dash ground black pepper', 'garlic salt', 'onion powder', 'ingredients', 'ingredients'],
    ['zucchini', 'summer squash', 'red onion chunks', 'pound asparagus inch pieces', 'pound portobello mushrooms', 'cloves garlic', 'tablespoons olive oil', 'white sugar', 'salt black pepper', 'cubes chicken bouillon', 'cup dry white wine', 'ounce package orzo pasta', 'tablespoons Parmesan cheese', 'ingredients', 'ingredients'],
    ['cups sweet potatoes', 'tablespoons vegetable oil', 'pound skinless boneless chicken breast halves', 'clove garlic', 'tablespoons soy sauce', 'cup water', 'tablespoons honey', 'tablespoons hot sauce', 'ripe mango cubed', 'teaspoon red pepper flakes', 'teaspoon cornstarch', 'tablespoon warm water', 'ingredients', 'ingredients'],
    ['cup butter', 'pound fresh mushrooms', 'cup seasoned croutons', 'cup mozzarella cheese', 'ounce', 'pound lobster tail', 'tablespoons garlic', 'cup mozzarella cheese optional', 'ingredients', 'ingredients'],
    ['tart green apples', 'cup almonds', 'cup cranberries', 'cup cherries', 'ounce container vanilla yogurt', 'ingredients', 'ingredients'],
    ['ounce package cream cheese', 'cup sour cream', 'cup mayonnaise', 'cup cocktail sauce', 'cup mild salsa', 'cup green onion', 'cup red bell pepper', 'cup green bell pepper', 'cups Cheddar cheese', 'cups lettuce', 'cucumber', 'ingredients', 'ingredients'],
    ['ounce black beans', 'cup salsa', 'tablespoon ground cumin', 'tablespoon chili powder', 'inch flour tortillas', 'cup Monterey Jack cheese', 'ingredients', 'ingredients'],
    ['ounce package corn bread mix', 'slices bacon', 'ounce package ranch dressing mix', 'cups sour cream', 'cups mayonnaise', 'ounce cans pinto beans', 'tomatoes', 'cup green bell pepper', 'cup green onion', 'cups Cheddar cheese', 'ounce cans whole kernel corn', 'ingredients', 'ingredients'],
    ['ounces cream cheese', 'cup margarine', 'cups white sugar', 'eggs', 'cup pumpkin', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon baking powder', 'teaspoon salt', 'teaspoon ground cinnamon', 'teaspoon ground cloves', 'cup walnuts', 'ingredients', 'ingredients'],
    ['ounce cans sweet potatoes', 'cup orange juice', 'cup purpose flour', 'cup white sugar', 'teaspoon ground cinnamon', 'pinch salt', 'cup margarine', 'cups miniature marshmallows', 'ingredients', 'ingredients'],
    ['inch flour tortillas', 'tablespoons vegetable oil', 'red onion', 'green bell pepper seeded strips', 'red bell pepper seeded strips', 'teaspoon garlic', 'yellow squash strips', 'cup salsa', 'teaspoon ground cumin', 'teaspoon salt', 'cup Monterey Jack cheese', 'cup fresh cilantro', 'ingredients', 'ingredients'],
    ['cloves garlic', 'tablespoons olive oil', 'cup white wine', 'cups asparagus', 'sprigs fresh thyme', 'ingredients', 'ingredients'],
    ['inch pie shell', 'ounce package cream cheese', 'cup white sugar', 'teaspoon vanilla extract', 'egg', 'cups pumpkin puree', 'cup milk', 'eggs', 'cup brown sugar', 'cup white sugar', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'teaspoon salt', 'tablespoons purpose flour', 'tablespoons brown sugar', 'tablespoons butter', 'cup pecans', 'ingredients', 'ingredients'],
    ['ounce package cream cheese', 'cup white sugar', 'tablespoon purpose flour', 'egg', 'tablespoon orange zest', 'cups purpose flour', 'teaspoon baking soda', 'teaspoon salt', 'teaspoon ground cinnamon', 'teaspoon ground cloves', 'teaspoon pumpkin pie spice', 'cup pumpkin puree', 'cup vegetable oil', 'eggs', 'cups white sugar', 'ingredients', 'ingredients'],
    ['pork tenderloin', 'cup olive oil', 'cloves garlic', 'teaspoon fresh thyme', 'tablespoon fresh rosemary', 'tablespoon salt', 'tablespoons black pepper', 'ingredients', 'ingredients'],
    ['ounce instant vanilla pudding mix', 'cup milk', 'ounce condensed milk', 'ounce container sour cream', 'ounce container topping', 'bananas', 'ounce package vanilla wafers', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'carrot', 'stalk celery', 'thin slice onion', 'teaspoon garlic', 'ounce cans tomato sauce', 'ounce chicken broth', 'black pepper', 'tablespoon parsley', 'tablespoon basil leaves', 'ounce beans', 'cups pasta', 'ingredients', 'ingredients'],
    ['pounds skinless boneless chicken breast halves', 'tablespoon lemon juice', 'tablespoon chili powder', 'salt', 'cup yogurt', 'salt', 'tablespoons garlic paste', 'tablespoon garam masala', 'tablespoons butter', 'tablespoon chili powder', 'tablespoons ginger paste', 'tablespoons lemon juice', 'tablespoons olive oil', 'tablespoon butter', 'tablespoon garam masala', 'tablespoon ginger paste', 'tablespoon garlic', 'tablespoon green chile pepper', 'tomato puree', 'tablespoon chili powder', 'salt', 'cup water', 'tablespoon honey', 'teaspoon fenugreek leaves', 'cup heavy cream', 'ingredients', 'ingredients'],
    ['cups ice', 'cup white sugar', 'ounces strawberries', 'cup lime juice', 'cup lemon juice', 'cup rum', 'cup lemon lime carbonated beverage', 'ingredients', 'ingredients'],
    ['boneless skinless chicken thighs breasts', 'ounce cream chicken soup', 'cloves garlic optional', 'ounce container sour cream', 'ounce green chile peppers', 'flour tortillas', 'cups Monterey Jack cheese', 'ounce black olives optional', 'chives garnish optional', 'black pepper', 'ingredients', 'ingredients'],
    ['cups graham cracker crumbs', 'cup sugar', 'cup butter margarine', 'ounce cream cheese', 'ounce EAGLE BRAND Condensed Milk', 'ounce pumpkin', 'large eggs', 'cup pure maple syrup', 'teaspoons ground cinnamon', 'teaspoon ground nutmeg', 'teaspoon salt', 'Maple Pecan Glaze', 'cup whipping cream', 'cup pure maple syrup', 'cup pecans', 'ingredients', 'ingredients'],
    ['ripe tomatoes', 'tablespoons mayonnaise', 'tablespoons Parmesan cheese', 'tablespoon Dijon mustard', 'tablespoon brown mustard', 'teaspoon oregano', 'salt pepper', 'teaspoon fresh parsley', 'cup mozzarella cheese', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'ounces skinless boneless chicken breast strips', 'red bell pepper', 'cup green onion', 'cloves garlic', 'cups brown rice', 'tablespoons soy sauce', 'tablespoon rice vinegar', 'cup frozen peas', 'ingredients', 'ingredients'],
    ['ounce low fat cream chicken herbs soup', 'ounce pieces', 'red onion', 'pounds skinless boneless chicken breast halves strips', 'Marsala wine', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'red onion', 'salt pepper', 'zucchini', 'pound fresh mushrooms', 'tomato', 'clove garlic', 'teaspoon Italian seasoning', 'ingredients', 'ingredients'],
    ['large yellow onion', 'tablespoons olive oil', 'teaspoon brown sugar', 'bunch red Swiss chard', 'cup kalamata olives', 'tablespoons capers', 'teaspoon coarse sea salt', 'black pepper', 'lemon', 'ingredients', 'ingredients'],
    ['tablespoon vegetable oil', 'boneless pork chops', 'ounce cans tomatoes juice', 'ounce whole kernel corn', 'ounce red kidney beans', 'cup uncooked long grain white rice', 'ounce green chilies', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'cups zucchini', 'teaspoon lemon zest', 'red pepper flakes', 'salt black pepper', 'cayenne pepper', 'tablespoons cream cheese', 'teaspoons fresh oregano', 'ingredients', 'ingredients'],
    ['tablespoons purpose flour', 'teaspoon garlic powder', 'teaspoon salt', 'teaspoon ground black pepper', 'pound skinless boneless chicken breast halves inch cubes', 'tablespoons vegetable oil', 'celery ribs', 'green bell peppers', 'onion', 'cup ketchup', 'cup lemon juice', 'cup pineapple syrup', 'cup brown sugar', 'ingredients', 'ingredients'],
    ['ounce skinless boneless salmon fillets', 'seasoned salt', 'teaspoon garlic powder', 'teaspoon onion powder', 'ounce package frozen puff pastry', 'cup pesto', 'ounce package spinach leaves', 'ingredients', 'ingredients'],
    ['ounce thin spaghetti', 'ounce sweet Italian sausage', 'assorted bell peppers seeded strips', 'clove garlic', 'cup extra virgin olive oil', 'teaspoon oregano', 'teaspoon basil', 'salt pepper', 'cup flat leaf parsley', 'Parmesan cheese', 'ingredients', 'ingredients'],
    ['large potatoes cubed', 'ounce package spinach', 'cup sour cream', 'cup butter', 'tablespoons green onions', 'teaspoons salt', 'teaspoon black pepper', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['ounce container tofu', 'cup % milk', 'teaspoon salt', 'teaspoon pepper', 'ounce package spinach', 'teaspoon garlic', 'cup diced onion', 'cup Cheddar cheese', 'cup Swiss cheese', 'unbaked inch pie crust', 'ingredients', 'ingredients'],
    ['ounce fillets tilapia bream', 'cup olive oil', 'cup raspberry vinegar', 'cup honey', 'teaspoon yellow mustard', 'teaspoon dill weed', 'ingredients', 'ingredients'],
    ['sugar pumpkin', 'ingredients', 'ingredients'],
    ['cup white sugar', 'tablespoons cornstarch', 'cups rhubarb', 'tablespoon water', 'tablespoon butter', 'teaspoon ground cinnamon', 'cup purpose flour', 'tablespoon white sugar', 'teaspoons powder', 'teaspoon salt', 'cup butter', 'cup milk', 'egg', 'tablespoons white sugar', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'tablespoon garlic salt', 'tablespoon fresh parsley', 'tablespoon fresh chives', 'teaspoon ground black pepper', 'teaspoon Old Bay Seasoning', 'teaspoon seasoning salt', 'skinless boneless chicken breast halves', 'medium green bell peppers', 'large tomatoes', 'red onion', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'medium onion', 'medium green bell pepper', 'cloves garlic', 'cup salsa', 'ounce whole kernel corn', 'pound ground turkey', 'ounce package taco seasoning mix', 'cup dry bread crumbs', 'inch flour tortillas', 'tablespoons sour cream', 'cups lettuce', 'ingredients', 'ingredients'],
    ['cup vegetable oil', 'sweet potatoes', 'tablespoons brown sugar', 'teaspoon ground cinnamon', 'teaspoon salt', 'ground pepper', 'tablespoon fresh lime juice optional', 'ingredients', 'ingredients'],
    ['ripe avocado', 'tablespoon lemon juice', 'tablespoon butter', 'large Vidalia onions rings', 'boneless skinless chicken breast halves', 'salt pepper', 'hamburger buns', 'tablespoons mayonnaise', 'provolone cheese', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'cups fresh lemon juice', 'cup purpose flour', 'teaspoons salt', 'teaspoons', 'teaspoon ground black pepper', 'cup vegetable oil', 'cup light brown sugar', 'cup chicken broth', 'lemon', 'cup fresh parsley', 'ingredients', 'ingredients'],
    ['acorn squash seeded', 'salt pepper', 'tablespoons butter', 'tablespoons brown sugar', 'ingredients', 'ingredients'],
    ['quarts apple cider', 'cups orange juice', 'cup lemon juice', 'whole cloves', 'cinnamon sticks', 'pinch ground ginger', 'pinch ground nutmeg', 'ingredients', 'ingredients'],
    ['small head red cabbage', 'apple diced', 'teaspoons salt', 'tablespoon lemon juice', 'cup water', 'tablespoon butter', 'small onion', 'teaspoon ground black pepper', 'tablespoon wine vinegar', 'pound kielbasa sausage inch pieces', 'ingredients', 'ingredients'],
    ['ounce whole kernel corn liquid', 'ounce cream style corn', 'eggs', 'ounce dry corn bread mix', 'cup sour cream', 'ounces Cheddar cheese', 'ingredients', 'ingredients'],
    ['cup shortening', 'cup white sugar', 'teaspoon vanilla extract', 'eggs', 'cups purpose flour', 'teaspoons powder', 'teaspoon salt', 'cup milk', 'pint blueberries', 'tablespoons white sugar', 'ingredients', 'ingredients'],
    ['cups pretzels', 'cup margarine', 'teaspoons white sugar', 'ounce package cream cheese', 'cup white sugar', 'ounces topping', 'ounce package strawberry Jell O r', 'cups water', 'ounce frozen strawberries', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'cup confectioners sugar', 'cup butter', 'cup white sugar', 'tablespoons purpose flour', 'teaspoon baking powder', 'eggs', 'tablespoons lemon juice', 'tablespoon lemon zest', 'cup confectioners sugar decoration', 'ingredients', 'ingredients'],
    ['ounce cream chicken soup', 'ounce cream mushroom soup', 'cups sour cream', 'cup dry white wine', 'onion', 'cup fresh mushrooms', 'teaspoon garlic powder', 'teaspoon salt', 'teaspoon ground black pepper', 'skinless boneless chicken breast halves', 'tablespoon fresh parsley garnish', 'ingredients', 'ingredients'],
    ['potatoes cubed', 'ounce chicken broth', 'leeks', 'teaspoons margarine', 'cups heavy whipping cream', 'ingredients', 'ingredients'],
    ['boneless skinless chicken breast halves', 'cup steak sauce', 'ounce package pasta', 'cubes chicken bouillon', 'Vidalia onion', 'avocados diced', 'cup cherry tomatoes', 'cup Ranch style salad dressing', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'large onions', 'ounce cans tomatoes', 'boneless chicken breast halves', 'pound turkey sausage links casings', 'teaspoon garlic powder', 'tablespoon hot sauce', 'salt pepper', 'cups uncooked long grain rice', 'chicken broth', 'large shrimp deveined', 'ingredients', 'ingredients'],
    ['pound lean ground beef', 'onion', 'medium head cabbage', 'ounce cans tomato soup', 'fluid ounces water', 'salt', 'ground black pepper', 'cup sour cream', 'ingredients', 'ingredients'],
    ['zucchini', 'onion rings', 'cup purpose flour', 'cup cornmeal', 'teaspoon salt', 'teaspoon ground black pepper', 'teaspoon garlic powder', 'cup vegetable oil frying', 'ingredients', 'ingredients'],
    ['cooking spray', 'medium onion', 'cups chicken broth', 'ounce package baby lima beans', 'ingredients', 'ingredients'],
    ['cup canola oil', 'cup unsalted butter', 'ounce bags fresh spinach leaves', 'cloves garlic', 'ingredients', 'ingredients'],
    ['pounds mahi chunks', 'ounce jar garlic', 'cup butter', 'large onion', 'lemons', 'cup dry white wine', 'ounce cans tomatoes green chile peppers', 'salt pepper', 'ounces pepperjack cheese', 'ingredients', 'ingredients'],
    ['cup red lentils', 'cup tomato puree', 'ounce container plain yogurt', 'teaspoon garam masala', 'teaspoon ground turmeric', 'teaspoon ground cumin', 'ancho', 'tablespoons vegetable oil', 'onion', 'cloves garlic', 'inch piece fresh ginger root', 'cups fresh spinach', 'tomatoes', 'sprigs fresh cilantro', 'ounce mixed beans', 'ingredients', 'ingredients'],
    ['medium potatoes paper thin', 'tablespoons', 'quart oil deep frying', 'ingredients', 'ingredients'],
    ['pound ground beef', 'cup teriyaki marinade sauce', 'ounce French onions', 'Cheddar cheese', 'hamburger buns split', 'ingredients', 'ingredients'],
    ['slices cut bacon inch strips more', 'pound Brussels sprouts more', 'clove garlic more', 'cup chicken stock more', 'tablespoon butter', 'tablespoon balsamic vinegar more', 'salt black pepper', 'ingredients', 'ingredients'],
    ['cooking spray', 'tablespoons butter', 'cloves garlic', 'tablespoons purpose flour', 'cup milk', 'cup Parmesan cheese', 'salt black pepper', 'large potatoes', 'ingredients', 'ingredients'],
    ['cups turnips', 'cup water', 'cube chicken bouillon', 'tablespoon butter more', 'tablespoons white sugar', 'ingredients', 'ingredients'],
    ['cup basmati rice', 'tablespoon coconut oil', 'teaspoon butter', 'cup coconut', 'cup coconut milk', 'cup chicken broth', 'salt', 'lime', 'ground black pepper', 'ingredients', 'ingredients'],
    ['cup vegetable oil', 'beef chuck roast inch cubes', 'onions', 'cloves garlic', 'ounce roma tomatoes juice', 'ounce cans green chile peppers', 'fluid ounce beer', 'cup beef broth', 'teaspoons oregano', 'teaspoons ground cumin', 'tablespoons Worcestershire sauce', 'salt', 'ground black pepper', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'chicken broth', 'leeks', 'carrots inch pieces', 'medium head cabbage', 'ounce uncooked egg noodles', 'teaspoon Thai chile sauce', 'ingredients', 'ingredients'],
    ['cups vegetable stock', 'cup oil sun tomatoes', 'onion', 'cups Arborio rice', 'cup mozzarella cheese', 'cup Parmesan cheese', 'cup fresh basil', 'salt pepper', 'ingredients', 'ingredients'],
    ['zucchinis', 'tablespoon olive oil', 'cup water', 'salt black pepper', 'ingredients', 'ingredients'],
    ['ounce Campbell Condensed Cream Mushroom Soup Regular % Fat Free Less Sodium', 'ounce container sour cream', 'cup butter margarine', 'ounce package hash brown potatoes', 'medium onion', 'ounce package Cheddar cheese', 'black pepper', 'cup corn flakes', 'ingredients', 'ingredients'],
    ['Roasted Winter Squash', 'tablespoons butter', 'cups raw winter squash butternut hubbard acorn', 'Salt pepper', 'Soup', 'tablespoons extra-virgin olive oil', 'cup onion inch', 'cup celery inch', 'cup carrot inch', 'cinnamon stick', 'Sea salt pepper', 'ounce carton Progresso chicken broth', 'teaspoon ground coriander optional', 'Roasted Winter Squash', 'cup half optional', 'tablespoons pumpkin seeds', 'cup Progresso plain Panko crispy bread crumbs toasted light brown saute pan medium heat', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'skinless boneless chicken breast halves', 'tablespoons rosemary', 'tablespoon olive oil', 'cloves garlic', 'ounce tomato sauce', 'teaspoon Italian seasoning', 'teaspoon cayenne pepper', 'teaspoons white sugar', 'bay leaf', 'teaspoon red pepper flakes', 'ounce beans', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'teaspoons soy sauce', 'teaspoons lemon juice', 'clove garlic', 'teaspoons red pepper flakes', 'teaspoon fresh ground black pepper', 'teaspoon fresh ginger root', 'tablespoons green onion optional', 'salt', 'ounce fillets', 'cups uncooked jasmine rice', 'cups water', 'cube chicken bouillon', 'tablespoon butter optional', 'ounce coconut milk', 'tablespoons white sugar', 'teaspoons butter', 'tablespoons white sugar', 'cups fresh mango', 'ingredients', 'ingredients'],
    ['ounce pumpkin puree', 'fluid ounce milk', 'eggs', 'cup white sugar', 'teaspoon salt', 'teaspoons pumpkin pie spice', 'ounce yellow cake mix', 'cup butter', 'cup walnuts', 'ingredients', 'ingredients'],
    ['pints strawberries', 'cups plain yogurt', 'cup orange juice', 'cup white sugar', 'cup water', 'teaspoon ground cardamom', 'ingredients', 'ingredients'],
    ['large onions', 'cup butter', 'cubes chicken bouillon', 'ingredients', 'ingredients'],
    ['ounces raisins', 'tablespoons olive oil', 'small onion', 'clove garlic', 'pound ripe tomatoes', 'green olives', 'ounces pine nuts', 'cup capers', 'swordfish steaks', 'salt pepper', 'ingredients', 'ingredients'],
    ['tablespoon butter', 'tablespoon olive oil', 'onion', 'ounce tomatoes', 'cup heavy whipping cream', 'cup vodka', 'teaspoon red pepper flakes', 'salt pepper', 'pound penne pasta', 'tablespoons Parmesan cheese', 'ingredients', 'ingredients'],
    ['pounds apples', 'cup butter', 'cup brown sugar', 'cup purpose flour', 'packets instant maple brown sugar oatmeal', 'pinch salt', 'cup walnuts', 'cup maple syrup', 'ingredients', 'ingredients'],
    ['ounces fresh green beans', 'tablespoons low sodium soy sauce', 'tablespoon miso paste', 'teaspoon red pepper flakes', 'cloves garlic', 'teaspoon fresh ginger root', 'tablespoon sesame seeds', 'ingredients', 'ingredients'],
    ['ground beef', 'jalapeno peppers', 'ounces blue cheese', 'tablespoons onion powder', 'tablespoons garlic powder', 'tablespoons tamari soy sauce', 'teaspoons salt', 'Swiss cheese', 'hamburger buns split', 'ingredients', 'ingredients'],
    ['medium yellow onions peel', 'cups chicken stock', 'tablespoon olive oil', 'salt pepper', 'sprigs fresh rosemary', 'cup heavy cream', 'ingredients', 'ingredients'],
    ['carrots sticks', 'cup butter', 'cup brown sugar', 'teaspoon salt', 'teaspoon ground white pepper', 'ingredients', 'ingredients'],
    ['ounce package cream cheese', 'egg', 'ounce pouch sugar cookie mix', 'cup raspberry baking chips', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'ounces pancetta', 'onion', 'cup diced celery', 'cloves garlic', 'chicken broth', 'cups water more', 'ounce plum tomatoes fine', 'cup cranberry beans', 'cups cabbage more', 'ounce beans', 'teaspoon red pepper flakes', 'teaspoon Italian seasoning', 'teaspoons salt', 'bunch Swiss chard', 'cup ditalini pasta', 'salt black pepper', 'cup extra virgin olive oil', 'cup Parmigiano Reggiano cheese', 'cup fresh Italian flat leaf parsley', 'ingredients', 'ingredients'],
    ['carrots diagonal', 'cup almonds', 'cloves garlic', 'cup olive oil', 'salt black pepper', 'teaspoon honey', 'tablespoon cider vinegar', 'cup cranberries', 'ounce package Danish blue cheese', 'cups arugula', 'ingredients', 'ingredients'],
    ['ears fresh corn husks', 'cup butter', 'roma plum tomatoes', 'jalapeno pepper seeded', 'small red onion', 'cloves garlic', 'bunch fresh cilantro', 'salt black pepper', 'salt free lemon herb seasoning such Mrs. Dash', 'ingredients', 'ingredients'],
    ['large green bell pepper', 'jalapeno peppers', 'cups apple cider vinegar', 'pinch salt', 'cups granulated sugar', 'ounces liquid pectin', 'jalapeno peppers seeded', 'ingredients', 'ingredients'],
    ['pounds zucchini', 'cup onion', 'ounce cream chicken soup', 'cup sour cream', 'cup carrots', 'unsalted butter', 'ounce package chicken dry bread stuffing mix', 'ingredients', 'ingredients'],
    ['fresh pears', 'cup white sugar', 'cup brown sugar', 'cup pecans', 'cup vegetable oil', 'eggs', 'cups purpose flour', 'teaspoon salt', 'teaspoons soda', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'cup purpose flour', 'teaspoon salt', 'teaspoon ground black pepper', 'tablespoons olive oil', 'cups fresh mushrooms', 'cups heavy cream', 'cup champagne', 'ingredients', 'ingredients'],
    ['cups brown sugar', 'cup vegetable oil', 'egg', 'teaspoon vanilla extract', 'cups purpose flour', 'teaspoon salt', 'teaspoon baking soda', 'cup milk', 'cups rhubarb', 'cup almonds', 'cup white sugar', 'tablespoon butter', 'cup almonds', 'ingredients', 'ingredients'],
    ['pounds broccoli', 'tablespoons olive oil', 'cup butter', 'cloves garlic', 'teaspoon red pepper flakes', 'cups chicken stock', 'salt', 'pound angel hair pasta', 'cup Parmigiano Reggiano cheese little more garnish', 'red pepper flakes garnish', 'ingredients', 'ingredients'],
    ['new potatoes', 'cups fine salt', 'tablespoons butter', 'ingredients', 'ingredients'],
    ['large eggplant', 'tablespoons extra virgin olive oil', 'salt pepper', 'tablespoons fresh lemon juice', 'ingredients', 'ingredients'],
    ['cup uncooked quinoa', 'cup water', 'ounce beans', 'cup broccoli', 'clove garlic', 'tablespoon fresh lemon juice', 'teaspoon tarragon', 'teaspoons coarse grain mustard', 'tablespoons extra virgin olive oil', 'salt black pepper', 'ingredients', 'ingredients'],
    ['large fresh strawberries', 'eggs', 'cup white sugar', 'cup vegetable oil', 'teaspoon vanilla extract', 'teaspoon lemon zest', 'cups purpose flour', 'teaspoons powder', 'teaspoon salt', 'tablespoons instant vanilla pudding mix optional', 'red food coloring optional', 'cup cream cheese', 'tablespoons butter', 'cup confectioners sugar', 'teaspoon vanilla extract', 'large fresh strawberries', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cups white sugar', 'eggs', 'teaspoons soda', 'teaspoon vanilla extract', 'ounce pineapple juice', 'ounce package cream cheese', 'cup butter', 'cups confectioners sugar', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['cup fresh strawberries', 'cup orange juice', 'cubes ice', 'tablespoon sugar', 'ingredients', 'ingredients'],
    ['tablespoons flax seed meal', 'tablespoons water', 'cups sugar', 'cup pumpkin puree', 'cup applesauce', 'cups purpose flour', 'cup whole wheat pastry flour', 'teaspoon baking soda', 'teaspoon ground cinnamon', 'teaspoon salt', 'teaspoon baking powder', 'teaspoon ground nutmeg', 'teaspoon ground cloves', 'ingredients', 'ingredients'],
    ['cups Basmati rice', 'ounce cans coconut milk', 'pinch salt', 'ingredients', 'ingredients'],
    ['cups water', 'pound Brussels sprouts', 'tablespoons olive oil', 'cloves garlic', 'ounces pancetta bacon', 'teaspoon salt', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['eggs', 'cup vegetable oil', 'salt pepper', 'teaspoon baking powder', 'cup purpose baking mix', 'cups zucchini', 'cups yellow squash', 'cup onion', 'large ripe tomato', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['ounce package cheese tortellini', 'pound flank steak', 'garlic powder', 'salt pepper', 'tablespoon olive oil', 'romaine lettuce bite size pieces', 'ounce cans small black olives', 'cup Caesar style croutons', 'small fresh tomatoes', 'ounce bottle Caesar salad dressing', 'ingredients', 'ingredients'],
    ['ounce package imitation crabmeat', 'large tomatoes', 'red onion', 'bunch cilantro', 'limes', 'serrano peppers', 'tablespoon olive oil', 'salt pepper', 'ingredients', 'ingredients'],
    ['slices bacon', 'large onion', 'cloves garlic', 'chicken broth', 'pinch dried oregano', 'pinch cayenne pepper', 'teaspoon ground cumin', 'salt', 'ounce cans black beans', 'fluid ounces water', 'teaspoon ground black pepper', 'Relish', 'cup green onion', 'teaspoons sambal paste red chiles', 'juice lime', 'Garnish', 'green onion optional', 'sour cream optional', 'ingredients', 'ingredients'],
    ['ounce thick cut pork chops', 'slices Gouda cheese', 'pound fresh spinach bite size pieces', 'tablespoons horseradish mustard', 'cup panko crumbs breadcrumbs', 'Creole style seasoning', 'ingredients', 'ingredients'],
    ['ounce whole kernel corn', 'ounce black olives', 'cups roma tomatoes', 'cup red onion', 'red bell pepper seeded diced', 'teaspoons jalapeno pepper', 'avocados diced', 'tablespoons olive oil', 'tablespoons fresh lime juice', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['ounce beans chickpeas', 'cup Spanish Manzanilla olives', 'teaspoon garlic', 'tablespoons olive oil', 'tablespoons lemon juice', 'teaspoons fresh basil', 'teaspoon cilantro leaves', 'salt pepper', 'ingredients', 'ingredients'],
    ['head cauliflower florets', 'cup mayonnaise', 'cup Dijon mustard', 'cup sharp Cheddar cheese', 'ingredients', 'ingredients'],
    ['ounce frozen whole green beans', 'bacon', 'medium onion', 'red bell pepper seeded diced', 'cup dry white wine vermouth', 'cup milk', 'tablespoons butter', 'ounce cream mushroom soup', 'tablespoon soy sauce', 'salt pepper', 'cup mozzarella cheese', 'cup French onions', 'ingredients', 'ingredients'],
    ['bacon', 'red new potatoes', 'cup olive oil', 'tablespoons white vinegar', 'bunch green onions', 'teaspoon salt', 'teaspoon ground black pepper', 'ounces blue cheese', 'ingredients', 'ingredients'],
    ['ounces dry fettuccine pasta', 'cup heavy whipping cream', 'tablespoons butter', 'ounce tomato sauce', 'teaspoon garlic', 'cup broccoli florets', 'cup fresh mushrooms', 'salt', 'ground black pepper', 'tablespoons Parmesan cheese', 'ingredients', 'ingredients'],
    ['French baguette', 'tomatoes', 'onion', 'clove garlic', 'teaspoon basil', 'teaspoon oregano', 'pinch salt', 'pinch ground black pepper', 'ounces mozzarella cheese', 'ingredients', 'ingredients'],
    ['cup dry brown lentils', 'cup carrots', 'cup red onion', 'cloves garlic', 'bay leaf', 'teaspoon thyme', 'tablespoons lemon juice', 'cup diced celery', 'cup parsley', 'teaspoon salt', 'teaspoon ground black pepper', 'cup olive oil', 'ingredients', 'ingredients'],
    ['cup ground Italian sausage', 'ounce package cream cheese', 'tablespoon garlic salt', 'tablespoons Romano cheese', 'teaspoon oregano', 'teaspoon basil', 'cup Italian style dry bread crumbs', 'tablespoon olive oil', 'Hungarian hot peppers seeded', 'ingredients', 'ingredients'],
    ['large green bell peppers', 'pound ground sausage', 'large onion', 'ounce cans tomato soup', 'cups water', 'tablespoon chili powder', 'tablespoon garlic powder', 'salt pepper', 'pound Cheddar cheese', 'cups uncooked instant rice', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cup white sugar', 'teaspoons powder', 'teaspoon salt', 'teaspoon baking soda', 'cup butter', 'egg', 'cup orange juice', 'tablespoon orange zest', 'cups fresh frozen cranberries', 'cup walnuts', 'ingredients', 'ingredients'],
    ['ounce package lemon cake mix', 'ounce package lemon Jell O mix', 'cup water', 'ounce container topping', 'ounce instant lemon pudding mix', 'teaspoon vanilla extract', 'cup milk', 'ingredients', 'ingredients'],
    ['cups coconut', 'ounce condensed milk', 'teaspoons vanilla extract', 'teaspoons almond extract', 'ingredients', 'ingredients'],
    ['Anaheim chile peppers charred', 'pound Cheddar cheese strips', 'cup milk', 'cup purpose flour', 'egg', 'teaspoon baking powder', 'teaspoon baking soda', 'teaspoon salt', 'teaspoon canola oil', 'cup purpose flour coating', 'ingredients', 'ingredients'],
    ['teaspoons olive oil', 'large russet potatoes wedges', 'cup butter', 'cup seasoned bread crumbs', 'ingredients', 'ingredients'],
    ['chicken breast tenderloins strips', 'cup margarine', 'salt pepper', 'ounce envelope dry onion soup mix', 'ingredients', 'ingredients'],
    ['pound ground beef', 'cup onion', 'teaspoon rosemary', 'teaspoon basil', 'ounce tomato sauce', 'ounce tomatoes', 'ounce cans vegetables', 'red potatoes', 'cup butter', 'cup Romano cheese', 'tablespoons garlic', 'teaspoon salt', 'teaspoon oregano', 'cup milk', 'ingredients', 'ingredients'],
    ['pounds red potatoes', 'pound fresh green beans', 'cup fresh basil', 'small red onion', 'salt pepper', 'cup balsamic vinegar', 'tablespoons Dijon mustard', 'tablespoons fresh lemon juice', 'clove garlic', 'Worcestershire sauce', 'cup extra virgin olive oil', 'ingredients', 'ingredients'],
    ['cup orange juice', 'cup soy sauce', 'cup olive oil', 'tablespoon rosemary', 'teaspoons garlic', 'red onion', 'pound boneless pork loin roast', 'ingredients', 'ingredients'],
    ['large tomatoes', 'avocados diced', 'red onion', 'teaspoon ground black pepper', 'ounce balsamic vinaigrette salad dressing', 'ingredients', 'ingredients'],
    ['large baking potatoes', 'teaspoons butter', 'cup milk', 'salt pepper', 'pounds beef sausage', 'cup diced onion', 'ounce packet dry brown gravy mix', 'cup water', 'ingredients', 'ingredients'],
    ['sirloin tips', 'medium onion', 'ounces mushrooms', 'tablespoons butter', 'ounce golden mushroom soup', 'cup Burgundy wine', 'ingredients', 'ingredients'],
    ['cup white sugar', 'cup butter', 'cup purpose flour', 'egg', 'teaspoon vanilla extract', 'cups fresh peaches skinned', 'recipe pastry inch single crust pie', 'ingredients', 'ingredients'],
    ['pound beef stew meat', 'tablespoon vegetable oil', 'cups water', 'tablespoons margarine', 'large carrots', 'cloves garlic', 'onion', 'stalks celery', 'pound fresh mushrooms', 'cups water', 'cubes beef bouillon cube', 'cup pearl barley', 'cup sour cream', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoons powder', 'teaspoon baking soda', 'teaspoon salt', 'cup white sugar', 'tablespoon orange zest', 'cup orange juice', 'cup butter', 'eggs', 'cup ground walnuts optional', 'tablespoon butter', 'cup brown sugar', 'teaspoon ground cinnamon', 'ingredients', 'ingredients'],
    ['pound frozen bread dough', 'cup margarine', 'egg', 'cup Parmesan cheese', 'teaspoon garlic powder', 'teaspoon salt', 'teaspoon parsley', 'ingredients', 'ingredients'],
    ['inch deep dish frozen pie crusts', 'pound ground beef', 'ounce beans', 'onion', 'cup tortilla chips', 'cup Cheddar cheese', 'cups lettuce', 'large tomato', 'sour cream', 'black olives', 'ingredients', 'ingredients'],
    ['ounce package garden herb tofu', 'tablespoons vegetable oil', 'clove garlic', 'cup onion', 'teaspoons chili powder', 'teaspoon paprika', 'teaspoon cayenne pepper', 'teaspoon ground cumin', 'teaspoon salt', 'lime', 'cup tomato sauce', 'cup fresh cilantro', 'medium taco shells heated', 'cups lettuce', 'tomatoes', 'avocado diced', 'cup Cheddar cheese', 'cup salsa', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'onion', 'teaspoons white sugar', 'pound calves livers', 'cup purpose flour', 'tablespoon butter', 'salt pepper', 'ingredients', 'ingredients'],
    ['cup white sugar', 'tablespoons sesame seeds', 'teaspoon sesame oil', 'tablespoon poppy seeds', 'teaspoons minced onion', 'teaspoon paprika', 'cup vegetable oil', 'cup balsamic vinegar', 'fresh spinach', 'pint strawberries', 'ingredients', 'ingredients'],
    ['tablespoons cornstarch', 'cup water', 'ounce chicken broth', 'pound Brussels sprouts', 'teaspoons Dijon style mustard', 'teaspoons lemon juice', 'ingredients', 'ingredients'],
    ['cup butter', 'cup white sugar', 'eggs', 'teaspoon almond extract', 'cups purpose flour', 'teaspoon baking powder', 'teaspoon baking soda', 'teaspoon salt', 'cup sour cream', 'ounce whole cranberry sauce', 'ingredients', 'ingredients'],
    ['cups French onions', 'egg', 'pound skinless boneless chicken breast halves', 'ingredients', 'ingredients'],
    ['pound boneless pork shoulder inch cubes', 'salt black pepper', 'tablespoon vegetable oil', 'tablespoon butter', 'large onion', 'cloves garlic', 'tablespoon purpose flour', 'bay leaf', 'teaspoon caraway seed', 'fluid ounce dark beer such Guinness', 'chicken broth', 'carrots inch pieces', 'stalks celery inch pieces', 'cup fresh flat leaf parsley', 'tablespoons balsamic vinegar', 'Brussels sprouts', 'cups potatoes', 'teaspoon fresh flat leaf parsley', 'ingredients', 'ingredients'],
    ['cup white sugar', 'tablespoons cornstarch', 'cups fresh blackberries', 'cup butter', 'cups purpose flour', 'cups white sugar', 'tablespoon baking powder', 'teaspoon salt', 'cups milk', 'tablespoon vanilla extract', 'cup butter', 'ingredients', 'ingredients'],
    ['butternut squash seeded', 'tablespoons vegetable oil', 'teaspoon salt', 'cup unsalted butter', 'tablespoons garlic', 'cup purpose flour', 'quart milk', 'salt black pepper', 'cup heavy cream', 'cook lasagna noodles', 'cups Parmesan cheese', 'ingredients', 'ingredients'],
    ['ounce package cheese tortellini', 'ounces pepperoni', 'green onions', 'ounce black olives', 'ounce jar artichoke hearts', 'ounces mozzarella cheese', 'cup olive oil', 'tablespoons balsamic vinegar', 'tablespoons white vinegar', 'teaspoon Italian herb seasoning', 'salt black pepper', 'ingredients', 'ingredients'],
    ['tablespoon butter', 'russet potato cubed', 'cup heavy cream', 'cup % reduced fat milk', 'cloves garlic', 'tablespoons purpose flour', 'salt black pepper', 'cup medium Cheddar cheese', 'ingredients', 'ingredients'],
    ['tablespoon butter', 'tablespoon olive oil', 'zucchini noodle shape strands', 'large yellow onion', 'tablespoon garlic', 'teaspoon kosher salt', 'tablespoons butter', 'large shrimp deveined', 'teaspoon garlic', 'ounce bag baby spinach', 'tablespoon fresh lemon juice', 'teaspoon red pepper flakes', 'teaspoon kosher salt', 'teaspoon black pepper', 'ingredients', 'ingredients'],
    ['lemons', 'cup Parmesan cheese', 'teaspoons garlic salt', 'cup mayonnaise', 'cup milk', 'ingredients', 'ingredients'],
    ['zucchini', 'tablespoon olive oil', 'onion', 'cloves garlic', 'ounce package button mushrooms', 'teaspoon ground coriander', 'teaspoons ground cumin', 'ounce chickpeas', 'lemon', 'tablespoons fresh parsley', 'sea salt', 'ground black pepper', 'ingredients', 'ingredients'],
    ['cup vegetable oil', 'cup cider vinegar', 'cup white sugar', 'teaspoons parsley', 'teaspoon salt', 'pinch ground black pepper', 'cup almonds', 'cup white sugar', 'red leaf lettuce', 'red onion', 'cup celery', 'ounce cans mandarin orange segments', 'ingredients', 'ingredients'],
    ['pound andouille sausage', 'cup butter', 'cups onion', 'teaspoon thyme', 'pounds pumpkin puree', 'cup praline liqueur', 'cups chicken stock', 'cup brown sugar', 'cup heavy cream', 'ingredients', 'ingredients'],
    ['ounce package pasta', 'roma plum tomatoes', 'mozzarella cheese', 'cup fresh basil', 'cloves garlic', 'cup olive oil', 'teaspoon garlic salt', 'ground black pepper', 'ingredients', 'ingredients'],
    ['red potatoes', 'eggs', 'stalks celery', 'green bell pepper', 'ounce jar sweet pickles', 'tablespoon prepared mustard', 'cup mayonnaise', 'onion', 'teaspoon white sugar', 'salt pepper', 'ingredients', 'ingredients'],
    ['tomatoes', 'cucumber diced', 'onion', 'tablespoon lemon juice', 'salt', 'ground black pepper', 'ingredients', 'ingredients'],
    ['cups potatoes', 'egg', 'cup purpose flour', 'salt', 'garlic salt', 'tablespoon butter', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['eggs', 'cups white sugar', 'cup vegetable oil', 'cups purpose flour', 'teaspoons ground cinnamon', 'teaspoon salt', 'teaspoons powder', 'teaspoon baking soda', 'ounce pineapple', 'cup walnuts', 'cups zucchini', 'teaspoons vanilla extract', 'cup confectioners sugar', 'tablespoons milk', 'ingredients', 'ingredients'],
    ['cups carrots', 'eggs', 'cups vegetable oil', 'cups white sugar', 'cups purpose flour', 'teaspoons powder', 'teaspoons soda', 'teaspoon ground cinnamon', 'teaspoon salt', 'teaspoon ground nutmeg', 'cup golden raisins', 'cups confectioners sugar', 'ounce package cream cheese', 'tablespoon light corn syrup', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['ounce halibut fillets', 'tablespoon dill weed', 'tablespoon onion powder', 'teaspoons parsley', 'teaspoon paprika', 'seasoned salt more', 'lemon pepper', 'garlic powder', 'tablespoons lemon juice', 'ingredients', 'ingredients'],
    ['ounce tri-colored pasta assortment', 'large tomatoes', 'large cucumber diced', 'red onion', 'ounce Italian style salad dressing', 'ingredients', 'ingredients'],
    ['gallon water', 'cup white sugar', 'tea bags', 'fluid ounce frozen cranberry juice concentrate', 'ingredients', 'ingredients'],
    ['medium baking potatoes', 'large white onion', 'tablespoons butter', 'cloves garlic', 'teaspoon fresh parsley', 'salt pepper', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'tablespoon baking powder', 'teaspoon cumin', 'cup sugar', 'teaspoon salt', 'fresh ground black pepper', 'eggs', 'cup milk', 'cup butter', 'cups zucchini', 'cups fresh corn kernels cob', 'cup Cheddar cheese', 'oil frying', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'boneless venison roast', 'large apple', 'small onions', 'cloves garlic', 'cup water', 'cube beef bouillon', 'ingredients', 'ingredients'],
    ['slices bacon bite size pieces', 'onion', 'ounce fresh Brussels sprouts', 'tablespoons white sugar optional', 'salt black pepper', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'pounds boneless pork loin chops bite size pieces', 'cups instant brown rice uncooked', 'ounce cream chicken soup', 'ounce black beans', 'ounce tomatoes', 'ounce cans green chile peppers', 'tablespoons chunky salsa', 'cup water', 'tablespoons ground cumin', 'cup Colby cheese', 'ingredients', 'ingredients'],
    ['ounce salmon fillets skin', 'tablespoon lime lemon juice', 'cup panko Japanese bread crumbs available Asian food aisle dry', 'cup coconut', 'Salt pepper', 'Cooking spray', 'ingredients', 'ingredients'],
    ['ounce garlic herb couscous mix flavor', 'pint cherry tomatoes half', 'ounce jar kalamata olives', 'cup mixed bell peppers green red yellow orange', 'cucumber', 'cup parsley', 'ounce package feta cheese', 'cup Greek vinaigrette salad dressing', 'ingredients', 'ingredients'],
    ['medium baking potatoes', 'teaspoon salt', 'tablespoons salt free herb seasoning blend', 'tablespoons butter', 'tablespoons Cheddar cheese', 'tablespoons Parmesan cheese', 'ingredients', 'ingredients'],
    ['cups white sugar', 'cup shortening', 'ounce pumpkin puree', 'teaspoons vanilla extract', 'cups purpose flour', 'teaspoons soda', 'teaspoons ground cinnamon', 'ounces chocolate chips', 'ingredients', 'ingredients'],
    ['egg', 'tablespoon purpose flour', 'cup white sugar', 'teaspoon salt', 'cups pumpkin puree', 'cups milk', 'teaspoon ground cinnamon', 'teaspoon ground ginger', 'teaspoon ground nutmeg', 'tablespoons corn syrup', 'recipe pastry inch single crust pie', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'teaspoon paprika', 'sweet potatoes quarters', 'ingredients', 'ingredients'],
    ['ounce package cream cheese', 'cup white sugar', 'egg', 'cups purpose flour', 'cups white sugar', 'teaspoon baking soda', 'teaspoon ground cinnamon', 'teaspoon salt', 'teaspoon ground nutmeg', 'cup pumpkin puree', 'cup butter', 'egg', 'cup water', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'onion', 'cloves garlic', 'cup bread crumbs', 'kale', 'ingredients', 'ingredients'],
    ['carrots', 'large zucchini', 'large yellow squash', 'red onion', 'red bell pepper', 'cup spaghetti sauce', 'tablespoons olive oil', 'garlic salt', 'ground black pepper', 'cups mozzarella cheese', 'cup Parmesan cheese', 'ounce prepared polenta', 'ingredients', 'ingredients'],
    ['butternut squash', 'egg', 'cup milk', 'cup purpose flour', 'cup cornmeal', 'pinch salt', 'pinch ground black pepper', 'garlic salt', 'cup oil frying', 'ingredients', 'ingredients'],
    ['ounce fresh salmon fillets', 'salt', 'cup cold water', 'cup seasoned rice vinegar', 'tablespoons brown sugar', 'tablespoon hot chile paste such sambal oelek', 'tablespoon fresh ginger', 'cloves garlic', 'teaspoon soy sauce', 'cup fresh basil', 'ingredients', 'ingredients'],
    ['carrot', 'ounce kidney beans', 'cup green bell pepper', 'cup onion', 'cups salsa', 'cup bread crumbs', 'cup whole wheat flour', 'teaspoon ground black pepper', 'salt', 'chili powder', 'ingredients', 'ingredients'],
    ['cups broccoli', 'cups chicken meat', 'ounce cans mushrooms', 'ounce water chestnuts optional', 'ounce cans cream chicken soup', 'cup mayonnaise', 'teaspoon lemon juice', 'teaspoon curry powder', 'tablespoon butter', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['yellow sweet potatoes', 'cup butter', 'cup brown sugar', 'cup water', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['cups warm water', 'tablespoons milk', 'tablespoons', 'teaspoon salt', 'bread flour', 'tablespoons white sugar', 'teaspoons active dry yeast', 'cup raisins', 'tablespoons butter', 'tablespoons brown sugar', 'teaspoon ground cinnamon', 'tablespoon butter', 'ingredients', 'ingredients'],
    ['tablespoons unsalted butter', 'cup onion', 'cloves garlic', 'cup chicken stock', 'tablespoons fresh rosemary', 'ounce package spaghetti', 'cup Parmesan cheese', 'kosher salt black pepper', 'ingredients', 'ingredients'],
    ['cup butter', 'medium eggplant inch slices', 'egg', 'teaspoon salt', 'teaspoon pepper', 'cup instant mashed potato flakes', 'ingredients', 'ingredients'],
    ['lemons', 'quart water', 'cup white sugar', 'ingredients', 'ingredients'],
    ['ounce dry penne pasta', 'head escarole', 'ounce beans liquid', 'ounce tomatoes garlic onion', 'salt black pepper', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'cup extra virgin olive oil', 'cup white sugar', 'tablespoon honey', 'teaspoon saffron', 'cloves garlic', 'teaspoon basil', 'teaspoon thyme', 'teaspoon cayenne pepper', 'teaspoon salt', 'teaspoon oregano', 'teaspoon parsley', 'sage', 'ingredients', 'ingredients'],
    ['zucchini', 'tablespoons olive oil', 'links Italian style chicken sausage casings', 'teaspoons red pepper flakes optional', 'salt black pepper', 'sweet onion such Vidalia', 'cloves garlic', 'ounce tomatoes', 'cup dry bread crumbs', 'cup Parmesan cheese', 'tablespoon fresh basil', 'ingredients', 'ingredients'],
    ['clove garlic', 'cup butter', 'skinless boneless chicken breast halves', 'cup purpose flour', 'salt black pepper', 'ounces mushrooms', 'capers', 'tablespoons lemon juice', 'cup white wine', 'tablespoons chicken demi glace', 'teaspoon fresh parsley', 'lemon slices', 'ingredients', 'ingredients'],
    ['grape tomatoes', 'cloves garlic', 'tablespoons olive oil', 'salt pepper', 'ingredients', 'ingredients'],
    ['ounces sour cream', 'cup sugar', 'egg', 'tablespoons flour', 'teaspoons vanilla extract', 'teaspoon salt', 'cups fresh blueberries', 'inch unbaked pie crust', 'Topping', 'tablespoons purpose flour', 'tablespoons white sugar', 'tablespoons butter small pieces', 'tablespoons pecans', 'ingredients', 'ingredients'],
    ['cups butter', 'cups white sugar', 'cups brown sugar', 'eggs', 'teaspoons vanilla extract', 'cups purpose flour', 'teaspoons salt', 'teaspoons soda', 'cups quick cooking oats', 'chocolate chips', 'cups toasted walnuts', 'cup coconut', 'cup raisins', 'ingredients', 'ingredients'],
    ['pound seedless green grapes', 'pound seedless red grapes', 'ounce package cream cheese', 'ounce jar marshmallow creme', 'ounce package almonds', 'ingredients', 'ingredients'],
    ['ounce package crescent rolls', 'cup margarine', 'cups zucchini', 'cup onion', 'tablespoons parsley', 'teaspoon oregano', 'teaspoon salt', 'teaspoon ground black pepper', 'eggs', 'cups mozzarella cheese', 'ingredients', 'ingredients'],
    ['cup butter', 'cup olive oil', 'cups onions', 'tomatoes', 'cup fresh basil leaves', 'salt', 'ground black pepper', 'quart chicken broth', 'cup heavy cream', 'sprigs fresh basil garnish', 'ingredients', 'ingredients'],
    ['ounce cans tomatoes', 'cups water', 'cup fresh lima beans', 'cup frozen corn kernels', 'cup carrots', 'tablespoons onion', 'tablespoons Old Bay Seasoning TM', 'beef broth', 'pound blue crab crabmeat', 'blue crab claws optional', 'gallon water', 'ingredients', 'ingredients'],
    ['cups potatoes', 'cup sour cream', 'ounces cream cheese', 'tablespoons butter', 'egg yolk', 'teaspoon kosher salt', 'teaspoon black pepper', 'teaspoons olive oil', 'pound ground lamb', 'pinch salt black pepper', 'ounce tomatoes juice', 'small onion', 'small carrot', 'cup peas', 'cup Irish stout beer such Guinness', 'cube beef bouillon', 'tablespoon purpose flour', 'tablespoon Worcestershire sauce', 'cup sharp Cheddar cheese', 'teaspoons fresh parsley', 'teaspoons', 'ingredients', 'ingredients'],
    ['cup brown rice', 'cups water', 'tablespoons butter', 'tablespoons olive oil', 'cloves garlic', 'cup white wine', 'tablespoons fresh lemon juice', 'pounds medium shrimp deveined', 'cup fresh flat leaf parsley', 'teaspoon cornstarch', 'ingredients', 'ingredients'],
    ['teaspoon ground turmeric', 'cups uncooked orzo pasta', 'tablespoons olive oil', 'cloves garlic', 'bunch kale', 'large lemon', 'teaspoon ground nutmeg', 'cup Parmesan cheese', 'salt black pepper', 'ingredients', 'ingredients'],
    ['teaspoon olive oil', 'cup onion', 'pounds skinless boneless chicken thighs', 'tablespoon garam masala', 'teaspoon curry powder', 'cup red wine', 'tablespoons red wine vinegar', 'cup fat free reduced sodium chicken broth', 'ingredients', 'ingredients'],
    ['teaspoons vegetable oil', 'inch corn tortillas strips', 'chicken broth', 'teaspoon ground cumin', 'teaspoon chili powder', 'teaspoon oregano', 'ounce black beans', 'ounce whole kernel corn', 'skinless boneless chicken breast halves bite size pieces', 'cup salsa', 'cup fresh cilantro', 'ingredients', 'ingredients'],
    ['ounces salad greens', 'cup walnuts', 'ounces Gorgonzola cheese', 'tart green apples diced', 'ounce bottle raspberry vinaigrette salad dressing', 'ingredients', 'ingredients'],
    ['cup vegetable oil', 'cups bananas', 'teaspoon vanilla extract', 'eggs', 'cups mix', 'cup white sugar', 'cup walnuts', 'ingredients', 'ingredients'],
    ['tablespoons', 'cup white sugar', 'eggs', 'cup milk', 'cups purpose flour', 'teaspoon salt', 'teaspoon baking powder', 'teaspoon lemon zest', 'cup walnuts', 'cup white sugar', 'lemon', 'ingredients', 'ingredients'],
    ['ounce frozen raspberries', 'tablespoons white sugar', 'tablespoons orange liqueur', 'cups heavy whipping cream', 'ounces white chocolate', 'red food coloring', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cup shortening', 'cups ice water', 'teaspoon salt', 'cups potatoes', 'carrots', 'onions', 'cup diced rutabaga', 'pounds ground beef', 'pound lean ground pork', 'tablespoon salt', 'teaspoon ground black pepper', 'teaspoons MSG', 'cube beef bouillon', 'cup hot water', 'ingredients', 'ingredients'],
    ['ears fresh corn', 'tomato', 'zucchini', 'cucumber', 'red onion', 'red bell pepper', 'cup Italian style salad dressing', 'ingredients', 'ingredients'],
    ['ounce jar Alfredo sauce', 'cup milk', 'potatoes inch thick', 'tablespoons Parmesan cheese', 'teaspoon salt', 'teaspoon ground black pepper', 'cups ham', 'ounce package frozen broccoli', 'cups Swiss cheese', 'ingredients', 'ingredients'],
    ['pounds new potatoes wedges', 'tablespoons butter', 'teaspoons fresh rosemary', 'salt pepper', 'ingredients', 'ingredients'],
    ['cups water', 'tablespoons chicken bouillon granules', 'bunches broccoli', 'onion', 'cups milk', 'cup margarine', 'tablespoons purpose flour', 'ounce package cream cheese', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['ounce package Hillshire Farm Sausage inch slices', 'cup olive oil', 'cloves garlic', 'large red bell pepper thin', 'small yellow onion thin', 'ounce frozen broccoli', 'cup chicken broth water', 'cup tomato sauce', 'instant rice', 'cup mozzarella cheese', 'ingredients', 'ingredients'],
    ['fluid ounce bottle tomato juice', 'tablespoons celery salt', 'teaspoons black pepper', 'tablespoons Worcestershire sauce', 'teaspoons hot pepper sauce Tabasco tm', 'ingredients', 'ingredients'],
    ['bacon', 'medium yellow squash', 'medium onion', 'jalapeno pepper optional', 'soy sauce', 'salt pepper', 'ingredients', 'ingredients'],
    ['teaspoon butter', 'ounces fresh mushrooms', 'ounces cream cheese', 'skinless boneless chicken breast halves', 'cup brown sugar', 'cup Dijon mustard', 'cup walnuts', 'ingredients', 'ingredients'],
    ['ounce frozen broccoli', 'ground beef', 'ounce fresh mushrooms', 'large onion', 'salt', 'ground black pepper', 'ounce cans cream celery soup', 'ounce container sour cream', 'ounce package Monterey Jack cheese', 'ingredients', 'ingredients'],
    ['large green bell peppers', 'tablespoons olive oil', 'onion', 'cloves garlic', 'teaspoon oregano', 'tablespoon Creole seasoning', 'black pepper', 'pound shrimp deveined', 'links andouille sausage', 'cup uncooked long grain white rice', 'cups chicken broth', 'ounce tomato sauce', 'lemon wedges garnish optional', 'Louisiana style hot sauce', 'ingredients', 'ingredients'],
    ['baking potatoes', 'ounce package cream cheese', 'ounce package dry Ranch style dressing mix', 'cup sour cream', 'cup butter', 'ingredients', 'ingredients'],
    ['potatoes', 'eggs', 'onion', 'tablespoons purpose flour', 'salt pepper', 'tablespoons vegetable oil', 'ingredients', 'ingredients'],
    ['pork chops', 'salt pepper', 'cup uncooked wild rice', 'cups water', 'ounce mushrooms', 'tablespoon chicken bouillon granules', 'ounce cream mushroom soup', 'ingredients', 'ingredients'],
    ['fluid ounce frozen raspberry lemonade concentrate', 'cups water', 'teaspoon lime juice', 'fluid ounce lemon lime carbonated beverage', 'cup ice', 'cup fresh raspberries garnish', 'Mint leaves garnish optional', 'ingredients', 'ingredients'],
    ['sweet potatoes', 'cup butter', 'tablespoon vanilla extract', 'cups white sugar', 'teaspoon ground nutmeg', 'eggs', 'cup milk', 'inch unbaked pie crusts', 'ingredients', 'ingredients'],
    ['pound package hash brown potatoes', 'ounce cream potato soup', 'ounce cream celery soup', 'ounce container sour cream', 'cup milk', 'cup onion', 'teaspoon salt', 'teaspoon pepper', 'teaspoon dill weed', 'cups Cheddar cheese', 'paprika', 'ingredients', 'ingredients'],
    ['ounce frozen corn kernels', 'cup butter', 'ounce package cream cheese', 'clove garlic', 'teaspoon white sugar', 'salt', 'teaspoon parsley', 'ingredients', 'ingredients'],
    ['cup plain yogurt', 'cup sour cream', 'teaspoon white vinegar', 'teaspoon lemon juice', 'small cucumber', 'green onion', 'garlic clove', 'cup feta cheese', 'teaspoon oregano', 'teaspoon lemon zest', 'salt pepper', 'ingredients', 'ingredients'],
    ['large portobello mushroom caps gills', 'tablespoon fat Italian salad dressing', 'egg', 'clove garlic', 'salt black pepper', 'ounce bag fresh spinach', 'cup pepperoni', 'cup Parmesan cheese', 'cup mozzarella cheese', 'tablespoons seasoned bread crumbs', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'ounces hot Italian ground sausage meat', 'cup mushrooms', 'onion', 'salt black pepper', 'pounds skinless boneless chicken thighs', 'cup Marsala wine', 'ounce whole Italian plum tomatoes such San Marzano', 'cup chicken broth', 'cup heavy whipping cream', 'cup water', 'cups hot sweet peppers', 'cup kalamata olives', 'cloves garlic', 'cup Italian flat leaf parsley', 'pound rigatoni', 'cup Parmigiano Reggiano Romano cheese', 'ingredients', 'ingredients'],
    ['ounce dry corn bread mix', 'ounce green chile peppers', 'ounce cans pinto beans', 'ounce bottle Ranch style salad dressing', 'green bell pepper', 'ounce cans whole kernel corn', 'tomatoes', 'ounce bacon bits', 'ounces Cheddar cheese', 'green onions', 'ingredients', 'ingredients'],
    ['pound ground beef', 'ounce package taco seasoning mix', 'ounce whole kernel corn', 'ounce black beans undrained', 'ounce tomato paste', 'cups water', 'cup sour cream', 'ounce packages Cheddar cheese', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breasts', 'cup fat free ranch dressing', 'tablespoons garlic', 'tablespoon fresh basil', 'ingredients', 'ingredients'],
    ['large potatoes', 'cup sour cream', 'ounce cream cheese', 'garlic salt', 'cups Cheddar cheese', 'ingredients', 'ingredients'],
    ['cups graham cracker crumbs', 'cup butter', 'cups confectioners sugar', 'ounce cream cheese', 'ounce pineapple', 'medium bananas', 'ounce container topping', 'maraschino cherries', 'cup chocolate syrup', 'cup pecan halves', 'ingredients', 'ingredients'],
    ['ounce pumpkin puree', 'cup vegetable oil', 'eggs', 'cups purpose flour', 'cups white sugar', 'teaspoons soda', 'teaspoons salt', 'teaspoons ground allspice', 'teaspoons ground nutmeg', 'teaspoons ground cinnamon', 'cup walnuts', 'teaspoon baking powder', 'ingredients', 'ingredients'],
    ['apples', 'cup purpose flour', 'cup white sugar', 'tablespoon baking powder', 'egg', 'tablespoons ground cinnamon', 'cup white sugar', 'tablespoon butter', 'ingredients', 'ingredients'],
    ['ounce peaches juice', 'ounce yellow cake mix', 'cup margarine', 'cup white sugar', 'teaspoon ground cinnamon', 'ingredients', 'ingredients'],
    ['ounce dry penne pasta', 'large cucumbers cubed', 'roma plum tomatoes', 'cups mayonnaise', 'ounce container sour cream', 'tablespoons dill weed', 'tablespoons lemon pepper', 'ingredients', 'ingredients'],
    ['cold water', 'cup kosher salt', 'cup white sugar', 'pound whole chicken pieces', 'pint cherry tomatoes', 'habanero peppers seeded', 'cloves garlic', 'teaspoon ground allspice', 'teaspoon thyme', 'teaspoon ground cumin', 'teaspoon ground black pepper', 'teaspoon cayenne pepper', 'tablespoon vegetable oil', 'tablespoons Thai sweet chili sauce', 'ingredients', 'ingredients'],
    ['pound fresh broccoli spears', 'cups chicken turkey', 'ounce Campbell Broccoli Cheese Soup % Fat Free', 'cup milk', 'cup Cheddar cheese optional', 'tablespoons dry bread crumbs', 'tablespoon butter margarine', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'tablespoons maple syrup', 'teaspoon cayenne pepper', 'large sweet potato inch slices', 'salt pepper', 'ingredients', 'ingredients'],
    ['tablespoon olive oil', 'boneless pork chops', 'ounce chicken broth', 'tablespoons soy sauce', 'tablespoon vinegar', 'tablespoons brown sugar', 'tablespoons cornstarch', 'cup pineapple juice', 'ingredients', 'ingredients'],
    ['cloves garlic', 'red chile pepper', 'tablespoon olive oil', 'teaspoon whole grain mustard', 'tablespoons fresh lime juice', 'sea salt', 'black pepper', 'ounce fillets salmon', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'teaspoons soda', 'teaspoons salt', 'teaspoon baking powder', 'teaspoon ground nutmeg', 'teaspoon ground allspice', 'teaspoons ground cinnamon', 'teaspoon ground cloves', 'cups pumpkin puree', 'cups white sugar', 'cup light brown sugar', 'cup applesauce', 'cup fat free vanilla yogurt', 'egg whites', 'egg', 'cup water', 'cup raisins', 'ingredients', 'ingredients'],
    ['teaspoons olive oil', 'pound sugar snap peas', 'green onions', 'clove garlic', 'teaspoon salt', 'teaspoon pepper', 'tablespoon fresh mint', 'ingredients', 'ingredients'],
    ['cup uncooked white rice', 'cups water', 'tablespoons sesame oil', 'green onions tops', 'cup diced ham', 'cup peas', 'ounce pineapple chunks', 'egg', 'tablespoon white sugar', 'teaspoon salt', 'teaspoon white pepper', 'teaspoon garlic powder', 'cup soy sauce', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'small onion', 'cup celery', 'cup carrots', 'clove garlic', 'cups water', 'cubes vegetable bouillon', 'cups corn', 'soy milk', 'tablespoon flour', 'teaspoon parsley', 'teaspoon garlic powder', 'teaspoon salt', 'teaspoon pepper', 'ingredients', 'ingredients'],
    ['fresh poblano chile peppers', 'pound lean ground beef', 'onion', 'clove garlic', 'salt pepper', 'eggs', 'cup mozzarella cheese', 'roma plum tomatoes', 'cup purpose flour', 'cup corn oil', 'ingredients', 'ingredients'],
    ['tablespoons vegetable oil', 'pound lamb stew meat', 'beef broth', 'cup dry red wine', 'cloves garlic', 'tablespoon fresh thyme', 'teaspoon salt', 'teaspoon black pepper', 'bay leaf', 'cups butternut squash', 'cup parsnips', 'cup sweet potatoes', 'cup celery', 'medium onion', 'cup sour cream', 'tablespoons purpose flour', 'ingredients', 'ingredients'],
    ['pounds fresh small beets', 'cups white sugar', 'tablespoon pickling salt', 'white vinegar', 'cup whole cloves', 'ingredients', 'ingredients'],
    ['ounce package pita bread half', 'tablespoons butter', 'teaspoon garlic', 'teaspoon Italian style seasoning', 'tablespoons Parmesan cheese', 'ingredients', 'ingredients'],
    ['ounce packages spinach', 'ounce dry onion soup mix', 'cups sour cream', 'cup Cheddar cheese', 'ingredients', 'ingredients'],
    ['ounces bratwurst casings', 'potatoes cubed', 'onion', 'cups water', 'medium head cabbage', 'cups milk', 'tablespoons purpose flour', 'ounces Swiss cheese', 'ingredients', 'ingredients'],
    ['cups seedless watermelon', 'ice cubes', 'cup fresh lime juice', 'cup white sugar', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['eggs', 'ounce cans whole pickled beets juice', 'onion', 'cup white sugar', 'cup cider vinegar', 'teaspoon salt', 'pinch ground black pepper', 'bay leaves', 'whole cloves', 'ingredients', 'ingredients'],
    ['ounce cream mushroom soup', 'cup water', 'pounds boneless pork chops', 'teaspoon ground black pepper', 'ounce green beans', 'potatoes cubed', 'ingredients', 'ingredients'],
    ['inch pie shell', 'tablespoons cornstarch', 'tablespoon ground cinnamon', 'fluid ounce apple juice concentrate', 'cups green apples', 'ingredients', 'ingredients'],
    ['medium head cabbage', 'water', 'pound ground beef', 'cup rice', 'garlic powder', 'egg', 'fluid ounce tomato juice', 'tablespoon vinegar', 'tablespoon white sugar', 'water', 'ingredients', 'ingredients'],
    ['cup white sugar', 'tablespoons butter', 'eggs', 'ounce pumpkin puree', 'fluid ounce milk', 'teaspoons vanilla extract', 'cup purpose flour', 'teaspoon baking powder', 'teaspoon salt', 'teaspoon ground cinnamon', 'teaspoon ground allspice', 'teaspoon ground ginger', 'teaspoon ground nutmeg', 'ingredients', 'ingredients'],
    ['apple', 'stalk celery leaves', 'pound whole chicken', 'salt', 'ground black pepper', 'onion', 'teaspoon rosemary', 'lemon', 'cup hot water', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves', 'ounce cans cream mushroom soup', 'cups milk', 'cups uncooked white rice', 'ounce cans mushrooms', 'ounce dry onion soup mix', 'ingredients', 'ingredients'],
    ['fresh broccoli florets', 'cup olive oil', 'cloves garlic', 'pounds pasta', 'teaspoon salt', 'teaspoon red pepper flakes', 'tablespoons Parmesan cheese', 'ingredients', 'ingredients'],
    ['ounce unsweetened strawberry drink mix powder', 'fluid ounce cans pineapple juice', 'cups white sugar', 'quarts water', 'liters ginger ale', 'ingredients', 'ingredients'],
    ['corn tortilla chips', 'vegetarian chili beans', 'cup Cheddar cheese', 'cups lettuce', 'roma plum tomatoes', 'cup salsa', 'cup sour cream', 'ingredients', 'ingredients'],
    ['dry pinto beans', 'cloves garlic', 'teaspoon salt', 'pound bacon', 'onion', 'fresh tomatoes', 'ounce jalapeno peppers', 'fluid ounce beer', 'cup fresh cilantro', 'ingredients', 'ingredients'],
    ['cups pears', 'cup unsweetened pineapple juice', 'cups white sugar', 'ingredients', 'ingredients'],
    ['cup vanilla yogurt', 'tablespoons butter', 'tablespoons vegetable oil', 'egg', 'cups purpose flour', 'cup brown sugar', 'teaspoon baking soda', 'teaspoon salt', 'cup diced rhubarb', 'cup brown sugar', 'teaspoon ground cinnamon', 'teaspoon ground nutmeg', 'cup almonds', 'teaspoons butter', 'ingredients', 'ingredients'],
    ['quart oil frying', 'summer squash chunks', 'medium onion', 'eggs', 'cup purpose flour', 'cup dry corn muffin mix', 'teaspoons powder', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['pound whole beef tenderloin', 'tablespoons olive oil', 'large garlic cloves', 'tablespoons fresh rosemary', 'tablespoon thyme leaves', 'tablespoons black pepper', 'tablespoon salt', 'ingredients', 'ingredients'],
    ['large mushrooms', 'tablespoon olive oil', 'cloves garlic', 'tablespoons green onions', 'ounce package cream cheese', 'tablespoons port wine', 'teaspoon Italian style seasoning', 'cup Parmesan cheese', 'cup Cheddar cheese', 'teaspoon ground black pepper', 'hot pepper sauce', 'ingredients', 'ingredients'],
    ['acorn squash seeded', 'salt pepper', 'cup butter', 'tablespoons brown sugar', 'teaspoon ground cinnamon', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cup white sugar', 'teaspoon baking soda', 'teaspoon salt', 'teaspoon ground nutmeg', 'teaspoon ground cinnamon', 'cup shortening', 'egg', 'cup ripe bananas', 'cups oats', 'cup walnuts', 'ingredients', 'ingredients'],
    ['apples', 'cups white sugar', 'teaspoons ground cinnamon', 'teaspoon ground cloves', 'teaspoon salt', 'ingredients', 'ingredients'],
    ['cup olive oil', 'onion', 'cloves garlic', 'green bell pepper', 'fresh red chile pepper seeded', 'cup fresh parsley', 'salt pepper', 'teaspoons basil', 'teaspoon oregano', 'teaspoon thyme', 'ounce tomatoes', 'ounce tomato sauce', 'cup water', 'pinch paprika', 'cayenne pepper', 'cup white wine', 'ounce clams juice', 'mussels', 'shrimp', 'ounces scallops', 'pound cod fillets', 'ingredients', 'ingredients'],
    ['ounce yellow cake mix', 'egg', 'cup butter', 'ounce pumpkin puree', 'eggs', 'teaspoon ground nutmeg', 'teaspoon ground cinnamon', 'cup white sugar', 'cup milk', 'cup white sugar', 'cup butter', 'ingredients', 'ingredients'],
    ['cups shredded apple', 'cups white sugar', 'eggs', 'cup walnuts', 'cups purpose flour', 'teaspoons ground cinnamon', 'teaspoon ground nutmeg', 'teaspoon salt', 'teaspoons soda', 'cup vegetable oil', 'ingredients', 'ingredients'],
    ['ounce package self cornmeal', 'egg', 'cup milk', 'ounce cream style corn', 'cup Monterey Jack cheese', 'ounce green chile peppers', 'ingredients', 'ingredients'],
    ['eggplant seeded inch slices', 'red bell pepper thin strips', 'salt pepper', 'French baguette', 'ounces soft goat cheese', 'cup tapenade olive spread', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['shallots', 'teaspoons fresh ginger root', 'lemon', 'cup dry white wine', 'cup heavy cream', 'cup unsalted butter chilled small cubes', 'tablespoons soy sauce', 'shiso leaves', 'coarse kosher salt', 'white pepper', 'tablespoons canola oil', 'ounce fillets', 'tablespoons sesame seeds', 'tablespoons black sesame seeds', 'ingredients', 'ingredients'],
    ['pastry inch double crust pie', 'cup white sugar', 'teaspoons tapioca', 'teaspoon ground cinnamon', 'cups fresh blackberries', 'cups apples', 'tablespoons butter small pieces', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'large onion', 'medium potatoes', 'cups raw radish greens', 'chicken broth', 'cup heavy cream', 'radishes', 'ingredients', 'ingredients'],
    ['cup elbow macaroni', 'pound ground beef', 'small onion', 'cup celery', 'large green bell pepper', 'ounce kidney beans', 'ounce cans tomato soup', 'ounce cans tomatoes', 'cup brown sugar', 'salt pepper', 'ingredients', 'ingredients'],
    ['ounce package lasagna noodles', 'tablespoons olive oil', 'clove garlic', 'pound baby portobello mushrooms', 'ounce jars Alfredo style pasta sauce', 'pound shrimp deveined', 'pound bay scallops', 'pound imitation crabmeat', 'ounces cheese', 'egg', 'black pepper', 'cups Italian cheese blend', 'ingredients', 'ingredients'],
    ['ounce packages Oriental ramen noodles', 'eggs', 'vegetable oil', 'green onions', 'small carrot grated', 'cup green peas', 'cup red bell pepper', 'tablespoons sesame oil', 'soy sauce', 'ingredients', 'ingredients'],
    ['banana', 'tablespoon chocolate syrup', 'cup milk', 'cup ice', 'ingredients', 'ingredients'],
    ['ounce package lemon cake mix', 'ounce package lemon Jell O mix', 'cup vegetable oil', 'cup water', 'eggs', 'cup lemon juice', 'cups confectioners sugar', 'tablespoons butter', 'tablespoon water', 'ingredients', 'ingredients'],
    ['quart water', 'orange spice tea bags', 'cup light brown sugar', 'cups apple cider', 'cups light rum', 'cinnamon sticks', 'teaspoons butter', 'ingredients', 'ingredients'],
    ['large fresh mushrooms', 'ounces spinach', 'tablespoon feta cheese', 'ounce fillets', 'ingredients', 'ingredients'],
    ['apples', 'cups white sugar', 'cup light corn syrup', 'cups water', 'red food coloring', 'ingredients', 'ingredients'],
    ['ounce vegetarian ground beef Gimme Lean TM', 'ounce vegetarian burger crumbles', 'onion', 'eggs', 'tablespoons vegetarian Worcestershire sauce', 'teaspoon salt', 'teaspoon pepper', 'teaspoon ground sage', 'teaspoon garlic powder', 'teaspoons prepared mustard', 'tablespoon vegetable oil', 'slices bread', 'cup milk', 'ounce tomato sauce', 'carrots inch pieces', 'potatoes', 'cooking spray', 'ingredients', 'ingredients'],
    ['cup walnuts', 'tablespoons olive oil', 'onions', 'pound fresh mushrooms', 'chicken broth', 'cups uncooked orzo pasta', 'salt pepper', 'ingredients', 'ingredients'],
    ['cups dates', 'teaspoon baking soda', 'cup water', 'cup butter', 'cup white sugar', 'eggs', 'cups self flour', 'cup brown sugar', 'cup butter', 'cup milk', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['skinless boneless chicken breast halves thin', 'tablespoons olive oil', 'onion', 'cloves garlic', 'pound tomatillos', 'jalapeno peppers', 'cups chicken stock', 'teaspoon cayenne pepper', 'teaspoon hot pepper sauce', 'tablespoons fresh cilantro', 'cup sour cream optional', 'salt', 'ground black pepper', 'ingredients', 'ingredients'],
    ['ground pork', 'Granny Smith apple', 'sweet onion', 'cloves garlic', 'cup teriyaki sauce', 'egg', 'hamburger buns', 'ounce pineapple', 'ingredients', 'ingredients'],
    ['cup butter', 'cup mayonnaise', 'bunch green onions', 'cloves garlic', 'cups Parmesan cheese', 'cups Monterey Jack cheese', 'pound loaf French bread', 'ingredients', 'ingredients'],
    ['new potatoes', 'cup Italian style salad dressing', 'cup mayonnaise', 'cup green onions', 'teaspoons fresh dill', 'teaspoon Dijon mustard optional', 'teaspoon lemon juice', 'teaspoon pepper', 'ingredients', 'ingredients'],
    ['pound ground beef', 'cup onion', 'cup green bell pepper', 'cup carrots', 'cloves garlic', 'ounce tomato sauce', 'teaspoon oregano', 'teaspoon basil', 'salt pepper', 'medium zucchini inch thick strips', 'cup cottage cheese', 'egg', 'cup mozzarella cheese', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['head cauliflower large florets', 'tablespoons butter more', 'salt black pepper', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'cloves garlic', 'large tomatoes', 'teaspoon oregano', 'pound Alaskan cod', 'pound large shrimp deveined', 'salt', 'tablespoon oregano', 'ingredients', 'ingredients'],
    ['ounces whole wheat rotini pasta', 'ounce frozen broccoli', 'tablespoon olive oil', 'cups ham', 'ounce jar Alfredo sauce', 'cup % milk', 'ground black pepper', 'cup Colby Monterey Jack cheese', 'ingredients', 'ingredients'],
    ['pound loaf Italian bread', 'fresh basil leaves', 'tomatoes', 'ounces fresh mozzarella cheese', 'teaspoon red pepper flakes', 'cup balsamic vinegar', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'cup oats', 'teaspoons powder', 'teaspoon baking soda', 'teaspoon ground cinnamon', 'teaspoon salt', 'egg', 'cup milk', 'cup light olive oil', 'cup sugar', 'pint fresh strawberries', 'ingredients', 'ingredients'],
    ['cup butter', 'teaspoon garlic salt', 'teaspoon garlic pepper', 'tablespoons almonds', 'tablespoons white wine', 'medium onion', 'medium red bell pepper', 'cups broccoli', 'cups cauliflower', 'ingredients', 'ingredients'],
    ['pound whole chicken', 'tablespoons', 'lemon', 'tablespoon paprika', 'cup water', 'ingredients', 'ingredients'],
    ['ounce packages broccoli', 'ounce cans cream mushroom soup', 'instant rice', 'cup onion', 'cup butter', 'ounce jar process cheese sauce', 'salt', 'ground black pepper', 'ingredients', 'ingredients'],
    ['ounce package potato flakes', 'pounds ground beef', 'onion', 'ounce whole kernel corn', 'ounce green beans', 'ounce cans brown gravy', 'salt pepper', 'pound American cheese slices', 'ingredients', 'ingredients'],
    ['ounce package pound cake mix', 'eggs', 'tablespoons butter', 'teaspoons pumpkin pie spice', 'ounce package cream cheese', 'ounce condensed milk', 'ounce pumpkin puree', 'teaspoon salt', 'cup pecans', 'ingredients', 'ingredients'],
    ['cooking spray', 'eggplants inch thick', 'teaspoons salt', 'eggs', 'cups bread crumbs', 'tablespoons Italian herb seasoning', 'tablespoons olive oil', 'onion', 'cloves garlic', 'cups pasta sauce', 'cups Italian cheese blend', 'cup Parmesan cheese', 'ingredients', 'ingredients'],
    ['cup butternut squash', 'teaspoon salt', 'teaspoon black pepper', 'cayenne pepper', 'cup mascarpone cheese', 'egg yolk', 'cup Parmesan cheese', 'ounce package wonton wrappers', 'tablespoons butter', 'clove garlic unpeeled', 'fresh sage', 'tablespoon Parmesan cheese', 'ingredients', 'ingredients'],
    ['cup apricot preserves', 'cup fruity white wine such Gewurztraminer', 'teaspoon ground ginger', 'salt pepper', 'tablespoons olive oil', 'boneless pork chops inch thick', 'ingredients', 'ingredients'],
    ['teaspoon canola oil', 'tablespoon garlic', 'cup diced yellow purple onion', 'cups water', 'teaspoons salt', 'teaspoon ground black pepper', 'cups quinoa', 'cup fresh tomato', 'cup carrots', 'cup yellow bell pepper', 'cup diced cucumber', 'cup frozen corn kernels', 'cup red onion', 'tablespoons fresh cilantro', 'tablespoon fresh mint', 'teaspoon salt', 'teaspoon ground black pepper', 'tablespoons olive oil', 'tablespoons balsamic vinegar', 'ingredients', 'ingredients'],
    ['potatoes cubed', 'pounds ground beef', 'ounce tomato sauce', 'tablespoons fresh parsley', 'Worcestershire sauce', 'teaspoon salt', 'teaspoon ground black pepper', 'ounce cans green beans', 'tablespoons purpose flour', 'tablespoons milk', 'ingredients', 'ingredients'],
    ['tablespoons purpose flour', 'skinless boneless chicken breast halves thin', 'cup unsalted butter', 'fresh mushrooms', 'cup dry white wine', 'cup chicken stock', 'tablespoons fresh parsley', 'teaspoon white pepper', 'slices prosciutto', 'fontina cheese', 'ingredients', 'ingredients'],
    ['cup uncooked white rice', 'ounce dry onion soup mix', 'ounce cans cream mushroom soup', 'ounce mushrooms', 'cup milk', 'skinless boneless chicken breast halves', 'ingredients', 'ingredients'],
    ['cups purpose flour', 'cups white sugar', 'teaspoon baking soda', 'teaspoon salt', 'cups canola oil', 'eggs', 'ounce pineapple', 'cups bananas', 'cup black walnuts', 'ounce package cream cheese', 'pound butter', 'pound confectioners sugar', 'teaspoon vanilla extract', 'ingredients', 'ingredients'],
    ['pounds ground beef', 'onion', 'cup instant rice', 'cup water', 'ounce tomatoes juice', 'ounce tomatoes', 'salt pepper', 'large green bell peppers', 'ounce jar spaghetti sauce', 'ingredients', 'ingredients'],
    ['zucchini inch slices', 'tablespoons butter', 'tablespoons lemon pepper', 'ingredients', 'ingredients'],
    ['pound sausage inch pieces', 'tablespoon margarine', 'zucchini half', 'tablespoon instant minced onion', 'teaspoon garlic salt', 'teaspoon oregano', 'teaspoon ground black pepper', 'tomatoes', 'ingredients', 'ingredients'],
    ['cups cabbage', 'carrot', 'onion', 'red bell pepper', 'jalapeno pepper seeded diced', 'tablespoon fresh cilantro', 'cup canola oil', 'tablespoons apple cider vinegar', 'tablespoon white sugar', 'teaspoon cayenne pepper', 'salt pepper', 'ingredients', 'ingredients'],
    ['cup purpose flour', 'pinch salt', 'tablespoons white sugar', 'cup milk', 'eggs', 'large bananas', 'tablespoons vegetable oil', 'tablespoon butter', 'ingredients', 'ingredients'],
    ['pounds skinless boneless chicken breast halves cut inch strips', 'tablespoons bacon bits', 'cup green olives', 'ounce tomatoes', 'ounce mushrooms', 'ounce envelope dry chicken gravy mix', 'cup red wine', 'tablespoons Dijon mustard', 'cup balsamic vinegar', 'ingredients', 'ingredients'],
    ['tablespoons olive oil', 'large heads escarole', 'salt pepper', 'teaspoon red pepper flakes', 'clove garlic', 'ounce cans beans undrained', 'sprigs fresh parsley', 'ingredients', 'ingredients'],
    ['large portobello mushroom', 'tablespoon spaghetti sauce', 'cup mozzarella cheese', 'tablespoon black olives', 'slices sausage', 'clove garlic', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'tablespoons extra-virgin olive oil', 'large onion', 'large Yukon Gold potatoes inch cubes', 'large carrot', 'pounds corned beef', 'tablespoons fresh parsley', 'teaspoon thyme leaves', 'salt optional', 'teaspoon ground black pepper', 'ingredients', 'ingredients'],
    ['slices whole wheat bread', 'teaspoons olive oil', 'Granny Smith apple', 'cup Swiss cheese', 'ingredients', 'ingredients'],
    ['pound penne pasta', 'tablespoons extra-virgin olive oil', 'pound bulk Italian sausage', 'pound cremini mushrooms', 'cloves garlic', 'cups heavy cream', 'teaspoon salt', 'teaspoon black pepper', 'cup Parmesan cheese', 'ounce package whole milk mozzarella', 'ingredients', 'ingredients'],
    ['tablespoons butter', 'cloves garlic', 'teaspoon red pepper flakes', 'lemon', 'cups white wine', 'black pepper', 'mussels', 'cup fresh flat leaf parsley', 'slices bread', 'lemon wedges garnish', 'ingredients', 'ingredients'],
    ['large potatoes', 'cup onion', 'tablespoons butter', 'cup purpose flour', 'cups low sodium chicken broth', 'tablespoons mayonnaise', 'cloves garlic', 'teaspoon salt', 'teaspoon parsley', 'teaspoon poultry seasoning', 'teaspoon ground black pepper', 'cup fresh chives', 'ingredients', 'ingredients'],
]

In [10]:
len(ingredients_google_nouns_adjs)

2119

In [12]:
urls_all = df['url'].values
print len(urls_all)
print urls_all.shape

print urls_all[2119]

# url_to_noun_adj_dict = {}
# for i, ingred_noun_adj_list in enumerate(ingredients_google_nouns_adjs):
#     url_to_noun_adj_dict[]

89061
(89061,)
http://allrecipes.com/recipe/76963/sausage-white-bean-soup/


In [28]:
df['POS_cleaned_ingredients'] = np.empty((len(df),0)).tolist()

for idx, row in ProgressBar(df.iterrows(), desc="Processing recipes"):
    ingreds_list = row['ingredients']
    noun_adj_ingreds_list = reduce_ingreds_list(ingreds_list)
    df['POS_cleaned_ingredients'].iloc[idx] = noun_adj_ingreds_list


# class BreakoutException(Exception): pass  # declare a label

# try:
#     for idx, row in ProgressBar(df.iterrows(), desc="Processing recipes"):
#         ingreds_list = row['ingredients']
#         noun_adj_ingreds_list = reduce_ingreds_list(ingreds_list)
#         df['POS_cleaned_ingredients'].iloc[idx] = noun_adj_ingreds_list
#         try:
#             noun_adj_ingreds_list = reduce_ingreds_list(ingreds_list)
#             df['POS_cleaned_ingredients'].iloc[idx] = noun_adj_ingreds_list
#         except KeyboardInterrupt:
#             raise BreakoutException
#         except: 
#             print idx 
#             print ingreds_list
#             raise BreakoutException
# except BreakoutException:
#     pass





Processing recipes: 0it [00:00, ?it/s]




Processing recipes: 1it [00:05,  5.47s/it]


Processing recipes: 2it [00:11,  5.76s/it]


Processing recipes: 3it [00:16,  5.56s/it]


Processing recipes: 4it [00:24,  6.28s/it]


Processing recipes: 5it [00:31,  6.28s/it]


Processing recipes: 6it [00:40,  7.31s/it]


Processing recipes: 7it [00:48,  7.39s/it]


Processing recipes: 8it [00:57,  7.75s/it]


Processing recipes: 9it [01:01,  6.65s/it]


Processing recipes: 10it [01:07,  6.48s/it]


Processing recipes: 11it [01:12,  6.01s/it]


Processing recipes: 12it [01:18,  6.19s/it]


Processing recipes: 13it [01:27,  6.96s/it]


Processing recipes: 14it [01:34,  6.92s/it]


Processing recipes: 15it [01:41,  6.99s/it]


Processing recipes: 16it [01:49,  7.37s/it]


Processing recipes: 17it [01:55,  6.75s/it]


Processing recipes: 18it [02:01,  6.64s/it]


Processing recipes: 19it [02:09,  7.02s/it]


Processing recipes: 20it [02:19,  8.09s/it]


Processing recipes: 21it [02:24,  7.02s/it]

RetryError: RetryError(Exception occurred in retry method that was not classified as transient, caused by <_Rendezvous of RPC that terminated with (StatusCode.INVALID_ARGUMENT, The language tr is not supported for syntax analysis.)>)

In [29]:
df.head(100)

,categories,cookingTime,description,ingredients,instructionSteps,name,rating,ratingCount,url,cookingTimeMinutes,cleanedIngredients,POS_cleaned_ingredients
0,"[Tomato Salsa, Salsa, Dips and Spreads, Appetizers and Snacks, Mexican Recipes, Everyday Cooking, Recipes, Summer Appetizers, Mexican Appetizers, Cilantro]",PT15M,NaN,"[6 roma (plum) tomatoes, diced, 1 sweet onion, diced, 1 medium red bell pepper, diced, 1 medium yellow bell pepper, diced, 1 bunch cilantro, finely minced, 1 lime, juiced, 1 teaspoon salt, or to taste, Add all ingredients to list, Add all ingredients to list]","[In a bowl, mix the tomatoes, onion, red bell pepper, yellow bell pepper, cilantro, lime juice, and salt. Cover and refrigerate until ready to serve.]",Fresco Salsa,4.61,61,http://allrecipes.com/recipe/88328/fresco-salsa/,15.0,"[roma plum tomatoes, sweet onion, red bell pepper, yellow bell pepper, cilantro, lime, salt]","[roma plum tomatoes, sweet onion, medium red bell pepper, medium yellow bell pepper, bunch cilantro, lime, teaspoon salt, ingredients, ingredients]"
1,"[Kids Healthy Snacks, Kids' Snacks, Snack Recipes, Salsa, Dips and Spreads, Appetizers and Snacks, Mexican Recipes, Everyday Cooking, Recipes, Kids' Nut-Free Snacks]",PT45M,"Easy to make, tasty fruit salsa and cinnamon tortilla chips. Great as an appetizer or a snack. Great for anytime!","[2 kiwis, peeled and diced, 2 Golden Delicious apples - peeled, cored and diced, 8 ounces raspberries, 1 pound strawberries, 2 tablespoons white sugar, 1 tablespoon brown sugar, 3 tablespoons fruit preserves, any flavor, 10 (10 inch) flour tortillas, butter flavored cooking spray, 2 tablespoons cinnamon sugar, Add all ingredients to list, Add all ingredients to list]","[In a large bowl, thoroughly mix kiwis, Golden Delicious apples, raspberries, strawberries, white sugar, brown sugar and fruit preserves. Cover and chill in the refrigerator at least 15 minutes., Preheat oven to 350 degrees F (175 degrees C)., Coat one side of each flour tortilla with butter flavored cooking spray. Cut into wedges and arrange in a single layer on a large baking sheet. Sprinkle wedges with desired amount of cinnamon sugar. Spray again with cooking spray., Bake in the preheated oven 8 to 10 minutes. Repeat with any remaining tortilla wedges. Allow to cool approximately 15 minutes. Serve with chilled fruit mixture.]",Annie's Fruit Salsa and Cinnamon Chips,4.78,3239,http://allrecipes.com/recipe/26692/annies-fruit-salsa-and-cinnamon-chips/,45.0,"[kiwis, golden delicious apples, raspberries, strawberries, white sugar, brown sugar, fruit preserves flavor, flour tortillas, butter flavored cooking spray, cinnamon sugar]","[kiwis diced, Golden Delicious apples diced, ounces raspberries, pound strawberries, tablespoons white sugar, tablespoon brown sugar, tablespoons fruit preserves flavor, inch flour tortillas, butter cooking spray, tablespoons cinnamon sugar, ingredients, ingredients]"
2,"[Potatoes Au Gratin, Potato Side Dishes, Side Dishes, French Recipes, Everyday Cooking, Recipes, Easter Side Dishes, Allrecipes Magazine Side Dishes, Gourmet Side Dishes, Potato Recipes]",PT2H,"This is my husband's favorite dish, and he considers it a special occasion every time I make it. The creamy cheese sauce and the tender potatoes in this classic French dish combine to make a deliciously addictive experience. It's a great side dish with a roast pork loin or beef tenderloin. Add a green salad and French bread, and you have found the magic path to a man's heart. To avoid lumps in your sauce, add the milk just a little at a time as you stir the flour and butter. Experiment with different cheeses for variety.","[4 russet potatoes, sliced into 1/4 inch slices, 1 onion, sliced into rings, salt and pepper to taste, 3 tablespoons butter, 3 tablespoons all-purpose flour, 1/2 teaspoon salt, 2 cups milk, 1 1/2 cups shredded Cheddar cheese, Add all ingredients to list, Add all ingredients to list]","[Preheat oven to 400 degrees F (200 degrees C). Bu

In [30]:
df.to_pickle("google_pos_possible_error.pkl")

In [26]:
#%%time

df['further_cleaned_ingredients'] = df['POS_cleaned_ingredients'].apply(extract_ingredients)

df.sample(5)

length of cleaned_ingred_list is zero
full_ingred_list:
[]


IndexError: list index out of range